In [ ]:
!pip install flwr["simulation"] torch pandas scikit-learn -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.1/250.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.0/732.0 kB 41.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 44.0.3 which is i

##Imports and Model Architecture Definitions

In [ ]:
# Cell 1 (or 4): Model Architecture Definitions

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import pandas as pd
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from collections import OrderedDict
import flwr as fl
import numpy as np
from typing import Dict, List, Tuple
from flwr.common import Context

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# ========== Model Components ==========
class SensorLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=32):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
    def forward(self, x):
        if x.ndim == 2: x = x.unsqueeze(1)
        out, _ = self.lstm(x)
        return out[:, -1, :]

class DomainScalingMLP(nn.Module):
    def __init__(self, input_dim=1, n_sensor_stream=5, hidden=16):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(input_dim, hidden), nn.ReLU(), nn.Linear(hidden, n_sensor_stream), nn.Sigmoid())
    def forward(self, domain_disc): return self.net(domain_disc)

class CrossSensorAttention(nn.Module):
    def __init__(self, n_sensor_stream=5, hidden_dim=32, fusion_dim=64):
        super().__init__()
        self.query, self.key, self.value = nn.Linear(hidden_dim, fusion_dim), nn.Linear(hidden_dim, fusion_dim), nn.Linear(hidden_dim, fusion_dim)
        self.scaler = DomainScalingMLP(input_dim=1, n_sensor_stream=n_sensor_stream, hidden=16)
    def forward(self, features, domain_disc, debug=False):
        x = torch.stack(features, dim=1)
        Q, K, V = self.query(x), self.key(x), self.value(x)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / (K.size(-1) ** 0.5)
        scaling = self.scaler(domain_disc).unsqueeze(1)
        scaled_scores = scores * scaling.transpose(-2, -1)
        attn = F.softmax(scaled_scores, dim=-1)
        fused = torch.matmul(attn, V).mean(dim=1)
        return fused

class DomainAdaptiveLayer(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.fc = nn.Linear(dim, dim)
        self.bn = nn.Identity()
    def forward(self, x):
        return self.bn(self.fc(x))

class FMAL_Daf(nn.Module):
    def __init__(self, sensor_dim, lstm_hidden=32, fusion_dim=64, num_sessions=3, use_time_and_session=True):
        super().__init__()
        self.use_time_and_session = use_time_and_session
        self.sensor_lstms = nn.ModuleList([SensorLSTM(d, lstm_hidden) for d in sensor_dim])
        self.attn_fusion = CrossSensorAttention(n_sensor_stream=len(sensor_dim), hidden_dim=lstm_hidden, fusion_dim=fusion_dim)
        self.domain_adapt = DomainAdaptiveLayer(fusion_dim)
        self.session_emb = nn.Embedding(num_sessions, 8)
        self.time_enc = nn.Sequential(nn.Linear(1, 8), nn.ReLU(), nn.Linear(8, 8))
        global_input_dim = fusion_dim + 16 if use_time_and_session else fusion_dim
        self.global_lstm = nn.LSTM(global_input_dim, 32, batch_first=True)
        self.reg_phys = nn.Linear(32, 1)
        self.reg_ment = nn.Linear(32, 1)
        self.apply(self._init_weights)
    @staticmethod
    def _init_weights(m):
        if isinstance(m, (nn.Linear)):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None: nn.init.zeros_(m.bias)
        elif isinstance(m, nn.LSTM):
            for name, param in m.named_parameters():
                if 'weight' in name: nn.init.xavier_uniform_(param)
                elif 'bias' in name: nn.init.zeros_(param)
    def forward(self, sensor_inputs, domain_disc, session_feat=None, elapsed_feat=None, debug=False):
        feats = [m(inp) for m, inp in zip(self.sensor_lstms, sensor_inputs)]
        fused = self.attn_fusion(feats, domain_disc, debug=debug)
        adapted = self.domain_adapt(fused)
        if self.use_time_and_session:
            session_emb = self.session_emb(session_feat.squeeze(-1).long())
            time_emb = self.time_enc(elapsed_feat)
            adapted = torch.cat([adapted, session_emb, time_emb], dim=1)
        glstm_out, _ = self.global_lstm(adapted.unsqueeze(1))
        feat = glstm_out[:, -1, :]
        return self.reg_phys(feat), self.reg_ment(feat)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


##Data Pipeline for New Dataset

In [ ]:
# Cell 2: Data Pipeline (Robust 20-Person Train / 5-Person Test Split)

from google.colab import drive
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# --- Configuration for your new dataset ---
NEW_DATA_PATH = '/content/drive/MyDrive/0_SWELL/final_SWELL_like_main_dataset.csv'
sensor_dim = [2, 2, 2, 2, 2]
NUM_SENSORS = len(sensor_dim)
NUM_CLIENTS = 10 # We will create 10 clients from the training data
# ---

class NewFatigueDataset(Dataset):
    def __init__(self, df, session_map):
        self.data = df.reset_index(drop=True)
        self.session_map = session_map
        self.noise_level = 0.010 # The winning data augmentation strategy

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx].copy() # Use .copy()

        # Apply On-the-fly Data Augmentation
        row['HR_mean'] += np.random.normal(0, self.noise_level)
        row['RMSSD_mean'] += np.random.normal(0, self.noise_level)
        row['SCL_mean'] += np.random.normal(0, self.noise_level)
        row['Svalence_mean'] += np.random.normal(0, self.noise_level)
        row['Sneutral_mean'] += np.random.normal(0, self.noise_level)

        HR = torch.tensor([row['HR_mean'], row['HR_std']], dtype=torch.float32)
        RMSSD = torch.tensor([row['RMSSD_mean'], row['RMSSD_std']], dtype=torch.float32)
        SCL = torch.tensor([row['SCL_mean'], row['SCL_std']], dtype=torch.float32)
        SVALENCE = torch.tensor([row['Svalence_mean'], row['Svalence_std']], dtype=torch.float32)
        SNEUTRAL = torch.tensor([row['Sneutral_mean'], row['Sneutral_std']], dtype=torch.float32)
        sensor_inputs = [HR, RMSSD, SCL, SVALENCE, SNEUTRAL]

        phys = torch.tensor([row['physicalFatigueScore']], dtype=torch.float32)
        ment = torch.tensor([row['mentalFatigueScore']], dtype=torch.float32)
        session_index = self.session_map.get(row['session'], 0)
        session_feat = torch.tensor([float(session_index)], dtype=torch.float32)
        elapsed = torch.tensor([0.0], dtype=torch.float32)
        domain_disc = torch.tensor([0.0], dtype=torch.float32)
        return sensor_inputs, domain_disc, session_feat, elapsed, (phys, ment)

def collate(batch):
    sensor_inputs = [torch.stack([b[0][i] for b in batch]) for i in range(NUM_SENSORS)]
    domain_disc  = torch.stack([b[1] for b in batch])
    session_feat = torch.stack([b[2] for b in batch])
    elapsed_feat = torch.stack([b[3] for b in batch])
    phys         = torch.stack([b[4][0] for b in batch])
    ment         = torch.stack([b[4][1] for b in batch])
    return sensor_inputs, domain_disc, session_feat, elapsed_feat, (phys, ment)

def load_data(is_federated=True):
    """
    Loads and prepares the new dataset using a robust 20-person train / 5-person test split.
    """
    drive.mount('/content/drive')
    df = pd.read_csv(NEW_DATA_PATH)

    df['window_start'] = pd.to_datetime(df['window_start'])
    all_sessions = sorted(df['session'].unique())
    session_map = {s: i for i, s in enumerate(all_sessions)}
    num_sessions = len(session_map)

    # --- NEW ROBUST SPLIT LOGIC ---
    all_persons = df['person'].unique()
    np.random.shuffle(all_persons) # Shuffle to ensure random split

    # Use 5 persons for test set, and the rest for training
    test_persons = all_persons[:5]
    train_persons = all_persons[5:]

    print(f"Total participants found: {len(all_persons)}")
    print(f"Training on {len(train_persons)} participants.")
    print(f"Testing on {len(test_persons)} holdout participants.")

    # Create the single, robust test dataset from the 5 holdout persons
    test_df = df[df['person'].isin(test_persons)]
    test_dataset = NewFatigueDataset(test_df, session_map)
    testloader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate)

    if is_federated:
        print(f"Loading FEDERATED data for {NUM_CLIENTS} clients...")
        trainloaders = []

        # Split the 20 training persons into 10 groups of 2
        client_person_groups = np.array_split(train_persons, NUM_CLIENTS)

        # Create a dataloader for each client group
        for i, group in enumerate(client_person_groups):
            print(f"  - Client {i}: persons {list(group)}")
            client_df = df[df['person'].isin(group)]
            client_dataset = NewFatigueDataset(client_df, session_map)
            client_trainloader = DataLoader(client_dataset, batch_size=32, shuffle=True, collate_fn=collate, drop_last=True)
            trainloaders.append(client_trainloader)

        valloaders = [testloader] * len(trainloaders)

        return trainloaders, valloaders, testloader, num_sessions

    else:
        # CENTRALIZED: Train on the *same* 20 people
        print("Loading CENTRALIZED data...")

        # Create one big training set from the 20 train persons
        train_df = df[df['person'].isin(train_persons)]
        train_dataset = NewFatigueDataset(train_df, session_map)

        trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate, drop_last=True)

        # Testloader is already created from the 5 holdout persons
        return trainloader, testloader, num_sessions
    # --- END OF NEW LOGIC ---

#Centralized (with AFDA)

##Centralized Training and Evaluation Functions

In [ ]:
# Cell 3: Centralized Training and Evaluation Functions (CORRECTED)

def train_centralized(net, trainloader, epochs):
    """A standard, centralized training loop."""
    print(f"Starting centralized training for {epochs} epochs...")
    net.train()
    optimizer = torch.optim.Adam(net.parameters(), lr=5e-4)
    criterion = nn.MSELoss()

    for epoch in range(1, epochs + 1):
        epoch_loss = 0.0
        for i, (sensors, dom, s_feat, e_feat, (phys, ment)) in enumerate(trainloader):
            sensors = [m.to(DEVICE) for m in sensors]
            dom, s_feat, e_feat = dom.to(DEVICE), s_feat.to(DEVICE), e_feat.to(DEVICE)
            phys, ment = phys.to(DEVICE), ment.to(DEVICE)

            optimizer.zero_grad()
            p_pred, m_pred = net(sensors, dom, s_feat, e_feat)
            loss = criterion(p_pred, phys) + 2 * criterion(m_pred, ment)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(trainloader)
        if epoch % 10 == 0 or epoch == 1:
            print(f"Epoch {epoch}/{epochs}, Average Training Loss: {avg_loss:.6f}")

def test_centralized(net, testloader):
    """Validates the model on the test set and calculates all metrics."""
    net.eval()
    criterion = nn.MSELoss()
    total_loss, all_phys_preds, all_phys_labels, all_ment_preds, all_ment_labels = 0.0, [], [], [], []

    # --- ADDED ROBUSTNESS CHECK ---
    if testloader is None or len(testloader) == 0:
        print("Warning: test_centralized received an empty testloader.")
        metrics = {"loss": 0.0, "rmse_physical": 99.0, "mae_physical": 99.0, "r2_physical": -99.0,
                   "rmse_mental": 99.0, "mae_mental": 99.0, "r2_mental": -99.0}
        return 0.0, metrics
    # --- END OF CHECK ---

    with torch.no_grad():
        for sensors, dom, s_feat, e_feat, (phys, ment) in testloader:
            sensors = [m.to(DEVICE) for m in sensors]
            dom, s_feat, e_feat = dom.to(DEVICE), s_feat.to(DEVICE), e_feat.to(DEVICE)
            phys, ment = phys.to(DEVICE), ment.to(DEVICE)

            p_pred, m_pred = net(sensors, dom, s_feat, e_feat)
            total_loss += (criterion(p_pred, phys) + criterion(m_pred, ment)).item()
            all_phys_preds.extend(p_pred.squeeze().cpu().numpy())
            all_phys_labels.extend(phys.squeeze().cpu().numpy())
            all_ment_preds.extend(m_pred.squeeze().cpu().numpy())
            all_ment_labels.extend(ment.squeeze().cpu().numpy())

    loss = total_loss / len(testloader)

    # --- ADDED NAN/EMPTY CHECK ---
    if not all_phys_labels or np.isnan(all_phys_preds).any():
        print("Warning: NaN or empty lists detected in test_centralized. Returning default metrics.")
        metrics = {"loss": loss, "rmse_physical": 99.0, "mae_physical": 99.0, "r2_physical": -99.0,
                   "rmse_mental": 99.0, "mae_mental": 99.0, "r2_mental": -99.0}
        return loss, metrics
    # --- END OF CHECK ---

    # Calculate all metrics after the loop
    rmse_phys = math.sqrt(mean_squared_error(all_phys_labels, all_phys_preds))
    mae_phys = mean_absolute_error(all_phys_labels, all_phys_preds)
    r2_phys = r2_score(all_phys_labels, all_phys_preds)
    rmse_ment = math.sqrt(mean_squared_error(all_ment_labels, all_ment_preds))
    mae_ment = mean_absolute_error(all_ment_labels, all_ment_preds)
    r2_ment = r2_score(all_ment_labels, all_ment_preds)

    metrics = {
        "loss": loss,
        "rmse_physical": rmse_phys, "mae_physical": mae_phys, "r2_physical": r2_phys,
        "rmse_mental": rmse_ment, "mae_mental": mae_ment, "r2_mental": r2_ment
    }
    return loss, metrics

print("Centralized train and test functions defined.")

Centralized train and test functions defined.


##Train model and evaluate

In [ ]:
# Cell 4: Run Experiment 1 - Centralized with Novelty (with JSON Logging)
import json # <-- ADDED IMPORT

print("--- Starting Experiment 1: Centralized with Novelty ---")
NUM_EPOCHS = 260

# 1. Load data
try:
    trainloader, testloader, num_sessions = load_data(is_federated=False)
    print(f"Data loaded. Training samples: {len(trainloader.dataset)}, Test samples: {len(testloader.dataset)}")
    print(f"Discovered {num_sessions} unique sessions.")
except Exception as e:
    print(f"Error loading data: {e}. Please ensure Cell 2 ran successfully.")
    raise e

# 2. Initialize the model
model_with_novelty = FMAL_Daf(
    sensor_dim=sensor_dim,
    lstm_hidden=32,
    fusion_dim=64,
    num_sessions=num_sessions,
    use_time_and_session=True
).to(DEVICE)

# --- NEW: Initialize history dictionary ---
metrics_to_track = ["rmse_physical", "mae_physical", "r2_physical", "rmse_mental", "mae_mental", "r2_mental", "loss"]
history = {metric: [] for metric in metrics_to_track}
# ---

# 3. Train the model (MODIFIED to log every epoch)
print(f"Starting centralized training for {NUM_EPOCHS} epochs...")
net = model_with_novelty
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4) # Using the 5e-4 LR for centralized
criterion = nn.MSELoss()

for epoch in range(1, NUM_EPOCHS + 1):
    net.train()
    epoch_loss = 0.0
    for i, (sensors, dom, s_feat, e_feat, (phys, ment)) in enumerate(trainloader):
        sensors = [m.to(DEVICE) for m in sensors]
        dom, s_feat, e_feat = dom.to(DEVICE), s_feat.to(DEVICE), e_feat.to(DEVICE)
        phys, ment = phys.to(DEVICE), ment.to(DEVICE)
        phys_weight = 0.6
        ment_weight = 0.4
        optimizer.zero_grad()
        p_pred, m_pred = net(sensors, dom, s_feat, e_feat)
        loss_phys = criterion(p_pred, phys)
        loss_ment = criterion(m_pred, ment)
        loss = (phys_weight * loss_phys) + (ment_weight * loss_ment)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(trainloader)

    # --- NEW: Evaluate and log metrics EVERY epoch ---
    loss_val, metrics = test_centralized(net, testloader)
    for metric_name in metrics_to_track:
        history[metric_name].append((epoch, metrics[metric_name]))

    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch}/{NUM_EPOCHS}, Train Loss: {avg_loss:.6f}, Val Loss: {loss_val:.4f}")
        print(f"  - Val R² Phys: {metrics['r2_physical']:.4f}, Val R² Mental: {metrics['r2_mental']:.4f}")
    # ---

print("--- Centralized Training Complete ---")

# 4. Save the final history to a JSON file
with open("centralized_with_novelty_results.json", "w") as f:
    json.dump(history, f, indent=4)
print("Saved final results to 'centralized_with_novelty_results.json'")

# 5. Print the final report from the last epoch
final_metrics = {metric: history[metric][-1][1] for metric in metrics_to_track} # Get last value
print("\n" + "="*50)
print(" FINAL TEST PERFORMANCE (Centralized with Novelty) ")
print("="*50)
print(f"  - Final Loss: {final_metrics['loss']:.4f}")
print(f"  - Physical Fatigue:")
print(f"    - RMSE: {final_metrics['rmse_physical']:.4f} | MAE: {final_metrics['mae_physical']:.4f} | R² Score: {final_metrics['r2_physical']:.4f}")
print(f"  - Mental Fatigue:")
print(f"    - RMSE: {final_metrics['rmse_mental']:.4f} | MAE: {final_metrics['mae_mental']:.4f} | R² Score: {final_metrics['r2_mental']:.4f}")
print("="*50 + "\n")

--- Starting Experiment 1: Centralized with Novelty ---
Mounted at /content/drive
Total participants found: 23
Training on 18 participants.
Testing on 5 holdout participants.
Loading CENTRALIZED data...
Data loaded. Training samples: 529, Test samples: 148
Discovered 3 unique sessions.
Starting centralized training for 260 epochs...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Epoch 1/260, Train Loss: 26.248718, Val Loss: 56.2440
  - Val R² Phys: -2.1885, Val R² Mental: -5.9182


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Epoch 10/260, Train Loss: 4.626800, Val Loss: 9.8352
  - Val R² Phys: -0.1480, Val R² Mental: -0.0186


...

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Epoch 250/260, Train Loss: 3.787750, Val Loss: 10.1493
  - Val R² Phys: -0.3977, Val R² Mental: 0.0054


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Epoch 260/260, Train Loss: 3.794939, Val Loss: 10.2490
  - Val R² Phys: -0.4272, Val R² Mental: 0.0055
--- Centralized Training Complete ---
Saved final results to 'centralized_with_novelty_results.json'

 FINAL TEST PERFORMANCE (Centralized with Novelty) 
  - Final Loss: 10.2490
  - Physical Fatigue:
    - RMSE: 2.0821 | MAE: 1.7222 | R² Score: -0.4272
  - Mental Fatigue:
    - RMSE: 2.5283 | MAE: 2.1460 | R² Score: 0.0055



/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


# Centralized (Baseline)

## Model Definition

In [ ]:
# Cell 5: Baseline Model Architecture and Train/Test Functions (CORRECTED)

# ========== 1. Baseline Model (No Novelty) ==========
# This model uses simple concatenation instead of your attention fusion.

class BaselineModel_NoFusion(nn.Module):
    def __init__(self, sensor_dim, lstm_hidden=32, num_sessions=3, use_time_and_session=True):
        super().__init__()
        self.use_time_and_session = use_time_and_session
        self.sensor_lstms = nn.ModuleList([SensorLSTM(d, lstm_hidden) for d in sensor_dim])

        self.session_emb = nn.Embedding(num_sessions, 8)
        self.time_enc = nn.Sequential(nn.Linear(1, 8), nn.ReLU(), nn.Linear(8, 8))

        # --- KEY DIFFERENCE ---
        # Input dimension is the sum of all LSTM outputs
        concatenated_dim = len(sensor_dim) * lstm_hidden
        global_input_dim = concatenated_dim + 16 if use_time_and_session else concatenated_dim
        # --- End of Difference ---

        self.global_lstm = nn.LSTM(global_input_dim, 32, batch_first=True)
        self.reg_phys = nn.Linear(32, 1)
        self.reg_ment = nn.Linear(32, 1)
        self.apply(self._init_weights)

    @staticmethod
    def _init_weights(m):
        if isinstance(m, (nn.Linear)):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None: nn.init.zeros_(m.bias)
        elif isinstance(m, nn.LSTM):
            for name, param in m.named_parameters():
                if 'weight' in name: nn.init.xavier_uniform_(param)
                elif 'bias' in name: nn.init.zeros_(param)

    def forward(self, sensor_inputs, domain_disc=None, session_feat=None, elapsed_feat=None, debug=False):
        feats = [m(inp) for m, inp in zip(self.sensor_lstms, sensor_inputs)]

        # --- KEY DIFFERENCE ---
        # Simple concatenation instead of attention fusion
        fused = torch.cat(feats, dim=1)
        # --- End of Difference ---

        if self.use_time_and_session:
            session_emb = self.session_emb(session_feat.squeeze(-1).long())
            time_emb = self.time_enc(elapsed_feat)
            fused = torch.cat([fused, session_emb, time_emb], dim=1)

        glstm_out, _ = self.global_lstm(fused.unsqueeze(1))
        feat = glstm_out[:, -1, :]
        return self.reg_phys(feat), self.reg_ment(feat)

# ========== 2. Train/Test Functions for Baseline Model ==========
# We need new functions because the model's 'forward' signature is simpler

def train_baseline_centralized(net, trainloader, epochs):
    """A centralized training loop for the baseline model."""
    print(f"Starting baseline centralized training for {epochs} epochs...")
    net.train()
    optimizer = torch.optim.Adam(net.parameters(), lr=5e-4)
    criterion = nn.MSELoss()

    for epoch in range(1, epochs + 1):
        epoch_loss = 0.0
        for i, (sensors, dom, s_feat, e_feat, (phys, ment)) in enumerate(trainloader):
            sensors = [m.to(DEVICE) for m in sensors]
            s_feat, e_feat = s_feat.to(DEVICE), e_feat.to(DEVICE)
            phys, ment = phys.to(DEVICE), ment.to(DEVICE)

            optimizer.zero_grad()
            p_pred, m_pred = net(sensors, session_feat=s_feat, elapsed_feat=e_feat)
            loss = criterion(p_pred, phys) + 2 * criterion(m_pred, ment)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(trainloader)
        if epoch % 10 == 0 or epoch == 1:
            print(f"Epoch {epoch}/{epochs}, Average Training Loss: {avg_loss:.6f}")

def test_baseline_centralized(net, testloader):
    """Validates the baseline model on the test set."""
    net.eval()
    criterion = nn.MSELoss()
    total_loss, all_phys_preds, all_phys_labels, all_ment_preds, all_ment_labels = 0.0, [], [], [], []

    # --- ADDED ROBUSTNESS CHECK ---
    if testloader is None or len(testloader) == 0:
        print("Warning: test_baseline_centralized received an empty testloader.")
        metrics = {"loss": 0.0, "rmse_physical": 99.0, "mae_physical": 99.0, "r2_physical": -99.0,
                   "rmse_mental": 99.0, "mae_mental": 99.0, "r2_mental": -99.0}
        return 0.0, metrics
    # --- END OF CHECK ---

    with torch.no_grad():
        for sensors, dom, s_feat, e_feat, (phys, ment) in testloader:
            sensors = [m.to(DEVICE) for m in sensors]
            s_feat, e_feat = s_feat.to(DEVICE), e_feat.to(DEVICE)
            phys, ment = phys.to(DEVICE), ment.to(DEVICE)

            p_pred, m_pred = net(sensors, session_feat=s_feat, elapsed_feat=e_feat)
            total_loss += (criterion(p_pred, phys) + criterion(m_pred, ment)).item()
            all_phys_preds.extend(p_pred.squeeze().cpu().numpy())
            all_phys_labels.extend(phys.squeeze().cpu().numpy())
            all_ment_preds.extend(m_pred.squeeze().cpu().numpy())
            all_ment_labels.extend(ment.squeeze().cpu().numpy())

    loss = total_loss / len(testloader)

    # --- ADDED NAN/EMPTY CHECK ---
    if not all_phys_labels or np.isnan(all_phys_preds).any():
        print("Warning: NaN or empty lists detected in test_baseline_centralized. Returning default metrics.")
        metrics = {"loss": loss, "rmse_physical": 99.0, "mae_physical": 99.0, "r2_physical": -99.0,
                   "rmse_mental": 99.0, "mae_mental": 99.0, "r2_mental": -99.0}
        return loss, metrics
    # --- END OF CHECK ---

    rmse_phys = math.sqrt(mean_squared_error(all_phys_labels, all_phys_preds))
    mae_phys = mean_absolute_error(all_phys_labels, all_phys_preds)
    r2_phys = r2_score(all_phys_labels, all_phys_preds)
    rmse_ment = math.sqrt(mean_squared_error(all_ment_labels, all_ment_preds))
    mae_ment = mean_absolute_error(all_ment_labels, all_ment_preds)
    r2_ment = r2_score(all_ment_labels, all_ment_preds)

    metrics = {
        "loss": loss,
        "rmse_physical": rmse_phys, "mae_physical": mae_phys, "r2_physical": r2_phys,
        "rmse_mental": rmse_ment, "mae_mental": mae_ment, "r2_mental": r2_ment
    }
    return loss, metrics

print("Baseline model and its train/test functions defined.")

Baseline model and its train/test functions defined.


## Train Model and Evaluate

In [ ]:
# Cell 6: Run Experiment 2 - Centralized without Novelty (with JSON Logging)
import json # <-- ADDED IMPORT

print("--- Starting Experiment 2: Centralized without Novelty ---")
NUM_EPOCHS = 260

# 1. Load data
try:
    trainloader, testloader, num_sessions = load_data(is_federated=False)
    print(f"Data loaded. Training samples: {len(trainloader.dataset)}, Test samples: {len(testloader.dataset)}")
    print(f"Discovered {num_sessions} unique sessions.")
except Exception as e:
    print(f"Error loading data: {e}. Please ensure Cell 2 ran successfully.")
    raise e

# 2. Initialize the baseline model
baseline_model = BaselineModel_NoFusion(
    sensor_dim=sensor_dim,
    lstm_hidden=32,
    num_sessions=num_sessions,
    use_time_and_session=True
).to(DEVICE)

# --- NEW: Initialize history dictionary ---
metrics_to_track = ["rmse_physical", "mae_physical", "r2_physical", "rmse_mental", "mae_mental", "r2_mental", "loss"]
history = {metric: [] for metric in metrics_to_track}
# ---

# 3. Train the model (MODIFIED to log every epoch)
print(f"Starting baseline centralized training for {NUM_EPOCHS} epochs...")
net = baseline_model
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4)
criterion = nn.MSELoss()

for epoch in range(1, NUM_EPOCHS + 1):
    net.train()
    epoch_loss = 0.0
    for i, (sensors, dom, s_feat, e_feat, (phys, ment)) in enumerate(trainloader):
        sensors = [m.to(DEVICE) for m in sensors]
        s_feat, e_feat = s_feat.to(DEVICE), e_feat.to(DEVICE)
        phys, ment = phys.to(DEVICE), ment.to(DEVICE)
        phys_weight = 0.6
        ment_weight = 0.4
        optimizer.zero_grad()
        p_pred, m_pred = net(sensors, session_feat=s_feat, elapsed_feat=e_feat)
        loss_phys = criterion(p_pred, phys)
        loss_ment = criterion(m_pred, ment)
        loss = (phys_weight * loss_phys) + (ment_weight * loss_ment)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(trainloader)

    # --- NEW: Evaluate and log metrics EVERY epoch ---
    loss_val, metrics = test_baseline_centralized(net, testloader)
    for metric_name in metrics_to_track:
        history[metric_name].append((epoch, metrics[metric_name]))

    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch}/{NUM_EPOCHS}, Train Loss: {avg_loss:.6f}, Val Loss: {loss_val:.4f}")
        print(f"  - Val R² Phys: {metrics['r2_physical']:.4f}, Val R² Mental: {metrics['r2_mental']:.4f}")
    # ---

print("--- Baseline Centralized Training Complete ---")

# 4. Save the final history to a JSON file
with open("centralized_baseline_results.json", "w") as f:
    json.dump(history, f, indent=4)
print("Saved final results to 'centralized_baseline_results.json'")

# 5. Print the final report from the last epoch
final_metrics = {metric: history[metric][-1][1] for metric in metrics_to_track} # Get last value
print("\n" + "="*50)
print(" FINAL TEST PERFORMANCE (Centralized WITHOUT Novelty) ")
print("="*50)
print(f"  - Final Loss: {final_metrics['loss']:.4f}")
print(f"  - Physical Fatigue:")
print(f"    - RMSE: {final_metrics['rmse_physical']:.4f} | MAE: {final_metrics['mae_physical']:.4f} | R² Score: {final_metrics['r2_physical']:.4f}")
print(f"  - Mental Fatigue:")
print(f"    - RMSE: {final_metrics['rmse_mental']:.4f} | MAE: {final_metrics['mae_mental']:.4f} | R² Score: {final_metrics['r2_mental']:.4f}")
print("="*50 + "\n")

--- Starting Experiment 2: Centralized without Novelty ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total participants found: 23
Training on 18 participants.
Testing on 5 holdout participants.
Loading CENTRALIZED data...
Data loaded. Training samples: 529, Test samples: 148
Discovered 3 unique sessions.
Starting baseline centralized training for 260 epochs...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Epoch 1/260, Train Loss: 23.335566, Val Loss: 73.4934
  - Val R² Phys: -3.7660, Val R² Mental: -17.9429


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Epoch 10/260, Train Loss: 5.879765, Val Loss: 17.2950
  - Val R² Phys: -0.0991, Val R² Mental: -3.5928


...

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Epoch 250/260, Train Loss: 2.313440, Val Loss: 12.1990
  - Val R² Phys: -0.9856, Val R² Mental: -1.1811


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Epoch 260/260, Train Loss: 2.192048, Val Loss: 12.3338
  - Val R² Phys: -1.0069, Val R² Mental: -1.2016
--- Baseline Centralized Training Complete ---
Saved final results to 'centralized_baseline_results.json'

 FINAL TEST PERFORMANCE (Centralized WITHOUT Novelty) 
  - Final Loss: 12.3338
  - Physical Fatigue:
    - RMSE: 2.4302 | MAE: 1.9295 | R² Score: -1.0069
  - Mental Fatigue:
    - RMSE: 2.6599 | MAE: 2.3282 | R² Score: -1.2016



/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


# Federated Model (with AFDA)

## Federated Training and Evaluation Functions

In [ ]:
# Cell 7: Federated Training and Evaluation Functions (STABILIZED v4 - FINAL)
phys_weight = 0.6
ment_weight = 0.4
def train(net, trainloader, epochs):
    """Train the model on the client's local training set."""
    net.train()
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)
    criterion = nn.MSELoss()

    for _ in range(epochs):
        for sensors, dom, s_feat, e_feat, (phys, ment) in trainloader:
            sensors = [m.to(DEVICE) for m in sensors]
            dom, s_feat, e_feat = dom.to(DEVICE), s_feat.to(DEVICE), e_feat.to(DEVICE)
            phys, ment = phys.to(DEVICE), ment.to(DEVICE)

            optimizer.zero_grad()
            p_pred, m_pred = net(sensors, dom, s_feat, e_feat)

            # --- FIX: Use the simplest possible loss function ---
            loss_phys = criterion(p_pred, phys)
            loss_ment = criterion(m_pred, ment)
            loss = (phys_weight * loss_phys) + (ment_weight * loss_ment)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=1.0)
            optimizer.step()

def test(net, testloader):
    """Validate the model on the (global) test set. This version is robust."""
    net.eval()
    criterion = nn.MSELoss()
    total_loss, all_phys_preds, all_phys_labels, all_ment_preds, all_ment_labels = 0.0, [], [], [], []

    if testloader is None or len(testloader) == 0:
        print("Warning: test() function received an empty or None testloader.")
        metrics = {"loss": 0.0, "rmse_physical": 99.0, "mae_physical": 99.0, "r2_physical": -99.0,
                   "rmse_mental": 99.0, "mae_mental": 99.0, "r2_mental": -99.0}
        return 0.0, metrics

    with torch.no_grad():
        for sensors, dom, s_feat, e_feat, (phys, ment) in testloader:
            sensors = [m.to(DEVICE) for m in sensors]
            dom, s_feat, e_feat = dom.to(DEVICE), s_feat.to(DEVICE), e_feat.to(DEVICE)
            phys, ment = phys.to(DEVICE), ment.to(DEVICE)

            p_pred, m_pred = net(sensors, dom, s_feat, e_feat)

            total_loss += (criterion(p_pred, phys) + criterion(m_pred, ment)).item()
            all_phys_preds.extend(p_pred.squeeze().cpu().numpy())
            all_phys_labels.extend(phys.squeeze().cpu().numpy())
            all_ment_preds.extend(m_pred.squeeze().cpu().numpy())
            all_ment_labels.extend(ment.squeeze().cpu().numpy())

    loss = total_loss / len(testloader)

    if not all_phys_labels or np.isnan(all_phys_preds).any():
        print("Warning: NaN or empty lists detected in test(). Returning default metrics.")
        metrics = {"loss": loss, "rmse_physical": 999, "mae_physical": 999, "r2_physical": -99,
                   "rmse_mental": 999, "mae_mental": 999, "r2_mental": -99}
        return loss, metrics

    rmse_phys = math.sqrt(mean_squared_error(all_phys_labels, all_phys_preds))
    mae_phys = mean_absolute_error(all_phys_labels, all_phys_preds)
    r2_phys = r2_score(all_phys_labels, all_phys_preds)
    rmse_ment = math.sqrt(mean_squared_error(all_ment_labels, all_ment_preds))
    mae_ment = mean_absolute_error(all_ment_labels, all_ment_preds)
    r2_ment = r2_score(all_ment_labels, all_ment_preds)

    metrics = {
        "loss": loss,
        "rmse_physical": rmse_phys, "mae_physical": mae_phys, "r2_physical": r2_phys,
        "rmse_mental": rmse_ment, "mae_mental": mae_ment, "r2_mental": r2_ment
    }
    return loss, metrics

print("Federated train and test functions (STABILIZED v4, No-BatchNorm) defined.")

Federated train and test functions (STABILIZED v4, No-BatchNorm) defined.


## Flower Client Definition

In [ ]:
# Cell 8: Flower Client Definition

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.net.state_dict().items()]

    def set_parameters(self, parameters):
        params_dict = zip(self.net.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        self.net.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        # Use 8 local epochs, just like our champion model
        train(self.net, self.trainloader, epochs=3)
        return self.get_parameters(config={}), len(self.trainloader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        # The valloader is the global test set
        loss, metrics = test(self.net, self.valloader)
        # Return the 3 items Flower expects
        return loss, len(self.valloader.dataset), metrics

def client_fn(cid: str) -> fl.client.Client:
    """Create a Flower client instance."""

    # Each client gets its own instance of the Novelty Model
    net = FMAL_Daf(
        sensor_dim=sensor_dim,
        lstm_hidden=32,
        fusion_dim=64,
        num_sessions=num_sessions, # num_sessions from Cell 2
        use_time_and_session=True
    ).to(DEVICE)

    # Get the data partition for this specific client ID
    client_trainloader = trainloaders[int(cid)]
    client_valloader = valloaders[int(cid)]

    return FlowerClient(net, client_trainloader, client_valloader).to_client()

print("FlowerClient and client_fn defined.")

FlowerClient and client_fn defined.


## Run Simulation

In [ ]:
# Cell 9: Run Experiment 3 - Federated with Novelty (with JSON Logging)
import json # <-- 1. ADD THIS IMPORT

print("--- Starting Experiment 3: Federated with Novelty ---")
NUM_ROUNDS = 260 # Use 80 rounds for a fair comparison

# 1. Load data in FEDERATED mode
try:
    # We re-use NUM_CLIENTS from Cell 2
    trainloaders, valloaders, testloader, num_sessions = load_data(is_federated=True)

    # Check if data loading was successful
    if not trainloaders or not testloader:
        raise ValueError("Data loading failed. trainloaders or testloader is empty.")

    print(f"Data loaded. {len(trainloaders)} training clients.")
    print(f"Test samples: {len(testloader.dataset)}")
    print(f"Discovered {num_sessions} unique sessions.")
except Exception as e:
    print(f"Error loading data: {e}. Please ensure Cell 2 ran successfully and your CSV path is correct.")
    raise e

# 2. Define the server-side evaluation function
def get_evaluate_fn(testloader, num_sessions):
    """Return an evaluation function for server-side evaluation."""
    def evaluate(server_round: int, parameters: fl.common.NDArrays, config: Dict[str, fl.common.Scalar]):

        # The server creates an instance of the Novelty Model
        net = FMAL_Daf(
            sensor_dim=sensor_dim,
            lstm_hidden=32,
            fusion_dim=64,
            num_sessions=num_sessions,
            use_time_and_session=True
        ).to(DEVICE)

        params_dict = zip(net.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        net.load_state_dict(state_dict, strict=True)

        # The server runs the global test function
        loss, metrics = test(net, testloader) # This returns the full metrics dict

        # Print the full report card
        print("\n" + "="*50)
        print(f"Server-side evaluation round {server_round} / {NUM_ROUNDS}")
        print(f"  - Loss: {metrics['loss']:.4f}")
        print(f"  - Physical Fatigue:")
        print(f"    - RMSE: {metrics['rmse_physical']:.4f} | MAE: {metrics['mae_physical']:.4f} | R² Score: {metrics['r2_physical']:.4f}")
        print(f"  - Mental Fatigue:")
        print(f"    - RMSE: {metrics['rmse_mental']:.4f} | MAE: {metrics['mae_mental']:.4f} | R² Score: {metrics['r2_mental']:.4f}")
        print("="*50 + "\n")

        # This correctly returns all metrics to the history object
        return loss, metrics
    return evaluate

# 3. Define the FedAvg strategy
# Get initial parameters from a model instance
net = FMAL_Daf(
    sensor_dim=sensor_dim,
    lstm_hidden=32,
    fusion_dim=64,
    num_sessions=num_sessions,
    use_time_and_session=True
).to(DEVICE)
initial_parameters = fl.common.ndarrays_to_parameters([val.cpu().numpy() for _, val in net.state_dict().items()])

strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # Use all available clients for training
    fraction_evaluate=1.0, # Use all available clients for evaluation
    min_fit_clients=len(trainloaders),
    min_evaluate_clients=len(trainloaders),
    min_available_clients=len(trainloaders),
    evaluate_fn=get_evaluate_fn(testloader, num_sessions),
    initial_parameters=initial_parameters,
)

# 4. Start the simulation
print(f"Starting Federated Simulation with {len(trainloaders)} clients for {NUM_ROUNDS} rounds...")
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=len(trainloaders),
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy,
)

print("--- Federated (with Novelty) Simulation Finished ---")


# --- 2. ADD THIS CODE BLOCK TO THE VERY END OF YOUR CELL ---
print("\nProcessing and saving history...")

# 'history.metrics_centralized' already contains all metrics from our 'test' function
# (because we returned the full 'metrics' dict in 'evaluate_fn')
results_to_save = history.metrics_centralized

# Add the 'loss' from its separate history list
results_to_save['loss'] = history.loss_centralized

# Save the final results to a JSON file
file_name = "federated_with_novelty_results.json"
with open(file_name, "w") as f:
    json.dump(results_to_save, f, indent=4)

print(f"Saved all metric results to '{file_name}'")
# --- END OF ADDED CODE ---

--- Starting Experiment 3: Federated with Novelty ---


	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
INFO :      Starting Flower simulation, config: num_rounds=260, no round_timeout


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total participants found: 23
Training on 18 participants.
Testing on 5 holdout participants.
Loading FEDERATED data for 10 clients...
  - Client 0: persons ['PP4', 'PP14']
  - Client 1: persons ['PP24', 'PP2']
  - Client 2: persons ['PP20', 'PP5']
  - Client 3: persons ['PP12', 'PP18']
  - Client 4: persons ['PP9', 'PP19']
  - Client 5: persons ['PP1', 'PP6']
  - Client 6: persons ['PP16', 'PP3']
  - Client 7: persons ['PP10', 'PP25']
  - Client 8: persons ['PP15']
  - Client 9: persons ['PP13']
Data loaded. 10 training clients.
Test samples: 146
Discovered 3 unique sessions.
Starting Federated Simulation with 10 clients for 260 rounds...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
2025-11-10 16:54:21,963	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 3937846886.0, 'node:172.28.0.12': 1.0, 'memory': 7875693774.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Using initial global


Server-side evaluation round 0 / 260
  - Loss: 59.4970
  - Physical Fatigue:
    - RMSE: 4.4405 | MAE: 4.0281 | R² Score: -4.7152
  - Mental Fatigue:
    - RMSE: 6.5934 | MAE: 6.1611 | R² Score: -6.8584



...

(ClientAppActor pid=14064) 
(ClientAppActor pid=14064)         
(ClientAppActor pid=14063) 
(ClientAppActor pid=14063)         
(ClientAppActor pid=14064) 
(ClientAppActor pid=14064)         
(ClientAppActor pid=14063) 
(ClientAppActor pid=14063)         
(ClientAppActor pid=14064) 
(ClientAppActor pid=14064)         
(ClientAppActor pid=14063) 
(ClientAppActor pid=14063)         
(ClientAppActor pid=14064) 
(ClientAppActor pid=14064)         
(ClientAppActor pid=14063) 
(ClientAppActor pid=14063)         
(ClientAppActor pid=14064) 
(ClientAppActor pid=14064)         
(ClientAppActor pid=14063) 
(ClientAppActor pid=14063)         
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 260]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)
(ClientAppActor pid=14063) 
(ClientAppActor pid=14063)         
(ClientAppActor pid=14064) 
(ClientAppActor pid=14064)         
(ClientAppActor pid=14063) 
(ClientAppActor pid=14063)       


Server-side evaluation round 260 / 260
  - Loss: 10.6515
  - Physical Fatigue:
    - RMSE: 2.2292 | MAE: 1.8769 | R² Score: -0.4404
  - Mental Fatigue:
    - RMSE: 2.5284 | MAE: 2.0566 | R² Score: -0.1556



(ClientAppActor pid=14064) 
(ClientAppActor pid=14064)         
(ClientAppActor pid=14064) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 33x across cluster]
(ClientAppActor pid=14064)             This is a deprecated feature. It will be removed [repeated 33x across cluster]
(ClientAppActor pid=14064)             entirely in future versions of Flower. [repeated 33x across cluster]
(ClientAppActor pid=14063) 
(ClientAppActor pid=14063)         
(ClientAppActor pid=14064) 
(ClientAppActor pid=14064)         
(ClientAppActor pid=14063) 
(ClientAppActor pid=14063)         
(ClientAppActor pid=14064) 
(ClientAppActor pid=14064)         
(ClientAppActor pid=14063) 
(ClientAppActor pid=14063)         
(ClientAppActor pid=14064) 
(ClientAppActor pid=14064)         
(ClientAppActor pid=

--- Federated (with Novelty) Simulation Finished ---

Processing and saving history...
Saved all metric results to 'federated_with_novelty_results.json'


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


# Federated Model (Baseline)

## Federated Training & Evaluation Logic

In [ ]:
# Cell 10: Federated Training and Evaluation Functions (BASELINE - STABILIZED)

def train_baseline_federated(net, trainloader, epochs):
    """Train the BASELINE model on the client's local training set."""
    net.train()

    # --- FIX 1: Use a stable, lower learning rate ---
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)
    criterion = nn.MSELoss()

    for _ in range(epochs):
        for sensors, dom, s_feat, e_feat, (phys, ment) in trainloader:
            sensors = [m.to(DEVICE) for m in sensors]
            s_feat, e_feat = s_feat.to(DEVICE), e_feat.to(DEVICE)
            phys, ment = phys.to(DEVICE), ment.to(DEVICE)

            optimizer.zero_grad()
            p_pred, m_pred = net(sensors, session_feat=s_feat, elapsed_feat=e_feat)

            # Use the 2x weighted loss for a fair comparison
            loss = criterion(p_pred, phys) + 2 * criterion(m_pred, ment)
            loss.backward()

            # --- FIX 2: Add Gradient Clipping to prevent explosion ---
            torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=1.0)

            optimizer.step()

def test_baseline_federated(net, testloader):
    """Validate the BASELINE model on the (global) test set. This version is robust."""
    net.eval()
    criterion = nn.MSELoss()
    total_loss, all_phys_preds, all_phys_labels, all_ment_preds, all_ment_labels = 0.0, [], [], [], []

    # --- FIX 3: Add robustness checks for empty/None testloader ---
    if testloader is None or len(testloader) == 0:
        print("Warning: test_baseline_federated received an empty or None testloader.")
        metrics = {
            "loss": 0.0, "rmse_physical": 99.0, "mae_physical": 99.0, "r2_physical": -99.0,
            "rmse_mental": 99.0, "mae_mental": 99.0, "r2_mental": -99.0
        }
        return 0.0, metrics
    # --- End of Fix 3 ---

    with torch.no_grad():
        for sensors, dom, s_feat, e_feat, (phys, ment) in testloader:
            sensors = [m.to(DEVICE) for m in sensors]
            s_feat, e_feat = s_feat.to(DEVICE), e_feat.to(DEVICE)
            phys, ment = phys.to(DEVICE), ment.to(DEVICE)

            p_pred, m_pred = net(sensors, session_feat=s_feat, elapsed_feat=e_feat)
            total_loss += (criterion(p_pred, phys) + criterion(m_pred, ment)).item()
            all_phys_preds.extend(p_pred.squeeze().cpu().numpy())
            all_phys_labels.extend(phys.squeeze().cpu().numpy())
            all_ment_preds.extend(m_pred.squeeze().cpu().numpy())
            all_ment_labels.extend(ment.squeeze().cpu().numpy())

    loss = total_loss / len(testloader)

    # --- FIX 3 (continued): Check for NaN/empty lists after loop ---
    if not all_phys_labels or np.isnan(all_phys_preds).any():
        print("Warning: NaN or empty lists detected in test_baseline_federated. Returning default metrics.")
        metrics = {
            "loss": loss, "rmse_physical": 999, "mae_physical": 999, "r2_physical": -99,
            "rmse_mental": 999, "mae_mental": 999, "r2_mental": -99
        }
        return loss, metrics
    # --- End of Fix 3 ---

    rmse_phys = math.sqrt(mean_squared_error(all_phys_labels, all_phys_preds))
    mae_phys = mean_absolute_error(all_phys_labels, all_phys_preds)
    r2_phys = r2_score(all_phys_labels, all_phys_preds)
    rmse_ment = math.sqrt(mean_squared_error(all_ment_labels, all_ment_preds))
    mae_ment = mean_absolute_error(all_ment_labels, all_ment_preds)
    r2_ment = r2_score(all_ment_labels, all_ment_preds)

    metrics = {
        "loss": loss,
        "rmse_physical": rmse_phys, "mae_physical": mae_phys, "r2_physical": r2_phys,
        "rmse_mental": rmse_ment, "mae_mental": mae_ment, "r2_mental": r2_ment
    }
    return loss, metrics

print("Federated BASELINE train and test functions (STABILIZED) defined.")

## Flower Client Definition

In [ ]:
# Cell 11: Flower Client Definition (BASELINE)

class BaselineFlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.net.state_dict().items()]

    def set_parameters(self, parameters):
        params_dict = zip(self.net.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        self.net.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        # Call the new baseline train function
        train_baseline_federated(self.net, self.trainloader, epochs=3)
        return self.get_parameters(config={}), len(self.trainloader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        # Call the new baseline test function
        loss, metrics = test_baseline_federated(self.net, self.valloader)
        return loss, len(self.valloader.dataset), metrics

def baseline_client_fn(cid: str) -> fl.client.Client:
    """Create a Flower client instance for the BASELINE model."""

    # Instantiate the Baseline model
    net = BaselineModel_NoFusion(
        sensor_dim=sensor_dim,
        lstm_hidden=32,
        num_sessions=num_sessions, # num_sessions from Cell 2
        use_time_and_session=True
    ).to(DEVICE)

    client_trainloader = trainloaders[int(cid)]
    client_valloader = valloaders[int(cid)]

    return BaselineFlowerClient(net, client_trainloader, client_valloader).to_client()

print("BaselineFlowerClient and baseline_client_fn defined.")

## Run Simulation

In [ ]:
# Cell 12: Run Experiment 4 - Federated without Novelty (with JSON Logging)
import json # <-- ADDED IMPORT

print("--- Starting Experiment 4: Federated without Novelty ---")
NUM_ROUNDS = 260

# 1. Load data
try:
    trainloaders, valloaders, testloader, num_sessions = load_data(is_federated=True)
    if not trainloaders or not testloader:
        raise ValueError("Data loading failed. trainloaders or testloader is empty.")
    print(f"Data loaded. {len(trainloaders)} training clients.")
    print(f"Test samples: {len(testloader.dataset)}")
    print(f"Discovered {num_sessions} unique sessions.")
except Exception as e:
    print(f"Error loading data: {e}. Please ensure Cell 2 ran successfully.")
    raise e

# 2. Define the server-side evaluation function (for BASELINE)
def get_baseline_evaluate_fn(testloader, num_sessions):
    """Return an evaluation function for server-side evaluation of the BASELINE model."""
    def evaluate(server_round: int, parameters: fl.common.NDArrays, config: Dict[str, fl.common.Scalar]):
        net = BaselineModel_NoFusion(
            sensor_dim=sensor_dim,
            lstm_hidden=32,
            num_sessions=num_sessions,
            use_time_and_session=True
        ).to(DEVICE)
        params_dict = zip(net.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        net.load_state_dict(state_dict, strict=True)
        loss, metrics = test_baseline_federated(net, testloader)

        # Print the full report card
        print("\n" + "="*50)
        print(f"Server-side evaluation round {server_round} / {NUM_ROUNDS} (BASELINE)")
        print(f"  - Loss: {metrics['loss']:.4f}")
        print(f"  - Physical Fatigue:")
        print(f"    - RMSE: {metrics['rmse_physical']:.4f} | MAE: {metrics['mae_physical']:.4f} | R² Score: {metrics['r2_physical']:.4f}")
        print(f"  - Mental Fatigue:")
        print(f"    - RMSE: {metrics['rmse_mental']:.4f} | MAE: {metrics['mae_mental']:.4f} | R² Score: {metrics['r2_mental']:.4f}")
        print("="*50 + "\n")

        # --- KEY CHANGE: Return the FULL metrics dict to Flower ---
        return loss, metrics
    return evaluate

# 3. Define the FedAvg strategy (for BASELINE)
baseline_net = BaselineModel_NoFusion(
    sensor_dim=sensor_dim,
    lstm_hidden=32,
    num_sessions=num_sessions,
    use_time_and_session=True
).to(DEVICE)
baseline_initial_parameters = fl.common.ndarrays_to_parameters([val.cpu().numpy() for _, val in baseline_net.state_dict().items()])

baseline_strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=1.0,
    min_fit_clients=len(trainloaders),
    min_evaluate_clients=len(trainloaders),
    min_available_clients=len(trainloaders),
    evaluate_fn=get_baseline_evaluate_fn(testloader, num_sessions),
    initial_parameters=baseline_initial_parameters,
)

# 4. Start the simulation
print(f"Starting Federated Simulation (Baseline) with {len(trainloaders)} clients for {NUM_ROUNDS} rounds...")
history_baseline = fl.simulation.start_simulation(
    client_fn=baseline_client_fn, # From Cell 11
    num_clients=len(trainloaders),
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=baseline_strategy,
)

print("--- Federated (without Novelty) Simulation Finished ---")

# --- ADD THIS CODE BLOCK TO THE END OF CELL 12 ---
print("\nProcessing and saving baseline history...")

# Combine the loss and metrics histories into one dictionary
results_to_save = history_baseline.metrics_centralized
results_to_save['loss'] = history_baseline.loss_centralized

# Save the final results to a JSON file
file_name = "federated_baseline_results.json"
with open(file_name, "w") as f:
    json.dump(results_to_save, f, indent=4)

print(f"Saved all metric results to '{file_name}'")
# --- END OF ADDED CODE ---

--- Starting Experiment 4: Federated without Novelty ---


	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
INFO :      Starting Flower simulation, config: num_rounds=260, no round_timeout


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total participants found: 23
Training on 18 participants.
Testing on 5 holdout participants.
Loading FEDERATED data for 10 clients...
  - Client 0: persons ['PP20', 'PP9']
  - Client 1: persons ['PP12', 'PP14']
  - Client 2: persons ['PP1', 'PP3']
  - Client 3: persons ['PP4', 'PP21']
  - Client 4: persons ['PP7', 'PP19']
  - Client 5: persons ['PP5', 'PP25']
  - Client 6: persons ['PP18', 'PP16']
  - Client 7: persons ['PP23', 'PP15']
  - Client 8: persons ['PP2']
  - Client 9: persons ['PP6']
Data loaded. 10 training clients.
Test samples: 149
Discovered 3 unique sessions.
Starting Federated Simulation (Baseline) with 10 clients for 260 rounds...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
2025-11-10 17:21:24,550	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7900753920.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3950376960.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Using initial global


Server-side evaluation round 0 / 260 (BASELINE)
  - Loss: 56.1030
  - Physical Fatigue:
    - RMSE: 3.5954 | MAE: 2.9345 | R² Score: -2.0099
  - Mental Fatigue:
    - RMSE: 6.6023 | MAE: 6.2747 | R² Score: -9.2260



...

(ClientAppActor pid=24086) 
(ClientAppActor pid=24086)         
(ClientAppActor pid=24089) 
(ClientAppActor pid=24089)         
(ClientAppActor pid=24089) 
(ClientAppActor pid=24089)         
(ClientAppActor pid=24086) 
(ClientAppActor pid=24086)         
(ClientAppActor pid=24089) 
(ClientAppActor pid=24089)         
(ClientAppActor pid=24089) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 40x across cluster]
(ClientAppActor pid=24089)             This is a deprecated feature. It will be removed [repeated 40x across cluster]
(ClientAppActor pid=24089)             entirely in future versions of Flower. [repeated 40x across cluster]
(ClientAppActor pid=24086) 
(ClientAppActor pid=24086)         
(ClientAppActor pid=24086) 
(ClientAppActor pid=24086)         
(ClientAppActor pid=


Server-side evaluation round 260 / 260 (BASELINE)
  - Loss: 26.5195
  - Physical Fatigue:
    - RMSE: 3.0617 | MAE: 2.2584 | R² Score: -1.1826
  - Mental Fatigue:
    - RMSE: 4.1604 | MAE: 3.6349 | R² Score: -3.0606



(ClientAppActor pid=24086) 
(ClientAppActor pid=24086)         
(ClientAppActor pid=24089) 
(ClientAppActor pid=24089)         
(ClientAppActor pid=24089) 
(ClientAppActor pid=24089)         
(ClientAppActor pid=24086) 
(ClientAppActor pid=24086)         
(ClientAppActor pid=24089) 
(ClientAppActor pid=24089)         
(ClientAppActor pid=24086) 
(ClientAppActor pid=24086)         
(ClientAppActor pid=24089) 
(ClientAppActor pid=24089)         
(ClientAppActor pid=24086) 
(ClientAppActor pid=24086)         
(ClientAppActor pid=24089) 
(ClientAppActor pid=24089)         
(ClientAppActor pid=24086) 
(ClientAppActor pid=24086)         
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 260 round(s) in 772.47s
INFO :      	History (loss, distributed):
INFO :      		round 1: 56.05098098754883
INFO :      		round 2: 55.99972198486328
INFO :      		round 3: 55.9485009765625
INFO :      		round 4: 55.89660247802735
INFO

--- Federated (without Novelty) Simulation Finished ---

Processing and saving baseline history...
Saved all metric results to 'federated_baseline_results.json'


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


# Graph code

In [ ]:
import matplotlib.pyplot as plt

afda_fl_metrics = {
    "loss": [
        [
            0,
            59.497015380859374
        ],
        [
            1,
            59.42250289916992
        ],
        [
            2,
            59.34786605834961
        ],
        [
            3,
            59.27279052734375
        ],
        [
            4,
            59.197571182250975
        ],
        [
            5,
            59.12158203125
        ],
        [
            6,
            59.045066833496094
        ],
        [
            7,
            58.968092346191405
        ],
        [
            8,
            58.89048690795899
        ],
        [
            9,
            58.81190299987793
        ],
        [
            10,
            58.73283462524414
        ],
        [
            11,
            58.65310134887695
        ],
        [
            12,
            58.57203712463379
        ],
        [
            13,
            58.490299987792966
        ],
        [
            14,
            58.40748138427735
        ],
        [
            15,
            58.324187469482425
        ],
        [
            16,
            58.2401626586914
        ],
        [
            17,
            58.15499267578125
        ],
        [
            18,
            58.06870384216309
        ],
        [
            19,
            57.98136940002441
        ],
        [
            20,
            57.89321174621582
        ],
        [
            21,
            57.80400009155274
        ],
        [
            22,
            57.71378593444824
        ],
        [
            23,
            57.622476196289064
        ],
        [
            24,
            57.530269622802734
        ],
        [
            25,
            57.436666870117186
        ],
        [
            26,
            57.34194145202637
        ],
        [
            27,
            57.24579544067383
        ],
        [
            28,
            57.148677825927734
        ],
        [
            29,
            57.050255584716794
        ],
        [
            30,
            56.950799560546876
        ],
        [
            31,
            56.84980850219726
        ],
        [
            32,
            56.747676086425784
        ],
        [
            33,
            56.64370574951172
        ],
        [
            34,
            56.538475799560544
        ],
        [
            35,
            56.43181228637695
        ],
        [
            36,
            56.32428131103516
        ],
        [
            37,
            56.21481475830078
        ],
        [
            38,
            56.10382843017578
        ],
        [
            39,
            55.99149017333984
        ],
        [
            40,
            55.877431106567386
        ],
        [
            41,
            55.76166763305664
        ],
        [
            42,
            55.64440574645996
        ],
        [
            43,
            55.52552032470703
        ],
        [
            44,
            55.40500106811523
        ],
        [
            45,
            55.28298110961914
        ],
        [
            46,
            55.15932846069336
        ],
        [
            47,
            55.033303833007814
        ],
        [
            48,
            54.90534210205078
        ],
        [
            49,
            54.77579536437988
        ],
        [
            50,
            54.64445419311524
        ],
        [
            51,
            54.51125564575195
        ],
        [
            52,
            54.376024627685545
        ],
        [
            53,
            54.23863334655762
        ],
        [
            54,
            54.098473739624026
        ],
        [
            55,
            53.95732498168945
        ],
        [
            56,
            53.8133129119873
        ],
        [
            57,
            53.666843032836915
        ],
        [
            58,
            53.51794013977051
        ],
        [
            59,
            53.36739921569824
        ],
        [
            60,
            53.21450500488281
        ],
        [
            61,
            53.05913391113281
        ],
        [
            62,
            52.900787353515625
        ],
        [
            63,
            52.740033340454104
        ],
        [
            64,
            52.57706756591797
        ],
        [
            65,
            52.411332321166995
        ],
        [
            66,
            52.243981170654294
        ],
        [
            67,
            52.072787857055665
        ],
        [
            68,
            51.900091552734374
        ],
        [
            69,
            51.7234992980957
        ],
        [
            70,
            51.544807052612306
        ],
        [
            71,
            51.36332130432129
        ],
        [
            72,
            51.17888145446777
        ],
        [
            73,
            50.99284896850586
        ],
        [
            74,
            50.80100936889649
        ],
        [
            75,
            50.608347702026364
        ],
        [
            76,
            50.411852073669436
        ],
        [
            77,
            50.2130012512207
        ],
        [
            78,
            50.01072654724121
        ],
        [
            79,
            49.80541820526123
        ],
        [
            80,
            49.59814987182617
        ],
        [
            81,
            49.386789321899414
        ],
        [
            82,
            49.172997665405276
        ],
        [
            83,
            48.95540008544922
        ],
        [
            84,
            48.735811424255374
        ],
        [
            85,
            48.512923049926755
        ],
        [
            86,
            48.28582935333252
        ],
        [
            87,
            48.05702896118164
        ],
        [
            88,
            47.82444801330566
        ],
        [
            89,
            47.58868522644043
        ],
        [
            90,
            47.34873275756836
        ],
        [
            91,
            47.10707588195801
        ],
        [
            92,
            46.86179695129395
        ],
        [
            93,
            46.61299839019775
        ],
        [
            94,
            46.36189937591553
        ],
        [
            95,
            46.10621109008789
        ],
        [
            96,
            45.84848613739014
        ],
        [
            97,
            45.587775230407715
        ],
        [
            98,
            45.323378562927246
        ],
        [
            99,
            45.05654373168945
        ],
        [
            100,
            44.786399841308594
        ],
        [
            101,
            44.51306571960449
        ],
        [
            102,
            44.23729572296143
        ],
        [
            103,
            43.95943756103516
        ],
        [
            104,
            43.677757263183594
        ],
        [
            105,
            43.39223442077637
        ],
        [
            106,
            43.10588474273682
        ],
        [
            107,
            42.81530838012695
        ],
        [
            108,
            42.522495651245116
        ],
        [
            109,
            42.22800617218017
        ],
        [
            110,
            41.9306037902832
        ],
        [
            111,
            41.630191040039065
        ],
        [
            112,
            41.326857566833496
        ],
        [
            113,
            41.020853996276855
        ],
        [
            114,
            40.714182662963864
        ],
        [
            115,
            40.40419979095459
        ],
        [
            116,
            40.092999649047854
        ],
        [
            117,
            39.77848510742187
        ],
        [
            118,
            39.46052989959717
        ],
        [
            119,
            39.143486976623535
        ],
        [
            120,
            38.8231840133667
        ],
        [
            121,
            38.500671005249025
        ],
        [
            122,
            38.177589416503906
        ],
        [
            123,
            37.85233116149902
        ],
        [
            124,
            37.52604942321777
        ],
        [
            125,
            37.195336532592776
        ],
        [
            126,
            36.86569700241089
        ],
        [
            127,
            36.53475456237793
        ],
        [
            128,
            36.202262020111085
        ],
        [
            129,
            35.86990585327148
        ],
        [
            130,
            35.53469734191894
        ],
        [
            131,
            35.199665260314944
        ],
        [
            132,
            34.86424837112427
        ],
        [
            133,
            34.52696447372436
        ],
        [
            134,
            34.18991584777832
        ],
        [
            135,
            33.85095262527466
        ],
        [
            136,
            33.51209888458252
        ],
        [
            137,
            33.174486351013186
        ],
        [
            138,
            32.836042404174805
        ],
        [
            139,
            32.497887992858885
        ],
        [
            140,
            32.159615993499756
        ],
        [
            141,
            31.820684814453124
        ],
        [
            142,
            31.484034729003906
        ],
        [
            143,
            31.143876457214354
        ],
        [
            144,
            30.807726860046387
        ],
        [
            145,
            30.469775390625
        ],
        [
            146,
            30.133347606658937
        ],
        [
            147,
            29.79852714538574
        ],
        [
            148,
            29.466189193725587
        ],
        [
            149,
            29.132904148101808
        ],
        [
            150,
            28.800647830963136
        ],
        [
            151,
            28.469296264648438
        ],
        [
            152,
            28.139830112457275
        ],
        [
            153,
            27.812685108184816
        ],
        [
            154,
            27.484974241256715
        ],
        [
            155,
            27.159177780151367
        ],
        [
            156,
            26.837712287902832
        ],
        [
            157,
            26.51402349472046
        ],
        [
            158,
            26.195464086532592
        ],
        [
            159,
            25.87820119857788
        ],
        [
            160,
            25.563736057281496
        ],
        [
            161,
            25.253372383117675
        ],
        [
            162,
            24.94572744369507
        ],
        [
            163,
            24.637546920776366
        ],
        [
            164,
            24.336734580993653
        ],
        [
            165,
            24.03711280822754
        ],
        [
            166,
            23.741225910186767
        ],
        [
            167,
            23.445183038711548
        ],
        [
            168,
            23.153436183929443
        ],
        [
            169,
            22.863087224960328
        ],
        [
            170,
            22.579748392105103
        ],
        [
            171,
            22.294479250907898
        ],
        [
            172,
            22.01575622558594
        ],
        [
            173,
            21.736372470855713
        ],
        [
            174,
            21.462302494049073
        ],
        [
            175,
            21.189022040367128
        ],
        [
            176,
            20.92041597366333
        ],
        [
            177,
            20.65581817626953
        ],
        [
            178,
            20.397494554519653
        ],
        [
            179,
            20.144118857383727
        ],
        [
            180,
            19.893408441543578
        ],
        [
            181,
            19.647991800308226
        ],
        [
            182,
            19.403077554702758
        ],
        [
            183,
            19.16271893978119
        ],
        [
            184,
            18.925452947616577
        ],
        [
            185,
            18.693117713928224
        ],
        [
            186,
            18.465329241752624
        ],
        [
            187,
            18.239773392677307
        ],
        [
            188,
            18.01725606918335
        ],
        [
            189,
            17.80028533935547
        ],
        [
            190,
            17.585278165340423
        ],
        [
            191,
            17.376096618175506
        ],
        [
            192,
            17.170591175556183
        ],
        [
            193,
            16.96990724802017
        ],
        [
            194,
            16.772562289237975
        ],
        [
            195,
            16.57982733249664
        ],
        [
            196,
            16.391529774665834
        ],
        [
            197,
            16.2036514043808
        ],
        [
            198,
            16.02025238275528
        ],
        [
            199,
            15.84366533756256
        ],
        [
            200,
            15.665454387664795
        ],
        [
            201,
            15.495593059062958
        ],
        [
            202,
            15.330891942977905
        ],
        [
            203,
            15.167679977416991
        ],
        [
            204,
            15.00886926651001
        ],
        [
            205,
            14.856748485565186
        ],
        [
            206,
            14.70471420288086
        ],
        [
            207,
            14.56083586215973
        ],
        [
            208,
            14.430425691604615
        ],
        [
            209,
            14.288251757621765
        ],
        [
            210,
            14.155773043632507
        ],
        [
            211,
            14.032456302642823
        ],
        [
            212,
            13.904521226882935
        ],
        [
            213,
            13.786489152908326
        ],
        [
            214,
            13.666686058044434
        ],
        [
            215,
            13.557199501991272
        ],
        [
            216,
            13.441503286361694
        ],
        [
            217,
            13.335164165496826
        ],
        [
            218,
            13.231816005706786
        ],
        [
            219,
            13.130808544158935
        ],
        [
            220,
            13.032982873916627
        ],
        [
            221,
            12.9376699924469
        ],
        [
            222,
            12.845876431465149
        ],
        [
            223,
            12.760246181488037
        ],
        [
            224,
            12.673237371444703
        ],
        [
            225,
            12.593268394470215
        ],
        [
            226,
            12.51784725189209
        ],
        [
            227,
            12.438291645050048
        ],
        [
            228,
            12.361939978599548
        ],
        [
            229,
            12.286595249176026
        ],
        [
            230,
            12.209203958511353
        ],
        [
            231,
            12.138452792167664
        ],
        [
            232,
            12.068420267105102
        ],
        [
            233,
            11.99515676498413
        ],
        [
            234,
            11.927405452728271
        ],
        [
            235,
            11.862112617492675
        ],
        [
            236,
            11.797830867767335
        ],
        [
            237,
            11.734135055541993
        ],
        [
            238,
            11.673254156112671
        ],
        [
            239,
            11.613497066497803
        ],
        [
            240,
            11.552998161315918
        ],
        [
            241,
            11.499847507476806
        ],
        [
            242,
            11.445398616790772
        ],
        [
            243,
            11.392206001281739
        ],
        [
            244,
            11.340223693847657
        ],
        [
            245,
            11.288409423828124
        ],
        [
            246,
            11.235559511184693
        ],
        [
            247,
            11.188125944137573
        ],
        [
            248,
            11.137593841552734
        ],
        [
            249,
            11.091151189804076
        ],
        [
            250,
            11.046013021469117
        ],
        [
            251,
            11.001793622970581
        ],
        [
            252,
            10.959609699249267
        ],
        [
            253,
            10.919145727157593
        ],
        [
            254,
            10.878639078140258
        ],
        [
            255,
            10.837990665435791
        ],
        [
            256,
            10.798628091812134
        ],
        [
            257,
            10.762860012054443
        ],
        [
            258,
            10.724998617172242
        ],
        [
            259,
            10.687517070770264
        ],
        [
            260,
            10.65149998664856
        ]
    ],
    "rmse_physical": [
        [
            0,
            4.440450703200741
        ],
        [
            1,
            4.437760595025352
        ],
        [
            2,
            4.435032127804951
        ],
        [
            3,
            4.4323156454705686
        ],
        [
            4,
            4.429603146989632
        ],
        [
            5,
            4.42685345238342
        ],
        [
            6,
            4.424094914999851
        ],
        [
            7,
            4.42128519192374
        ],
        [
            8,
            4.41848572965347
        ],
        [
            9,
            4.415644903490724
        ],
        [
            10,
            4.412761479714501
        ],
        [
            11,
            4.409886673284086
        ],
        [
            12,
            4.406969955627194
        ],
        [
            13,
            4.404011055960202
        ],
        [
            14,
            4.401000570857454
        ],
        [
            15,
            4.397969162607074
        ],
        [
            16,
            4.394926260718484
        ],
        [
            17,
            4.391818870220186
        ],
        [
            18,
            4.388685848747894
        ],
        [
            19,
            4.385520516370264
        ],
        [
            20,
            4.382325169085177
        ],
        [
            21,
            4.379106792525376
        ],
        [
            22,
            4.375823938262085
        ],
        [
            23,
            4.372520845038449
        ],
        [
            24,
            4.369167991881998
        ],
        [
            25,
            4.365763525277545
        ],
        [
            26,
            4.36235488809762
        ],
        [
            27,
            4.358883818038754
        ],
        [
            28,
            4.355374863829805
        ],
        [
            29,
            4.351813217488691
        ],
        [
            30,
            4.348201698689623
        ],
        [
            31,
            4.344538411601566
        ],
        [
            32,
            4.34088990430198
        ],
        [
            33,
            4.337147040204305
        ],
        [
            34,
            4.333389087856782
        ],
        [
            35,
            4.32957155504949
        ],
        [
            36,
            4.325697547566883
        ],
        [
            37,
            4.321763659117391
        ],
        [
            38,
            4.317771191376657
        ],
        [
            39,
            4.313744660472227
        ],
        [
            40,
            4.309621274295856
        ],
        [
            41,
            4.305478676970884
        ],
        [
            42,
            4.301276858760463
        ],
        [
            43,
            4.297007333170549
        ],
        [
            44,
            4.292673920829421
        ],
        [
            45,
            4.288318685179127
        ],
        [
            46,
            4.283872759734463
        ],
        [
            47,
            4.279384403367215
        ],
        [
            48,
            4.274819686407786
        ],
        [
            49,
            4.270226318296773
        ],
        [
            50,
            4.265508487927177
        ],
        [
            51,
            4.260732061663724
        ],
        [
            52,
            4.255860924513618
        ],
        [
            53,
            4.250962020557209
        ],
        [
            54,
            4.245998595071472
        ],
        [
            55,
            4.240954966990344
        ],
        [
            56,
            4.235811882805675
        ],
        [
            57,
            4.230583665055684
        ],
        [
            58,
            4.2252794807974
        ],
        [
            59,
            4.219891582635291
        ],
        [
            60,
            4.214453434805062
        ],
        [
            61,
            4.208917833150443
        ],
        [
            62,
            4.203291692941881
        ],
        [
            63,
            4.19755762044642
        ],
        [
            64,
            4.191775175885958
        ],
        [
            65,
            4.185871942048055
        ],
        [
            66,
            4.179900029948423
        ],
        [
            67,
            4.17377315746788
        ],
        [
            68,
            4.167570239361744
        ],
        [
            69,
            4.161214872084341
        ],
        [
            70,
            4.154867627491945
        ],
        [
            71,
            4.14834326233989
        ],
        [
            72,
            4.14176761524167
        ],
        [
            73,
            4.1351016467895425
        ],
        [
            74,
            4.1282171311090226
        ],
        [
            75,
            4.121311764499238
        ],
        [
            76,
            4.114286695079847
        ],
        [
            77,
            4.1071246374381465
        ],
        [
            78,
            4.099831314704585
        ],
        [
            79,
            4.09242167203898
        ],
        [
            80,
            4.084921772820867
        ],
        [
            81,
            4.077274755376443
        ],
        [
            82,
            4.069541359385867
        ],
        [
            83,
            4.061592666523913
        ],
        [
            84,
            4.053608661311622
        ],
        [
            85,
            4.045391761342295
        ],
        [
            86,
            4.037060086256374
        ],
        [
            87,
            4.028698850232295
        ],
        [
            88,
            4.020202279222302
        ],
        [
            89,
            4.01157891107882
        ],
        [
            90,
            4.002804303402181
        ],
        [
            91,
            3.993774058320868
        ],
        [
            92,
            3.98468347953285
        ],
        [
            93,
            3.9754481619652884
        ],
        [
            94,
            3.966102373891774
        ],
        [
            95,
            3.956692522227822
        ],
        [
            96,
            3.9471513254987407
        ],
        [
            97,
            3.9374775236124373
        ],
        [
            98,
            3.9276151605817824
        ],
        [
            99,
            3.9176635934972377
        ],
        [
            100,
            3.9074990820487923
        ],
        [
            101,
            3.8972944408254406
        ],
        [
            102,
            3.8869012569360684
        ],
        [
            103,
            3.876297822797441
        ],
        [
            104,
            3.865679752969501
        ],
        [
            105,
            3.8548240656622417
        ],
        [
            106,
            3.844068762353415
        ],
        [
            107,
            3.832883999086281
        ],
        [
            108,
            3.821660128265896
        ],
        [
            109,
            3.810312673228585
        ],
        [
            110,
            3.7987169488949735
        ],
        [
            111,
            3.7871303349763803
        ],
        [
            112,
            3.7754311804518603
        ],
        [
            113,
            3.763494733460289
        ],
        [
            114,
            3.751572460929845
        ],
        [
            115,
            3.739505859087928
        ],
        [
            116,
            3.727281665874493
        ],
        [
            117,
            3.714994572158717
        ],
        [
            118,
            3.702571569416561
        ],
        [
            119,
            3.689915387878605
        ],
        [
            120,
            3.6772195452904195
        ],
        [
            121,
            3.6644109441605184
        ],
        [
            122,
            3.651596160585368
        ],
        [
            123,
            3.638543915262816
        ],
        [
            124,
            3.625504099820192
        ],
        [
            125,
            3.6122573371310547
        ],
        [
            126,
            3.5988496618993935
        ],
        [
            127,
            3.5853617127632114
        ],
        [
            128,
            3.571888554340078
        ],
        [
            129,
            3.5583669314385284
        ],
        [
            130,
            3.5447774500157663
        ],
        [
            131,
            3.5310858917564003
        ],
        [
            132,
            3.517275768580054
        ],
        [
            133,
            3.503193944033841
        ],
        [
            134,
            3.489437332722873
        ],
        [
            135,
            3.475384836835798
        ],
        [
            136,
            3.4611362617236665
        ],
        [
            137,
            3.446926574487065
        ],
        [
            138,
            3.432517119801553
        ],
        [
            139,
            3.4183859447380467
        ],
        [
            140,
            3.4041667563765006
        ],
        [
            141,
            3.389890729564456
        ],
        [
            142,
            3.375330742405596
        ],
        [
            143,
            3.3608469868150572
        ],
        [
            144,
            3.346352505001892
        ],
        [
            145,
            3.3317432352465013
        ],
        [
            146,
            3.3172211681095987
        ],
        [
            147,
            3.302744978540091
        ],
        [
            148,
            3.2885820295825146
        ],
        [
            149,
            3.2738719393442275
        ],
        [
            150,
            3.259173372249661
        ],
        [
            151,
            3.2445623525221774
        ],
        [
            152,
            3.229868704336813
        ],
        [
            153,
            3.21545915125169
        ],
        [
            154,
            3.20072505153949
        ],
        [
            155,
            3.185873551194905
        ],
        [
            156,
            3.171496152051683
        ],
        [
            157,
            3.1567544948872897
        ],
        [
            158,
            3.1424989376136168
        ],
        [
            159,
            3.1279745178784806
        ],
        [
            160,
            3.113724965507428
        ],
        [
            161,
            3.099604208060535
        ],
        [
            162,
            3.085386554894771
        ],
        [
            163,
            3.0709201123417573
        ],
        [
            164,
            3.0569871353681326
        ],
        [
            165,
            3.0434433534404453
        ],
        [
            166,
            3.029804083047139
        ],
        [
            167,
            3.016174605165843
        ],
        [
            168,
            3.002693057486414
        ],
        [
            169,
            2.9892766262671593
        ],
        [
            170,
            2.9763085789229202
        ],
        [
            171,
            2.962780748186889
        ],
        [
            172,
            2.9497222817756548
        ],
        [
            173,
            2.9360710646628605
        ],
        [
            174,
            2.922840248777662
        ],
        [
            175,
            2.90945792095914
        ],
        [
            176,
            2.8960205511704666
        ],
        [
            177,
            2.8831348519240576
        ],
        [
            178,
            2.8704560161809334
        ],
        [
            179,
            2.858141700265475
        ],
        [
            180,
            2.845693203509672
        ],
        [
            181,
            2.8335271961643733
        ],
        [
            182,
            2.8209103995277838
        ],
        [
            183,
            2.8085758311323312
        ],
        [
            184,
            2.7965437743000017
        ],
        [
            185,
            2.785058416512657
        ],
        [
            186,
            2.7732838159403395
        ],
        [
            187,
            2.761502295105885
        ],
        [
            188,
            2.749829040167402
        ],
        [
            189,
            2.7384908824128993
        ],
        [
            190,
            2.7268184288599624
        ],
        [
            191,
            2.7153774355152174
        ],
        [
            192,
            2.7040941446110973
        ],
        [
            193,
            2.6929544452490153
        ],
        [
            194,
            2.6818937309023183
        ],
        [
            195,
            2.670970342542292
        ],
        [
            196,
            2.6604460603133524
        ],
        [
            197,
            2.649729603699875
        ],
        [
            198,
            2.63940376565291
        ],
        [
            199,
            2.629223718939482
        ],
        [
            200,
            2.6188144347494897
        ],
        [
            201,
            2.608400917931857
        ],
        [
            202,
            2.5987100202124873
        ],
        [
            203,
            2.5889277779421658
        ],
        [
            204,
            2.5793715485532545
        ],
        [
            205,
            2.56969031907537
        ],
        [
            206,
            2.560174185156462
        ],
        [
            207,
            2.55087786151483
        ],
        [
            208,
            2.5426530824387403
        ],
        [
            209,
            2.5333926321050524
        ],
        [
            210,
            2.524600919535723
        ],
        [
            211,
            2.5164718731174025
        ],
        [
            212,
            2.5078124890921902
        ],
        [
            213,
            2.4996664084492335
        ],
        [
            214,
            2.4914162040457364
        ],
        [
            215,
            2.483529450213495
        ],
        [
            216,
            2.4753175414983972
        ],
        [
            217,
            2.467471324933341
        ],
        [
            218,
            2.460100655253664
        ],
        [
            219,
            2.4529375092167793
        ],
        [
            220,
            2.445874858836482
        ],
        [
            221,
            2.438871797537309
        ],
        [
            222,
            2.4320223351032126
        ],
        [
            223,
            2.4255558185468606
        ],
        [
            224,
            2.4189756413917722
        ],
        [
            225,
            2.41260858468754
        ],
        [
            226,
            2.4067007892239816
        ],
        [
            227,
            2.4003022414626245
        ],
        [
            228,
            2.394075493384594
        ],
        [
            229,
            2.387859498545144
        ],
        [
            230,
            2.3814369384033953
        ],
        [
            231,
            2.3756857063503856
        ],
        [
            232,
            2.36964646081931
        ],
        [
            233,
            2.363319037551873
        ],
        [
            234,
            2.357340191142082
        ],
        [
            235,
            2.351602563389113
        ],
        [
            236,
            2.3458035456085145
        ],
        [
            237,
            2.3403649653454126
        ],
        [
            238,
            2.334855578495727
        ],
        [
            239,
            2.329364615062361
        ],
        [
            240,
            2.3235988155866782
        ],
        [
            241,
            2.318450453833931
        ],
        [
            242,
            2.3134136104888476
        ],
        [
            243,
            2.308334928288816
        ],
        [
            244,
            2.3035327755008037
        ],
        [
            245,
            2.2983183733404244
        ],
        [
            246,
            2.293311314911603
        ],
        [
            247,
            2.2885164557354143
        ],
        [
            248,
            2.2832850840388046
        ],
        [
            249,
            2.278432399815958
        ],
        [
            250,
            2.273821334502346
        ],
        [
            251,
            2.269216185695075
        ],
        [
            252,
            2.2643274114963776
        ],
        [
            253,
            2.2601073097805444
        ],
        [
            254,
            2.2557200336319645
        ],
        [
            255,
            2.2509692910553665
        ],
        [
            256,
            2.246550375284796
        ],
        [
            257,
            2.2425559202726757
        ],
        [
            258,
            2.2379546790328653
        ],
        [
            259,
            2.233529950435382
        ],
        [
            260,
            2.229205107357377
        ]
    ],
    "mae_physical": [
        [
            0,
            4.0280922355372715
        ],
        [
            1,
            4.025070257193875
        ],
        [
            2,
            4.022001769551561
        ],
        [
            3,
            4.01895478588556
        ],
        [
            4,
            4.015912938078432
        ],
        [
            5,
            4.012826026053716
        ],
        [
            6,
            4.009730599897161
        ],
        [
            7,
            4.00657121163525
        ],
        [
            8,
            4.003418104512591
        ],
        [
            9,
            4.00022187211538
        ],
        [
            10,
            3.996979104227399
        ],
        [
            11,
            3.9937455045212418
        ],
        [
            12,
            3.990477878115561
        ],
        [
            13,
            3.9871453083413715
        ],
        [
            14,
            3.983763766953441
        ],
        [
            15,
            3.980359458575051
        ],
        [
            16,
            3.9769327451238943
        ],
        [
            17,
            3.973437768046757
        ],
        [
            18,
            3.9699102018972576
        ],
        [
            19,
            3.9663537324769806
        ],
        [
            20,
            3.9627544906864274
        ],
        [
            21,
            3.959129714643047
        ],
        [
            22,
            3.955431923550898
        ],
        [
            23,
            3.9517107687680024
        ],
        [
            24,
            3.9479405390200752
        ],
        [
            25,
            3.944107705267937
        ],
        [
            26,
            3.9402645504140383
        ],
        [
            27,
            3.9363547261797964
        ],
        [
            28,
            3.932397723848587
        ],
        [
            29,
            3.9283842905424535
        ],
        [
            30,
            3.9243122858556676
        ],
        [
            31,
            3.9201820104906004
        ],
        [
            32,
            3.9160605731917775
        ],
        [
            33,
            3.9118381838684213
        ],
        [
            34,
            3.907593693813845
        ],
        [
            35,
            3.9032838130242204
        ],
        [
            36,
            3.8989070033948
        ],
        [
            37,
            3.8944705251591563
        ],
        [
            38,
            3.889959534336749
        ],
        [
            39,
            3.885406008985353
        ],
        [
            40,
            3.88074562720291
        ],
        [
            41,
            3.8760749051117735
        ],
        [
            42,
            3.871320067777311
        ],
        [
            43,
            3.8664987338324117
        ],
        [
            44,
            3.8615955093824494
        ],
        [
            45,
            3.8566738686026776
        ],
        [
            46,
            3.8516339918164766
        ],
        [
            47,
            3.8465617131186676
        ],
        [
            48,
            3.8413944277810317
        ],
        [
            49,
            3.8361983184945094
        ],
        [
            50,
            3.8308545264275105
        ],
        [
            51,
            3.825458667662046
        ],
        [
            52,
            3.8199230085497033
        ],
        [
            53,
            3.8143898111908405
        ],
        [
            54,
            3.8087628442436863
        ],
        [
            55,
            3.8030567547842247
        ],
        [
            56,
            3.7972258096688414
        ],
        [
            57,
            3.7913042599820113
        ],
        [
            58,
            3.7853002200694115
        ],
        [
            59,
            3.779187610194291
        ],
        [
            60,
            3.773016354298755
        ],
        [
            61,
            3.766741162599766
        ],
        [
            62,
            3.7603690866125774
        ],
        [
            63,
            3.7538761527162707
        ],
        [
            64,
            3.7473069538604724
        ],
        [
            65,
            3.740626486605161
        ],
        [
            66,
            3.7338512611817825
        ],
        [
            67,
            3.7269027561765826
        ],
        [
            68,
            3.719862312792915
        ],
        [
            69,
            3.7126808268566656
        ],
        [
            70,
            3.7054860561269605
        ],
        [
            71,
            3.698090365079984
        ],
        [
            72,
            3.69063840612565
        ],
        [
            73,
            3.6830827022252017
        ],
        [
            74,
            3.675297805299498
        ],
        [
            75,
            3.667455690262252
        ],
        [
            76,
            3.6594917808901775
        ],
        [
            77,
            3.6513676112645292
        ],
        [
            78,
            3.6431127785615725
        ],
        [
            79,
            3.6347181396941615
        ],
        [
            80,
            3.6262012971182394
        ],
        [
            81,
            3.617539146583374
        ],
        [
            82,
            3.608759558160011
        ],
        [
            83,
            3.599739582979516
        ],
        [
            84,
            3.5906648446027547
        ],
        [
            85,
            3.5813450137638068
        ],
        [
            86,
            3.5718865523191345
        ],
        [
            87,
            3.5623912868434435
        ],
        [
            88,
            3.5527536742491264
        ],
        [
            89,
            3.542931921882172
        ],
        [
            90,
            3.532953579948373
        ],
        [
            91,
            3.5226970941644824
        ],
        [
            92,
            3.512338786909025
        ],
        [
            93,
            3.5018362315141993
        ],
        [
            94,
            3.491196373145874
        ],
        [
            95,
            3.480468118435716
        ],
        [
            96,
            3.469611244658901
        ],
        [
            97,
            3.4585548894454354
        ],
        [
            98,
            3.4473109265712845
        ],
        [
            99,
            3.435958656546188
        ],
        [
            100,
            3.4243468960670578
        ],
        [
            101,
            3.4127026220707046
        ],
        [
            102,
            3.4008114450190163
        ],
        [
            103,
            3.3886869572205085
        ],
        [
            104,
            3.3765460060883874
        ],
        [
            105,
            3.3641230194535976
        ],
        [
            106,
            3.3517710295033782
        ],
        [
            107,
            3.3389510165338647
        ],
        [
            108,
            3.326074771685143
        ],
        [
            109,
            3.313091448929212
        ],
        [
            110,
            3.2999484306328917
        ],
        [
            111,
            3.286922334808193
        ],
        [
            112,
            3.2740314676337046
        ],
        [
            113,
            3.261062548176883
        ],
        [
            114,
            3.248825238583839
        ],
        [
            115,
            3.236432478035966
        ],
        [
            116,
            3.22391233093118
        ],
        [
            117,
            3.2114356895015663
        ],
        [
            118,
            3.1988439906949866
        ],
        [
            119,
            3.185979231579663
        ],
        [
            120,
            3.1730406802811033
        ],
        [
            121,
            3.1600092200383747
        ],
        [
            122,
            3.1469419700642156
        ],
        [
            123,
            3.133608446545797
        ],
        [
            124,
            3.1202591318790227
        ],
        [
            125,
            3.1067226794484544
        ],
        [
            126,
            3.0929898680889445
        ],
        [
            127,
            3.0791277146502716
        ],
        [
            128,
            3.0652794017367166
        ],
        [
            129,
            3.0513315943822468
        ],
        [
            130,
            3.0373674133052564
        ],
        [
            131,
            3.0232290699057383
        ],
        [
            132,
            3.0089595938382083
        ],
        [
            133,
            2.994353697316287
        ],
        [
            134,
            2.9800691776079673
        ],
        [
            135,
            2.965487475672813
        ],
        [
            136,
            2.9506414116245425
        ],
        [
            137,
            2.9358370340850257
        ],
        [
            138,
            2.9208313381835205
        ],
        [
            139,
            2.906065149666512
        ],
        [
            140,
            2.891156250075118
        ],
        [
            141,
            2.87627304294338
        ],
        [
            142,
            2.8611788574146897
        ],
        [
            143,
            2.846174997417894
        ],
        [
            144,
            2.8316455895770085
        ],
        [
            145,
            2.8176024666387742
        ],
        [
            146,
            2.8035878788118493
        ],
        [
            147,
            2.789626613463441
        ],
        [
            148,
            2.7759485689744556
        ],
        [
            149,
            2.7616919617946833
        ],
        [
            150,
            2.747415797759409
        ],
        [
            151,
            2.733164125517623
        ],
        [
            152,
            2.7188431902291024
        ],
        [
            153,
            2.7047894217380106
        ],
        [
            154,
            2.6906549109988016
        ],
        [
            155,
            2.6765073501083947
        ],
        [
            156,
            2.663103180797133
        ],
        [
            157,
            2.649559062229444
        ],
        [
            158,
            2.6365820214356464
        ],
        [
            159,
            2.6235625356027525
        ],
        [
            160,
            2.610783233626248
        ],
        [
            161,
            2.5981093763488614
        ],
        [
            162,
            2.585286070630975
        ],
        [
            163,
            2.5722398933482498
        ],
        [
            164,
            2.5596329369773603
        ],
        [
            165,
            2.5473413022413647
        ],
        [
            166,
            2.5349300083232253
        ],
        [
            167,
            2.5224379247998536
        ],
        [
            168,
            2.510107894874599
        ],
        [
            169,
            2.4977288184917135
        ],
        [
            170,
            2.485817565901639
        ],
        [
            171,
            2.473336037299404
        ],
        [
            172,
            2.4612657431870293
        ],
        [
            173,
            2.448535727719738
        ],
        [
            174,
            2.436249299816889
        ],
        [
            175,
            2.4237409393264824
        ],
        [
            176,
            2.411127753861963
        ],
        [
            177,
            2.3989903930931877
        ],
        [
            178,
            2.387043679821981
        ],
        [
            179,
            2.375346287880858
        ],
        [
            180,
            2.3635511647348535
        ],
        [
            181,
            2.351934797551534
        ],
        [
            182,
            2.3398822778708315
        ],
        [
            183,
            2.328075412609806
        ],
        [
            184,
            2.316429893039677
        ],
        [
            185,
            2.3053757184988832
        ],
        [
            186,
            2.2938949665794635
        ],
        [
            187,
            2.2824799479687052
        ],
        [
            188,
            2.2710944271250946
        ],
        [
            189,
            2.2599796207800305
        ],
        [
            190,
            2.2485070012203634
        ],
        [
            191,
            2.237227194521525
        ],
        [
            192,
            2.226025554823549
        ],
        [
            193,
            2.2149975671343607
        ],
        [
            194,
            2.203916213692051
        ],
        [
            195,
            2.1938178412718314
        ],
        [
            196,
            2.1851123126402294
        ],
        [
            197,
            2.1763233317904276
        ],
        [
            198,
            2.1678489507877665
        ],
        [
            199,
            2.159831856619822
        ],
        [
            200,
            2.1519038068921597
        ],
        [
            201,
            2.1440770654645687
        ],
        [
            202,
            2.1367266467989308
        ],
        [
            203,
            2.1294390334658426
        ],
        [
            204,
            2.122411859770344
        ],
        [
            205,
            2.1152019317019475
        ],
        [
            206,
            2.1081790299448246
        ],
        [
            207,
            2.1018064279262334
        ],
        [
            208,
            2.096271511218319
        ],
        [
            209,
            2.0901907515035916
        ],
        [
            210,
            2.0844565756516915
        ],
        [
            211,
            2.0791704977211887
        ],
        [
            212,
            2.0735669507555765
        ],
        [
            213,
            2.0682456587275415
        ],
        [
            214,
            2.0628210677676004
        ],
        [
            215,
            2.057625897534906
        ],
        [
            216,
            2.05219543755871
        ],
        [
            217,
            2.0469766050985414
        ],
        [
            218,
            2.0420637551235825
        ],
        [
            219,
            2.0372866616673666
        ],
        [
            220,
            2.0324829812735725
        ],
        [
            221,
            2.0277757428280294
        ],
        [
            222,
            2.0231257787306016
        ],
        [
            223,
            2.018733734545642
        ],
        [
            224,
            2.014263830364567
        ],
        [
            225,
            2.00988063052909
        ],
        [
            226,
            2.005867451429367
        ],
        [
            227,
            2.0015066694723416
        ],
        [
            228,
            1.997208941064469
        ],
        [
            229,
            1.9929090456603324
        ],
        [
            230,
            1.9884418261377779
        ],
        [
            231,
            1.9844367793978077
        ],
        [
            232,
            1.980225827187708
        ],
        [
            233,
            1.9757740656807
        ],
        [
            234,
            1.9715664562297195
        ],
        [
            235,
            1.9675089639343628
        ],
        [
            236,
            1.963363771160988
        ],
        [
            237,
            1.9594726166496539
        ],
        [
            238,
            1.9555382806144348
        ],
        [
            239,
            1.951604908051556
        ],
        [
            240,
            1.947434862590816
        ],
        [
            241,
            1.9437033570792577
        ],
        [
            242,
            1.9400251364871248
        ],
        [
            243,
            1.9363239917853108
        ],
        [
            244,
            1.9328022668622944
        ],
        [
            245,
            1.9289840978302368
        ],
        [
            246,
            1.925293505600054
        ],
        [
            247,
            1.9216924689403951
        ],
        [
            248,
            1.9178047894615016
        ],
        [
            249,
            1.91414889203359
        ],
        [
            250,
            1.9106672981830493
        ],
        [
            251,
            1.9072466251784808
        ],
        [
            252,
            1.9035213234489912
        ],
        [
            253,
            1.900308030517134
        ],
        [
            254,
            1.8969655343114513
        ],
        [
            255,
            1.8933107963163558
        ],
        [
            256,
            1.8899595929335242
        ],
        [
            257,
            1.8869455607786572
        ],
        [
            258,
            1.8834929903076119
        ],
        [
            259,
            1.8801444960783606
        ],
        [
            260,
            1.8768944621902623
        ]
    ],
    "r2_physical": [
        [
            0,
            -4.715246132823159
        ],
        [
            1,
            -4.708323424182014
        ],
        [
            2,
            -4.701306287933014
        ],
        [
            3,
            -4.6943242619734376
        ],
        [
            4,
            -4.68735674392918
        ],
        [
            5,
            -4.680298034688912
        ],
        [
            6,
            -4.673221029506763
        ],
        [
            7,
            -4.666017242707525
        ],
        [
            8,
            -4.6588443148685315
        ],
        [
            9,
            -4.651570046682687
        ],
        [
            10,
            -4.644191486501559
        ],
        [
            11,
            -4.63683977594725
        ],
        [
            12,
            -4.629385782579772
        ],
        [
            13,
            -4.621829027671976
        ],
        [
            14,
            -4.614145737263259
        ],
        [
            15,
            -4.6064143561052315
        ],
        [
            16,
            -4.598659018822229
        ],
        [
            17,
            -4.590744861206005
        ],
        [
            18,
            -4.582771091573274
        ],
        [
            19,
            -4.574720866975495
        ],
        [
            20,
            -4.566600198212199
        ],
        [
            21,
            -4.558426985878409
        ],
        [
            22,
            -4.550096215768337
        ],
        [
            23,
            -4.541720391550603
        ],
        [
            24,
            -4.533224857023034
        ],
        [
            25,
            -4.5246052113182635
        ],
        [
            26,
            -4.5159817377033304
        ],
        [
            27,
            -4.50720723755867
        ],
        [
            28,
            -4.498344069564884
        ],
        [
            29,
            -4.489355107420602
        ],
        [
            30,
            -4.480247785049478
        ],
        [
            31,
            -4.471017640906403
        ],
        [
            32,
            -4.461832468928367
        ],
        [
            33,
            -4.452417771102447
        ],
        [
            34,
            -4.442973291055748
        ],
        [
            35,
            -4.433387455999052
        ],
        [
            36,
            -4.423668449716763
        ],
        [
            37,
            -4.413808118678079
        ],
        [
            38,
            -4.40381013072696
        ],
        [
            39,
            -4.3937362004279255
        ],
        [
            40,
            -4.383429689715618
        ],
        [
            41,
            -4.373085085640598
        ],
        [
            42,
            -4.362602762130527
        ],
        [
            43,
            -4.351962011909455
        ],
        [
            44,
            -4.3411728453752785
        ],
        [
            45,
            -4.330340312425101
        ],
        [
            46,
            -4.319293553670407
        ],
        [
            47,
            -4.30815299036588
        ],
        [
            48,
            -4.296834870374849
        ],
        [
            49,
            -4.2854579036278
        ],
        [
            50,
            -4.273785398648755
        ],
        [
            51,
            -4.261981064592003
        ],
        [
            52,
            -4.249956286166892
        ],
        [
            53,
            -4.237876836419079
        ],
        [
            54,
            -4.225652481684186
        ],
        [
            55,
            -4.213245226563012
        ],
        [
            56,
            -4.200608496635633
        ],
        [
            57,
            -4.187778307621844
        ],
        [
            58,
            -4.174777887706985
        ],
        [
            59,
            -4.1615889850806935
        ],
        [
            60,
            -4.148294142354792
        ],
        [
            61,
            -4.134778658253414
        ],
        [
            62,
            -4.121060321336825
        ],
        [
            63,
            -4.107097692075827
        ],
        [
            64,
            -4.093036577959474
        ],
        [
            65,
            -4.07870173688747
        ],
        [
            66,
            -4.064220679922834
        ],
        [
            67,
            -4.049385351857557
        ],
        [
            68,
            -4.034388058711398
        ],
        [
            69,
            -4.019045311022256
        ],
        [
            70,
            -4.003745542654192
        ],
        [
            71,
            -3.988043174323021
        ],
        [
            72,
            -3.972242352745573
        ],
        [
            73,
            -3.956250080229923
        ],
        [
            74,
            -3.939760532922884
        ],
        [
            75,
            -3.9232486454217117
        ],
        [
            76,
            -3.906478894338462
        ],
        [
            77,
            -3.8894115865017023
        ],
        [
            78,
            -3.8720620304339217
        ],
        [
            79,
            -3.8544673472073496
        ],
        [
            80,
            -3.836690755474793
        ],
        [
            81,
            -3.8185990311781115
        ],
        [
            82,
            -3.800337424057041
        ],
        [
            83,
            -3.781603546894906
        ],
        [
            84,
            -3.762823315332132
        ],
        [
            85,
            -3.7435338471516246
        ],
        [
            86,
            -3.7240149047000335
        ],
        [
            87,
            -3.7044671647200538
        ],
        [
            88,
            -3.684644541940277
        ],
        [
            89,
            -3.6645688913612453
        ],
        [
            90,
            -3.644185396652282
        ],
        [
            91,
            -3.623254657414039
        ],
        [
            92,
            -3.6022318212530635
        ],
        [
            93,
            -3.5809233194197088
        ],
        [
            94,
            -3.5594102655351456
        ],
        [
            95,
            -3.5378008996512387
        ],
        [
            96,
            -3.5159423151623246
        ],
        [
            97,
            -3.4938338152466297
        ],
        [
            98,
            -3.4713502249912063
        ],
        [
            99,
            -3.448720426500203
        ],
        [
            100,
            -3.4256656606066542
        ],
        [
            101,
            -3.4025801230887502
        ],
        [
            102,
            -3.3791301045897866
        ],
        [
            103,
            -3.355270233165193
        ],
        [
            104,
            -3.331442741921043
        ],
        [
            105,
            -3.3071495937100153
        ],
        [
            106,
            -3.2831484593059574
        ],
        [
            107,
            -3.2582600868637606
        ],
        [
            108,
            -3.233357594068945
        ],
        [
            109,
            -3.208255143702673
        ],
        [
            110,
            -3.1826805943619725
        ],
        [
            111,
            -3.1572039946341492
        ],
        [
            112,
            -3.131558904486157
        ],
        [
            113,
            -3.1054754336070616
        ],
        [
            114,
            -3.0795053860043122
        ],
        [
            115,
            -3.053304851402868
        ],
        [
            116,
            -3.026848202660225
        ],
        [
            117,
            -3.000342715604976
        ],
        [
            118,
            -2.9736330259978487
        ],
        [
            119,
            -2.9465139995223146
        ],
        [
            120,
            -2.919403283628313
        ],
        [
            121,
            -2.8921464717746064
        ],
        [
            122,
            -2.864971679878498
        ],
        [
            123,
            -2.8373911926807462
        ],
        [
            124,
            -2.809935589874537
        ],
        [
            125,
            -2.782145186349444
        ],
        [
            126,
            -2.7541207908205227
        ],
        [
            127,
            -2.726033758519074
        ],
        [
            128,
            -2.698082816305743
        ],
        [
            129,
            -2.6701371323095913
        ],
        [
            130,
            -2.64215799245103
        ],
        [
            131,
            -2.6140769550546223
        ],
        [
            132,
            -2.5858628314365046
        ],
        [
            133,
            -2.557207467311024
        ],
        [
            134,
            -2.5293248892043865
        ],
        [
            135,
            -2.5009558693305864
        ],
        [
            136,
            -2.472307890953243
        ],
        [
            137,
            -2.443855312420509
        ],
        [
            138,
            -2.4151222644096255
        ],
        [
            139,
            -2.387061019656602
        ],
        [
            140,
            -2.3589418469666907
        ],
        [
            141,
            -2.330828185908281
        ],
        [
            142,
            -2.302277020306377
        ],
        [
            143,
            -2.273997270866227
        ],
        [
            144,
            -2.2458183245376806
        ],
        [
            145,
            -2.217539459075021
        ],
        [
            146,
            -2.1895520123072822
        ],
        [
            147,
            -2.1617746590041444
        ],
        [
            148,
            -2.1347159307706587
        ],
        [
            149,
            -2.1067349817957193
        ],
        [
            150,
            -2.078901245904398
        ],
        [
            151,
            -2.051357421230606
        ],
        [
            152,
            -2.0237826397002663
        ],
        [
            153,
            -1.9968625590067255
        ],
        [
            154,
            -1.9694606221820785
        ],
        [
            155,
            -1.9419677075368194
        ],
        [
            156,
            -1.9154742505870441
        ],
        [
            157,
            -1.8884339937972041
        ],
        [
            158,
            -1.8624051957948997
        ],
        [
            159,
            -1.8360066528347976
        ],
        [
            160,
            -1.8102265343317385
        ],
        [
            161,
            -1.7847955489516596
        ],
        [
            162,
            -1.7593068420439657
        ],
        [
            163,
            -1.7334923951212895
        ],
        [
            164,
            -1.708744575893565
        ],
        [
            165,
            -1.6847959131624983
        ],
        [
            166,
            -1.6607858696456517
        ],
        [
            167,
            -1.6369007916586606
        ],
        [
            168,
            -1.6133808966343905
        ],
        [
            169,
            -1.590079205121628
        ],
        [
            170,
            -1.5676554432458683
        ],
        [
            171,
            -1.5443676224515528
        ],
        [
            172,
            -1.5219884313376073
        ],
        [
            173,
            -1.4986990894681949
        ],
        [
            174,
            -1.4762300564409037
        ],
        [
            175,
            -1.4536069512170111
        ],
        [
            176,
            -1.430995256139795
        ],
        [
            177,
            -1.4094101993283137
        ],
        [
            178,
            -1.3882656134209443
        ],
        [
            179,
            -1.367818149576106
        ],
        [
            180,
            -1.347237234083274
        ],
        [
            181,
            -1.3272101542074046
        ],
        [
            182,
            -1.306531639036983
        ],
        [
            183,
            -1.2864048937153254
        ],
        [
            184,
            -1.2668567469344696
        ],
        [
            185,
            -1.248275098385558
        ],
        [
            186,
            -1.2293048813671863
        ],
        [
            187,
            -1.2104039575980785
        ],
        [
            188,
            -1.1917560836865948
        ],
        [
            189,
            -1.1737191484087592
        ],
        [
            190,
            -1.1552282619026313
        ],
        [
            191,
            -1.1371806919669405
        ],
        [
            192,
            -1.1194562095947256
        ],
        [
            193,
            -1.1020296911297853
        ],
        [
            194,
            -1.0847979084005668
        ],
        [
            195,
            -1.0678496755430613
        ],
        [
            196,
            -1.0515861040812013
        ],
        [
            197,
            -1.0350915366550666
        ],
        [
            198,
            -1.0192611814464194
        ],
        [
            199,
            -1.003714843776459
        ],
        [
            200,
            -0.9878805505971828
        ],
        [
            201,
            -0.9721026706250564
        ],
        [
            202,
            -0.9574761282781288
        ],
        [
            203,
            -0.9427669337023055
        ],
        [
            204,
            -0.9284511523471357
        ],
        [
            205,
            -0.9140020969209275
        ],
        [
            206,
            -0.8998523956268709
        ],
        [
            207,
            -0.8860802257576721
        ],
        [
            208,
            -0.8739372809047532
        ],
        [
            209,
            -0.8603122196324682
        ],
        [
            210,
            -0.8474228230918113
        ],
        [
            211,
            -0.8355448208038634
        ],
        [
            212,
            -0.8229340380542074
        ],
        [
            213,
            -0.811110467007131
        ],
        [
            214,
            -0.7991749757379862
        ],
        [
            215,
            -0.7878021742427712
        ],
        [
            216,
            -0.7759988142326713
        ],
        [
            217,
            -0.764757601069866
        ],
        [
            218,
            -0.7542302099597438
        ],
        [
            219,
            -0.7440293972101844
        ],
        [
            220,
            -0.734000819452951
        ],
        [
            221,
            -0.724085406537339
        ],
        [
            222,
            -0.7144149715181156
        ],
        [
            223,
            -0.7053101586734123
        ],
        [
            224,
            -0.6960701963772131
        ],
        [
            225,
            -0.6871533947520527
        ],
        [
            226,
            -0.6789007891558132
        ],
        [
            227,
            -0.6699854751413861
        ],
        [
            228,
            -0.6613323222817828
        ],
        [
            229,
            -0.652716531477799
        ],
        [
            230,
            -0.6438379552197664
        ],
        [
            231,
            -0.6359077199529581
        ],
        [
            232,
            -0.6276009891404233
        ],
        [
            233,
            -0.6189205627570915
        ],
        [
            234,
            -0.6107396665910207
        ],
        [
            235,
            -0.6029083169886045
        ],
        [
            236,
            -0.5950125671322279
        ],
        [
            237,
            -0.5876252919665415
        ],
        [
            238,
            -0.580159322608077
        ],
        [
            239,
            -0.5727358274615479
        ],
        [
            240,
            -0.5649595811910215
        ],
        [
            241,
            -0.5580323507776301
        ],
        [
            242,
            -0.5512700406918976
        ],
        [
            243,
            -0.5444664499321246
        ],
        [
            244,
            -0.5380470616257116
        ],
        [
            245,
            -0.5310917287070935
        ],
        [
            246,
            -0.5244277999252747
        ],
        [
            247,
            -0.5180599114427038
        ],
        [
            248,
            -0.5111275087795555
        ],
        [
            249,
            -0.5047111114548792
        ],
        [
            250,
            -0.498626840211003
        ],
        [
            251,
            -0.49256267779726515
        ],
        [
            252,
            -0.4861384843858798
        ],
        [
            253,
            -0.48060411582204754
        ],
        [
            254,
            -0.47486145544820246
        ],
        [
            255,
            -0.468655624351789
        ],
        [
            256,
            -0.46289499905712006
        ],
        [
            257,
            -0.4576974540858534
        ],
        [
            258,
            -0.45172182895374946
        ],
        [
            259,
            -0.4459870166022597
        ],
        [
            260,
            -0.4403926323692431
        ]
    ],
    "rmse_mental": [
        [
            0,
            6.593446975872525
        ],
        [
            1,
            6.589380447464783
        ],
        [
            2,
            6.585330515589379
        ],
        [
            3,
            6.5812295429899415
        ],
        [
            4,
            6.5771124086039645
        ],
        [
            5,
            6.5729569288400445
        ],
        [
            6,
            6.568761377040165
        ],
        [
            7,
            6.564557767170877
        ],
        [
            8,
            6.560294445597346
        ],
        [
            9,
            6.555975116236076
        ],
        [
            10,
            6.551644528517969
        ],
        [
            11,
            6.547252939462844
        ],
        [
            12,
            6.542777513019448
        ],
        [
            13,
            6.538270954704741
        ],
        [
            14,
            6.533708573978524
        ],
        [
            15,
            6.529120127198244
        ],
        [
            16,
            6.5244762062156205
        ],
        [
            17,
            6.519779863217102
        ],
        [
            18,
            6.515007897654935
        ],
        [
            19,
            6.510168935438074
        ],
        [
            20,
            6.505277490235353
        ],
        [
            21,
            6.500312989665225
        ],
        [
            22,
            6.495309705154294
        ],
        [
            23,
            6.490225871863145
        ],
        [
            24,
            6.48509582150089
        ],
        [
            25,
            6.479883894804991
        ],
        [
            26,
            6.474580932572174
        ],
        [
            27,
            6.469200074681143
        ],
        [
            28,
            6.463756505860278
        ],
        [
            29,
            6.458238624466818
        ],
        [
            30,
            6.452667006691283
        ],
        [
            31,
            6.446997463347405
        ],
        [
            32,
            6.441218742231025
        ],
        [
            33,
            6.435348773600934
        ],
        [
            34,
            6.429377062406634
        ],
        [
            35,
            6.423325108798958
        ],
        [
            36,
            6.417231535092793
        ],
        [
            37,
            6.411008784841178
        ],
        [
            38,
            6.404699544433413
        ],
        [
            39,
            6.398288998371035
        ],
        [
            40,
            6.391799730109719
        ],
        [
            41,
            6.385174454378546
        ],
        [
            42,
            6.3784551868857875
        ],
        [
            43,
            6.371641649828949
        ],
        [
            44,
            6.364731314071961
        ],
        [
            45,
            6.3577018790104916
        ],
        [
            46,
            6.35058918541212
        ],
        [
            47,
            6.343296295662313
        ],
        [
            48,
            6.335885225064323
        ],
        [
            49,
            6.328350643282125
        ],
        [
            50,
            6.320743631432316
        ],
        [
            51,
            6.313007591416426
        ],
        [
            52,
            6.3051521921724065
        ],
        [
            53,
            6.297128979434488
        ],
        [
            54,
            6.288903944971734
        ],
        [
            55,
            6.280640691375894
        ],
        [
            56,
            6.272182219395112
        ],
        [
            57,
            6.26356642564746
        ],
        [
            58,
            6.254784221335809
        ],
        [
            59,
            6.2459034800740945
        ],
        [
            60,
            6.236848194489927
        ],
        [
            61,
            6.227635297093664
        ],
        [
            62,
            6.218215541308319
        ],
        [
            63,
            6.208646399567445
        ],
        [
            64,
            6.198907193112215
        ],
        [
            65,
            6.188994028711229
        ],
        [
            66,
            6.178967557198585
        ],
        [
            67,
            6.168698860752657
        ],
        [
            68,
            6.158343112474034
        ],
        [
            69,
            6.147719210404086
        ],
        [
            70,
            6.13690642471396
        ],
        [
            71,
            6.125940150380056
        ],
        [
            72,
            6.114744350517855
        ],
        [
            73,
            6.103431343011821
        ],
        [
            74,
            6.091747532378824
        ],
        [
            75,
            6.0799794749341975
        ],
        [
            76,
            6.067934215080958
        ],
        [
            77,
            6.055742139194619
        ],
        [
            78,
            6.043320920500018
        ],
        [
            79,
            6.030673990504565
        ],
        [
            80,
            6.017884160556287
        ],
        [
            81,
            6.004799268420624
        ],
        [
            82,
            5.991535859362062
        ],
        [
            83,
            5.97804109541306
        ],
        [
            84,
            5.96435428532513
        ],
        [
            85,
            5.950487129498708
        ],
        [
            86,
            5.936294519882437
        ],
        [
            87,
            5.921933689159217
        ],
        [
            88,
            5.907273423924936
        ],
        [
            89,
            5.8923719397753915
        ],
        [
            90,
            5.877153455036491
        ],
        [
            91,
            5.8619093437113765
        ],
        [
            92,
            5.846314393203752
        ],
        [
            93,
            5.830455162800605
        ],
        [
            94,
            5.814395731921547
        ],
        [
            95,
            5.797910207895727
        ],
        [
            96,
            5.781270003380369
        ],
        [
            97,
            5.764396029431992
        ],
        [
            98,
            5.7472278321299335
        ],
        [
            99,
            5.729844479914737
        ],
        [
            100,
            5.712221404773186
        ],
        [
            101,
            5.6942674448035575
        ],
        [
            102,
            5.676139015073723
        ],
        [
            103,
            5.6578747719489355
        ],
        [
            104,
            5.639193014499172
        ],
        [
            105,
            5.620224311600134
        ],
        [
            106,
            5.601022942296355
        ],
        [
            107,
            5.581636021291848
        ],
        [
            108,
            5.561978338290559
        ],
        [
            109,
            5.542126284791814
        ],
        [
            110,
            5.522089828557451
        ],
        [
            111,
            5.501662086829402
        ],
        [
            112,
            5.480913527992189
        ],
        [
            113,
            5.459985887145736
        ],
        [
            114,
            5.438834035670906
        ],
        [
            115,
            5.417384944478007
        ],
        [
            116,
            5.395797933953845
        ],
        [
            117,
            5.373800353237378
        ],
        [
            118,
            5.35144762010473
        ],
        [
            119,
            5.329208241643099
        ],
        [
            120,
            5.306555480547274
        ],
        [
            121,
            5.283619076702595
        ],
        [
            122,
            5.260485020493434
        ],
        [
            123,
            5.237154620002
        ],
        [
            124,
            5.213594540256141
        ],
        [
            125,
            5.189556385051833
        ],
        [
            126,
            5.1655872152103735
        ],
        [
            127,
            5.141387415220551
        ],
        [
            128,
            5.116853663918253
        ],
        [
            129,
            5.092245342379761
        ],
        [
            130,
            5.067161272016926
        ],
        [
            131,
            5.042011927973561
        ],
        [
            132,
            5.01676281858555
        ],
        [
            133,
            4.9913020259000636
        ],
        [
            134,
            4.965449155677184
        ],
        [
            135,
            4.939446180969425
        ],
        [
            136,
            4.9133821414719705
        ],
        [
            137,
            4.8872226205782585
        ],
        [
            138,
            4.86093325841823
        ],
        [
            139,
            4.834263316541223
        ],
        [
            140,
            4.807470392160042
        ],
        [
            141,
            4.780408034712588
        ],
        [
            142,
            4.753605568009643
        ],
        [
            143,
            4.72611769252437
        ],
        [
            144,
            4.698912730978463
        ],
        [
            145,
            4.671303952187384
        ],
        [
            146,
            4.6436375686472955
        ],
        [
            147,
            4.615859017730775
        ],
        [
            148,
            4.587927947157812
        ],
        [
            149,
            4.560010754189347
        ],
        [
            150,
            4.5320062395569884
        ],
        [
            151,
            4.503806213064435
        ],
        [
            152,
            4.475610704699628
        ],
        [
            153,
            4.447261585490875
        ],
        [
            154,
            4.418800553398905
        ],
        [
            155,
            4.3904374503776955
        ],
        [
            156,
            4.362006989632954
        ],
        [
            157,
            4.333298848184197
        ],
        [
            158,
            4.304581075824496
        ],
        [
            159,
            4.275962966741439
        ],
        [
            160,
            4.247232846896259
        ],
        [
            161,
            4.218665464414792
        ],
        [
            162,
            4.190254987237238
        ],
        [
            163,
            4.161673824190781
        ],
        [
            164,
            4.133385505498222
        ],
        [
            165,
            4.104696547090747
        ],
        [
            166,
            4.076294954264164
        ],
        [
            167,
            4.047574858208413
        ],
        [
            168,
            4.019050137982361
        ],
        [
            169,
            3.9903872487467287
        ],
        [
            170,
            3.9620684107360025
        ],
        [
            171,
            3.9336063760032993
        ],
        [
            172,
            3.905408344338761
        ],
        [
            173,
            3.877308696039703
        ],
        [
            174,
            3.8493234421258467
        ],
        [
            175,
            3.8212900687975893
        ],
        [
            176,
            3.7936814620305555
        ],
        [
            177,
            3.7659241578557467
        ],
        [
            178,
            3.7386350907651162
        ],
        [
            179,
            3.711516801137128
        ],
        [
            180,
            3.6845993220799844
        ],
        [
            181,
            3.6579996820841614
        ],
        [
            182,
            3.631528549018464
        ],
        [
            183,
            3.605253152643127
        ],
        [
            184,
            3.57894743192388
        ],
        [
            185,
            3.5526719213068687
        ],
        [
            186,
            3.5270952980244483
        ],
        [
            187,
            3.501573458891736
        ],
        [
            188,
            3.4761891082569085
        ],
        [
            189,
            3.4511458832190716
        ],
        [
            190,
            3.4264215189556078
        ],
        [
            191,
            3.402186080900436
        ],
        [
            192,
            3.378181746860212
        ],
        [
            193,
            3.3545818772803924
        ],
        [
            194,
            3.331248933566062
        ],
        [
            195,
            3.3083001489882724
        ],
        [
            196,
            3.285546364785606
        ],
        [
            197,
            3.2627924761619473
        ],
        [
            198,
            3.240222688936057
        ],
        [
            199,
            3.2184820561363403
        ],
        [
            200,
            3.1964359705046514
        ],
        [
            201,
            3.175608469736299
        ],
        [
            202,
            3.1548689933784866
        ],
        [
            203,
            3.134275876005241
        ],
        [
            204,
            3.1140489659911537
        ],
        [
            205,
            3.0949444570046483
        ],
        [
            206,
            3.0755188811162215
        ],
        [
            207,
            3.057203680420638
        ],
        [
            208,
            3.0402884856208607
        ],
        [
            209,
            3.0219180360892914
        ],
        [
            210,
            3.0047661632117864
        ],
        [
            211,
            2.9886320466202507
        ],
        [
            212,
            2.9719100354074732
        ],
        [
            213,
            2.956485615902998
        ],
        [
            214,
            2.9406610649015286
        ],
        [
            215,
            2.9263985939258794
        ],
        [
            216,
            2.9110596547378025
        ],
        [
            217,
            2.897118392067695
        ],
        [
            218,
            2.883217948271322
        ],
        [
            219,
            2.8694701003528493
        ],
        [
            220,
            2.8561787421060703
        ],
        [
            221,
            2.843198940012589
        ],
        [
            222,
            2.8307084676492504
        ],
        [
            223,
            2.8190065107583235
        ],
        [
            224,
            2.807054356720851
        ],
        [
            225,
            2.7962466454586554
        ],
        [
            226,
            2.785905995217864
        ],
        [
            227,
            2.7750517129719365
        ],
        [
            228,
            2.76464305016461
        ],
        [
            229,
            2.754361077590607
        ],
        [
            230,
            2.743742672440393
        ],
        [
            231,
            2.7338318998707885
        ],
        [
            232,
            2.7242771689092855
        ],
        [
            233,
            2.7142049573329508
        ],
        [
            234,
            2.7049345919000007
        ],
        [
            235,
            2.695905826524817
        ],
        [
            236,
            2.6870942540566833
        ],
        [
            237,
            2.6780210561976325
        ],
        [
            238,
            2.6695469841099606
        ],
        [
            239,
            2.661255223433075
        ],
        [
            240,
            2.6529803528422726
        ],
        [
            241,
            2.6457266354435207
        ],
        [
            242,
            2.638021131284423
        ],
        [
            243,
            2.630592848506342
        ],
        [
            244,
            2.623098188865653
        ],
        [
            245,
            2.6159969050899607
        ],
        [
            246,
            2.608404740443858
        ],
        [
            247,
            2.6018256177795354
        ],
        [
            248,
            2.5948689599723123
        ],
        [
            249,
            2.5884894066296917
        ],
        [
            250,
            2.5821019985388034
        ],
        [
            251,
            2.575947839332668
        ],
        [
            252,
            2.5704303659707275
        ],
        [
            253,
            2.5647013345035594
        ],
        [
            254,
            2.5590261015398363
        ],
        [
            255,
            2.55365442274895
        ],
        [
            256,
            2.5482334079529325
        ],
        [
            257,
            2.5432422192820527
        ],
        [
            258,
            2.5382978934945233
        ],
        [
            259,
            2.5332291966962748
        ],
        [
            260,
            2.5283926079810515
        ]
    ],
    "mae_mental": [
        [
            0,
            6.161077595838945
        ],
        [
            1,
            6.156695230513147
        ],
        [
            2,
            6.152332068982291
        ],
        [
            3,
            6.14791466012495
        ],
        [
            4,
            6.143474092018114
        ],
        [
            5,
            6.138997895643115
        ],
        [
            6,
            6.134474585687563
        ],
        [
            7,
            6.12994330054889
        ],
        [
            8,
            6.125346467052013
        ],
        [
            9,
            6.120690894976565
        ],
        [
            10,
            6.116020876123277
        ],
        [
            11,
            6.111284172187929
        ],
        [
            12,
            6.1064584517264615
        ],
        [
            13,
            6.10159960686395
        ],
        [
            14,
            6.096678442589632
        ],
        [
            15,
            6.0917265239771305
        ],
        [
            16,
            6.086720013847472
        ],
        [
            17,
            6.081653368442434
        ],
        [
            18,
            6.076504796666846
        ],
        [
            19,
            6.07128428453452
        ],
        [
            20,
            6.066005485834018
        ],
        [
            21,
            6.060645188472859
        ],
        [
            22,
            6.055246713609524
        ],
        [
            23,
            6.049755639792101
        ],
        [
            24,
            6.044218329559654
        ],
        [
            25,
            6.038591728481936
        ],
        [
            26,
            6.032864599552465
        ],
        [
            27,
            6.027053686037456
        ],
        [
            28,
            6.021175443717878
        ],
        [
            29,
            6.0152141568187165
        ],
        [
            30,
            6.009194668280344
        ],
        [
            31,
            6.0030704350912405
        ],
        [
            32,
            5.996826516949151
        ],
        [
            33,
            5.99048450553458
        ],
        [
            34,
            5.984031426477922
        ],
        [
            35,
            5.977489178719586
        ],
        [
            36,
            5.970896060642314
        ],
        [
            37,
            5.964173151409789
        ],
        [
            38,
            5.957349414694799
        ],
        [
            39,
            5.950417289688979
        ],
        [
            40,
            5.943402218798252
        ],
        [
            41,
            5.936239433513112
        ],
        [
            42,
            5.928976875155756
        ],
        [
            43,
            5.92160174189365
        ],
        [
            44,
            5.914124861665784
        ],
        [
            45,
            5.906512372195721
        ],
        [
            46,
            5.8988174935930395
        ],
        [
            47,
            5.89093097248306
        ],
        [
            48,
            5.88290658674828
        ],
        [
            49,
            5.874747396127819
        ],
        [
            50,
            5.866511177424699
        ],
        [
            51,
            5.858135527841849
        ],
        [
            52,
            5.849621237140813
        ],
        [
            53,
            5.840940475463867
        ],
        [
            54,
            5.832029058304552
        ],
        [
            55,
            5.823068797384223
        ],
        [
            56,
            5.8139042954330575
        ],
        [
            57,
            5.804565557061809
        ],
        [
            58,
            5.7950467708176125
        ],
        [
            59,
            5.785421280011739
        ],
        [
            60,
            5.77559815877921
        ],
        [
            61,
            5.765600534130449
        ],
        [
            62,
            5.7553829099625755
        ],
        [
            63,
            5.745014479919655
        ],
        [
            64,
            5.734431898348952
        ],
        [
            65,
            5.723678082227707
        ],
        [
            66,
            5.7127763071697055
        ],
        [
            67,
            5.7016292144990945
        ],
        [
            68,
            5.690372510315621
        ],
        [
            69,
            5.678841215902811
        ],
        [
            70,
            5.667088008498492
        ],
        [
            71,
            5.655161184807346
        ],
        [
            72,
            5.642982431266406
        ],
        [
            73,
            5.630686373947418
        ],
        [
            74,
            5.617977072523065
        ],
        [
            75,
            5.605167426066856
        ],
        [
            76,
            5.592055198060323
        ],
        [
            77,
            5.5787787849772466
        ],
        [
            78,
            5.5652356586635925
        ],
        [
            79,
            5.551467673215147
        ],
        [
            80,
            5.537536025863805
        ],
        [
            81,
            5.52327600897175
        ],
        [
            82,
            5.508794588993673
        ],
        [
            83,
            5.494057070310801
        ],
        [
            84,
            5.479122803635793
        ],
        [
            85,
            5.463980316299281
        ],
        [
            86,
            5.448488226900362
        ],
        [
            87,
            5.4327774378534865
        ],
        [
            88,
            5.416756465418698
        ],
        [
            89,
            5.400439895995675
        ],
        [
            90,
            5.383808185384698
        ],
        [
            91,
            5.367135227134783
        ],
        [
            92,
            5.350048190110351
        ],
        [
            93,
            5.332664693055087
        ],
        [
            94,
            5.3150480291614794
        ],
        [
            95,
            5.296984024652063
        ],
        [
            96,
            5.278729273031836
        ],
        [
            97,
            5.26018081625847
        ],
        [
            98,
            5.241337624722964
        ],
        [
            99,
            5.222211272340932
        ],
        [
            100,
            5.2028250665697335
        ],
        [
            101,
            5.183072744983516
        ],
        [
            102,
            5.1631001234054565
        ],
        [
            103,
            5.142987224745424
        ],
        [
            104,
            5.122399549369943
        ],
        [
            105,
            5.101466256461731
        ],
        [
            106,
            5.080235252641652
        ],
        [
            107,
            5.058837455021192
        ],
        [
            108,
            5.037096401191738
        ],
        [
            109,
            5.015129472294899
        ],
        [
            110,
            4.992943180342243
        ],
        [
            111,
            4.970270488768408
        ],
        [
            112,
            4.94728647356164
        ],
        [
            113,
            4.9240397280209685
        ],
        [
            114,
            4.900536895614781
        ],
        [
            115,
            4.876677856053392
        ],
        [
            116,
            4.852628421293546
        ],
        [
            117,
            4.828152957027906
        ],
        [
            118,
            4.803236840522453
        ],
        [
            119,
            4.7784221294808065
        ],
        [
            120,
            4.753092754377078
        ],
        [
            121,
            4.7274250094204735
        ],
        [
            122,
            4.701541169048989
        ],
        [
            123,
            4.6754009903293765
        ],
        [
            124,
            4.648881619107233
        ],
        [
            125,
            4.62192660325194
        ],
        [
            126,
            4.594966761053425
        ],
        [
            127,
            4.567699042085099
        ],
        [
            128,
            4.540057269677724
        ],
        [
            129,
            4.512159892141002
        ],
        [
            130,
            4.483842910969094
        ],
        [
            131,
            4.455379766144165
        ],
        [
            132,
            4.426705184864671
        ],
        [
            133,
            4.397805919385936
        ],
        [
            134,
            4.368388044507536
        ],
        [
            135,
            4.338725767723502
        ],
        [
            136,
            4.30906604904018
        ],
        [
            137,
            4.27917201714973
        ],
        [
            138,
            4.249099413826041
        ],
        [
            139,
            4.21854186711246
        ],
        [
            140,
            4.187735580418208
        ],
        [
            141,
            4.156631117814208
        ],
        [
            142,
            4.125729256296811
        ],
        [
            143,
            4.094039448320049
        ],
        [
            144,
            4.06248888005949
        ],
        [
            145,
            4.030613828195285
        ],
        [
            146,
            3.9983595389209383
        ],
        [
            147,
            3.9660345103642713
        ],
        [
            148,
            3.933465309339027
        ],
        [
            149,
            3.900898975052246
        ],
        [
            150,
            3.868002551875702
        ],
        [
            151,
            3.8348500908237613
        ],
        [
            152,
            3.801744284695142
        ],
        [
            153,
            3.768236612620419
        ],
        [
            154,
            3.7346760116211355
        ],
        [
            155,
            3.7009527683258057
        ],
        [
            156,
            3.667178607966802
        ],
        [
            157,
            3.6329276235136265
        ],
        [
            158,
            3.598570088817649
        ],
        [
            159,
            3.5643321471671534
        ],
        [
            160,
            3.529772652338629
        ],
        [
            161,
            3.495319421977213
        ],
        [
            162,
            3.4609884970808684
        ],
        [
            163,
            3.426290743971524
        ],
        [
            164,
            3.3917994760487176
        ],
        [
            165,
            3.356739473669496
        ],
        [
            166,
            3.3219197384298664
        ],
        [
            167,
            3.286655925724604
        ],
        [
            168,
            3.251457531158238
        ],
        [
            169,
            3.216038360987624
        ],
        [
            170,
            3.1810487786384476
        ],
        [
            171,
            3.1473352402856904
        ],
        [
            172,
            3.1147097267516672
        ],
        [
            173,
            3.082476398716234
        ],
        [
            174,
            3.051371025712523
        ],
        [
            175,
            3.0216262275225496
        ],
        [
            176,
            2.992272241474831
        ],
        [
            177,
            2.962713491426755
        ],
        [
            178,
            2.933382965114019
        ],
        [
            179,
            2.903954466728315
        ],
        [
            180,
            2.874826770939239
        ],
        [
            181,
            2.845718747948947
        ],
        [
            182,
            2.816686800081436
        ],
        [
            183,
            2.7878012689825606
        ],
        [
            184,
            2.758861923870975
        ],
        [
            185,
            2.7301898198584986
        ],
        [
            186,
            2.7018254276824325
        ],
        [
            187,
            2.673705894653111
        ],
        [
            188,
            2.6454209056619096
        ],
        [
            189,
            2.6174982116646963
        ],
        [
            190,
            2.590092043354087
        ],
        [
            191,
            2.563423223691444
        ],
        [
            192,
            2.536905969658943
        ],
        [
            193,
            2.5107046382067955
        ],
        [
            194,
            2.4861298685204494
        ],
        [
            195,
            2.462132050566477
        ],
        [
            196,
            2.438273650326141
        ],
        [
            197,
            2.414494759415927
        ],
        [
            198,
            2.3912726183460182
        ],
        [
            199,
            2.369850465696152
        ],
        [
            200,
            2.350374311616976
        ],
        [
            201,
            2.334462817401102
        ],
        [
            202,
            2.3206298596238435
        ],
        [
            203,
            2.3074544129306322
        ],
        [
            204,
            2.295367634459718
        ],
        [
            205,
            2.2843044783971083
        ],
        [
            206,
            2.2734191874935203
        ],
        [
            207,
            2.263453671376999
        ],
        [
            208,
            2.2546190301032913
        ],
        [
            209,
            2.2456635514350785
        ],
        [
            210,
            2.239015278750903
        ],
        [
            211,
            2.232841645201592
        ],
        [
            212,
            2.2263610591627145
        ],
        [
            213,
            2.2204890055199193
        ],
        [
            214,
            2.214184689195189
        ],
        [
            215,
            2.2084893494436186
        ],
        [
            216,
            2.202346302058599
        ],
        [
            217,
            2.1968312916690356
        ],
        [
            218,
            2.1911595945488918
        ],
        [
            219,
            2.185500892874313
        ],
        [
            220,
            2.1799783478044485
        ],
        [
            221,
            2.1745946341997957
        ],
        [
            222,
            2.1692580066315115
        ],
        [
            223,
            2.1643475930984706
        ],
        [
            224,
            2.159096975849099
        ],
        [
            225,
            2.1544613184994215
        ],
        [
            226,
            2.149913444910964
        ],
        [
            227,
            2.1451406054300803
        ],
        [
            228,
            2.1409531194869786
        ],
        [
            229,
            2.136698546474927
        ],
        [
            230,
            2.1326171012773907
        ],
        [
            231,
            2.1288120093410963
        ],
        [
            232,
            2.125107324286683
        ],
        [
            233,
            2.1214570378604
        ],
        [
            234,
            2.1182202084423745
        ],
        [
            235,
            2.1150119239336824
        ],
        [
            236,
            2.1122497728426164
        ],
        [
            237,
            2.109328808849805
        ],
        [
            238,
            2.106506249676012
        ],
        [
            239,
            2.103822358667034
        ],
        [
            240,
            2.100964794420216
        ],
        [
            241,
            2.0985667378935093
        ],
        [
            242,
            2.0959209709951323
        ],
        [
            243,
            2.0933583599247343
        ],
        [
            244,
            2.0907053882128572
        ],
        [
            245,
            2.0881836185716605
        ],
        [
            246,
            2.0854503716508
        ],
        [
            247,
            2.083010866217417
        ],
        [
            248,
            2.0804286884934933
        ],
        [
            249,
            2.077971334326757
        ],
        [
            250,
            2.075623280381503
        ],
        [
            251,
            2.073188892782551
        ],
        [
            252,
            2.0711088474482704
        ],
        [
            253,
            2.0689652913237273
        ],
        [
            254,
            2.066943789181644
        ],
        [
            255,
            2.0650021637955756
        ],
        [
            256,
            2.0630262126661325
        ],
        [
            257,
            2.0613658624152613
        ],
        [
            258,
            2.0597524577624178
        ],
        [
            259,
            2.0581265083731037
        ],
        [
            260,
            2.056570693238141
        ]
    ],
    "r2_mental": [
        [
            0,
            -6.858443487482915
        ],
        [
            1,
            -6.8487530390302425
        ],
        [
            2,
            -6.839108081659417
        ],
        [
            3,
            -6.829347612568324
        ],
        [
            4,
            -6.819554793970034
        ],
        [
            5,
            -6.809676982952845
        ],
        [
            6,
            -6.79971025188939
        ],
        [
            7,
            -6.789730760693888
        ],
        [
            8,
            -6.779616037265419
        ],
        [
            9,
            -6.7693751366014805
        ],
        [
            10,
            -6.759114314243387
        ],
        [
            11,
            -6.748715880350342
        ],
        [
            12,
            -6.7381261091020574
        ],
        [
            13,
            -6.727469990428681
        ],
        [
            14,
            -6.716689354459998
        ],
        [
            15,
            -6.7058547158727855
        ],
        [
            16,
            -6.694896836924237
        ],
        [
            17,
            -6.683823191269888
        ],
        [
            18,
            -6.672579400293208
        ],
        [
            19,
            -6.661186157362905
        ],
        [
            20,
            -6.6496779478478665
        ],
        [
            21,
            -6.638006704127076
        ],
        [
            22,
            -6.626253296675
        ],
        [
            23,
            -6.614319938785293
        ],
        [
            24,
            -6.60228756648855
        ],
        [
            25,
            -6.590072899281687
        ],
        [
            26,
            -6.577654960664786
        ],
        [
            27,
            -6.565065005674692
        ],
        [
            28,
            -6.552338972944469
        ],
        [
            29,
            -6.539450144811885
        ],
        [
            30,
            -6.526446966426875
        ],
        [
            31,
            -6.513226767430427
        ],
        [
            32,
            -6.4997639471863335
        ],
        [
            33,
            -6.486100905666559
        ],
        [
            34,
            -6.472213829106896
        ],
        [
            35,
            -6.458153304246211
        ],
        [
            36,
            -6.44400946153554
        ],
        [
            37,
            -6.429579640104411
        ],
        [
            38,
            -6.4149635507508025
        ],
        [
            39,
            -6.400127514685412
        ],
        [
            40,
            -6.385124422295886
        ],
        [
            41,
            -6.369822588755103
        ],
        [
            42,
            -6.354319878757609
        ],
        [
            43,
            -6.338616337353136
        ],
        [
            44,
            -6.32270684156963
        ],
        [
            45,
            -6.306540859205039
        ],
        [
            46,
            -6.290201585341445
        ],
        [
            47,
            -6.2734673557722385
        ],
        [
            48,
            -6.256481647210417
        ],
        [
            49,
            -6.239233214443964
        ],
        [
            50,
            -6.22183979297775
        ],
        [
            51,
            -6.204172804462529
        ],
        [
            52,
            -6.1862553716920585
        ],
        [
            53,
            -6.167978200853772
        ],
        [
            54,
            -6.149265430484399
        ],
        [
            55,
            -6.130490336301504
        ],
        [
            56,
            -6.111297250026761
        ],
        [
            57,
            -6.091773777402141
        ],
        [
            58,
            -6.071900836221867
        ],
        [
            59,
            -6.051833273864786
        ],
        [
            60,
            -6.031400657497535
        ],
        [
            61,
            -6.010642824211864
        ],
        [
            62,
            -5.989450639351705
        ],
        [
            63,
            -5.967955218154407
        ],
        [
            64,
            -5.946111768274126
        ],
        [
            65,
            -5.9239133750222805
        ],
        [
            66,
            -5.901497393995545
        ],
        [
            67,
            -5.878577547617634
        ],
        [
            68,
            -5.855502008383755
        ],
        [
            69,
            -5.831869236257837
        ],
        [
            70,
            -5.8078581935625255
        ],
        [
            71,
            -5.78354948431275
        ],
        [
            72,
            -5.758776842166553
        ],
        [
            73,
            -5.733790887966398
        ],
        [
            74,
            -5.708034545992221
        ],
        [
            75,
            -5.682142374109438
        ],
        [
            76,
            -5.655692149180559
        ],
        [
            77,
            -5.628972946780008
        ],
        [
            78,
            -5.601806837276065
        ],
        [
            79,
            -5.574204389204266
        ],
        [
            80,
            -5.546348864295396
        ],
        [
            81,
            -5.5179119114955215
        ],
        [
            82,
            -5.489150165137284
        ],
        [
            83,
            -5.459951997885925
        ],
        [
            84,
            -5.430405556246536
        ],
        [
            85,
            -5.4005388616873455
        ],
        [
            86,
            -5.370043201422882
        ],
        [
            87,
            -5.33926020687151
        ],
        [
            88,
            -5.307912270120458
        ],
        [
            89,
            -5.276128165726813
        ],
        [
            90,
            -5.243750773840581
        ],
        [
            91,
            -5.211402803515463
        ],
        [
            92,
            -5.178397288625469
        ],
        [
            93,
            -5.144922617695303
        ],
        [
            94,
            -5.111118034855203
        ],
        [
            95,
            -5.076513523232648
        ],
        [
            96,
            -5.041683964588481
        ],
        [
            97,
            -5.006467328972641
        ],
        [
            98,
            -4.97074228394368
        ],
        [
            99,
            -4.9346780969532045
        ],
        [
            100,
            -4.898228088315788
        ],
        [
            101,
            -4.861209169435083
        ],
        [
            102,
            -4.8239487554647695
        ],
        [
            103,
            -4.786529347268474
        ],
        [
            104,
            -4.748379310369864
        ],
        [
            105,
            -4.709772399834963
        ],
        [
            106,
            -4.670824429467218
        ],
        [
            107,
            -4.6316353177523375
        ],
        [
            108,
            -4.592037623304889
        ],
        [
            109,
            -4.552190178256276
        ],
        [
            110,
            -4.512117067138722
        ],
        [
            111,
            -4.471410799111272
        ],
        [
            112,
            -4.430219675655181
        ],
        [
            113,
            -4.3888306886389765
        ],
        [
            114,
            -4.347159166557789
        ],
        [
            115,
            -4.305067222069338
        ],
        [
            116,
            -4.262872556431508
        ],
        [
            117,
            -4.22004868539324
        ],
        [
            118,
            -4.176712620424498
        ],
        [
            119,
            -4.1337755839494
        ],
        [
            120,
            -4.090224263813607
        ],
        [
            121,
            -4.046316639080859
        ],
        [
            122,
            -4.002223301865379
        ],
        [
            123,
            -3.957951688114699
        ],
        [
            124,
            -3.9134439341703393
        ],
        [
            125,
            -3.8682398648263696
        ],
        [
            126,
            -3.8233735275208334
        ],
        [
            127,
            -3.77828620123298
        ],
        [
            128,
            -3.732792805144812
        ],
        [
            129,
            -3.687379730458087
        ],
        [
            130,
            -3.641314011742045
        ],
        [
            131,
            -3.595356786665098
        ],
        [
            132,
            -3.549447277977843
        ],
        [
            133,
            -3.503386260236633
        ],
        [
            134,
            -3.4568557388632977
        ],
        [
            135,
            -3.4102987994399205
        ],
        [
            136,
            -3.3638778371314775
        ],
        [
            137,
            -3.317533768408304
        ],
        [
            138,
            -3.271208919352297
        ],
        [
            139,
            -3.224468760217257
        ],
        [
            140,
            -3.1777720001205747
        ],
        [
            141,
            -3.1308691058285225
        ],
        [
            142,
            -3.084677609522031
        ],
        [
            143,
            -3.0375746350896042
        ],
        [
            144,
            -2.99122542088891
        ],
        [
            145,
            -2.944461778638151
        ],
        [
            146,
            -2.8978770012533803
        ],
        [
            147,
            -2.8513817647512734
        ],
        [
            148,
            -2.8049125174096248
        ],
        [
            149,
            -2.7587481920995733
        ],
        [
            150,
            -2.7127225572933416
        ],
        [
            151,
            -2.6666621023860633
        ],
        [
            152,
            -2.620896459835638
        ],
        [
            153,
            -2.575171245939755
        ],
        [
            154,
            -2.529557796715387
        ],
        [
            155,
            -2.484392633441519
        ],
        [
            156,
            -2.439412083703669
        ],
        [
            157,
            -2.394288722985214
        ],
        [
            158,
            -2.3494483300519295
        ],
        [
            159,
            -2.3050601582795234
        ],
        [
            160,
            -2.2607960823506006
        ],
        [
            161,
            -2.2170786148827326
        ],
        [
            162,
            -2.1738938787187556
        ],
        [
            163,
            -2.1307441390561914
        ],
        [
            164,
            -2.088327316624736
        ],
        [
            165,
            -2.045605236265177
        ],
        [
            166,
            -2.0036041873021944
        ],
        [
            167,
            -1.9614286774295802
        ],
        [
            168,
            -1.9198352451489882
        ],
        [
            169,
            -1.878336642384462
        ],
        [
            170,
            -1.8376278533151678
        ],
        [
            171,
            -1.7970053488866315
        ],
        [
            172,
            -1.7570484502146817
        ],
        [
            173,
            -1.7175169201903215
        ],
        [
            174,
            -1.6784300419205764
        ],
        [
            175,
            -1.6395598253657488
        ],
        [
            176,
            -1.6015562660398457
        ],
        [
            177,
            -1.563625822406002
        ],
        [
            178,
            -1.5266067631401579
        ],
        [
            179,
            -1.4900860776737863
        ],
        [
            180,
            -1.4540987495682134
        ],
        [
            181,
            -1.4187936802648422
        ],
        [
            182,
            -1.3839131200220791
        ],
        [
            183,
            -1.3495410057484305
        ],
        [
            184,
            -1.3153792523862466
        ],
        [
            185,
            -1.2815064742293036
        ],
        [
            186,
            -1.2487743882196227
        ],
        [
            187,
            -1.216348152011696
        ],
        [
            188,
            -1.1843301856446926
        ],
        [
            189,
            -1.152970771238273
        ],
        [
            190,
            -1.122233062774737
        ],
        [
            191,
            -1.0923176823845235
        ],
        [
            192,
            -1.0628968866574193
        ],
        [
            193,
            -1.0341749004864402
        ],
        [
            194,
            -1.0059757247443506
        ],
        [
            195,
            -0.9784328178899684
        ],
        [
            196,
            -0.9513119260249026
        ],
        [
            197,
            -0.9243780863833382
        ],
        [
            198,
            -0.8978470796362463
        ],
        [
            199,
            -0.872464888461461
        ],
        [
            200,
            -0.8469005770360865
        ],
        [
            201,
            -0.8229107313927793
        ],
        [
            202,
            -0.799178107740532
        ],
        [
            203,
            -0.7757668259712511
        ],
        [
            204,
            -0.7529211169696519
        ],
        [
            205,
            -0.7314789546959459
        ],
        [
            206,
            -0.709811735472204
        ],
        [
            207,
            -0.6895079741979666
        ],
        [
            208,
            -0.6708639459166272
        ],
        [
            209,
            -0.6507331006989863
        ],
        [
            210,
            -0.6320477401421909
        ],
        [
            211,
            -0.614568207230495
        ],
        [
            212,
            -0.5965510707966892
        ],
        [
            213,
            -0.5800216549080783
        ],
        [
            214,
            -0.5631528302104247
        ],
        [
            215,
            -0.5480267368568108
        ],
        [
            216,
            -0.5318410694692137
        ],
        [
            217,
            -0.5172040203813253
        ],
        [
            218,
            -0.5026797851520259
        ],
        [
            219,
            -0.4883837027809037
        ],
        [
            220,
            -0.47462727820808737
        ],
        [
            221,
            -0.46125495063807853
        ],
        [
            222,
            -0.4484442581757997
        ],
        [
            223,
            -0.4364934704870893
        ],
        [
            224,
            -0.42433827055930484
        ],
        [
            225,
            -0.4133914201131872
        ],
        [
            226,
            -0.402957174505437
        ],
        [
            227,
            -0.39204623366323554
        ],
        [
            228,
            -0.38162324479722054
        ],
        [
            229,
            -0.3713655763284056
        ],
        [
            230,
            -0.36081238720348097
        ],
        [
            231,
            -0.35099926273560467
        ],
        [
            232,
            -0.3415722917333335
        ],
        [
            233,
            -0.3316704941525839
        ],
        [
            234,
            -0.3225893893594445
        ],
        [
            235,
            -0.3137748177361115
        ],
        [
            236,
            -0.3052007027779957
        ],
        [
            237,
            -0.2964013457232477
        ],
        [
            238,
            -0.28820991105833627
        ],
        [
            239,
            -0.28021983660857885
        ],
        [
            240,
            -0.272270818038717
        ],
        [
            241,
            -0.265323102410415
        ],
        [
            242,
            -0.2579634952224197
        ],
        [
            243,
            -0.25088898617494393
        ],
        [
            244,
            -0.24377147853294634
        ],
        [
            245,
            -0.23704628715247056
        ],
        [
            246,
            -0.22987637765535784
        ],
        [
            247,
            -0.22368002100233864
        ],
        [
            248,
            -0.21714511518596713
        ],
        [
            249,
            -0.21116770571083543
        ],
        [
            250,
            -0.20519767751235407
        ],
        [
            251,
            -0.19945960860111223
        ],
        [
            252,
            -0.1943268192328247
        ],
        [
            253,
            -0.18900886870913025
        ],
        [
            254,
            -0.1837525558007409
        ],
        [
            255,
            -0.17878811667589622
        ],
        [
            256,
            -0.17378865783716768
        ],
        [
            257,
            -0.16919499459290277
        ],
        [
            258,
            -0.1646533417345719
        ],
        [
            259,
            -0.16000662111912356
        ],
        [
            260,
            -0.1555813452555317
        ]
    ]
}

fl_base_metrics = {
    "loss": [
        [
            0,
            56.10302505493164
        ],
        [
            1,
            56.05101699829102
        ],
        [
            2,
            55.999718475341794
        ],
        [
            3,
            55.948403930664064
        ],
        [
            4,
            55.896636199951175
        ],
        [
            5,
            55.845220947265624
        ],
        [
            6,
            55.79327392578125
        ],
        [
            7,
            55.741494750976564
        ],
        [
            8,
            55.68951644897461
        ],
        [
            9,
            55.63712844848633
        ],
        [
            10,
            55.585209655761716
        ],
        [
            11,
            55.53326416015625
        ],
        [
            12,
            55.48096466064453
        ],
        [
            13,
            55.428304290771486
        ],
        [
            14,
            55.37637557983398
        ],
        [
            15,
            55.323577880859375
        ],
        [
            16,
            55.271061706542966
        ],
        [
            17,
            55.21785125732422
        ],
        [
            18,
            55.16500473022461
        ],
        [
            19,
            55.11168670654297
        ],
        [
            20,
            55.057709503173825
        ],
        [
            21,
            55.004364013671875
        ],
        [
            22,
            54.95085220336914
        ],
        [
            23,
            54.89635009765625
        ],
        [
            24,
            54.841268157958986
        ],
        [
            25,
            54.78662872314453
        ],
        [
            26,
            54.73131637573242
        ],
        [
            27,
            54.67621994018555
        ],
        [
            28,
            54.62049102783203
        ],
        [
            29,
            54.56465835571289
        ],
        [
            30,
            54.50848159790039
        ],
        [
            31,
            54.45148696899414
        ],
        [
            32,
            54.39506378173828
        ],
        [
            33,
            54.3372688293457
        ],
        [
            34,
            54.280765533447266
        ],
        [
            35,
            54.22302474975586
        ],
        [
            36,
            54.16498794555664
        ],
        [
            37,
            54.106729888916014
        ],
        [
            38,
            54.048214721679685
        ],
        [
            39,
            53.989226531982425
        ],
        [
            40,
            53.92936401367187
        ],
        [
            41,
            53.869904327392575
        ],
        [
            42,
            53.80939865112305
        ],
        [
            43,
            53.74945755004883
        ],
        [
            44,
            53.68724899291992
        ],
        [
            45,
            53.626374053955075
        ],
        [
            46,
            53.564631652832034
        ],
        [
            47,
            53.502323913574216
        ],
        [
            48,
            53.4399917602539
        ],
        [
            49,
            53.37613372802734
        ],
        [
            50,
            53.31304397583008
        ],
        [
            51,
            53.2483627319336
        ],
        [
            52,
            53.18360214233398
        ],
        [
            53,
            53.11796646118164
        ],
        [
            54,
            53.05348892211914
        ],
        [
            55,
            52.988060760498044
        ],
        [
            56,
            52.92115707397461
        ],
        [
            57,
            52.85438690185547
        ],
        [
            58,
            52.786827850341794
        ],
        [
            59,
            52.71958465576172
        ],
        [
            60,
            52.651502990722655
        ],
        [
            61,
            52.58191452026367
        ],
        [
            62,
            52.51225357055664
        ],
        [
            63,
            52.442160034179686
        ],
        [
            64,
            52.37234725952148
        ],
        [
            65,
            52.30156631469727
        ],
        [
            66,
            52.230648040771484
        ],
        [
            67,
            52.158270263671874
        ],
        [
            68,
            52.08560638427734
        ],
        [
            69,
            52.01262588500977
        ],
        [
            70,
            51.93813781738281
        ],
        [
            71,
            51.86355209350586
        ],
        [
            72,
            51.78927383422852
        ],
        [
            73,
            51.713069915771484
        ],
        [
            74,
            51.63770523071289
        ],
        [
            75,
            51.55991821289062
        ],
        [
            76,
            51.48289260864258
        ],
        [
            77,
            51.4059944152832
        ],
        [
            78,
            51.3273422241211
        ],
        [
            79,
            51.24875335693359
        ],
        [
            80,
            51.16895370483398
        ],
        [
            81,
            51.08905181884766
        ],
        [
            82,
            51.00703048706055
        ],
        [
            83,
            50.92578811645508
        ],
        [
            84,
            50.84265289306641
        ],
        [
            85,
            50.76054458618164
        ],
        [
            86,
            50.677458953857425
        ],
        [
            87,
            50.59182586669922
        ],
        [
            88,
            50.50722732543945
        ],
        [
            89,
            50.42249984741211
        ],
        [
            90,
            50.33552856445313
        ],
        [
            91,
            50.24910049438476
        ],
        [
            92,
            50.16212921142578
        ],
        [
            93,
            50.073360443115234
        ],
        [
            94,
            49.98456344604492
        ],
        [
            95,
            49.89466018676758
        ],
        [
            96,
            49.805198669433594
        ],
        [
            97,
            49.71377410888672
        ],
        [
            98,
            49.62294311523438
        ],
        [
            99,
            49.53105697631836
        ],
        [
            100,
            49.43700332641602
        ],
        [
            101,
            49.34337158203125
        ],
        [
            102,
            49.25088806152344
        ],
        [
            103,
            49.154875946044925
        ],
        [
            104,
            49.05888900756836
        ],
        [
            105,
            48.96302108764648
        ],
        [
            106,
            48.86493072509766
        ],
        [
            107,
            48.76804046630859
        ],
        [
            108,
            48.66906051635742
        ],
        [
            109,
            48.56979522705078
        ],
        [
            110,
            48.46971130371094
        ],
        [
            111,
            48.368883514404295
        ],
        [
            112,
            48.26728210449219
        ],
        [
            113,
            48.16560440063476
        ],
        [
            114,
            48.06189804077148
        ],
        [
            115,
            47.95810470581055
        ],
        [
            116,
            47.85471649169922
        ],
        [
            117,
            47.74924240112305
        ],
        [
            118,
            47.642398834228516
        ],
        [
            119,
            47.53635368347168
        ],
        [
            120,
            47.429248046875
        ],
        [
            121,
            47.32042922973633
        ],
        [
            122,
            47.21170501708984
        ],
        [
            123,
            47.10181350708008
        ],
        [
            124,
            46.9918025970459
        ],
        [
            125,
            46.879817962646484
        ],
        [
            126,
            46.76863746643066
        ],
        [
            127,
            46.65558967590332
        ],
        [
            128,
            46.54195899963379
        ],
        [
            129,
            46.428793334960936
        ],
        [
            130,
            46.31408462524414
        ],
        [
            131,
            46.198492431640624
        ],
        [
            132,
            46.082837677001955
        ],
        [
            133,
            45.96605224609375
        ],
        [
            134,
            45.84884796142578
        ],
        [
            135,
            45.72913665771485
        ],
        [
            136,
            45.610131454467776
        ],
        [
            137,
            45.49067306518555
        ],
        [
            138,
            45.37028923034668
        ],
        [
            139,
            45.24912567138672
        ],
        [
            140,
            45.12538528442383
        ],
        [
            141,
            45.0045768737793
        ],
        [
            142,
            44.880496978759766
        ],
        [
            143,
            44.75527801513672
        ],
        [
            144,
            44.630516815185544
        ],
        [
            145,
            44.504400634765624
        ],
        [
            146,
            44.37762107849121
        ],
        [
            147,
            44.25053825378418
        ],
        [
            148,
            44.121768188476565
        ],
        [
            149,
            43.99166488647461
        ],
        [
            150,
            43.863424682617186
        ],
        [
            151,
            43.73326950073242
        ],
        [
            152,
            43.60284538269043
        ],
        [
            153,
            43.46963806152344
        ],
        [
            154,
            43.33801040649414
        ],
        [
            155,
            43.20477523803711
        ],
        [
            156,
            43.07088088989258
        ],
        [
            157,
            42.935237884521484
        ],
        [
            158,
            42.80015182495117
        ],
        [
            159,
            42.66360740661621
        ],
        [
            160,
            42.52734909057617
        ],
        [
            161,
            42.3897762298584
        ],
        [
            162,
            42.251562881469724
        ],
        [
            163,
            42.11186294555664
        ],
        [
            164,
            41.97301177978515
        ],
        [
            165,
            41.833364486694336
        ],
        [
            166,
            41.69184951782226
        ],
        [
            167,
            41.550398635864255
        ],
        [
            168,
            41.40963287353516
        ],
        [
            169,
            41.26631202697754
        ],
        [
            170,
            41.123652648925784
        ],
        [
            171,
            40.97794418334961
        ],
        [
            172,
            40.83335304260254
        ],
        [
            173,
            40.68864212036133
        ],
        [
            174,
            40.540564346313474
        ],
        [
            175,
            40.39536094665527
        ],
        [
            176,
            40.24934196472168
        ],
        [
            177,
            40.09895210266113
        ],
        [
            178,
            39.951525115966795
        ],
        [
            179,
            39.802768325805665
        ],
        [
            180,
            39.651872634887695
        ],
        [
            181,
            39.502069091796876
        ],
        [
            182,
            39.35306930541992
        ],
        [
            183,
            39.20156555175781
        ],
        [
            184,
            39.048866271972656
        ],
        [
            185,
            38.89675598144531
        ],
        [
            186,
            38.74413223266602
        ],
        [
            187,
            38.59024543762207
        ],
        [
            188,
            38.434591674804686
        ],
        [
            189,
            38.2808334350586
        ],
        [
            190,
            38.124911880493165
        ],
        [
            191,
            37.970018768310545
        ],
        [
            192,
            37.81213645935058
        ],
        [
            193,
            37.655059814453125
        ],
        [
            194,
            37.49828109741211
        ],
        [
            195,
            37.34124069213867
        ],
        [
            196,
            37.182329940795896
        ],
        [
            197,
            37.02304573059082
        ],
        [
            198,
            36.863860321044925
        ],
        [
            199,
            36.70393676757813
        ],
        [
            200,
            36.54547462463379
        ],
        [
            201,
            36.38417854309082
        ],
        [
            202,
            36.22366104125977
        ],
        [
            203,
            36.06177024841308
        ],
        [
            204,
            35.901160049438474
        ],
        [
            205,
            35.73750228881836
        ],
        [
            206,
            35.575631332397464
        ],
        [
            207,
            35.411146545410155
        ],
        [
            208,
            35.25042915344238
        ],
        [
            209,
            35.085321044921876
        ],
        [
            210,
            34.91856689453125
        ],
        [
            211,
            34.75543212890625
        ],
        [
            212,
            34.59048957824707
        ],
        [
            213,
            34.42487907409668
        ],
        [
            214,
            34.260776138305665
        ],
        [
            215,
            34.09619407653808
        ],
        [
            216,
            33.92878952026367
        ],
        [
            217,
            33.76281089782715
        ],
        [
            218,
            33.59521751403808
        ],
        [
            219,
            33.43022842407227
        ],
        [
            220,
            33.26115455627441
        ],
        [
            221,
            33.09456024169922
        ],
        [
            222,
            32.92672462463379
        ],
        [
            223,
            32.76041030883789
        ],
        [
            224,
            32.59118843078613
        ],
        [
            225,
            32.42334671020508
        ],
        [
            226,
            32.25740280151367
        ],
        [
            227,
            32.0869800567627
        ],
        [
            228,
            31.919467544555665
        ],
        [
            229,
            31.748671340942384
        ],
        [
            230,
            31.58157386779785
        ],
        [
            231,
            31.414400100708008
        ],
        [
            232,
            31.243903732299806
        ],
        [
            233,
            31.07468185424805
        ],
        [
            234,
            30.90459442138672
        ],
        [
            235,
            30.7355224609375
        ],
        [
            236,
            30.566745376586915
        ],
        [
            237,
            30.396533966064453
        ],
        [
            238,
            30.227786254882812
        ],
        [
            239,
            30.059377670288086
        ],
        [
            240,
            29.88832817077637
        ],
        [
            241,
            29.720541381835936
        ],
        [
            242,
            29.55100860595703
        ],
        [
            243,
            29.381827926635744
        ],
        [
            244,
            29.214018630981446
        ],
        [
            245,
            29.04361457824707
        ],
        [
            246,
            28.87275505065918
        ],
        [
            247,
            28.70403633117676
        ],
        [
            248,
            28.536485862731933
        ],
        [
            249,
            28.36398105621338
        ],
        [
            250,
            28.19676570892334
        ],
        [
            251,
            28.02874641418457
        ],
        [
            252,
            27.860643196105958
        ],
        [
            253,
            27.692024612426756
        ],
        [
            254,
            27.523739433288576
        ],
        [
            255,
            27.3554780960083
        ],
        [
            256,
            27.188383865356446
        ],
        [
            257,
            27.019388008117676
        ],
        [
            258,
            26.853246879577636
        ],
        [
            259,
            26.687685775756837
        ],
        [
            260,
            26.519503784179687
        ]
    ],
    "rmse_physical": [
        [
            0,
            3.5954339898462195
        ],
        [
            1,
            3.5950871265291213
        ],
        [
            2,
            3.5947143662673566
        ],
        [
            3,
            3.5943610961106054
        ],
        [
            4,
            3.5939817890446744
        ],
        [
            5,
            3.5936157434503286
        ],
        [
            6,
            3.593267135892306
        ],
        [
            7,
            3.5928922312879887
        ],
        [
            8,
            3.59248899211281
        ],
        [
            9,
            3.592115559322221
        ],
        [
            10,
            3.591734477318111
        ],
        [
            11,
            3.5913434558284494
        ],
        [
            12,
            3.590921957759894
        ],
        [
            13,
            3.5905283656667213
        ],
        [
            14,
            3.590131750101771
        ],
        [
            15,
            3.5897031561252177
        ],
        [
            16,
            3.589318134031137
        ],
        [
            17,
            3.5888974226809633
        ],
        [
            18,
            3.5884713179328043
        ],
        [
            19,
            3.588088947554853
        ],
        [
            20,
            3.587670297269488
        ],
        [
            21,
            3.5872401959109355
        ],
        [
            22,
            3.5867964554790435
        ],
        [
            23,
            3.586350375153938
        ],
        [
            24,
            3.5859041053040075
        ],
        [
            25,
            3.5854439653561823
        ],
        [
            26,
            3.5849751591957593
        ],
        [
            27,
            3.584503446079118
        ],
        [
            28,
            3.584030995527997
        ],
        [
            29,
            3.583539976206368
        ],
        [
            30,
            3.5830461143660526
        ],
        [
            31,
            3.5825435679401876
        ],
        [
            32,
            3.5820630437053973
        ],
        [
            33,
            3.5815567572424594
        ],
        [
            34,
            3.5810670055354272
        ],
        [
            35,
            3.580538745188443
        ],
        [
            36,
            3.5800325552758108
        ],
        [
            37,
            3.5795207085367737
        ],
        [
            38,
            3.578973142410854
        ],
        [
            39,
            3.5784147428645645
        ],
        [
            40,
            3.5779096370659387
        ],
        [
            41,
            3.5773500951536987
        ],
        [
            42,
            3.5767884444548117
        ],
        [
            43,
            3.5762451137914173
        ],
        [
            44,
            3.5756461076214
        ],
        [
            45,
            3.575061191960012
        ],
        [
            46,
            3.574469084064947
        ],
        [
            47,
            3.573851539560614
        ],
        [
            48,
            3.573239713229919
        ],
        [
            49,
            3.5726069514413337
        ],
        [
            50,
            3.5719820865579512
        ],
        [
            51,
            3.57135345296588
        ],
        [
            52,
            3.5707086042038743
        ],
        [
            53,
            3.570047268354707
        ],
        [
            54,
            3.569398922417453
        ],
        [
            55,
            3.568752285979453
        ],
        [
            56,
            3.5680680043448834
        ],
        [
            57,
            3.567376343352883
        ],
        [
            58,
            3.5666699701159703
        ],
        [
            59,
            3.5659987670531907
        ],
        [
            60,
            3.5652854421300475
        ],
        [
            61,
            3.5645624965713574
        ],
        [
            62,
            3.563823448459345
        ],
        [
            63,
            3.5630457723396196
        ],
        [
            64,
            3.562274956261453
        ],
        [
            65,
            3.5615108021938777
        ],
        [
            66,
            3.560752291332757
        ],
        [
            67,
            3.5599484338608924
        ],
        [
            68,
            3.5591616558292567
        ],
        [
            69,
            3.558378498762105
        ],
        [
            70,
            3.5575330384292934
        ],
        [
            71,
            3.556714796935043
        ],
        [
            72,
            3.5558455528420407
        ],
        [
            73,
            3.555002165270997
        ],
        [
            74,
            3.5541422522829436
        ],
        [
            75,
            3.5532854787247303
        ],
        [
            76,
            3.5523951057702243
        ],
        [
            77,
            3.5515228769977973
        ],
        [
            78,
            3.55061890873371
        ],
        [
            79,
            3.5496880605710484
        ],
        [
            80,
            3.548786356861601
        ],
        [
            81,
            3.5478621141943907
        ],
        [
            82,
            3.5468728812207164
        ],
        [
            83,
            3.545930436139434
        ],
        [
            84,
            3.544943363961307
        ],
        [
            85,
            3.543956155966186
        ],
        [
            86,
            3.5429750584982753
        ],
        [
            87,
            3.5419769407395156
        ],
        [
            88,
            3.540988078897725
        ],
        [
            89,
            3.539949552900661
        ],
        [
            90,
            3.538890669869734
        ],
        [
            91,
            3.537833597295848
        ],
        [
            92,
            3.536768587302939
        ],
        [
            93,
            3.535681355508322
        ],
        [
            94,
            3.53459288662977
        ],
        [
            95,
            3.5334345972011447
        ],
        [
            96,
            3.5323197160193285
        ],
        [
            97,
            3.531171983948002
        ],
        [
            98,
            3.530049529228317
        ],
        [
            99,
            3.5288571483058577
        ],
        [
            100,
            3.5276569218153937
        ],
        [
            101,
            3.526468590136584
        ],
        [
            102,
            3.5252972151858866
        ],
        [
            103,
            3.52405849000669
        ],
        [
            104,
            3.5227806801519757
        ],
        [
            105,
            3.521496817488908
        ],
        [
            106,
            3.5202201642770765
        ],
        [
            107,
            3.5189128755829353
        ],
        [
            108,
            3.517586637572654
        ],
        [
            109,
            3.5162493474090177
        ],
        [
            110,
            3.51491452654096
        ],
        [
            111,
            3.5135821245016374
        ],
        [
            112,
            3.512200013648869
        ],
        [
            113,
            3.510796086595273
        ],
        [
            114,
            3.509400690044709
        ],
        [
            115,
            3.5079616361280666
        ],
        [
            116,
            3.506538343647862
        ],
        [
            117,
            3.5050581807960293
        ],
        [
            118,
            3.503562187260867
        ],
        [
            119,
            3.502074522072133
        ],
        [
            120,
            3.5006025711609405
        ],
        [
            121,
            3.4990639884861245
        ],
        [
            122,
            3.497498044961464
        ],
        [
            123,
            3.4959400986328317
        ],
        [
            124,
            3.494348514236533
        ],
        [
            125,
            3.492710123116698
        ],
        [
            126,
            3.49114436181516
        ],
        [
            127,
            3.489504714276147
        ],
        [
            128,
            3.4878336546018476
        ],
        [
            129,
            3.486209347633423
        ],
        [
            130,
            3.484503749273462
        ],
        [
            131,
            3.482819057549903
        ],
        [
            132,
            3.481083379887991
        ],
        [
            133,
            3.4793190361610318
        ],
        [
            134,
            3.477550933347515
        ],
        [
            135,
            3.475732977278826
        ],
        [
            136,
            3.4739276306251528
        ],
        [
            137,
            3.472127730439839
        ],
        [
            138,
            3.4702985598710514
        ],
        [
            139,
            3.4684071546118527
        ],
        [
            140,
            3.466538822663041
        ],
        [
            141,
            3.4646350966291366
        ],
        [
            142,
            3.462687952036292
        ],
        [
            143,
            3.4607255560798507
        ],
        [
            144,
            3.458771918519602
        ],
        [
            145,
            3.4567874196150714
        ],
        [
            146,
            3.4547722716271307
        ],
        [
            147,
            3.4527160236996095
        ],
        [
            148,
            3.4506422442302127
        ],
        [
            149,
            3.448523702372636
        ],
        [
            150,
            3.446428391101535
        ],
        [
            151,
            3.4443045883240857
        ],
        [
            152,
            3.442170843003914
        ],
        [
            153,
            3.4399591256562205
        ],
        [
            154,
            3.4377751621374175
        ],
        [
            155,
            3.435575459216769
        ],
        [
            156,
            3.4333117464579996
        ],
        [
            157,
            3.431057971113641
        ],
        [
            158,
            3.428710573671795
        ],
        [
            159,
            3.4264123089139398
        ],
        [
            160,
            3.4240936691557904
        ],
        [
            161,
            3.4217522765918362
        ],
        [
            162,
            3.4193728534313577
        ],
        [
            163,
            3.4169182403990415
        ],
        [
            164,
            3.4145160182741208
        ],
        [
            165,
            3.412085981884871
        ],
        [
            166,
            3.409623432982442
        ],
        [
            167,
            3.4070840353059024
        ],
        [
            168,
            3.4046369227702864
        ],
        [
            169,
            3.40207449250106
        ],
        [
            170,
            3.399524058156149
        ],
        [
            171,
            3.3968634027743803
        ],
        [
            172,
            3.3942369639250645
        ],
        [
            173,
            3.39163300089652
        ],
        [
            174,
            3.388888079181157
        ],
        [
            175,
            3.3862387221039265
        ],
        [
            176,
            3.383531950349737
        ],
        [
            177,
            3.380724182188973
        ],
        [
            178,
            3.377976389623506
        ],
        [
            179,
            3.3751715924085017
        ],
        [
            180,
            3.3722919959013566
        ],
        [
            181,
            3.369444195314846
        ],
        [
            182,
            3.3666193169271685
        ],
        [
            183,
            3.3637079742626605
        ],
        [
            184,
            3.36073282734669
        ],
        [
            185,
            3.3578757872512783
        ],
        [
            186,
            3.3548486608115953
        ],
        [
            187,
            3.3518726923382003
        ],
        [
            188,
            3.3487691684997616
        ],
        [
            189,
            3.3457675399581914
        ],
        [
            190,
            3.34267438099824
        ],
        [
            191,
            3.339599861591735
        ],
        [
            192,
            3.336445116957291
        ],
        [
            193,
            3.3332810675749536
        ],
        [
            194,
            3.33008891371497
        ],
        [
            195,
            3.3268399293161117
        ],
        [
            196,
            3.3235811814412863
        ],
        [
            197,
            3.3203154283947964
        ],
        [
            198,
            3.317023223653604
        ],
        [
            199,
            3.313663171257741
        ],
        [
            200,
            3.3103801449884567
        ],
        [
            201,
            3.3069826820886217
        ],
        [
            202,
            3.303610385930637
        ],
        [
            203,
            3.3001770715242658
        ],
        [
            204,
            3.2967039219038594
        ],
        [
            205,
            3.293174034927718
        ],
        [
            206,
            3.2896950877281315
        ],
        [
            207,
            3.2861258617989426
        ],
        [
            208,
            3.2826038102376347
        ],
        [
            209,
            3.278956478481607
        ],
        [
            210,
            3.2753276330841743
        ],
        [
            211,
            3.2717166195368454
        ],
        [
            212,
            3.2680066126995055
        ],
        [
            213,
            3.264291336537685
        ],
        [
            214,
            3.2605415636056696
        ],
        [
            215,
            3.2568545670030313
        ],
        [
            216,
            3.253007219954659
        ],
        [
            217,
            3.249237135129075
        ],
        [
            218,
            3.245336916161759
        ],
        [
            219,
            3.2414988061445955
        ],
        [
            220,
            3.23752878582275
        ],
        [
            221,
            3.2336391397190654
        ],
        [
            222,
            3.229664851587667
        ],
        [
            223,
            3.225760447686884
        ],
        [
            224,
            3.2217636424474083
        ],
        [
            225,
            3.2177238822122898
        ],
        [
            226,
            3.213727632418004
        ],
        [
            227,
            3.2096134564449996
        ],
        [
            228,
            3.2055970293518947
        ],
        [
            229,
            3.201375235429602
        ],
        [
            230,
            3.1972719743451514
        ],
        [
            231,
            3.193164111047254
        ],
        [
            232,
            3.1888605145607665
        ],
        [
            233,
            3.18464057000302
        ],
        [
            234,
            3.1804133192604813
        ],
        [
            235,
            3.17613652941649
        ],
        [
            236,
            3.1718316561077957
        ],
        [
            237,
            3.1675418697011053
        ],
        [
            238,
            3.1631950182082775
        ],
        [
            239,
            3.1588585107390523
        ],
        [
            240,
            3.154469577990692
        ],
        [
            241,
            3.150076063425199
        ],
        [
            242,
            3.145658467402868
        ],
        [
            243,
            3.141175296589296
        ],
        [
            244,
            3.136719518899519
        ],
        [
            245,
            3.132177825962508
        ],
        [
            246,
            3.127588050994956
        ],
        [
            247,
            3.12302939105967
        ],
        [
            248,
            3.1184646920909946
        ],
        [
            249,
            3.11378810026774
        ],
        [
            250,
            3.1091670949240475
        ],
        [
            251,
            3.104530505092396
        ],
        [
            252,
            3.0998676831914898
        ],
        [
            253,
            3.0951579357392016
        ],
        [
            254,
            3.090417786298552
        ],
        [
            255,
            3.085644175109751
        ],
        [
            256,
            3.0809267737086214
        ],
        [
            257,
            3.07608451195539
        ],
        [
            258,
            3.0713153758471026
        ],
        [
            259,
            3.0666168168401393
        ],
        [
            260,
            3.061679166850599
        ]
    ],
    "mae_physical": [
        [
            0,
            2.934458800947806
        ],
        [
            1,
            2.934035283039307
        ],
        [
            2,
            2.9335785735759496
        ],
        [
            3,
            2.9331354396796674
        ],
        [
            4,
            2.9326756170375785
        ],
        [
            5,
            2.932217055591036
        ],
        [
            6,
            2.9317770167910213
        ],
        [
            7,
            2.931333598252636
        ],
        [
            8,
            2.930826403875679
        ],
        [
            9,
            2.93035862951902
        ],
        [
            10,
            2.9298827909136302
        ],
        [
            11,
            2.9294007393193975
        ],
        [
            12,
            2.9288847843472587
        ],
        [
            13,
            2.9284059022453257
        ],
        [
            14,
            2.927889929678066
        ],
        [
            15,
            2.9273592211343598
        ],
        [
            16,
            2.9268790025126514
        ],
        [
            17,
            2.9263391420713956
        ],
        [
            18,
            2.9258242809032433
        ],
        [
            19,
            2.9253404399461345
        ],
        [
            20,
            2.9248196986603907
        ],
        [
            21,
            2.9242875010850247
        ],
        [
            22,
            2.9237334264173556
        ],
        [
            23,
            2.9231770786350504
        ],
        [
            24,
            2.9226187129892507
        ],
        [
            25,
            2.922053942272452
        ],
        [
            26,
            2.921477110659541
        ],
        [
            27,
            2.92088475876219
        ],
        [
            28,
            2.920282419245915
        ],
        [
            29,
            2.9196786891837796
        ],
        [
            30,
            2.9190756723428484
        ],
        [
            31,
            2.918463118105577
        ],
        [
            32,
            2.9178513101764625
        ],
        [
            33,
            2.917235572434862
        ],
        [
            34,
            2.916621092813177
        ],
        [
            35,
            2.915961376425729
        ],
        [
            36,
            2.9153195127669798
        ],
        [
            37,
            2.914678506871948
        ],
        [
            38,
            2.9140024500205213
        ],
        [
            39,
            2.913320987472848
        ],
        [
            40,
            2.9126838503667973
        ],
        [
            41,
            2.91199184980731
        ],
        [
            42,
            2.911306038633264
        ],
        [
            43,
            2.9106207348300055
        ],
        [
            44,
            2.909879593128626
        ],
        [
            45,
            2.9091539231172745
        ],
        [
            46,
            2.908418056564468
        ],
        [
            47,
            2.907654635108157
        ],
        [
            48,
            2.9068860836427084
        ],
        [
            49,
            2.9060928220079347
        ],
        [
            50,
            2.9053281860132296
        ],
        [
            51,
            2.9045356485536384
        ],
        [
            52,
            2.903743210492418
        ],
        [
            53,
            2.9029234296911013
        ],
        [
            54,
            2.902098506176532
        ],
        [
            55,
            2.9012998472211646
        ],
        [
            56,
            2.900455978999052
        ],
        [
            57,
            2.899608759790809
        ],
        [
            58,
            2.898733993060887
        ],
        [
            59,
            2.897894778251398
        ],
        [
            60,
            2.8970000535599736
        ],
        [
            61,
            2.896108225338011
        ],
        [
            62,
            2.895185379073924
        ],
        [
            63,
            2.8942317709501757
        ],
        [
            64,
            2.893269968537876
        ],
        [
            65,
            2.8923162793703128
        ],
        [
            66,
            2.8913636421472235
        ],
        [
            67,
            2.8903683456248483
        ],
        [
            68,
            2.8893751863502715
        ],
        [
            69,
            2.8884025660887263
        ],
        [
            70,
            2.8873490706186167
        ],
        [
            71,
            2.8863367304993455
        ],
        [
            72,
            2.8852627337753973
        ],
        [
            73,
            2.8842135552979458
        ],
        [
            74,
            2.8831379101080383
        ],
        [
            75,
            2.882075829075137
        ],
        [
            76,
            2.880979690269486
        ],
        [
            77,
            2.8798800591916947
        ],
        [
            78,
            2.878767877817154
        ],
        [
            79,
            2.8776075805306136
        ],
        [
            80,
            2.8764936214800447
        ],
        [
            81,
            2.8753424086389225
        ],
        [
            82,
            2.8741299497438657
        ],
        [
            83,
            2.8729391830829565
        ],
        [
            84,
            2.871715026638852
        ],
        [
            85,
            2.870473077595734
        ],
        [
            86,
            2.8692507191292393
        ],
        [
            87,
            2.8680033676276273
        ],
        [
            88,
            2.8667798835234155
        ],
        [
            89,
            2.8654843217774526
        ],
        [
            90,
            2.864165839857903
        ],
        [
            91,
            2.862861660202878
        ],
        [
            92,
            2.861527089179562
        ],
        [
            93,
            2.860170895446267
        ],
        [
            94,
            2.8588158113994453
        ],
        [
            95,
            2.857398844342804
        ],
        [
            96,
            2.8560128033461187
        ],
        [
            97,
            2.854569939046398
        ],
        [
            98,
            2.853177096157166
        ],
        [
            99,
            2.8517026762484305
        ],
        [
            100,
            2.850208777934313
        ],
        [
            101,
            2.8487356720654757
        ],
        [
            102,
            2.847280448715159
        ],
        [
            103,
            2.8457333806327125
        ],
        [
            104,
            2.8441625319971333
        ],
        [
            105,
            2.8425678284526272
        ],
        [
            106,
            2.8409663327288306
        ],
        [
            107,
            2.8393445732259512
        ],
        [
            108,
            2.837703343730845
        ],
        [
            109,
            2.8360280764832995
        ],
        [
            110,
            2.8343720184326573
        ],
        [
            111,
            2.8327026141669926
        ],
        [
            112,
            2.830996087883543
        ],
        [
            113,
            2.829243681649834
        ],
        [
            114,
            2.8275104468100825
        ],
        [
            115,
            2.8257235917168977
        ],
        [
            116,
            2.823945116376717
        ],
        [
            117,
            2.8221006802964532
        ],
        [
            118,
            2.8202566387469337
        ],
        [
            119,
            2.818406954457696
        ],
        [
            120,
            2.816535052587122
        ],
        [
            121,
            2.8146259605184496
        ],
        [
            122,
            2.812690406637704
        ],
        [
            123,
            2.8107291103509446
        ],
        [
            124,
            2.808748709475434
        ],
        [
            125,
            2.806714168330967
        ],
        [
            126,
            2.8047455141208317
        ],
        [
            127,
            2.8027022235945567
        ],
        [
            128,
            2.800624457231704
        ],
        [
            129,
            2.7985755130148573
        ],
        [
            130,
            2.7964753724584646
        ],
        [
            131,
            2.794353385949695
        ],
        [
            132,
            2.7921818420790987
        ],
        [
            133,
            2.7899927469907992
        ],
        [
            134,
            2.787771690661875
        ],
        [
            135,
            2.78550314463225
        ],
        [
            136,
            2.7832350050843
        ],
        [
            137,
            2.780984004871957
        ],
        [
            138,
            2.778705040630478
        ],
        [
            139,
            2.776324791246213
        ],
        [
            140,
            2.7739908093674073
        ],
        [
            141,
            2.771593097422347
        ],
        [
            142,
            2.7691524100283647
        ],
        [
            143,
            2.766714476749001
        ],
        [
            144,
            2.7642542443819496
        ],
        [
            145,
            2.761759913307708
        ],
        [
            146,
            2.759223534656851
        ],
        [
            147,
            2.756645240839696
        ],
        [
            148,
            2.7540319501953636
        ],
        [
            149,
            2.7513915875914114
        ],
        [
            150,
            2.748739522075493
        ],
        [
            151,
            2.74609917332262
        ],
        [
            152,
            2.7433918956542174
        ],
        [
            153,
            2.7406019878467456
        ],
        [
            154,
            2.737850951218365
        ],
        [
            155,
            2.735096123294542
        ],
        [
            156,
            2.732242093490274
        ],
        [
            157,
            2.7293978527488325
        ],
        [
            158,
            2.726447485537337
        ],
        [
            159,
            2.7235316522169435
        ],
        [
            160,
            2.720618663418213
        ],
        [
            161,
            2.7176484216579655
        ],
        [
            162,
            2.7146484267791644
        ],
        [
            163,
            2.7115335965516585
        ],
        [
            164,
            2.708488097626891
        ],
        [
            165,
            2.705463473328808
        ],
        [
            166,
            2.702324941254302
        ],
        [
            167,
            2.699129817169785
        ],
        [
            168,
            2.6960020438336687
        ],
        [
            169,
            2.6927693568220072
        ],
        [
            170,
            2.6895218145527298
        ],
        [
            171,
            2.6861913117186336
        ],
        [
            172,
            2.6828465235713344
        ],
        [
            173,
            2.6795293424753535
        ],
        [
            174,
            2.676076467805261
        ],
        [
            175,
            2.67269196806338
        ],
        [
            176,
            2.6692637236126315
        ],
        [
            177,
            2.665686013314548
        ],
        [
            178,
            2.66219388448552
        ],
        [
            179,
            2.658613994317567
        ],
        [
            180,
            2.654997305982065
        ],
        [
            181,
            2.6513269112614175
        ],
        [
            182,
            2.6476941368723876
        ],
        [
            183,
            2.6440079931244753
        ],
        [
            184,
            2.6402563170698663
        ],
        [
            185,
            2.6365589325460013
        ],
        [
            186,
            2.6327216383194765
        ],
        [
            187,
            2.628916858226661
        ],
        [
            188,
            2.624962191053685
        ],
        [
            189,
            2.621107749890961
        ],
        [
            190,
            2.617192089257624
        ],
        [
            191,
            2.613221084611528
        ],
        [
            192,
            2.6092207347786665
        ],
        [
            193,
            2.605121599147784
        ],
        [
            194,
            2.601039382135308
        ],
        [
            195,
            2.596884578666431
        ],
        [
            196,
            2.592694373738846
        ],
        [
            197,
            2.588479733707121
        ],
        [
            198,
            2.5842306033876916
        ],
        [
            199,
            2.5799173958749577
        ],
        [
            200,
            2.5757262046705156
        ],
        [
            201,
            2.5713618445716446
        ],
        [
            202,
            2.5669707617903716
        ],
        [
            203,
            2.5625577477000703
        ],
        [
            204,
            2.5580610526888163
        ],
        [
            205,
            2.5535058367172345
        ],
        [
            206,
            2.549013673259108
        ],
        [
            207,
            2.5443831988229046
        ],
        [
            208,
            2.539840024789708
        ],
        [
            209,
            2.535149435068937
        ],
        [
            210,
            2.5304597772207837
        ],
        [
            211,
            2.525755947868296
        ],
        [
            212,
            2.520991607400395
        ],
        [
            213,
            2.516118624866409
        ],
        [
            214,
            2.511284035124235
        ],
        [
            215,
            2.5064784286406216
        ],
        [
            216,
            2.5014878201404676
        ],
        [
            217,
            2.496555641793565
        ],
        [
            218,
            2.4915120741664962
        ],
        [
            219,
            2.4864859524989287
        ],
        [
            220,
            2.481314011068152
        ],
        [
            221,
            2.476234700855793
        ],
        [
            222,
            2.471060833274918
        ],
        [
            223,
            2.465900186924326
        ],
        [
            224,
            2.46061624776597
        ],
        [
            225,
            2.455374213073078
        ],
        [
            226,
            2.450113726942331
        ],
        [
            227,
            2.4447610768295776
        ],
        [
            228,
            2.4393964839861697
        ],
        [
            229,
            2.433913273899347
        ],
        [
            230,
            2.4284738074213066
        ],
        [
            231,
            2.4230408242484867
        ],
        [
            232,
            2.417438861707713
        ],
        [
            233,
            2.411857912604441
        ],
        [
            234,
            2.406278864849334
        ],
        [
            235,
            2.4006954373529292
        ],
        [
            236,
            2.3951818376979572
        ],
        [
            237,
            2.3896788614708306
        ],
        [
            238,
            2.384163481877154
        ],
        [
            239,
            2.378679947565066
        ],
        [
            240,
            2.373093042397659
        ],
        [
            241,
            2.3675705368486826
        ],
        [
            242,
            2.3620994707081944
        ],
        [
            243,
            2.3565599958368595
        ],
        [
            244,
            2.3511264710218316
        ],
        [
            245,
            2.3455625334442063
        ],
        [
            246,
            2.3399195527070322
        ],
        [
            247,
            2.334320644004233
        ],
        [
            248,
            2.3287935340964556
        ],
        [
            249,
            2.3230279320038405
        ],
        [
            250,
            2.3172971104215456
        ],
        [
            251,
            2.3116224028120107
        ],
        [
            252,
            2.305818181709955
        ],
        [
            253,
            2.2999813274649163
        ],
        [
            254,
            2.294172083371438
        ],
        [
            255,
            2.288251672015094
        ],
        [
            256,
            2.282334734929488
        ],
        [
            257,
            2.276316961186044
        ],
        [
            258,
            2.270451636522408
        ],
        [
            259,
            2.2645446438117314
        ],
        [
            260,
            2.258402668789729
        ]
    ],
    "r2_physical": [
        [
            0,
            -2.0099434472641873
        ],
        [
            1,
            -2.009362717032226
        ],
        [
            2,
            -2.0087386917228565
        ],
        [
            3,
            -2.0081473538568217
        ],
        [
            4,
            -2.007512497585219
        ],
        [
            5,
            -2.00689990091564
        ],
        [
            6,
            -2.0063165457304817
        ],
        [
            7,
            -2.0056892485798987
        ],
        [
            8,
            -2.005014614573704
        ],
        [
            9,
            -2.0043899152791966
        ],
        [
            10,
            -2.0037524869004075
        ],
        [
            11,
            -2.0030985032356785
        ],
        [
            12,
            -2.002393627216977
        ],
        [
            13,
            -2.0017354933763944
        ],
        [
            14,
            -2.001072376897099
        ],
        [
            15,
            -2.0003558768592278
        ],
        [
            16,
            -1.9997122908575862
        ],
        [
            17,
            -1.9990091271928856
        ],
        [
            18,
            -1.9982970331889547
        ],
        [
            19,
            -1.9976580987862134
        ],
        [
            20,
            -1.9969586193567794
        ],
        [
            21,
            -1.9962400925120924
        ],
        [
            22,
            -1.9954988705322583
        ],
        [
            23,
            -1.9947538324266798
        ],
        [
            24,
            -1.994008570502202
        ],
        [
            25,
            -1.9932402428968121
        ],
        [
            26,
            -1.9924575461151646
        ],
        [
            27,
            -1.9916700993159768
        ],
        [
            28,
            -1.9908815253548893
        ],
        [
            29,
            -1.9900620681458059
        ],
        [
            30,
            -1.9892379803376365
        ],
        [
            31,
            -1.9883995174956195
        ],
        [
            32,
            -1.9875979070982033
        ],
        [
            33,
            -1.986753436487247
        ],
        [
            34,
            -1.9859366589339338
        ],
        [
            35,
            -1.9850557842718777
        ],
        [
            36,
            -1.9842118340024686
        ],
        [
            37,
            -1.983358573659268
        ],
        [
            38,
            -1.982445903342195
        ],
        [
            39,
            -1.981515319929846
        ],
        [
            40,
            -1.9806736766337112
        ],
        [
            41,
            -1.979741466709413
        ],
        [
            42,
            -1.9788058901202379
        ],
        [
            43,
            -1.9779009701460737
        ],
        [
            44,
            -1.9769034816573856
        ],
        [
            45,
            -1.9759296184741442
        ],
        [
            46,
            -1.9749439427561786
        ],
        [
            47,
            -1.9739160969192162
        ],
        [
            48,
            -1.9728979435870775
        ],
        [
            49,
            -1.9718451344771761
        ],
        [
            50,
            -1.970805647464085
        ],
        [
            51,
            -1.9697600745327373
        ],
        [
            52,
            -1.9686877229769566
        ],
        [
            53,
            -1.9675881553442656
        ],
        [
            54,
            -1.9665103830509807
        ],
        [
            55,
            -1.9654356475044383
        ],
        [
            56,
            -1.964298556191752
        ],
        [
            57,
            -1.9631494239690919
        ],
        [
            58,
            -1.961976078617678
        ],
        [
            59,
            -1.9608613691739647
        ],
        [
            60,
            -1.9596769352445826
        ],
        [
            61,
            -1.9584767685850344
        ],
        [
            62,
            -1.9572501215499116
        ],
        [
            63,
            -1.9559596356936653
        ],
        [
            64,
            -1.9546808113967393
        ],
        [
            65,
            -1.9534133128340985
        ],
        [
            66,
            -1.9521554435492425
        ],
        [
            67,
            -1.9508226665704798
        ],
        [
            68,
            -1.9495184982987457
        ],
        [
            69,
            -1.948220618437658
        ],
        [
            70,
            -1.9468198076647338
        ],
        [
            71,
            -1.9454644117066509
        ],
        [
            72,
            -1.9440248727943232
        ],
        [
            73,
            -1.9426284907007147
        ],
        [
            74,
            -1.9412050887993448
        ],
        [
            75,
            -1.93978722601442
        ],
        [
            76,
            -1.9383141222575215
        ],
        [
            77,
            -1.9368713956418042
        ],
        [
            78,
            -1.9353765435675854
        ],
        [
            79,
            -1.9338376390757368
        ],
        [
            80,
            -1.9323473017651969
        ],
        [
            81,
            -1.930820104972939
        ],
        [
            82,
            -1.9291859604652415
        ],
        [
            83,
            -1.927629530205798
        ],
        [
            84,
            -1.9259998425972218
        ],
        [
            85,
            -1.9243703845595044
        ],
        [
            86,
            -1.9227514620231618
        ],
        [
            87,
            -1.9211049140399283
        ],
        [
            88,
            -1.9194740926014355
        ],
        [
            89,
            -1.9177618557570537
        ],
        [
            90,
            -1.9160165730124823
        ],
        [
            91,
            -1.9142747951127572
        ],
        [
            92,
            -1.9125204646569287
        ],
        [
            93,
            -1.9107300743295088
        ],
        [
            94,
            -1.9089381982491398
        ],
        [
            95,
            -1.907031987041751
        ],
        [
            96,
            -1.9051978034092087
        ],
        [
            97,
            -1.903310178703245
        ],
        [
            98,
            -1.9014647198673171
        ],
        [
            99,
            -1.8995049361024479
        ],
        [
            100,
            -1.8975329260878717
        ],
        [
            101,
            -1.8955811205143789
        ],
        [
            102,
            -1.8936578095405427
        ],
        [
            103,
            -1.8916246096057465
        ],
        [
            104,
            -1.8895280061888684
        ],
        [
            105,
            -1.8874222372210703
        ],
        [
            106,
            -1.885329054193654
        ],
        [
            107,
            -1.883186428171593
        ],
        [
            108,
            -1.8810135577452347
        ],
        [
            109,
            -1.8788234091384188
        ],
        [
            110,
            -1.8766381351111274
        ],
        [
            111,
            -1.874457648484905
        ],
        [
            112,
            -1.872196686542161
        ],
        [
            113,
            -1.8699009467255605
        ],
        [
            114,
            -1.8676200657280653
        ],
        [
            115,
            -1.865268773203344
        ],
        [
            116,
            -1.8629441821099957
        ],
        [
            117,
            -1.860527708181853
        ],
        [
            118,
            -1.8580864261788195
        ],
        [
            119,
            -1.8556597685371194
        ],
        [
            120,
            -1.8532597581365118
        ],
        [
            121,
            -1.850752183396664
        ],
        [
            122,
            -1.8482011480232994
        ],
        [
            123,
            -1.8456642738542954
        ],
        [
            124,
            -1.843073792467032
        ],
        [
            125,
            -1.8404083600841723
        ],
        [
            126,
            -1.8378622543598766
        ],
        [
            127,
            -1.835197224962363
        ],
        [
            128,
            -1.832482427465079
        ],
        [
            129,
            -1.8298448305587103
        ],
        [
            130,
            -1.8270765527306039
        ],
        [
            131,
            -1.8243435373789452
        ],
        [
            132,
            -1.8215291916418943
        ],
        [
            133,
            -1.8186698026353034
        ],
        [
            134,
            -1.8158057757028137
        ],
        [
            135,
            -1.8128625128917641
        ],
        [
            136,
            -1.8099411877414027
        ],
        [
            137,
            -1.8070301867160978
        ],
        [
            138,
            -1.804073391963704
        ],
        [
            139,
            -1.8010176353728085
        ],
        [
            140,
            -1.7980007916124836
        ],
        [
            141,
            -1.7949284681191506
        ],
        [
            142,
            -1.7917878195669026
        ],
        [
            143,
            -1.7886243576707592
        ],
        [
            144,
            -1.7854767960016238
        ],
        [
            145,
            -1.7822813324226328
        ],
        [
            146,
            -1.7790383939818404
        ],
        [
            147,
            -1.775731263489388
        ],
        [
            148,
            -1.7723979306587396
        ],
        [
            149,
            -1.7689947160512896
        ],
        [
            150,
            -1.76563087475013
        ],
        [
            151,
            -1.7622233791650839
        ],
        [
            152,
            -1.7588020466847887
        ],
        [
            153,
            -1.7552579285413015
        ],
        [
            154,
            -1.7517605191703693
        ],
        [
            155,
            -1.7482401498592184
        ],
        [
            156,
            -1.74461969280841
        ],
        [
            157,
            -1.7410174997364876
        ],
        [
            158,
            -1.7372681859762569
        ],
        [
            159,
            -1.7335998340102021
        ],
        [
            160,
            -1.7299014532212111
        ],
        [
            161,
            -1.7261693207293503
        ],
        [
            162,
            -1.7223791836721798
        ],
        [
            163,
            -1.7184720409195662
        ],
        [
            164,
            -1.714651008237981
        ],
        [
            165,
            -1.7107884676407168
        ],
        [
            166,
            -1.706877053731533
        ],
        [
            167,
            -1.7028465338166234
        ],
        [
            168,
            -1.6989653296682161
        ],
        [
            169,
            -1.6949042165134491
        ],
        [
            170,
            -1.6908651515388362
        ],
        [
            171,
            -1.6866547603027575
        ],
        [
            172,
            -1.6825017487515401
        ],
        [
            173,
            -1.678387447922002
        ],
        [
            174,
            -1.6740538488477053
        ],
        [
            175,
            -1.6698744519111952
        ],
        [
            176,
            -1.665607858195704
        ],
        [
            177,
            -1.6611856723990268
        ],
        [
            178,
            -1.6568615015272106
        ],
        [
            179,
            -1.6524512492751167
        ],
        [
            180,
            -1.6479271941919733
        ],
        [
            181,
            -1.6434568905242726
        ],
        [
            182,
            -1.6390262998843927
        ],
        [
            183,
            -1.6344639855356968
        ],
        [
            184,
            -1.629805762405581
        ],
        [
            185,
            -1.6253363402854055
        ],
        [
            186,
            -1.6206049902661284
        ],
        [
            187,
            -1.6159577590001204
        ],
        [
            188,
            -1.6111157322086411
        ],
        [
            189,
            -1.6064369463357275
        ],
        [
            190,
            -1.6016198770176033
        ],
        [
            191,
            -1.5968362508791296
        ],
        [
            192,
            -1.5919323784845623
        ],
        [
            193,
            -1.5870186975291074
        ],
        [
            194,
            -1.5820660953972094
        ],
        [
            195,
            -1.5770301937800322
        ],
        [
            196,
            -1.5719840966778462
        ],
        [
            197,
            -1.5669321134217906
        ],
        [
            198,
            -1.5618442375275228
        ],
        [
            199,
            -1.5566567121527606
        ],
        [
            200,
            -1.5515931841293584
        ],
        [
            201,
            -1.5463584411058973
        ],
        [
            202,
            -1.541167790124395
        ],
        [
            203,
            -1.5358886600938728
        ],
        [
            204,
            -1.5305538638261202
        ],
        [
            205,
            -1.525137674114681
        ],
        [
            206,
            -1.5198053228003174
        ],
        [
            207,
            -1.514340454312158
        ],
        [
            208,
            -1.5089536302015047
        ],
        [
            209,
            -1.5033812840364513
        ],
        [
            210,
            -1.4978433289993074
        ],
        [
            211,
            -1.4923386741840456
        ],
        [
            212,
            -1.4866894374391624
        ],
        [
            213,
            -1.4810386004043257
        ],
        [
            214,
            -1.4753418123765387
        ],
        [
            215,
            -1.4697467806669802
        ],
        [
            216,
            -1.4639151658895866
        ],
        [
            217,
            -1.4582073480508178
        ],
        [
            218,
            -1.4523094758277861
        ],
        [
            219,
            -1.4465124395874835
        ],
        [
            220,
            -1.4405233852184143
        ],
        [
            221,
            -1.4346626991953855
        ],
        [
            222,
            -1.4286817566719443
        ],
        [
            223,
            -1.4228131460644629
        ],
        [
            224,
            -1.4168130023058692
        ],
        [
            225,
            -1.410755932309407
        ],
        [
            226,
            -1.4047715776587033
        ],
        [
            227,
            -1.3986183987291811
        ],
        [
            228,
            -1.3926190170204449
        ],
        [
            229,
            -1.386320974757039
        ],
        [
            230,
            -1.3802077126917887
        ],
        [
            231,
            -1.3740954476980787
        ],
        [
            232,
            -1.3677003716069245
        ],
        [
            233,
            -1.3614379759098343
        ],
        [
            234,
            -1.3551730521485483
        ],
        [
            235,
            -1.3488431781723667
        ],
        [
            236,
            -1.3424803409551531
        ],
        [
            237,
            -1.3361483876005713
        ],
        [
            238,
            -1.3297409444212565
        ],
        [
            239,
            -1.3233575164277434
        ],
        [
            240,
            -1.3169058340025441
        ],
        [
            241,
            -1.3104564006378756
        ],
        [
            242,
            -1.303980680050676
        ],
        [
            243,
            -1.2974181250003363
        ],
        [
            244,
            -1.2909049425213048
        ],
        [
            245,
            -1.2842756890782647
        ],
        [
            246,
            -1.2775860116064917
        ],
        [
            247,
            -1.27095139529724
        ],
        [
            248,
            -1.2643176865243926
        ],
        [
            249,
            -1.2575314307657957
        ],
        [
            250,
            -1.25083584093343
        ],
        [
            251,
            -1.2441276643148411
        ],
        [
            252,
            -1.2373916316213127
        ],
        [
            253,
            -1.2305980871464897
        ],
        [
            254,
            -1.2237711192936205
        ],
        [
            255,
            -1.2169065327676707
        ],
        [
            256,
            -1.2101332023702884
        ],
        [
            257,
            -1.203191373424941
        ],
        [
            258,
            -1.1963650497262237
        ],
        [
            259,
            -1.1896501045996937
        ],
        [
            260,
            -1.1826045411515431
        ]
    ],
    "rmse_mental": [
        [
            0,
            6.602330414529382
        ],
        [
            1,
            6.598573156451919
        ],
        [
            2,
            6.594877368185114
        ],
        [
            3,
            6.591164287557556
        ],
        [
            4,
            6.587431349725545
        ],
        [
            5,
            6.583724913968683
        ],
        [
            6,
            6.579958707285205
        ],
        [
            7,
            6.576225998698554
        ],
        [
            8,
            6.572489868940678
        ],
        [
            9,
            6.568703441177601
        ],
        [
            10,
            6.564947788776211
        ],
        [
            11,
            6.561187978390389
        ],
        [
            12,
            6.557417113929506
        ],
        [
            13,
            6.553601226223078
        ],
        [
            14,
            6.5498570922163655
        ],
        [
            15,
            6.546051217945518
        ],
        [
            16,
            6.542242287622771
        ],
        [
            17,
            6.538399548749777
        ],
        [
            18,
            6.534579248145847
        ],
        [
            19,
            6.530695135554785
        ],
        [
            20,
            6.526774873823587
        ],
        [
            21,
            6.522922785375325
        ],
        [
            22,
            6.519053529570084
        ],
        [
            23,
            6.51510195884251
        ],
        [
            24,
            6.511108708258857
        ],
        [
            25,
            6.507154765082274
        ],
        [
            26,
            6.503150885055339
        ],
        [
            27,
            6.499161015641721
        ],
        [
            28,
            6.495121880852987
        ],
        [
            29,
            6.491075526007456
        ],
        [
            30,
            6.487003512285874
        ],
        [
            31,
            6.482868991559205
        ],
        [
            32,
            6.4787653336924516
        ],
        [
            33,
            6.474568567966426
        ],
        [
            34,
            6.470453221249318
        ],
        [
            35,
            6.466268668383253
        ],
        [
            36,
            6.462046050244949
        ],
        [
            37,
            6.457802460244415
        ],
        [
            38,
            6.453554965927174
        ],
        [
            39,
            6.44927038860505
        ],
        [
            40,
            6.444888201699214
        ],
        [
            41,
            6.44055764886356
        ],
        [
            42,
            6.436153780442776
        ],
        [
            43,
            6.431765641585068
        ],
        [
            44,
            6.4272391632106
        ],
        [
            45,
            6.422808425568314
        ],
        [
            46,
            6.418303299767091
        ],
        [
            47,
            6.413762573697151
        ],
        [
            48,
            6.409212727083826
        ],
        [
            49,
            6.404556342659987
        ],
        [
            50,
            6.399954952665023
        ],
        [
            51,
            6.39522654035757
        ],
        [
            52,
            6.390498043054097
        ],
        [
            53,
            6.385706698919763
        ],
        [
            54,
            6.38099797857831
        ],
        [
            55,
            6.376201937535734
        ],
        [
            56,
            6.371304401434973
        ],
        [
            57,
            6.366423971097632
        ],
        [
            58,
            6.361480172585547
        ],
        [
            59,
            6.356534198281135
        ],
        [
            60,
            6.351544749995339
        ],
        [
            61,
            6.346449116248891
        ],
        [
            62,
            6.341348484062757
        ],
        [
            63,
            6.3362231932364805
        ],
        [
            64,
            6.331109612149111
        ],
        [
            65,
            6.3259190890079235
        ],
        [
            66,
            6.320704211649508
        ],
        [
            67,
            6.315396601479388
        ],
        [
            68,
            6.31005522674988
        ],
        [
            69,
            6.304682199924467
        ],
        [
            70,
            6.299219786421116
        ],
        [
            71,
            6.293731878277113
        ],
        [
            72,
            6.2882807474861515
        ],
        [
            73,
            6.282652748759393
        ],
        [
            74,
            6.277109471157695
        ],
        [
            75,
            6.271357144884353
        ],
        [
            76,
            6.265681682341539
        ],
        [
            77,
            6.259989899774079
        ],
        [
            78,
            6.2541789200250015
        ],
        [
            79,
            6.248383166135346
        ],
        [
            80,
            6.242469611238897
        ],
        [
            81,
            6.236558593274123
        ],
        [
            82,
            6.230502825511816
        ],
        [
            83,
            6.2244654139843245
        ],
        [
            84,
            6.218310579057607
        ],
        [
            85,
            6.212223852470597
        ],
        [
            86,
            6.206051579463985
        ],
        [
            87,
            6.199668768981614
        ],
        [
            88,
            6.193368997607809
        ],
        [
            89,
            6.187078158731261
        ],
        [
            90,
            6.180599971840958
        ],
        [
            91,
            6.17416531619867
        ],
        [
            92,
            6.1676932814804575
        ],
        [
            93,
            6.1610656853288726
        ],
        [
            94,
            6.154428061910318
        ],
        [
            95,
            6.147756517998957
        ],
        [
            96,
            6.141056605530076
        ],
        [
            97,
            6.1342095251453745
        ],
        [
            98,
            6.127403484661257
        ],
        [
            99,
            6.120538405799877
        ],
        [
            100,
            6.113482038950832
        ],
        [
            101,
            6.106451930192137
        ],
        [
            102,
            6.099506620532764
        ],
        [
            103,
            6.092286151308986
        ],
        [
            104,
            6.085083297934585
        ],
        [
            105,
            6.077894722230678
        ],
        [
            106,
            6.07049967804765
        ],
        [
            107,
            6.063202069382269
        ],
        [
            108,
            6.05575417823993
        ],
        [
            109,
            6.048278290940611
        ],
        [
            110,
            6.040712329210183
        ],
        [
            111,
            6.033071152420128
        ],
        [
            112,
            6.0253934223519705
        ],
        [
            113,
            6.017706448120808
        ],
        [
            114,
            6.00983220396235
        ],
        [
            115,
            6.001964356724657
        ],
        [
            116,
            5.994104615026736
        ],
        [
            117,
            5.986115051337148
        ],
        [
            118,
            5.977989105778375
        ],
        [
            119,
            5.969919321190718
        ],
        [
            120,
            5.961734054333703
        ],
        [
            121,
            5.9534429394946065
        ],
        [
            122,
            5.945158278276056
        ],
        [
            123,
            5.936765162280871
        ],
        [
            124,
            5.9283581668662375
        ],
        [
            125,
            5.919796853789207
        ],
        [
            126,
            5.911268765361156
        ],
        [
            127,
            5.902593935445339
        ],
        [
            128,
            5.893878365863392
        ],
        [
            129,
            5.885157394558706
        ],
        [
            130,
            5.87633678643418
        ],
        [
            131,
            5.867424399358524
        ],
        [
            132,
            5.858493610417952
        ],
        [
            133,
            5.849510712842084
        ],
        [
            134,
            5.8404611599122305
        ],
        [
            135,
            5.831201081587003
        ],
        [
            136,
            5.821972471130433
        ],
        [
            137,
            5.812718510610998
        ],
        [
            138,
            5.80336059087543
        ],
        [
            139,
            5.7939715944030965
        ],
        [
            140,
            5.784296576807409
        ],
        [
            141,
            5.774911600700797
        ],
        [
            142,
            5.765250055284644
        ],
        [
            143,
            5.755459936914762
        ],
        [
            144,
            5.745719134693465
        ],
        [
            145,
            5.735839883430446
        ],
        [
            146,
            5.725909839334796
        ],
        [
            147,
            5.715955893693278
        ],
        [
            148,
            5.70582787329824
        ],
        [
            149,
            5.695610415453616
        ],
        [
            150,
            5.685512109929685
        ],
        [
            151,
            5.675264619314408
        ],
        [
            152,
            5.664946702584422
        ],
        [
            153,
            5.654429196528488
        ],
        [
            154,
            5.644016622176678
        ],
        [
            155,
            5.633451423639002
        ],
        [
            156,
            5.622837563450562
        ],
        [
            157,
            5.612049383737678
        ],
        [
            158,
            5.601317925382474
        ],
        [
            159,
            5.59044209114901
        ],
        [
            160,
            5.579555325475813
        ],
        [
            161,
            5.568552214603436
        ],
        [
            162,
            5.557479648437522
        ],
        [
            163,
            5.546306903305076
        ],
        [
            164,
            5.535156698815756
        ],
        [
            165,
            5.523910020714443
        ],
        [
            166,
            5.512499157286072
        ],
        [
            167,
            5.501137815969599
        ],
        [
            168,
            5.489729206435058
        ],
        [
            169,
            5.47814837592126
        ],
        [
            170,
            5.466593918774296
        ],
        [
            171,
            5.454755600557167
        ],
        [
            172,
            5.443025642402866
        ],
        [
            173,
            5.431215812934749
        ],
        [
            174,
            5.419188461850628
        ],
        [
            175,
            5.407304777116944
        ],
        [
            176,
            5.395357360806103
        ],
        [
            177,
            5.383047218708333
        ],
        [
            178,
            5.370959096341244
        ],
        [
            179,
            5.358718281047788
        ],
        [
            180,
            5.346321769783334
        ],
        [
            181,
            5.333981596278036
        ],
        [
            182,
            5.321626781684929
        ],
        [
            183,
            5.309110225480033
        ],
        [
            184,
            5.296455518908958
        ],
        [
            185,
            5.283779680157681
        ],
        [
            186,
            5.271086488470064
        ],
        [
            187,
            5.258235203173876
        ],
        [
            188,
            5.245275163526025
        ],
        [
            189,
            5.232388217711686
        ],
        [
            190,
            5.219275264177852
        ],
        [
            191,
            5.206269688739179
        ],
        [
            192,
            5.192968373070642
        ],
        [
            193,
            5.179708681634043
        ],
        [
            194,
            5.16647736055612
        ],
        [
            195,
            5.15319523083899
        ],
        [
            196,
            5.139738080449198
        ],
        [
            197,
            5.126167185440137
        ],
        [
            198,
            5.112614281746221
        ],
        [
            199,
            5.098985026835478
        ],
        [
            200,
            5.085416554306764
        ],
        [
            201,
            5.071616067103911
        ],
        [
            202,
            5.057773555240846
        ],
        [
            203,
            5.043829265145082
        ],
        [
            204,
            5.03004995283713
        ],
        [
            205,
            5.0159087516542336
        ],
        [
            206,
            5.001874448751021
        ],
        [
            207,
            4.987609490980443
        ],
        [
            208,
            4.973599089134941
        ],
        [
            209,
            4.959248216599214
        ],
        [
            210,
            4.944589382279449
        ],
        [
            211,
            4.930330463120341
        ],
        [
            212,
            4.9158794998564055
        ],
        [
            213,
            4.901281933789929
        ],
        [
            214,
            4.886852485998767
        ],
        [
            215,
            4.872258653044157
        ],
        [
            216,
            4.857450607675372
        ],
        [
            217,
            4.842697420284661
        ],
        [
            218,
            4.827822389707948
        ],
        [
            219,
            4.813097950312327
        ],
        [
            220,
            4.7979815864592625
        ],
        [
            221,
            4.783031212326058
        ],
        [
            222,
            4.7679948043507645
        ],
        [
            223,
            4.75296707654953
        ],
        [
            224,
            4.7376976919630875
        ],
        [
            225,
            4.722463961027901
        ],
        [
            226,
            4.707415814942334
        ],
        [
            227,
            4.691859546673418
        ],
        [
            228,
            4.676560278926748
        ],
        [
            229,
            4.660965612907692
        ],
        [
            230,
            4.645678611747415
        ],
        [
            231,
            4.630249839911683
        ],
        [
            232,
            4.614599278640882
        ],
        [
            233,
            4.598948185196417
        ],
        [
            234,
            4.583138504319004
        ],
        [
            235,
            4.567424966112127
        ],
        [
            236,
            4.551721935270164
        ],
        [
            237,
            4.535726946945337
        ],
        [
            238,
            4.519935961918894
        ],
        [
            239,
            4.504099987164448
        ],
        [
            240,
            4.4879629438696025
        ],
        [
            241,
            4.472104168074485
        ],
        [
            242,
            4.456018526980446
        ],
        [
            243,
            4.439924378949748
        ],
        [
            244,
            4.4239444734033775
        ],
        [
            245,
            4.407634906591231
        ],
        [
            246,
            4.3912647309106125
        ],
        [
            247,
            4.375048268969342
        ],
        [
            248,
            4.35889413627716
        ],
        [
            249,
            4.342177921165588
        ],
        [
            250,
            4.325946471606763
        ],
        [
            251,
            4.3096074009844445
        ],
        [
            252,
            4.293186229774626
        ],
        [
            253,
            4.276691143123313
        ],
        [
            254,
            4.2601495531745845
        ],
        [
            255,
            4.243610538530614
        ],
        [
            256,
            4.227045968361637
        ],
        [
            257,
            4.210308447237133
        ],
        [
            258,
            4.193772640160168
        ],
        [
            259,
            4.1772138196915884
        ],
        [
            260,
            4.160420299135557
        ]
    ],
    "mae_mental": [
        [
            0,
            6.274680614346426
        ],
        [
            1,
            6.270809101863552
        ],
        [
            2,
            6.2669824323548164
        ],
        [
            3,
            6.263136093709431
        ],
        [
            4,
            6.259288666688606
        ],
        [
            5,
            6.255448612748589
        ],
        [
            6,
            6.251565039921208
        ],
        [
            7,
            6.247695401635946
        ],
        [
            8,
            6.243840656343273
        ],
        [
            9,
            6.239925269062487
        ],
        [
            10,
            6.236037299086983
        ],
        [
            11,
            6.2321509132099235
        ],
        [
            12,
            6.228253487747588
        ],
        [
            13,
            6.224297372461765
        ],
        [
            14,
            6.220430273962701
        ],
        [
            15,
            6.21649846752118
        ],
        [
            16,
            6.212533517209375
        ],
        [
            17,
            6.20856298610018
        ],
        [
            18,
            6.204603773185471
        ],
        [
            19,
            6.200578846412417
        ],
        [
            20,
            6.196531888660068
        ],
        [
            21,
            6.1925434191905016
        ],
        [
            22,
            6.188532768117401
        ],
        [
            23,
            6.184431638540513
        ],
        [
            24,
            6.180303167583008
        ],
        [
            25,
            6.17620308342821
        ],
        [
            26,
            6.172067261682261
        ],
        [
            27,
            6.167920511950357
        ],
        [
            28,
            6.163743168096865
        ],
        [
            29,
            6.159554442703324
        ],
        [
            30,
            6.155334712724958
        ],
        [
            31,
            6.151048327477386
        ],
        [
            32,
            6.146795231548932
        ],
        [
            33,
            6.142434740120282
        ],
        [
            34,
            6.138158416125498
        ],
        [
            35,
            6.133818595532653
        ],
        [
            36,
            6.129435010366892
        ],
        [
            37,
            6.125034275320952
        ],
        [
            38,
            6.1206279682158025
        ],
        [
            39,
            6.116181342456265
        ],
        [
            40,
            6.1116447015509205
        ],
        [
            41,
            6.1071513141536675
        ],
        [
            42,
            6.102578051307008
        ],
        [
            43,
            6.098007311856986
        ],
        [
            44,
            6.093312505795152
        ],
        [
            45,
            6.088706041642483
        ],
        [
            46,
            6.08402771102702
        ],
        [
            47,
            6.079306513646105
        ],
        [
            48,
            6.074564789865641
        ],
        [
            49,
            6.069740917008595
        ],
        [
            50,
            6.0649386660114635
        ],
        [
            51,
            6.060035102969448
        ],
        [
            52,
            6.055128767636399
        ],
        [
            53,
            6.050144258636556
        ],
        [
            54,
            6.045231609711511
        ],
        [
            55,
            6.0402236315328
        ],
        [
            56,
            6.03512871965466
        ],
        [
            57,
            6.030049553703542
        ],
        [
            58,
            6.024899966214727
        ],
        [
            59,
            6.019738870579125
        ],
        [
            60,
            6.014538919765677
        ],
        [
            61,
            6.009217491985967
        ],
        [
            62,
            6.003924362461439
        ],
        [
            63,
            5.998566421386379
        ],
        [
            64,
            5.99323981320298
        ],
        [
            65,
            5.987830549228511
        ],
        [
            66,
            5.982381131124977
        ],
        [
            67,
            5.976848448932971
        ],
        [
            68,
            5.971278591344021
        ],
        [
            69,
            5.965675292899145
        ],
        [
            70,
            5.959971917255613
        ],
        [
            71,
            5.954242736501182
        ],
        [
            72,
            5.948534923411856
        ],
        [
            73,
            5.942672006635858
        ],
        [
            74,
            5.936854860126572
        ],
        [
            75,
            5.93087780995657
        ],
        [
            76,
            5.92493448151438
        ],
        [
            77,
            5.918967390720476
        ],
        [
            78,
            5.912898823718897
        ],
        [
            79,
            5.906834608356425
        ],
        [
            80,
            5.900642110297344
        ],
        [
            81,
            5.894467670945513
        ],
        [
            82,
            5.888128904488263
        ],
        [
            83,
            5.881797365494222
        ],
        [
            84,
            5.875364207681393
        ],
        [
            85,
            5.868974405947148
        ],
        [
            86,
            5.8625121724685565
        ],
        [
            87,
            5.855823211321895
        ],
        [
            88,
            5.849234295771426
        ],
        [
            89,
            5.842640363430017
        ],
        [
            90,
            5.835859468019248
        ],
        [
            91,
            5.829100170791549
        ],
        [
            92,
            5.82230520568438
        ],
        [
            93,
            5.815369794232733
        ],
        [
            94,
            5.808402086264335
        ],
        [
            95,
            5.8013955972738716
        ],
        [
            96,
            5.794371589518233
        ],
        [
            97,
            5.787183992814698
        ],
        [
            98,
            5.7800471072629
        ],
        [
            99,
            5.772795510172044
        ],
        [
            100,
            5.7654057647157835
        ],
        [
            101,
            5.7580241458928025
        ],
        [
            102,
            5.750705662989777
        ],
        [
            103,
            5.743116535196369
        ],
        [
            104,
            5.73556748392598
        ],
        [
            105,
            5.727957700922985
        ],
        [
            106,
            5.720198045440968
        ],
        [
            107,
            5.712531350603039
        ],
        [
            108,
            5.704692889980022
        ],
        [
            109,
            5.696788419213071
        ],
        [
            110,
            5.6888331424869945
        ],
        [
            111,
            5.680788807820954
        ],
        [
            112,
            5.672690826174397
        ],
        [
            113,
            5.6645627675840515
        ],
        [
            114,
            5.65629967187075
        ],
        [
            115,
            5.647988426205296
        ],
        [
            116,
            5.639691758275832
        ],
        [
            117,
            5.631247086412954
        ],
        [
            118,
            5.622702129935258
        ],
        [
            119,
            5.614156278187797
        ],
        [
            120,
            5.605521038074621
        ],
        [
            121,
            5.5967747097047384
        ],
        [
            122,
            5.5880483770930525
        ],
        [
            123,
            5.579148633568079
        ],
        [
            124,
            5.570246715473648
        ],
        [
            125,
            5.561194036430961
        ],
        [
            126,
            5.55218286922314
        ],
        [
            127,
            5.543022695203756
        ],
        [
            128,
            5.533784815129017
        ],
        [
            129,
            5.524528311002975
        ],
        [
            130,
            5.51522820327906
        ],
        [
            131,
            5.505790306417734
        ],
        [
            132,
            5.49631048488937
        ],
        [
            133,
            5.486769342582498
        ],
        [
            134,
            5.477170507379826
        ],
        [
            135,
            5.467362340664703
        ],
        [
            136,
            5.457577461764316
        ],
        [
            137,
            5.447733273842191
        ],
        [
            138,
            5.437810190172003
        ],
        [
            139,
            5.427837094604569
        ],
        [
            140,
            5.417601956217081
        ],
        [
            141,
            5.40759906872807
        ],
        [
            142,
            5.397310741395758
        ],
        [
            143,
            5.386948874332761
        ],
        [
            144,
            5.376567304934431
        ],
        [
            145,
            5.366067492721865
        ],
        [
            146,
            5.355517350187237
        ],
        [
            147,
            5.344912484588239
        ],
        [
            148,
            5.3341201727822325
        ],
        [
            149,
            5.323227821580516
        ],
        [
            150,
            5.312478229903535
        ],
        [
            151,
            5.301548724606533
        ],
        [
            152,
            5.290555644755395
        ],
        [
            153,
            5.279327254167339
        ],
        [
            154,
            5.268192554080247
        ],
        [
            155,
            5.256916183753302
        ],
        [
            156,
            5.245598754226761
        ],
        [
            157,
            5.234080985488507
        ],
        [
            158,
            5.222625731621813
        ],
        [
            159,
            5.210968918848358
        ],
        [
            160,
            5.199378509649494
        ],
        [
            161,
            5.187592359997282
        ],
        [
            162,
            5.175740162398191
        ],
        [
            163,
            5.163764220356141
        ],
        [
            164,
            5.151826478490894
        ],
        [
            165,
            5.139767463175242
        ],
        [
            166,
            5.127552368496889
        ],
        [
            167,
            5.115368493851399
        ],
        [
            168,
            5.103062049654506
        ],
        [
            169,
            5.090667207369068
        ],
        [
            170,
            5.078290913329028
        ],
        [
            171,
            5.065644208216828
        ],
        [
            172,
            5.052959802166727
        ],
        [
            173,
            5.040299797218118
        ],
        [
            174,
            5.02737704859484
        ],
        [
            175,
            5.014549161363768
        ],
        [
            176,
            5.001747045740985
        ],
        [
            177,
            4.988440468807348
        ],
        [
            178,
            4.975432177918069
        ],
        [
            179,
            4.96225370336699
        ],
        [
            180,
            4.948903343821532
        ],
        [
            181,
            4.935524782078378
        ],
        [
            182,
            4.922193767640414
        ],
        [
            183,
            4.908681476676224
        ],
        [
            184,
            4.8950783726352975
        ],
        [
            185,
            4.881368926307499
        ],
        [
            186,
            4.8676943194946185
        ],
        [
            187,
            4.853796533690203
        ],
        [
            188,
            4.83973378303067
        ],
        [
            189,
            4.82573319721542
        ],
        [
            190,
            4.811568406204249
        ],
        [
            191,
            4.797466132464825
        ],
        [
            192,
            4.783104024477453
        ],
        [
            193,
            4.768716796932605
        ],
        [
            194,
            4.754306818014824
        ],
        [
            195,
            4.739848188905908
        ],
        [
            196,
            4.725259925695074
        ],
        [
            197,
            4.710486594462555
        ],
        [
            198,
            4.695711467090069
        ],
        [
            199,
            4.680902686695124
        ],
        [
            200,
            4.6661215516544825
        ],
        [
            201,
            4.651076987285742
        ],
        [
            202,
            4.6359798100170675
        ],
        [
            203,
            4.620828854957683
        ],
        [
            204,
            4.605725894838372
        ],
        [
            205,
            4.590202089124078
        ],
        [
            206,
            4.574845249220829
        ],
        [
            207,
            4.5593099714125565
        ],
        [
            208,
            4.544064596195349
        ],
        [
            209,
            4.528265665841583
        ],
        [
            210,
            4.512282960366883
        ],
        [
            211,
            4.496618434886805
        ],
        [
            212,
            4.480805024204638
        ],
        [
            213,
            4.464792155579433
        ],
        [
            214,
            4.448875900883003
        ],
        [
            215,
            4.432859632792889
        ],
        [
            216,
            4.416548308110077
        ],
        [
            217,
            4.400331842819316
        ],
        [
            218,
            4.384035148076563
        ],
        [
            219,
            4.367683718668535
        ],
        [
            220,
            4.351036708626971
        ],
        [
            221,
            4.334528893432362
        ],
        [
            222,
            4.317922878585406
        ],
        [
            223,
            4.301348143775991
        ],
        [
            224,
            4.284353510645412
        ],
        [
            225,
            4.267564043902711
        ],
        [
            226,
            4.250847280425513
        ],
        [
            227,
            4.233690581065696
        ],
        [
            228,
            4.216621792556455
        ],
        [
            229,
            4.199313485382388
        ],
        [
            230,
            4.182218077198771
        ],
        [
            231,
            4.165136683707269
        ],
        [
            232,
            4.147797194903329
        ],
        [
            233,
            4.130219492336248
        ],
        [
            234,
            4.112637539037922
        ],
        [
            235,
            4.095099233940944
        ],
        [
            236,
            4.077518125508456
        ],
        [
            237,
            4.059749713680088
        ],
        [
            238,
            4.042082408130569
        ],
        [
            239,
            4.024317538178207
        ],
        [
            240,
            4.006162361810671
        ],
        [
            241,
            3.988340244997268
        ],
        [
            242,
            3.9702787967336257
        ],
        [
            243,
            3.952268778077708
        ],
        [
            244,
            3.9341690084278182
        ],
        [
            245,
            3.915799518559603
        ],
        [
            246,
            3.8973405577192373
        ],
        [
            247,
            3.8790150184759358
        ],
        [
            248,
            3.8608127640397756
        ],
        [
            249,
            3.842014132730113
        ],
        [
            250,
            3.823607315953146
        ],
        [
            251,
            3.805003062990688
        ],
        [
            252,
            3.7863727431969356
        ],
        [
            253,
            3.767483628836254
        ],
        [
            254,
            3.7488959459650437
        ],
        [
            255,
            3.7300015135899485
        ],
        [
            256,
            3.7110059045305186
        ],
        [
            257,
            3.6919546743367344
        ],
        [
            258,
            3.67303627769419
        ],
        [
            259,
            3.6541034455267374
        ],
        [
            260,
            3.6348847710846255
        ]
    ],
    "r2_mental": [
        [
            0,
            -9.226033916123402
        ],
        [
            1,
            -9.214398353055202
        ],
        [
            2,
            -9.20295961290944
        ],
        [
            3,
            -9.191473805173182
        ],
        [
            4,
            -9.179933093022495
        ],
        [
            5,
            -9.16848078294388
        ],
        [
            6,
            -9.15685039223376
        ],
        [
            7,
            -9.145330013230206
        ],
        [
            8,
            -9.13380562153225
        ],
        [
            9,
            -9.122132763627915
        ],
        [
            10,
            -9.110561425395518
        ],
        [
            11,
            -9.098983905013338
        ],
        [
            12,
            -9.087379007743062
        ],
        [
            13,
            -9.075642342172147
        ],
        [
            14,
            -9.064133012374857
        ],
        [
            15,
            -9.052440636132289
        ],
        [
            16,
            -9.040745675271603
        ],
        [
            17,
            -9.028953806237082
        ],
        [
            18,
            -9.017237659211709
        ],
        [
            19,
            -9.0053328328764
        ],
        [
            20,
            -8.99332438660776
        ],
        [
            21,
            -8.981531790634898
        ],
        [
            22,
            -8.969693647980558
        ],
        [
            23,
            -8.95761090948612
        ],
        [
            24,
            -8.945408168855987
        ],
        [
            25,
            -8.933332916947675
        ],
        [
            26,
            -8.92111263364384
        ],
        [
            27,
            -8.908942594349483
        ],
        [
            28,
            -8.896629891948287
        ],
        [
            29,
            -8.884302855358198
        ],
        [
            30,
            -8.87190540532467
        ],
        [
            31,
            -8.859325609296857
        ],
        [
            32,
            -8.846847648142198
        ],
        [
            33,
            -8.83409474661814
        ],
        [
            34,
            -8.821597281445495
        ],
        [
            35,
            -8.808897799063864
        ],
        [
            36,
            -8.796091122433303
        ],
        [
            37,
            -8.783229268952736
        ],
        [
            38,
            -8.770364042665063
        ],
        [
            39,
            -8.757395071555576
        ],
        [
            40,
            -8.744139556010419
        ],
        [
            41,
            -8.731049077836008
        ],
        [
            42,
            -8.717746003532723
        ],
        [
            43,
            -8.704499495302287
        ],
        [
            44,
            -8.690844846721953
        ],
        [
            45,
            -8.677488321457572
        ],
        [
            46,
            -8.663916995973285
        ],
        [
            47,
            -8.650248063365327
        ],
        [
            48,
            -8.636561378097204
        ],
        [
            49,
            -8.622564266031914
        ],
        [
            50,
            -8.608742460748717
        ],
        [
            51,
            -8.594549450752377
        ],
        [
            52,
            -8.580366675860482
        ],
        [
            53,
            -8.566006096982841
        ],
        [
            54,
            -8.551903651512859
        ],
        [
            55,
            -8.537550377649266
        ],
        [
            56,
            -8.522904493259642
        ],
        [
            57,
            -8.508320957844713
        ],
        [
            58,
            -8.49355946170028
        ],
        [
            59,
            -8.478802943894292
        ],
        [
            60,
            -8.463928348923242
        ],
        [
            61,
            -8.448749248398185
        ],
        [
            62,
            -8.433567458805646
        ],
        [
            63,
            -8.418324569088918
        ],
        [
            64,
            -8.403128787354133
        ],
        [
            65,
            -8.387716907469589
        ],
        [
            66,
            -8.372245443830638
        ],
        [
            67,
            -8.356511962875139
        ],
        [
            68,
            -8.340691736425299
        ],
        [
            69,
            -8.32479126710903
        ],
        [
            70,
            -8.308640159550231
        ],
        [
            71,
            -8.292427767809667
        ],
        [
            72,
            -8.276338012552435
        ],
        [
            73,
            -8.259740835059599
        ],
        [
            74,
            -8.243408030522044
        ],
        [
            75,
            -8.226474524363756
        ],
        [
            76,
            -8.20978250223906
        ],
        [
            77,
            -8.193057659581394
        ],
        [
            78,
            -8.175998246303566
        ],
        [
            79,
            -8.158999312828614
        ],
        [
            80,
            -8.141671113244689
        ],
        [
            81,
            -8.124366744358468
        ],
        [
            82,
            -8.106655624187358
        ],
        [
            83,
            -8.089015317834004
        ],
        [
            84,
            -8.071049524015297
        ],
        [
            85,
            -8.053300016240536
        ],
        [
            86,
            -8.035318797083526
        ],
        [
            87,
            -8.016743034330808
        ],
        [
            88,
            -7.998427681858997
        ],
        [
            89,
            -7.980156879831242
        ],
        [
            90,
            -7.9613613589371415
        ],
        [
            91,
            -7.9427116298098905
        ],
        [
            92,
            -7.923973159984564
        ],
        [
            93,
            -7.90480466047191
        ],
        [
            94,
            -7.885627815539751
        ],
        [
            95,
            -7.866373800499414
        ],
        [
            96,
            -7.847058930568247
        ],
        [
            97,
            -7.8273415571199845
        ],
        [
            98,
            -7.807764164421883
        ],
        [
            99,
            -7.788038966628875
        ],
        [
            100,
            -7.767787192640906
        ],
        [
            101,
            -7.747634010868881
        ],
        [
            102,
            -7.727746692621819
        ],
        [
            103,
            -7.707095472661505
        ],
        [
            104,
            -7.686519007204165
        ],
        [
            105,
            -7.666007598340508
        ],
        [
            106,
            -7.644932365806733
        ],
        [
            107,
            -7.624159969236345
        ],
        [
            108,
            -7.602985562211529
        ],
        [
            109,
            -7.581757734960396
        ],
        [
            110,
            -7.560300838777165
        ],
        [
            111,
            -7.53865789404975
        ],
        [
            112,
            -7.516939006830217
        ],
        [
            113,
            -7.495221676754159
        ],
        [
            114,
            -7.473004014665955
        ],
        [
            115,
            -7.450833457563007
        ],
        [
            116,
            -7.428714739811458
        ],
        [
            117,
            -7.406260385900669
        ],
        [
            118,
            -7.38345345678928
        ],
        [
            119,
            -7.360834813768683
        ],
        [
            120,
            -7.337923700615455
        ],
        [
            121,
            -7.314748358840644
        ],
        [
            122,
            -7.29162327147761
        ],
        [
            123,
            -7.2682282906282225
        ],
        [
            124,
            -7.244827755645833
        ],
        [
            125,
            -7.221031763008762
        ],
        [
            126,
            -7.197362306979713
        ],
        [
            127,
            -7.173320582998036
        ],
        [
            128,
            -7.149201508117136
        ],
        [
            129,
            -7.125103157610242
        ],
        [
            130,
            -7.100765781984849
        ],
        [
            131,
            -7.076212250178404
        ],
        [
            132,
            -7.051645404477092
        ],
        [
            133,
            -7.026972967913062
        ],
        [
            134,
            -7.002155737676631
        ],
        [
            135,
            -6.976800943436875
        ],
        [
            136,
            -6.951572337793503
        ],
        [
            137,
            -6.926314555576441
        ],
        [
            138,
            -6.900813885310882
        ],
        [
            139,
            -6.875269821413954
        ],
        [
            140,
            -6.848990866675806
        ],
        [
            141,
            -6.823541675999988
        ],
        [
            142,
            -6.797385683070864
        ],
        [
            143,
            -6.770926288283962
        ],
        [
            144,
            -6.744644808784054
        ],
        [
            145,
            -6.718035253723093
        ],
        [
            146,
            -6.6913350351170156
        ],
        [
            147,
            -6.66461698064541
        ],
        [
            148,
            -6.637479394174856
        ],
        [
            149,
            -6.610150930692683
        ],
        [
            150,
            -6.583189289546729
        ],
        [
            151,
            -6.555878247355918
        ],
        [
            152,
            -6.528429287928572
        ],
        [
            153,
            -6.500500764697856
        ],
        [
            154,
            -6.472902006473033
        ],
        [
            155,
            -6.444950708058399
        ],
        [
            156,
            -6.416923404583744
        ],
        [
            157,
            -6.38848995198371
        ],
        [
            158,
            -6.360260170793619
        ],
        [
            159,
            -6.331705728140985
        ],
        [
            160,
            -6.303178165543892
        ],
        [
            161,
            -6.274402236251502
        ],
        [
            162,
            -6.245502006017873
        ],
        [
            163,
            -6.216398606860589
        ],
        [
            164,
            -6.187412314010524
        ],
        [
            165,
            -6.158234317882171
        ],
        [
            166,
            -6.128691016813875
        ],
        [
            167,
            -6.099336625221554
        ],
        [
            168,
            -6.069921047055651
        ],
        [
            169,
            -6.040123877019732
        ],
        [
            170,
            -6.010457275011002
        ],
        [
            171,
            -5.980126812356701
        ],
        [
            172,
            -5.950138828547191
        ],
        [
            173,
            -5.92001186807935
        ],
        [
            174,
            -5.889397265410147
        ],
        [
            175,
            -5.85921501278538
        ],
        [
            176,
            -5.828937687305424
        ],
        [
            177,
            -5.797811188954489
        ],
        [
            178,
            -5.767315260932515
        ],
        [
            179,
            -5.7365039818684584
        ],
        [
            180,
            -5.705372449738137
        ],
        [
            181,
            -5.674454009708191
        ],
        [
            182,
            -5.643570460681518
        ],
        [
            183,
            -5.61235563260746
        ],
        [
            184,
            -5.580870999336309
        ],
        [
            185,
            -5.549409113765129
        ],
        [
            186,
            -5.517979699401828
        ],
        [
            187,
            -5.486235843542199
        ],
        [
            188,
            -5.454301829081002
        ],
        [
            189,
            -5.422626060372952
        ],
        [
            190,
            -5.390474751371265
        ],
        [
            191,
            -5.358666407569771
        ],
        [
            192,
            -5.326216845529043
        ],
        [
            193,
            -5.293951449982509
        ],
        [
            194,
            -5.261837316296129
        ],
        [
            195,
            -5.229682476123855
        ],
        [
            196,
            -5.197188340866449
        ],
        [
            197,
            -5.164505600648786
        ],
        [
            198,
            -5.13195242624012
        ],
        [
            199,
            -5.099302771085349
        ],
        [
            200,
            -5.066885295352873
        ],
        [
            201,
            -5.034002102081485
        ],
        [
            202,
            -5.001108540488015
        ],
        [
            203,
            -4.968064022828006
        ],
        [
            204,
            -4.9355000786105805
        ],
        [
            205,
            -4.90217352468204
        ],
        [
            206,
            -4.869191660917847
        ],
        [
            207,
            -4.835762439453818
        ],
        [
            208,
            -4.803022690501703
        ],
        [
            209,
            -4.769582804719667
        ],
        [
            210,
            -4.735525077274561
        ],
        [
            211,
            -4.702493226719701
        ],
        [
            212,
            -4.669113820362986
        ],
        [
            213,
            -4.635495260828121
        ],
        [
            214,
            -4.602362139175575
        ],
        [
            215,
            -4.568950917654698
        ],
        [
            216,
            -4.535151414332179
        ],
        [
            217,
            -4.501579435577172
        ],
        [
            218,
            -4.467833582985103
        ],
        [
            219,
            -4.434531608605819
        ],
        [
            220,
            -4.400449047089711
        ],
        [
            221,
            -4.3668461903509375
        ],
        [
            222,
            -4.333155737559706
        ],
        [
            223,
            -4.299590721023572
        ],
        [
            224,
            -4.265594481049878
        ],
        [
            225,
            -4.231786630702842
        ],
        [
            226,
            -4.198497544518538
        ],
        [
            227,
            -4.164196091604198
        ],
        [
            228,
            -4.130572065926423
        ],
        [
            229,
            -4.096411849282289
        ],
        [
            230,
            -4.063036322463829
        ],
        [
            231,
            -4.029462447721696
        ],
        [
            232,
            -3.9955200542383933
        ],
        [
            233,
            -3.9616914342912413
        ],
        [
            234,
            -3.9276367223216537
        ],
        [
            235,
            -3.8939053086540376
        ],
        [
            236,
            -3.8603121829852736
        ],
        [
            237,
            -3.826213419748318
        ],
        [
            238,
            -3.7926673080583155
        ],
        [
            239,
            -3.7591431147264123
        ],
        [
            240,
            -3.725102607643792
        ],
        [
            241,
            -3.6917681310073647
        ],
        [
            242,
            -3.65807733722606
        ],
        [
            243,
            -3.6244902225118354
        ],
        [
            244,
            -3.5912617598542678
        ],
        [
            245,
            -3.5574713451326208
        ],
        [
            246,
            -3.5236808602898035
        ],
        [
            247,
            -3.4903316311897825
        ],
        [
            248,
            -3.4572332549301397
        ],
        [
            249,
            -3.423112122563646
        ],
        [
            250,
            -3.3901059504037487
        ],
        [
            251,
            -3.3570057771603725
        ],
        [
            252,
            -3.323865483449418
        ],
        [
            253,
            -3.2907033908255245
        ],
        [
            254,
            -3.257576006425423
        ],
        [
            255,
            -3.224582129492079
        ],
        [
            256,
            -3.1916659091344144
        ],
        [
            257,
            -3.1585367701029803
        ],
        [
            258,
            -3.1259359631660715
        ],
        [
            259,
            -3.093418342832904
        ],
        [
            260,
            -3.0605712224397754
        ]
    ]
}

central_afda_metrics = {
    "rmse_physical": [
        [
            1,
            3.1120945241030396
        ],
        [
            2,
            2.9880372263834407
        ],
        [
            3,
            2.691640667657561
        ],
        [
            4,
            2.0755283353468728
        ],
        [
            5,
            1.7594185571022076
        ],
        [
            6,
            1.957134894426517
        ],
        [
            7,
            1.9468933929681091
        ],
        [
            8,
            1.8671497892789375
        ],
        [
            9,
            1.8563796747551489
        ],
        [
            10,
            1.867344193453846
        ],
        [
            11,
            1.885712721476546
        ],
        [
            12,
            1.8888232626022274
        ],
        [
            13,
            1.8725488516756155
        ],
        [
            14,
            1.8754722358322038
        ],
        [
            15,
            1.8731548055727283
        ],
        [
            16,
            1.8767075464867942
        ],
        [
            17,
            1.8817817516136335
        ],
        [
            18,
            1.8888265450889377
        ],
        [
            19,
            1.866333046863568
        ],
        [
            20,
            1.8785168206007485
        ],
        [
            21,
            1.8822162300445135
        ],
        [
            22,
            1.872048968918591
        ],
        [
            23,
            1.8636080975230618
        ],
        [
            24,
            1.8838685968547055
        ],
        [
            25,
            1.886347161175702
        ],
        [
            26,
            1.8431153124527584
        ],
        [
            27,
            1.88285426298357
        ],
        [
            28,
            1.8732093268579406
        ],
        [
            29,
            1.8938099064687999
        ],
        [
            30,
            1.8756296994709534
        ],
        [
            31,
            1.8435907075749596
        ],
        [
            32,
            1.8910692893944896
        ],
        [
            33,
            1.8803849135849093
        ],
        [
            34,
            1.8797627631602216
        ],
        [
            35,
            1.8737672196368427
        ],
        [
            36,
            1.8740976770802904
        ],
        [
            37,
            1.8698636058636078
        ],
        [
            38,
            1.876997086606434
        ],
        [
            39,
            1.8748114434901457
        ],
        [
            40,
            1.882770263828804
        ],
        [
            41,
            1.8911521388886101
        ],
        [
            42,
            1.8721824757913121
        ],
        [
            43,
            1.8695943184309682
        ],
        [
            44,
            1.8726272123288836
        ],
        [
            45,
            1.8764459489161092
        ],
        [
            46,
            1.8777354997404914
        ],
        [
            47,
            1.8988663432770623
        ],
        [
            48,
            1.8977970575073018
        ],
        [
            49,
            1.8765561095126524
        ],
        [
            50,
            1.8964942460304768
        ],
        [
            51,
            1.8920704477475705
        ],
        [
            52,
            1.8792852254425716
        ],
        [
            53,
            1.8766748788328484
        ],
        [
            54,
            1.9311255849003597
        ],
        [
            55,
            1.9221704618094373
        ],
        [
            56,
            1.8749882658165822
        ],
        [
            57,
            1.9136189073891474
        ],
        [
            58,
            1.8897026287547958
        ],
        [
            59,
            1.9235844434957599
        ],
        [
            60,
            1.8923957030902452
        ],
        [
            61,
            1.894490693723606
        ],
        [
            62,
            1.9166019307715774
        ],
        [
            63,
            1.8965190074513296
        ],
        [
            64,
            1.9207518098813237
        ],
        [
            65,
            1.9139151280795579
        ],
        [
            66,
            1.9126454228735112
        ],
        [
            67,
            1.9254067330308189
        ],
        [
            68,
            1.9479080601705223
        ],
        [
            69,
            1.8938156107829847
        ],
        [
            70,
            1.9340641120236384
        ],
        [
            71,
            1.949040831011705
        ],
        [
            72,
            1.9197438957228004
        ],
        [
            73,
            1.9361498946499938
        ],
        [
            74,
            1.9314028394190763
        ],
        [
            75,
            1.929427499801598
        ],
        [
            76,
            1.9498624263121578
        ],
        [
            77,
            1.9485262123791482
        ],
        [
            78,
            1.950369602039864
        ],
        [
            79,
            1.975619498418566
        ],
        [
            80,
            1.937225368820254
        ],
        [
            81,
            1.9698426929967228
        ],
        [
            82,
            1.9641947815577847
        ],
        [
            83,
            1.9684991755917867
        ],
        [
            84,
            1.9796972759268014
        ],
        [
            85,
            1.9450965154712447
        ],
        [
            86,
            1.9731631631982578
        ],
        [
            87,
            1.9631691884610871
        ],
        [
            88,
            2.0024219411452737
        ],
        [
            89,
            1.9682170256890235
        ],
        [
            90,
            1.9964176628490735
        ],
        [
            91,
            1.9809104411158978
        ],
        [
            92,
            1.9968920055813366
        ],
        [
            93,
            1.9695326259427668
        ],
        [
            94,
            1.990309755394277
        ],
        [
            95,
            1.986168311716919
        ],
        [
            96,
            1.9973888101642514
        ],
        [
            97,
            1.99348467206134
        ],
        [
            98,
            1.98891013370879
        ],
        [
            99,
            2.004586634595101
        ],
        [
            100,
            1.9832075287000825
        ],
        [
            101,
            2.024939185055841
        ],
        [
            102,
            2.0001236565218723
        ],
        [
            103,
            1.9889468043225722
        ],
        [
            104,
            2.0301551821553825
        ],
        [
            105,
            1.9908349722165148
        ],
        [
            106,
            1.9973454987105195
        ],
        [
            107,
            1.9995587309520693
        ],
        [
            108,
            2.0025425648978
        ],
        [
            109,
            1.99245140367879
        ],
        [
            110,
            2.0191103089689415
        ],
        [
            111,
            1.9777933632042255
        ],
        [
            112,
            2.0276380073960496
        ],
        [
            113,
            1.9930212461655379
        ],
        [
            114,
            2.023872401144179
        ],
        [
            115,
            2.0051593941824986
        ],
        [
            116,
            2.010820860784974
        ],
        [
            117,
            2.004211187826406
        ],
        [
            118,
            2.0209151283712914
        ],
        [
            119,
            2.009711451295223
        ],
        [
            120,
            1.9908419638638146
        ],
        [
            121,
            2.01593866018583
        ],
        [
            122,
            2.0321055180390974
        ],
        [
            123,
            2.003420641248479
        ],
        [
            124,
            2.0178810600518986
        ],
        [
            125,
            2.024718015406639
        ],
        [
            126,
            2.014274286321058
        ],
        [
            127,
            2.019001965163179
        ],
        [
            128,
            2.004354220430824
        ],
        [
            129,
            2.005261472642897
        ],
        [
            130,
            2.0021435177778497
        ],
        [
            131,
            2.0198617079266312
        ],
        [
            132,
            2.036955107178352
        ],
        [
            133,
            2.001338952115495
        ],
        [
            134,
            2.033626696893581
        ],
        [
            135,
            2.0134543598811967
        ],
        [
            136,
            2.030645058298681
        ],
        [
            137,
            2.0338870779635476
        ],
        [
            138,
            2.0000802469658785
        ],
        [
            139,
            2.0287927229292206
        ],
        [
            140,
            2.0029141894223135
        ],
        [
            141,
            2.0195208622337986
        ],
        [
            142,
            2.0029460595716757
        ],
        [
            143,
            2.0322671393395595
        ],
        [
            144,
            2.029158214198319
        ],
        [
            145,
            2.0011780130467836
        ],
        [
            146,
            2.024109798828717
        ],
        [
            147,
            2.0141386923199613
        ],
        [
            148,
            2.031542985331697
        ],
        [
            149,
            2.0465555972875835
        ],
        [
            150,
            2.0084314331879978
        ],
        [
            151,
            2.0082401177418165
        ],
        [
            152,
            2.0446890057152975
        ],
        [
            153,
            2.0081989814751546
        ],
        [
            154,
            2.0295401420508714
        ],
        [
            155,
            2.0040376864752605
        ],
        [
            156,
            2.029795132957516
        ],
        [
            157,
            1.9901264974583346
        ],
        [
            158,
            2.0472227692804568
        ],
        [
            159,
            2.0259831067289
        ],
        [
            160,
            2.016073769316086
        ],
        [
            161,
            2.021571420758456
        ],
        [
            162,
            2.023110194443177
        ],
        [
            163,
            2.024116361904399
        ],
        [
            164,
            2.0159722753504767
        ],
        [
            165,
            2.041756765277873
        ],
        [
            166,
            2.0170343069797054
        ],
        [
            167,
            2.0410253076510667
        ],
        [
            168,
            2.029400918543849
        ],
        [
            169,
            2.0279815010421585
        ],
        [
            170,
            2.0113459178975277
        ],
        [
            171,
            2.0344553537075623
        ],
        [
            172,
            2.0278403952718596
        ],
        [
            173,
            2.0080896310305834
        ],
        [
            174,
            2.0231160943139037
        ],
        [
            175,
            2.0300879450917684
        ],
        [
            176,
            2.0488083426558483
        ],
        [
            177,
            2.012324150497005
        ],
        [
            178,
            2.0327207130266394
        ],
        [
            179,
            2.0249874155586425
        ],
        [
            180,
            2.0221886020715543
        ],
        [
            181,
            2.0192635662761953
        ],
        [
            182,
            2.0374381138509694
        ],
        [
            183,
            2.0290024751753024
        ],
        [
            184,
            2.018486018215159
        ],
        [
            185,
            2.067399374551906
        ],
        [
            186,
            2.0228871122339913
        ],
        [
            187,
            2.034835940246706
        ],
        [
            188,
            2.019846890417956
        ],
        [
            189,
            2.0514445166963147
        ],
        [
            190,
            2.0360450357001216
        ],
        [
            191,
            2.0299261691634736
        ],
        [
            192,
            2.0348909313020735
        ],
        [
            193,
            2.0311376739653793
        ],
        [
            194,
            2.050909169893257
        ],
        [
            195,
            2.067419824013013
        ],
        [
            196,
            2.027196347438073
        ],
        [
            197,
            2.0389307006533435
        ],
        [
            198,
            2.033717186314242
        ],
        [
            199,
            2.019957843004214
        ],
        [
            200,
            2.04135666852796
        ],
        [
            201,
            2.030953110085994
        ],
        [
            202,
            2.031024538996325
        ],
        [
            203,
            2.045889046490753
        ],
        [
            204,
            2.027268601850332
        ],
        [
            205,
            2.0668094781241706
        ],
        [
            206,
            2.038509169171009
        ],
        [
            207,
            2.010505117661318
        ],
        [
            208,
            2.0686387045236954
        ],
        [
            209,
            2.0476844845517896
        ],
        [
            210,
            2.0097104781547586
        ],
        [
            211,
            2.0365251837279357
        ],
        [
            212,
            2.022863185280517
        ],
        [
            213,
            2.0392815439544396
        ],
        [
            214,
            2.0351401267051497
        ],
        [
            215,
            2.0424367046096985
        ],
        [
            216,
            2.0536306673767135
        ],
        [
            217,
            2.0491144873457268
        ],
        [
            218,
            2.039076110532491
        ],
        [
            219,
            2.0341819562771097
        ],
        [
            220,
            2.0567108775918626
        ],
        [
            221,
            2.02331699206555
        ],
        [
            222,
            2.0400951062380623
        ],
        [
            223,
            2.023767776017201
        ],
        [
            224,
            2.0514681810426567
        ],
        [
            225,
            2.014829125707982
        ],
        [
            226,
            2.054640799343284
        ],
        [
            227,
            2.029854064318817
        ],
        [
            228,
            2.0212126494122273
        ],
        [
            229,
            2.06483440768509
        ],
        [
            230,
            2.0369887381165057
        ],
        [
            231,
            2.0504117249229044
        ],
        [
            232,
            2.0391980021634764
        ],
        [
            233,
            2.0499273620281753
        ],
        [
            234,
            2.024260132711572
        ],
        [
            235,
            2.0334153267043806
        ],
        [
            236,
            2.0432889829051333
        ],
        [
            237,
            2.0805374002705967
        ],
        [
            238,
            2.029808513052165
        ],
        [
            239,
            2.0387927987066954
        ],
        [
            240,
            2.0281053782105287
        ],
        [
            241,
            2.0552791508854895
        ],
        [
            242,
            2.054835695570846
        ],
        [
            243,
            2.062490472433101
        ],
        [
            244,
            2.034635244526805
        ],
        [
            245,
            2.0598898873989735
        ],
        [
            246,
            2.067181112258602
        ],
        [
            247,
            2.04834212555134
        ],
        [
            248,
            2.0428497156184884
        ],
        [
            249,
            2.0508956201707043
        ],
        [
            250,
            2.0604581151658796
        ],
        [
            251,
            2.0389438093302426
        ],
        [
            252,
            2.0747393389014666
        ],
        [
            253,
            2.0724092979178286
        ],
        [
            254,
            2.0608158825410445
        ],
        [
            255,
            2.0617065035389146
        ],
        [
            256,
            2.0815536927279052
        ],
        [
            257,
            2.06063132420141
        ],
        [
            258,
            2.0847241980663695
        ],
        [
            259,
            2.067539023095168
        ],
        [
            260,
            2.0821269825091657
        ]
    ],
    "mae_physical": [
        [
            1,
            2.6039315357904984
        ],
        [
            2,
            2.4894091519149573
        ],
        [
            3,
            2.2084502843586176
        ],
        [
            4,
            1.603683604581936
        ],
        [
            5,
            1.445330742243174
        ],
        [
            6,
            1.6860442242106877
        ],
        [
            7,
            1.6771059551754512
        ],
        [
            8,
            1.5884221502252527
        ],
        [
            9,
            1.5779364044601853
        ],
        [
            10,
            1.5905433007188745
        ],
        [
            11,
            1.6124963438188709
        ],
        [
            12,
            1.6167300150201127
        ],
        [
            13,
            1.5971536539696358
        ],
        [
            14,
            1.6009591850074563
        ],
        [
            15,
            1.5982215533385407
        ],
        [
            16,
            1.6026892790923248
        ],
        [
            17,
            1.6091301070677269
        ],
        [
            18,
            1.6180668908196527
        ],
        [
            19,
            1.590895973347329
        ],
        [
            20,
            1.60561609912563
        ],
        [
            21,
            1.6104373561369407
        ],
        [
            22,
            1.5979402097495827
        ],
        [
            23,
            1.5882414643828933
        ],
        [
            24,
            1.6126575099455345
        ],
        [
            25,
            1.616123481376751
        ],
        [
            26,
            1.5653059337590192
        ],
        [
            27,
            1.6119057068953644
        ],
        [
            28,
            1.5996208319792877
        ],
        [
            29,
            1.624746649651914
        ],
        [
            30,
            1.6027374251468762
        ],
        [
            31,
            1.5648036695815422
        ],
        [
            32,
            1.6214709877967834
        ],
        [
            33,
            1.608031699786315
        ],
        [
            34,
            1.606907862263757
        ],
        [
            35,
            1.600295957681295
        ],
        [
            36,
            1.6005134147566717
        ],
        [
            37,
            1.5949245965158618
        ],
        [
            38,
            1.6032685846895784
        ],
        [
            39,
            1.5999845056920439
        ],
        [
            40,
            1.609301037079579
        ],
        [
            41,
            1.618453230406787
        ],
        [
            42,
            1.5955681027592838
        ],
        [
            43,
            1.5917903797046558
        ],
        [
            44,
            1.5951255994874078
        ],
        [
            45,
            1.5988917205784772
        ],
        [
            46,
            1.5998218494492609
        ],
        [
            47,
            1.6234976874815452
        ],
        [
            48,
            1.62168112799928
        ],
        [
            49,
            1.5952288186227954
        ],
        [
            50,
            1.6176556928737744
        ],
        [
            51,
            1.6113052658132605
        ],
        [
            52,
            1.5943132816134273
        ],
        [
            53,
            1.5901319545668524
        ],
        [
            54,
            1.651616861691346
        ],
        [
            55,
            1.640951390201981
        ],
        [
            56,
            1.5829416062380817
        ],
        [
            57,
            1.6289022129935187
        ],
        [
            58,
            1.5982133423959888
        ],
        [
            59,
            1.6369551033587069
        ],
        [
            60,
            1.5988716885850236
        ],
        [
            61,
            1.5990142709500081
        ],
        [
            62,
            1.624100751168019
        ],
        [
            63,
            1.5979108697659261
        ],
        [
            64,
            1.6259436075751845
        ],
        [
            65,
            1.6161760558953155
        ],
        [
            66,
            1.612453995524226
        ],
        [
            67,
            1.6264315843582153
        ],
        [
            68,
            1.6516649271990802
        ],
        [
            69,
            1.5820353917173438
        ],
        [
            70,
            1.630180695572415
        ],
        [
            71,
            1.6465143899659853
        ],
        [
            72,
            1.6069395171629417
        ],
        [
            73,
            1.6266838296039685
        ],
        [
            74,
            1.6188751894074518
        ],
        [
            75,
            1.612617953403576
        ],
        [
            76,
            1.6368749528317839
        ],
        [
            77,
            1.6329106559624542
        ],
        [
            78,
            1.634168831077782
        ],
        [
            79,
            1.6630140655749552
        ],
        [
            80,
            1.611427664756775
        ],
        [
            81,
            1.6518033517373574
        ],
        [
            82,
            1.6421560032947644
        ],
        [
            83,
            1.6451165128398586
        ],
        [
            84,
            1.6580873180080105
        ],
        [
            85,
            1.6120082897108954
        ],
        [
            86,
            1.649178371236131
        ],
        [
            87,
            1.6306739610594672
        ],
        [
            88,
            1.6826919784417023
        ],
        [
            89,
            1.635595991804793
        ],
        [
            90,
            1.6746711279894855
        ],
        [
            91,
            1.6502425638405052
        ],
        [
            92,
            1.6740209798555117
        ],
        [
            93,
            1.6317685343123771
        ],
        [
            94,
            1.6634755682300877
        ],
        [
            95,
            1.651853616173203
        ],
        [
            96,
            1.6689876817368172
        ],
        [
            97,
            1.6594114400245048
        ],
        [
            98,
            1.6571655064015776
        ],
        [
            99,
            1.6803665032257904
        ],
        [
            100,
            1.6470322737822662
        ],
        [
            101,
            1.7041528853210244
        ],
        [
            102,
            1.6700000440752185
        ],
        [
            103,
            1.6541884058230631
        ],
        [
            104,
            1.7140507069793907
        ],
        [
            105,
            1.6553523943230912
        ],
        [
            106,
            1.6635055912507546
        ],
        [
            107,
            1.6676144519367733
        ],
        [
            108,
            1.6744070681365761
        ],
        [
            109,
            1.657184059555466
        ],
        [
            110,
            1.6926354556470304
        ],
        [
            111,
            1.6360162045504596
        ],
        [
            112,
            1.7065588026433378
        ],
        [
            113,
            1.654891181636501
        ],
        [
            114,
            1.697670606342522
        ],
        [
            115,
            1.6761110859948236
        ],
        [
            116,
            1.6814581155776978
        ],
        [
            117,
            1.6732536602664638
        ],
        [
            118,
            1.6959296690451133
        ],
        [
            119,
            1.674462215320484
        ],
        [
            120,
            1.6448398892943923
        ],
        [
            121,
            1.6844947611963428
        ],
        [
            122,
            1.700368302899438
        ],
        [
            123,
            1.6587787560514502
        ],
        [
            124,
            1.6864558071703524
        ],
        [
            125,
            1.689806873733933
        ],
        [
            126,
            1.6809878703710195
        ],
        [
            127,
            1.685826562546395
        ],
        [
            128,
            1.662212871216439
        ],
        [
            129,
            1.6720329909711271
        ],
        [
            130,
            1.664982729666942
        ],
        [
            131,
            1.6849990487098694
        ],
        [
            132,
            1.7100674316689775
        ],
        [
            133,
            1.6564885007368553
        ],
        [
            134,
            1.7062911890648507
        ],
        [
            135,
            1.6752879313520483
        ],
        [
            136,
            1.698499484642132
        ],
        [
            137,
            1.704292039613466
        ],
        [
            138,
            1.6579509458026371
        ],
        [
            139,
            1.6971622093303784
        ],
        [
            140,
            1.6624325868245717
        ],
        [
            141,
            1.6884681324701052
        ],
        [
            142,
            1.6625131980792895
        ],
        [
            143,
            1.6998966352359668
        ],
        [
            144,
            1.6976122050671965
        ],
        [
            145,
            1.6581862407761652
        ],
        [
            146,
            1.6937025366602718
        ],
        [
            147,
            1.6765457891129159
        ],
        [
            148,
            1.6962340168050818
        ],
        [
            149,
            1.7172878851761688
        ],
        [
            150,
            1.6685235403679513
        ],
        [
            151,
            1.67061194213661
        ],
        [
            152,
            1.7207816823108777
        ],
        [
            153,
            1.6625784087825466
        ],
        [
            154,
            1.6978260246483055
        ],
        [
            155,
            1.6634692034205876
        ],
        [
            156,
            1.6978927786285813
        ],
        [
            157,
            1.6422592296793654
        ],
        [
            158,
            1.719994968659169
        ],
        [
            159,
            1.6884116900933754
        ],
        [
            160,
            1.6792796592454653
        ],
        [
            161,
            1.6806972961168032
        ],
        [
            162,
            1.6893801850241583
        ],
        [
            163,
            1.6849982335760787
        ],
        [
            164,
            1.6761689379408553
        ],
        [
            165,
            1.7068450144819312
        ],
        [
            166,
            1.6774416662551261
        ],
        [
            167,
            1.7115064923827712
        ],
        [
            168,
            1.698564817776551
        ],
        [
            169,
            1.6904011747321568
        ],
        [
            170,
            1.6768000947462547
        ],
        [
            171,
            1.7007618780071672
        ],
        [
            172,
            1.6940915858423389
        ],
        [
            173,
            1.6686598835764705
        ],
        [
            174,
            1.6856966808035567
        ],
        [
            175,
            1.693869411945343
        ],
        [
            176,
            1.7148435019158028
        ],
        [
            177,
            1.6675282595930874
        ],
        [
            178,
            1.6987327844709963
        ],
        [
            179,
            1.6880981430814073
        ],
        [
            180,
            1.6798831860761385
        ],
        [
            181,
            1.6814899476798806
        ],
        [
            182,
            1.703293161617743
        ],
        [
            183,
            1.6900864388491657
        ],
        [
            184,
            1.6741757441211391
        ],
        [
            185,
            1.7400994099475242
        ],
        [
            186,
            1.6826313959585655
        ],
        [
            187,
            1.7000802900340106
        ],
        [
            188,
            1.6813125481476654
        ],
        [
            189,
            1.7170227113607768
        ],
        [
            190,
            1.6936767576514065
        ],
        [
            191,
            1.6905467107489303
        ],
        [
            192,
            1.699400904210838
        ],
        [
            193,
            1.6904187532695565
        ],
        [
            194,
            1.7146007410577826
        ],
        [
            195,
            1.7353985148507196
        ],
        [
            196,
            1.6891995305950578
        ],
        [
            197,
            1.704183904705821
        ],
        [
            198,
            1.6951634279779486
        ],
        [
            199,
            1.6822792090274192
        ],
        [
            200,
            1.7063475887517672
        ],
        [
            201,
            1.6874262106579703
        ],
        [
            202,
            1.6917399780170337
        ],
        [
            203,
            1.7149280958079003
        ],
        [
            204,
            1.6907302705017295
        ],
        [
            205,
            1.7337388468755257
        ],
        [
            206,
            1.7010025527026202
        ],
        [
            207,
            1.6646529492494222
        ],
        [
            208,
            1.7402625708161175
        ],
        [
            209,
            1.7178097902923017
        ],
        [
            210,
            1.6611373529240892
        ],
        [
            211,
            1.6920821497569214
        ],
        [
            212,
            1.6780410280098785
        ],
        [
            213,
            1.6959774518335187
        ],
        [
            214,
            1.6897856117905796
        ],
        [
            215,
            1.7000543446959675
        ],
        [
            216,
            1.714989176473102
        ],
        [
            217,
            1.7105637531022768
        ],
        [
            218,
            1.696256338341816
        ],
        [
            219,
            1.6931625161622021
        ],
        [
            220,
            1.7187122903965615
        ],
        [
            221,
            1.680433443269214
        ],
        [
            222,
            1.6941686292757858
        ],
        [
            223,
            1.6796423961987366
        ],
        [
            224,
            1.713146672458262
        ],
        [
            225,
            1.6623526018213581
        ],
        [
            226,
            1.7145403772592545
        ],
        [
            227,
            1.6815039708807662
        ],
        [
            228,
            1.6669005544201747
        ],
        [
            229,
            1.726156109289543
        ],
        [
            230,
            1.6836781920613468
        ],
        [
            231,
            1.7081345398281071
        ],
        [
            232,
            1.6919910958087123
        ],
        [
            233,
            1.7091224306338542
        ],
        [
            234,
            1.6628344630872882
        ],
        [
            235,
            1.6794996537469529
        ],
        [
            236,
            1.695835731722213
        ],
        [
            237,
            1.7416173584155135
        ],
        [
            238,
            1.6775158698091637
        ],
        [
            239,
            1.6892598750623498
        ],
        [
            240,
            1.6767319191146541
        ],
        [
            241,
            1.708230889427501
        ],
        [
            242,
            1.7063626439084072
        ],
        [
            243,
            1.7133677878693954
        ],
        [
            244,
            1.6770365647216503
        ],
        [
            245,
            1.706925096871281
        ],
        [
            246,
            1.7165017566719167
        ],
        [
            247,
            1.684697983081679
        ],
        [
            248,
            1.6870038504083011
        ],
        [
            249,
            1.703400720162569
        ],
        [
            250,
            1.7129904412337251
        ],
        [
            251,
            1.6791288260672543
        ],
        [
            252,
            1.7202633732074015
        ],
        [
            253,
            1.7129388168655537
        ],
        [
            254,
            1.7016496721755814
        ],
        [
            255,
            1.7035074346774333
        ],
        [
            256,
            1.7351349004217096
        ],
        [
            257,
            1.698591138563446
        ],
        [
            258,
            1.7242592538933497
        ],
        [
            259,
            1.7036104321077064
        ],
        [
            260,
            1.7221546981584381
        ]
    ],
    "r2_physical": [
        [
            1,
            -2.188502872224697
        ],
        [
            2,
            -1.9393632591576533
        ],
        [
            3,
            -1.3851484614039546
        ],
        [
            4,
            -0.4182036125548432
        ],
        [
            5,
            -0.019106543915892837
        ],
        [
            6,
            -0.261022321421045
        ],
        [
            7,
            -0.24785923172515356
        ],
        [
            8,
            -0.14772957726446223
        ],
        [
            9,
            -0.13452707137655628
        ],
        [
            10,
            -0.14796858865858398
        ],
        [
            11,
            -0.1706641411444192
        ],
        [
            12,
            -0.1745294194698659
        ],
        [
            13,
            -0.1543767384017789
        ],
        [
            14,
            -0.1579839296480059
        ],
        [
            15,
            -0.15512396831906816
        ],
        [
            16,
            -0.15950988237614117
        ],
        [
            17,
            -0.16578847913446393
        ],
        [
            18,
            -0.17453350177935079
        ],
        [
            19,
            -0.14672570022103937
        ],
        [
            20,
            -0.16174665332585714
        ],
        [
            21,
            -0.16632687145539116
        ],
        [
            22,
            -0.1537604917248836
        ],
        [
            23,
            -0.14337957903670717
        ],
        [
            24,
            -0.16837556884736138
        ],
        [
            25,
            -0.17145200319655785
        ],
        [
            26,
            -0.11837195568077852
        ],
        [
            27,
            -0.16711772754992182
        ],
        [
            28,
            -0.15519121290525706
        ],
        [
            29,
            -0.18073930841203945
        ],
        [
            30,
            -0.1581783852248908
        ],
        [
            31,
            -0.1189489539137576
        ],
        [
            32,
            -0.1773243797534192
        ],
        [
            33,
            -0.16405840252362291
        ],
        [
            34,
            -0.16328824146198895
        ],
        [
            35,
            -0.1558794102118526
        ],
        [
            36,
            -0.1562871477716059
        ],
        [
            37,
            -0.1510683464796838
        ],
        [
            38,
            -0.15986769041996074
        ],
        [
            39,
            -0.15716807954081924
        ],
        [
            40,
            -0.1670135933981609
        ],
        [
            41,
            -0.17742754135252103
        ],
        [
            42,
            -0.1539250605478666
        ],
        [
            43,
            -0.150736829337057
        ],
        [
            44,
            -0.15447335495514913
        ],
        [
            45,
            -0.15918665265557874
        ],
        [
            46,
            -0.1607804568904443
        ],
        [
            47,
            -0.1870528291290916
        ],
        [
            48,
            -0.18571630397535643
        ],
        [
            49,
            -0.15932276149642566
        ],
        [
            50,
            -0.18408890701727554
        ],
        [
            51,
            -0.17857129349753764
        ],
        [
            52,
            -0.16269726960013364
        ],
        [
            53,
            -0.15946951579054103
        ],
        [
            54,
            -0.22772836337617353
        ],
        [
            55,
            -0.21636818396145885
        ],
        [
            56,
            -0.15738636581342513
        ],
        [
            57,
            -0.20556924544005128
        ],
        [
            58,
            -0.1756233088384338
        ],
        [
            59,
            -0.21815840493548078
        ],
        [
            60,
            -0.1789765316418419
        ],
        [
            61,
            -0.18158836591309213
        ],
        [
            62,
            -0.209330751155288
        ],
        [
            63,
            -0.18411982713761899
        ],
        [
            64,
            -0.21457337302688528
        ],
        [
            65,
            -0.20594250909542988
        ],
        [
            66,
            -0.2043429777915018
        ],
        [
            67,
            -0.22046751987120694
        ],
        [
            68,
            -0.24916027037942
        ],
        [
            69,
            -0.18074642139429398
        ],
        [
            70,
            -0.23146758985657878
        ],
        [
            71,
            -0.25061354612344133
        ],
        [
            72,
            -0.21329901316702937
        ],
        [
            73,
            -0.23412516332704159
        ],
        [
            74,
            -0.22808092218906606
        ],
        [
            75,
            -0.22557017060639217
        ],
        [
            76,
            -0.25166813129244847
        ],
        [
            77,
            -0.24995321714817442
        ],
        [
            78,
            -0.2523193549349334
        ],
        [
            79,
            -0.2849548303372975
        ],
        [
            80,
            -0.2354965843820891
        ],
        [
            81,
            -0.277451278635678
        ],
        [
            82,
            -0.2701363915857966
        ],
        [
            83,
            -0.27570931949832755
        ],
        [
            84,
            -0.290264726776134
        ],
        [
            85,
            -0.24555688128017628
        ],
        [
            86,
            -0.2817615862237377
        ],
        [
            87,
            -0.2688103489310989
        ],
        [
            88,
            -0.32005627033400486
        ],
        [
            89,
            -0.2753436445015853
        ],
        [
            90,
            -0.31215174033869664
        ],
        [
            91,
            -0.29184656848714163
        ],
        [
            92,
            -0.3127753408953875
        ],
        [
            93,
            -0.27704915070755654
        ],
        [
            94,
            -0.3041351397768357
        ],
        [
            95,
            -0.2987134881951945
        ],
        [
            96,
            -0.3134286300397573
        ],
        [
            97,
            -0.30829913770000905
        ],
        [
            98,
            -0.30230160203022627
        ],
        [
            99,
            -0.3229118739855876
        ],
        [
            100,
            -0.29484438728500884
        ],
        [
            101,
            -0.34991126924495
        ],
        [
            102,
            -0.31702781374064637
        ],
        [
            103,
            -0.3023496249534545
        ],
        [
            104,
            -0.35687464065736574
        ],
        [
            105,
            -0.30482351914814476
        ],
        [
            106,
            -0.31337166978616415
        ],
        [
            107,
            -0.3162839421176755
        ],
        [
            108,
            -0.3202153126752101
        ],
        [
            109,
            -0.30694324686997887
        ],
        [
            110,
            -0.3421508972658738
        ],
        [
            111,
            -0.28778417563084724
        ],
        [
            112,
            -0.35351196847745303
        ],
        [
            113,
            -0.30769092712780055
        ],
        [
            114,
            -0.34848931612583467
        ],
        [
            115,
            -0.3236679587506597
        ],
        [
            116,
            -0.3311531305434219
        ],
        [
            117,
            -0.3224163738491026
        ],
        [
            118,
            -0.34455138286231035
        ],
        [
            119,
            -0.3296846889174392
        ],
        [
            120,
            -0.3048326840281885
        ],
        [
            121,
            -0.3379376674151331
        ],
        [
            122,
            -0.35948294615244336
        ],
        [
            123,
            -0.32137334448792143
        ],
        [
            124,
            -0.34051717243850743
        ],
        [
            125,
            -0.3496164030096376
        ],
        [
            126,
            -0.33572935692705275
        ],
        [
            127,
            -0.342006863694176
        ],
        [
            128,
            -0.32260513180896244
        ],
        [
            129,
            -0.32380273250180025
        ],
        [
            130,
            -0.31968920587789107
        ],
        [
            131,
            -0.3431500288462317
        ],
        [
            132,
            -0.36597945988214176
        ],
        [
            133,
            -0.31862877911827203
        ],
        [
            134,
            -0.36151905178067056
        ],
        [
            135,
            -0.334642139641143
        ],
        [
            136,
            -0.357529546959914
        ],
        [
            137,
            -0.361867725884532
        ],
        [
            138,
            -0.316970646302994
        ],
        [
            139,
            -0.3550540251178391
        ],
        [
            140,
            -0.3207053595991243
        ],
        [
            141,
            -0.34269676189871423
        ],
        [
            142,
            -0.3207473897691344
        ],
        [
            143,
            -0.3596992047447396
        ],
        [
            144,
            -0.35554230075175774
        ],
        [
            145,
            -0.3184167107381526
        ],
        [
            146,
            -0.3488056868776781
        ],
        [
            147,
            -0.3355495295887052
        ],
        [
            148,
            -0.3587303791572376
        ],
        [
            149,
            -0.3788859560453215
        ],
        [
            150,
            -0.32799143244992823
        ],
        [
            151,
            -0.32773844579717437
        ],
        [
            152,
            -0.3763718360633872
        ],
        [
            153,
            -0.32768405225838526
        ],
        [
            154,
            -0.35605262870832344
        ],
        [
            155,
            -0.32218742482630214
        ],
        [
            156,
            -0.3563933983280134
        ],
        [
            157,
            -0.30389499413055887
        ],
        [
            158,
            -0.3797851293136387
        ],
        [
            159,
            -0.35130347389543415
        ],
        [
            160,
            -0.33811701181257425
        ],
        [
            161,
            -0.3454248110504974
        ],
        [
            162,
            -0.3474738034400435
        ],
        [
            163,
            -0.3488144337630097
        ],
        [
            164,
            -0.3379822871953255
        ],
        [
            165,
            -0.37242702168594777
        ],
        [
            166,
            -0.3393923798021288
        ],
        [
            167,
            -0.3714438561971565
        ],
        [
            168,
            -0.355866588606673
        ],
        [
            169,
            -0.3539705928864598
        ],
        [
            170,
            -0.3318483915522401
        ],
        [
            171,
            -0.3626288541901759
        ],
        [
            172,
            -0.3537821824727623
        ],
        [
            173,
            -0.3275394660993878
        ],
        [
            174,
            -0.34748166255998814
        ],
        [
            175,
            -0.356784765009865
        ],
        [
            176,
            -0.38192324325384663
        ],
        [
            177,
            -0.3331442147417356
        ],
        [
            178,
            -0.3603062042791514
        ],
        [
            179,
            -0.3499755750523532
        ],
        [
            180,
            -0.3462464469306452
        ],
        [
            181,
            -0.34235465259115894
        ],
        [
            182,
            -0.366627343986478
        ],
        [
            183,
            -0.3553342314837604
        ],
        [
            184,
            -0.34132106359386016
        ],
        [
            185,
            -0.40711636771742654
        ],
        [
            186,
            -0.3471766561460061
        ],
        [
            187,
            -0.36313871714294654
        ],
        [
            188,
            -0.343130322483042
        ],
        [
            189,
            -0.3854817350825199
        ],
        [
            190,
            -0.3647591470368643
        ],
        [
            191,
            -0.35656853164943403
        ],
        [
            192,
            -0.3632123952691104
        ],
        [
            193,
            -0.3581882750179828
        ],
        [
            194,
            -0.38475871631943726
        ],
        [
            195,
            -0.4071442045389917
        ],
        [
            196,
            -0.35292238894557415
        ],
        [
            197,
            -0.3686304056605503
        ],
        [
            198,
            -0.3616402203426887
        ],
        [
            199,
            -0.3432778860203527
        ],
        [
            200,
            -0.3718892007272214
        ],
        [
            201,
            -0.3579414565859025
        ],
        [
            202,
            -0.3580369762547144
        ],
        [
            203,
            -0.3779879126579824
        ],
        [
            204,
            -0.3530188338254987
        ],
        [
            205,
            -0.40631348994899597
        ],
        [
            206,
            -0.36806455889904854
        ],
        [
            207,
            -0.3307351226966446
        ],
        [
            208,
            -0.40880390250748455
        ],
        [
            209,
            -0.38040757227849564
        ],
        [
            210,
            -0.3296834012005767
        ],
        [
            211,
            -0.36540290851314783
        ],
        [
            212,
            -0.34714478719756525
        ],
        [
            213,
            -0.3691014526870309
        ],
        [
            214,
            -0.3635462972562171
        ],
        [
            215,
            -0.373341256435302
        ],
        [
            216,
            -0.38843622465617145
        ],
        [
            217,
            -0.3823362638986678
        ],
        [
            218,
            -0.36882562512365125
        ],
        [
            219,
            -0.3622626488614269
        ],
        [
            220,
            -0.3926043380091009
        ],
        [
            221,
            -0.34774928880038436
        ],
        [
            222,
            -0.3701940644357815
        ],
        [
            223,
            -0.3483498980290267
        ],
        [
            224,
            -0.38551369959188575
        ],
        [
            225,
            -0.3364653215710067
        ],
        [
            226,
            -0.3898024379343912
        ],
        [
            227,
            -0.35647216023703066
        ],
        [
            228,
            -0.34494730426244646
        ],
        [
            229,
            -0.40362699069395447
        ],
        [
            230,
            -0.3660245659817407
        ],
        [
            231,
            -0.38408705544826427
        ],
        [
            232,
            -0.3689892809813611
        ],
        [
            233,
            -0.3834332148354733
        ],
        [
            234,
            -0.34900605024365094
        ],
        [
            235,
            -0.36123604056312186
        ],
        [
            236,
            -0.37448764511117827
        ],
        [
            237,
            -0.42505723730955736
        ],
        [
            238,
            -0.3564112806567006
        ],
        [
            239,
            -0.3684452788050201
        ],
        [
            240,
            -0.35413600974623316
        ],
        [
            241,
            -0.3906661610223703
        ],
        [
            242,
            -0.3900661142899555
        ],
        [
            243,
            -0.4004420927540808
        ],
        [
            244,
            -0.36286983785950633
        ],
        [
            245,
            -0.39691269688519926
        ],
        [
            246,
            -0.4068192754020523
        ],
        [
            247,
            -0.3812943870202137
        ],
        [
            248,
            -0.37389673255369815
        ],
        [
            249,
            -0.3847404190347956
        ],
        [
            250,
            -0.3976834896042918
        ],
        [
            251,
            -0.3686480040923643
        ],
        [
            252,
            -0.4171255786793555
        ],
        [
            253,
            -0.4139443534738716
        ],
        [
            254,
            -0.3981689049227384
        ],
        [
            255,
            -0.39937765692497185
        ],
        [
            256,
            -0.42644978968092695
        ],
        [
            257,
            -0.39791848742631797
        ],
        [
            258,
            -0.43079847494276335
        ],
        [
            259,
            -0.4073064697267572
        ],
        [
            260,
            -0.42723562739766585
        ]
    ],
    "rmse_mental": [
        [
            1,
            6.668373651343851
        ],
        [
            2,
            6.4888284711096915
        ],
        [
            3,
            6.075378124631557
        ],
        [
            4,
            5.173044350633896
        ],
        [
            5,
            3.9370688824009292
        ],
        [
            6,
            3.09529312377254
        ],
        [
            7,
            2.7453934658838377
        ],
        [
            8,
            2.6229781334591515
        ],
        [
            9,
            2.5776697752034647
        ],
        [
            10,
            2.5587440413796183
        ],
        [
            11,
            2.5484461389055735
        ],
        [
            12,
            2.5429628158830866
        ],
        [
            13,
            2.5409489408221337
        ],
        [
            14,
            2.539829155785087
        ],
        [
            15,
            2.5390537175464556
        ],
        [
            16,
            2.538771506189229
        ],
        [
            17,
            2.538764230936933
        ],
        [
            18,
            2.538127284432984
        ],
        [
            19,
            2.5383881631391483
        ],
        [
            20,
            2.5383817776709705
        ],
        [
            21,
            2.537901187726369
        ],
        [
            22,
            2.5376315447013837
        ],
        [
            23,
            2.538389638176884
        ],
        [
            24,
            2.5382339453263207
        ],
        [
            25,
            2.537354019970257
        ],
        [
            26,
            2.5387010605843323
        ],
        [
            27,
            2.537696961442995
        ],
        [
            28,
            2.537429370596075
        ],
        [
            29,
            2.5366136987116765
        ],
        [
            30,
            2.5367456529307146
        ],
        [
            31,
            2.537753040829434
        ],
        [
            32,
            2.536230479408421
        ],
        [
            33,
            2.535836493184252
        ],
        [
            34,
            2.535254018011378
        ],
        [
            35,
            2.535568227060169
        ],
        [
            36,
            2.535651601396084
        ],
        [
            37,
            2.5354078969918943
        ],
        [
            38,
            2.5347628908816207
        ],
        [
            39,
            2.5343664780155772
        ],
        [
            40,
            2.534018510571213
        ],
        [
            41,
            2.532666475857594
        ],
        [
            42,
            2.5330133509694597
        ],
        [
            43,
            2.532560794185911
        ],
        [
            44,
            2.5325149081653375
        ],
        [
            45,
            2.5320216417519825
        ],
        [
            46,
            2.5310672807221866
        ],
        [
            47,
            2.529342611605868
        ],
        [
            48,
            2.529153231418774
        ],
        [
            49,
            2.5287858581643365
        ],
        [
            50,
            2.5275994637871597
        ],
        [
            51,
            2.527532569690133
        ],
        [
            52,
            2.5269117101628877
        ],
        [
            53,
            2.5268620264998094
        ],
        [
            54,
            2.5238150739260634
        ],
        [
            55,
            2.523194193671524
        ],
        [
            56,
            2.524326405570171
        ],
        [
            57,
            2.521149884303831
        ],
        [
            58,
            2.521094871374743
        ],
        [
            59,
            2.5192478990034117
        ],
        [
            60,
            2.519501414618356
        ],
        [
            61,
            2.5196152271320527
        ],
        [
            62,
            2.516581323779857
        ],
        [
            63,
            2.516448896357163
        ],
        [
            64,
            2.5141078723891237
        ],
        [
            65,
            2.5133471678437322
        ],
        [
            66,
            2.5119312667931104
        ],
        [
            67,
            2.5104147487736643
        ],
        [
            68,
            2.5104722436983455
        ],
        [
            69,
            2.5100007300391205
        ],
        [
            70,
            2.5074171023829286
        ],
        [
            71,
            2.5065191157051006
        ],
        [
            72,
            2.5057607105926976
        ],
        [
            73,
            2.5048610235859905
        ],
        [
            74,
            2.5051295165838594
        ],
        [
            75,
            2.503886519735375
        ],
        [
            76,
            2.5031755606378567
        ],
        [
            77,
            2.5028125274094224
        ],
        [
            78,
            2.5021798834519857
        ],
        [
            79,
            2.5012575355827407
        ],
        [
            80,
            2.501738963005635
        ],
        [
            81,
            2.50060995424248
        ],
        [
            82,
            2.500787535971996
        ],
        [
            83,
            2.4995305813650748
        ],
        [
            84,
            2.5002954354366267
        ],
        [
            85,
            2.501116242993942
        ],
        [
            86,
            2.5024203584940206
        ],
        [
            87,
            2.5021928416485797
        ],
        [
            88,
            2.5020108991546963
        ],
        [
            89,
            2.50429533395773
        ],
        [
            90,
            2.5064576611151943
        ],
        [
            91,
            2.5053755320342104
        ],
        [
            92,
            2.505916960879146
        ],
        [
            93,
            2.508934954985393
        ],
        [
            94,
            2.5080184504955536
        ],
        [
            95,
            2.511755725370707
        ],
        [
            96,
            2.5130575888582123
        ],
        [
            97,
            2.5174241870754788
        ],
        [
            98,
            2.5175284465103926
        ],
        [
            99,
            2.519130592564967
        ],
        [
            100,
            2.518752173954719
        ],
        [
            101,
            2.5193552165570923
        ],
        [
            102,
            2.5168012218401903
        ],
        [
            103,
            2.519703227948605
        ],
        [
            104,
            2.527376794429034
        ],
        [
            105,
            2.525271908049306
        ],
        [
            106,
            2.5250844538354067
        ],
        [
            107,
            2.526421320239133
        ],
        [
            108,
            2.5221656358460027
        ],
        [
            109,
            2.5248086190470738
        ],
        [
            110,
            2.5251479514570216
        ],
        [
            111,
            2.532442466111317
        ],
        [
            112,
            2.5254550400516482
        ],
        [
            113,
            2.5248788013450056
        ],
        [
            114,
            2.528258833742588
        ],
        [
            115,
            2.526839150808806
        ],
        [
            116,
            2.5249152578865774
        ],
        [
            117,
            2.5292726133798724
        ],
        [
            118,
            2.5267728738709665
        ],
        [
            119,
            2.532613152265764
        ],
        [
            120,
            2.530921003058321
        ],
        [
            121,
            2.5265033846235068
        ],
        [
            122,
            2.5235048459171368
        ],
        [
            123,
            2.5237741885529585
        ],
        [
            124,
            2.532440037266331
        ],
        [
            125,
            2.524347310475422
        ],
        [
            126,
            2.526502246685489
        ],
        [
            127,
            2.527752518481156
        ],
        [
            128,
            2.524860134783846
        ],
        [
            129,
            2.525667012846333
        ],
        [
            130,
            2.526362356520809
        ],
        [
            131,
            2.5240849617656993
        ],
        [
            132,
            2.522642155772152
        ],
        [
            133,
            2.527886460938996
        ],
        [
            134,
            2.522849773186392
        ],
        [
            135,
            2.5227307994185786
        ],
        [
            136,
            2.5219116754887545
        ],
        [
            137,
            2.517054365795673
        ],
        [
            138,
            2.521912716564606
        ],
        [
            139,
            2.5235649007921666
        ],
        [
            140,
            2.5224008922005203
        ],
        [
            141,
            2.523981504222707
        ],
        [
            142,
            2.529460400689855
        ],
        [
            143,
            2.521506124416737
        ],
        [
            144,
            2.520914963771866
        ],
        [
            145,
            2.5208330689557505
        ],
        [
            146,
            2.5253472279202698
        ],
        [
            147,
            2.5244915748400754
        ],
        [
            148,
            2.525180622921284
        ],
        [
            149,
            2.5188116856486205
        ],
        [
            150,
            2.5223220319247224
        ],
        [
            151,
            2.520034204121258
        ],
        [
            152,
            2.521594181880902
        ],
        [
            153,
            2.5204005159347083
        ],
        [
            154,
            2.5277059344937785
        ],
        [
            155,
            2.5206436969017485
        ],
        [
            156,
            2.5261923881520847
        ],
        [
            157,
            2.522059422117778
        ],
        [
            158,
            2.522103871077966
        ],
        [
            159,
            2.5233576636151036
        ],
        [
            160,
            2.523798941841595
        ],
        [
            161,
            2.5201829406415124
        ],
        [
            162,
            2.528787343964439
        ],
        [
            163,
            2.5206788060694043
        ],
        [
            164,
            2.5276634574650054
        ],
        [
            165,
            2.519372737186395
        ],
        [
            166,
            2.5236050951573574
        ],
        [
            167,
            2.5278418948580352
        ],
        [
            168,
            2.519778958229163
        ],
        [
            169,
            2.52125650555518
        ],
        [
            170,
            2.5239019286509157
        ],
        [
            171,
            2.517983825280787
        ],
        [
            172,
            2.523104582616552
        ],
        [
            173,
            2.5214248972019178
        ],
        [
            174,
            2.5231173733449843
        ],
        [
            175,
            2.5168948943777654
        ],
        [
            176,
            2.5228621356702874
        ],
        [
            177,
            2.5306782653430617
        ],
        [
            178,
            2.5256802883742036
        ],
        [
            179,
            2.523880740638863
        ],
        [
            180,
            2.522083923584913
        ],
        [
            181,
            2.5238331841188044
        ],
        [
            182,
            2.5271711952147276
        ],
        [
            183,
            2.5227064044530785
        ],
        [
            184,
            2.52178364166774
        ],
        [
            185,
            2.522822504338475
        ],
        [
            186,
            2.527669294599334
        ],
        [
            187,
            2.5228728775054376
        ],
        [
            188,
            2.524883928484303
        ],
        [
            189,
            2.5244485664196166
        ],
        [
            190,
            2.5235485612853394
        ],
        [
            191,
            2.530092742028517
        ],
        [
            192,
            2.525183465631838
        ],
        [
            193,
            2.5247210430563727
        ],
        [
            194,
            2.529063382069507
        ],
        [
            195,
            2.5253906302862723
        ],
        [
            196,
            2.5281809509024247
        ],
        [
            197,
            2.525305202393271
        ],
        [
            198,
            2.5280437735264845
        ],
        [
            199,
            2.527935263803206
        ],
        [
            200,
            2.5267845389181227
        ],
        [
            201,
            2.5280007038270855
        ],
        [
            202,
            2.5233392784725157
        ],
        [
            203,
            2.5323324580166315
        ],
        [
            204,
            2.5335361188880094
        ],
        [
            205,
            2.5261749585924207
        ],
        [
            206,
            2.5336521162086347
        ],
        [
            207,
            2.5267115322196374
        ],
        [
            208,
            2.529165587624962
        ],
        [
            209,
            2.5341225513531773
        ],
        [
            210,
            2.536094542866455
        ],
        [
            211,
            2.5316139384046115
        ],
        [
            212,
            2.5338461918945425
        ],
        [
            213,
            2.5258833891719723
        ],
        [
            214,
            2.530067993158641
        ],
        [
            215,
            2.5341164368164693
        ],
        [
            216,
            2.5339692285476865
        ],
        [
            217,
            2.5326825772274058
        ],
        [
            218,
            2.5257988431625042
        ],
        [
            219,
            2.5317398605197887
        ],
        [
            220,
            2.5302706307641154
        ],
        [
            221,
            2.534544168957027
        ],
        [
            222,
            2.5302953284795895
        ],
        [
            223,
            2.5336097007813954
        ],
        [
            224,
            2.5293282705169338
        ],
        [
            225,
            2.527518412144036
        ],
        [
            226,
            2.5336019995783565
        ],
        [
            227,
            2.53950669130861
        ],
        [
            228,
            2.5364426932178628
        ],
        [
            229,
            2.535384938086111
        ],
        [
            230,
            2.537605158329037
        ],
        [
            231,
            2.5364724852849165
        ],
        [
            232,
            2.5314865853661024
        ],
        [
            233,
            2.5348539905437417
        ],
        [
            234,
            2.532106681958772
        ],
        [
            235,
            2.53322672809031
        ],
        [
            236,
            2.534102211875877
        ],
        [
            237,
            2.532499359034013
        ],
        [
            238,
            2.537557966039704
        ],
        [
            239,
            2.533696723222914
        ],
        [
            240,
            2.5359707581491904
        ],
        [
            241,
            2.5379649930690626
        ],
        [
            242,
            2.533663122050307
        ],
        [
            243,
            2.5373280195771364
        ],
        [
            244,
            2.540891579685112
        ],
        [
            245,
            2.5368853418513466
        ],
        [
            246,
            2.531909237598485
        ],
        [
            247,
            2.5382442281336535
        ],
        [
            248,
            2.532775769279788
        ],
        [
            249,
            2.536855474568147
        ],
        [
            250,
            2.5283999506655013
        ],
        [
            251,
            2.536244218971219
        ],
        [
            252,
            2.528705106284117
        ],
        [
            253,
            2.526280446497201
        ],
        [
            254,
            2.532217684072576
        ],
        [
            255,
            2.5287234009712884
        ],
        [
            256,
            2.527980873608467
        ],
        [
            257,
            2.5250875166181204
        ],
        [
            258,
            2.5290209038536227
        ],
        [
            259,
            2.527755698432721
        ],
        [
            260,
            2.5282943372766105
        ]
    ],
    "mae_mental": [
        [
            1,
            6.172441057722412
        ],
        [
            2,
            5.977451916683364
        ],
        [
            3,
            5.523726606691206
        ],
        [
            4,
            4.571196286259471
        ],
        [
            5,
            3.286922630426046
        ],
        [
            6,
            2.5704769169962085
        ],
        [
            7,
            2.4137700586705595
        ],
        [
            8,
            2.3308443427085876
        ],
        [
            9,
            2.2854592171875208
        ],
        [
            10,
            2.2590093177718087
        ],
        [
            11,
            2.2406925107981706
        ],
        [
            12,
            2.233958909640441
        ],
        [
            13,
            2.23098744089539
        ],
        [
            14,
            2.2289867385013684
        ],
        [
            15,
            2.227432123712591
        ],
        [
            16,
            2.226854886557605
        ],
        [
            17,
            2.2267578985240006
        ],
        [
            18,
            2.225045911363653
        ],
        [
            19,
            2.2255278970744157
        ],
        [
            20,
            2.225485332914301
        ],
        [
            21,
            2.2242573354695296
        ],
        [
            22,
            2.2234517996375627
        ],
        [
            23,
            2.2253062773395227
        ],
        [
            24,
            2.2250572652430147
        ],
        [
            25,
            2.2230027156907157
        ],
        [
            26,
            2.225692850512427
        ],
        [
            27,
            2.2237512082666964
        ],
        [
            28,
            2.223148482876855
        ],
        [
            29,
            2.221140215525756
        ],
        [
            30,
            2.221353213529329
        ],
        [
            31,
            2.2234961003870577
        ],
        [
            32,
            2.220491871640489
        ],
        [
            33,
            2.2195011425662683
        ],
        [
            34,
            2.21789895038347
        ],
        [
            35,
            2.2185683008786796
        ],
        [
            36,
            2.219244272322268
        ],
        [
            37,
            2.2190832173502124
        ],
        [
            38,
            2.217875008647506
        ],
        [
            39,
            2.217116131975844
        ],
        [
            40,
            2.2165428222836674
        ],
        [
            41,
            2.2131373833965613
        ],
        [
            42,
            2.214834749698639
        ],
        [
            43,
            2.2137931988045976
        ],
        [
            44,
            2.2142182282499365
        ],
        [
            45,
            2.2133522371988037
        ],
        [
            46,
            2.210965193606712
        ],
        [
            47,
            2.2081135881913676
        ],
        [
            48,
            2.2070138019484444
        ],
        [
            49,
            2.206776933090107
        ],
        [
            50,
            2.2052875000077323
        ],
        [
            51,
            2.205317753392297
        ],
        [
            52,
            2.203521059977042
        ],
        [
            53,
            2.203848705098436
        ],
        [
            54,
            2.198614689143928
        ],
        [
            55,
            2.197290296490128
        ],
        [
            56,
            2.2001289435335107
        ],
        [
            57,
            2.1932387335880383
        ],
        [
            58,
            2.1935880554688945
        ],
        [
            59,
            2.1896747946739197
        ],
        [
            60,
            2.1912654815493404
        ],
        [
            61,
            2.191884280862035
        ],
        [
            62,
            2.186892208215353
        ],
        [
            63,
            2.186839881780985
        ],
        [
            64,
            2.1825602457330033
        ],
        [
            65,
            2.180623835808522
        ],
        [
            66,
            2.1779720734905554
        ],
        [
            67,
            2.1758015751838684
        ],
        [
            68,
            2.1705927639394194
        ],
        [
            69,
            2.1746862466270858
        ],
        [
            70,
            2.1704375792194055
        ],
        [
            71,
            2.1662052789249935
        ],
        [
            72,
            2.1709744334220886
        ],
        [
            73,
            2.16388393898268
        ],
        [
            74,
            2.1630607923945866
        ],
        [
            75,
            2.164783679150246
        ],
        [
            76,
            2.1599774151235014
        ],
        [
            77,
            2.162116116768605
        ],
        [
            78,
            2.1632135574882096
        ],
        [
            79,
            2.1571854046873145
        ],
        [
            80,
            2.166054363186295
        ],
        [
            81,
            2.1569324039124154
        ],
        [
            82,
            2.15588499243195
        ],
        [
            83,
            2.153211086182981
        ],
        [
            84,
            2.153955979927166
        ],
        [
            85,
            2.1619644438898242
        ],
        [
            86,
            2.1577270594803064
        ],
        [
            87,
            2.1603503597749247
        ],
        [
            88,
            2.1585599425676705
        ],
        [
            89,
            2.1638214249868652
        ],
        [
            90,
            2.157115470718693
        ],
        [
            91,
            2.164506849404928
        ],
        [
            92,
            2.15931348704003
        ],
        [
            93,
            2.1649118388021313
        ],
        [
            94,
            2.1603409519066683
        ],
        [
            95,
            2.1668499208785392
        ],
        [
            96,
            2.1617968485162065
        ],
        [
            97,
            2.1688722323726966
        ],
        [
            98,
            2.1670099384075887
        ],
        [
            99,
            2.1644811807452022
        ],
        [
            100,
            2.1736782837558435
        ],
        [
            101,
            2.164444177537351
        ],
        [
            102,
            2.1664575837753914
        ],
        [
            103,
            2.170377053119041
        ],
        [
            104,
            2.1633390204326526
        ],
        [
            105,
            2.1703537525357426
        ],
        [
            106,
            2.17546003412556
        ],
        [
            107,
            2.1750599484185913
        ],
        [
            108,
            2.1679241125648088
        ],
        [
            109,
            2.1717211410805985
        ],
        [
            110,
            2.1687488282049023
        ],
        [
            111,
            2.1795559751020894
        ],
        [
            112,
            2.1666016304815137
        ],
        [
            113,
            2.1740830927281767
        ],
        [
            114,
            2.168392524525926
        ],
        [
            115,
            2.177019739473188
        ],
        [
            116,
            2.17120972678468
        ],
        [
            117,
            2.1729984428431535
        ],
        [
            118,
            2.168643099230689
        ],
        [
            119,
            2.1727070663426375
        ],
        [
            120,
            2.1766169924993775
        ],
        [
            121,
            2.1683125866426005
        ],
        [
            122,
            2.1648246903677246
        ],
        [
            123,
            2.1719827958055444
        ],
        [
            124,
            2.172070263205348
        ],
        [
            125,
            2.1645735937195854
        ],
        [
            126,
            2.1678995715605245
        ],
        [
            127,
            2.168030009076402
        ],
        [
            128,
            2.172777557695234
        ],
        [
            129,
            2.164781981223338
        ],
        [
            130,
            2.168739897173804
        ],
        [
            131,
            2.1669665011199744
        ],
        [
            132,
            2.1598029152767078
        ],
        [
            133,
            2.169242528644768
        ],
        [
            134,
            2.161168863644471
        ],
        [
            135,
            2.163930691577293
        ],
        [
            136,
            2.162791546937582
        ],
        [
            137,
            2.1601875714353613
        ],
        [
            138,
            2.165635916026863
        ],
        [
            139,
            2.159126428333489
        ],
        [
            140,
            2.165150260603106
        ],
        [
            141,
            2.16301524478036
        ],
        [
            142,
            2.1683634406811483
        ],
        [
            143,
            2.160835380489762
        ],
        [
            144,
            2.1603091520232125
        ],
        [
            145,
            2.1662379197172217
        ],
        [
            146,
            2.165635081561836
        ],
        [
            147,
            2.1649095802693754
        ],
        [
            148,
            2.164783598603429
        ],
        [
            149,
            2.159413632508871
        ],
        [
            150,
            2.1654091477394104
        ],
        [
            151,
            2.165549141329688
        ],
        [
            152,
            2.1612273148588232
        ],
        [
            153,
            2.1630484010722184
        ],
        [
            154,
            2.165484297919918
        ],
        [
            155,
            2.163558259203627
        ],
        [
            156,
            2.165501225639034
        ],
        [
            157,
            2.1678342190948694
        ],
        [
            158,
            2.164621167891734
        ],
        [
            159,
            2.1654984290535384
        ],
        [
            160,
            2.167231516258137
        ],
        [
            161,
            2.164953626490928
        ],
        [
            162,
            2.1705126198562414
        ],
        [
            163,
            2.163548722460463
        ],
        [
            164,
            2.1677346084568954
        ],
        [
            165,
            2.163038503479313
        ],
        [
            166,
            2.1672456151730306
        ],
        [
            167,
            2.16992849111557
        ],
        [
            168,
            2.1667099240663887
        ],
        [
            169,
            2.1659432633503064
        ],
        [
            170,
            2.168744217705082
        ],
        [
            171,
            2.1619166474084595
        ],
        [
            172,
            2.165868261375943
        ],
        [
            173,
            2.164474158673673
        ],
        [
            174,
            2.1666059767877734
        ],
        [
            175,
            2.1632938529994035
        ],
        [
            176,
            2.165651993171589
        ],
        [
            177,
            2.168774664402008
        ],
        [
            178,
            2.1676945235278153
        ],
        [
            179,
            2.167563362701519
        ],
        [
            180,
            2.164703080783019
        ],
        [
            181,
            2.1678342384261056
        ],
        [
            182,
            2.169905773691229
        ],
        [
            183,
            2.1680499717995927
        ],
        [
            184,
            2.166799237599244
        ],
        [
            185,
            2.1660715776520805
        ],
        [
            186,
            2.168366401582151
        ],
        [
            187,
            2.165530166110477
        ],
        [
            188,
            2.169358192263423
        ],
        [
            189,
            2.1680694383543893
        ],
        [
            190,
            2.1654866337776184
        ],
        [
            191,
            2.1696872115135193
        ],
        [
            192,
            2.1671788144756006
        ],
        [
            193,
            2.1637249427872733
        ],
        [
            194,
            2.1693027728312724
        ],
        [
            195,
            2.1670802815540418
        ],
        [
            196,
            2.1686558707340344
        ],
        [
            197,
            2.1687668996888236
        ],
        [
            198,
            2.168900989197396
        ],
        [
            199,
            2.1719341664700895
        ],
        [
            200,
            2.1688697918041333
        ],
        [
            201,
            2.166721532473693
        ],
        [
            202,
            2.1637414484410673
        ],
        [
            203,
            2.1725109364535355
        ],
        [
            204,
            2.1732031541901664
        ],
        [
            205,
            2.166449440492166
        ],
        [
            206,
            2.17204472342053
        ],
        [
            207,
            2.1666294078569153
        ],
        [
            208,
            2.1690978842812614
        ],
        [
            209,
            2.174321799664884
        ],
        [
            210,
            2.168346097340455
        ],
        [
            211,
            2.16683279179238
        ],
        [
            212,
            2.169549846971357
        ],
        [
            213,
            2.165941064422195
        ],
        [
            214,
            2.168332712070362
        ],
        [
            215,
            2.1691402100227974
        ],
        [
            216,
            2.1719627203168095
        ],
        [
            217,
            2.1698591403059058
        ],
        [
            218,
            2.164687048744511
        ],
        [
            219,
            2.170509882875391
        ],
        [
            220,
            2.1686805390022896
        ],
        [
            221,
            2.1715230555147738
        ],
        [
            222,
            2.1661337050231726
        ],
        [
            223,
            2.1693290165952734
        ],
        [
            224,
            2.1648718875807687
        ],
        [
            225,
            2.163191651170318
        ],
        [
            226,
            2.16795615247778
        ],
        [
            227,
            2.1695052610861287
        ],
        [
            228,
            2.165197208121016
        ],
        [
            229,
            2.1670658000417657
        ],
        [
            230,
            2.165092022032351
        ],
        [
            231,
            2.169047176032453
        ],
        [
            232,
            2.1639802488120825
        ],
        [
            233,
            2.1671859098447337
        ],
        [
            234,
            2.157804680031699
        ],
        [
            235,
            2.161627806521751
        ],
        [
            236,
            2.161742337652155
        ],
        [
            237,
            2.1619837090775773
        ],
        [
            238,
            2.1632047037820556
        ],
        [
            239,
            2.162492073870994
        ],
        [
            240,
            2.1645673972529336
        ],
        [
            241,
            2.1645518501062653
        ],
        [
            242,
            2.161638430646948
        ],
        [
            243,
            2.1595123184693827
        ],
        [
            244,
            2.1625183860997894
        ],
        [
            245,
            2.157892363296973
        ],
        [
            246,
            2.153695230548446
        ],
        [
            247,
            2.155361041829393
        ],
        [
            248,
            2.156953312255241
        ],
        [
            249,
            2.1629828488504566
        ],
        [
            250,
            2.1567206471352964
        ],
        [
            251,
            2.155922346018456
        ],
        [
            252,
            2.1506394421732105
        ],
        [
            253,
            2.1456496377249024
        ],
        [
            254,
            2.1537456528560535
        ],
        [
            255,
            2.1486474629994987
        ],
        [
            256,
            2.1485778641056372
        ],
        [
            257,
            2.143830147143957
        ],
        [
            258,
            2.147125906235463
        ],
        [
            259,
            2.142469628437145
        ],
        [
            260,
            2.145995459846548
        ]
    ],
    "r2_mental": [
        [
            1,
            -5.918170870006452
        ],
        [
            2,
            -5.550644311067579
        ],
        [
            3,
            -4.742461033788562
        ],
        [
            4,
            -3.1633587438520756
        ],
        [
            5,
            -1.4115565873316385
        ],
        [
            6,
            -0.4905789936921552
        ],
        [
            7,
            -0.1726289776383798
        ],
        [
            8,
            -0.07038686445681686
        ],
        [
            9,
            -0.03372730776228905
        ],
        [
            10,
            -0.01860339498610397
        ],
        [
            11,
            -0.010420966174857327
        ],
        [
            12,
            -0.00607753200439709
        ],
        [
            13,
            -0.0044846560429667726
        ],
        [
            14,
            -0.003599507175167327
        ],
        [
            15,
            -0.0029867804244021467
        ],
        [
            16,
            -0.0027638323803047538
        ],
        [
            17,
            -0.002758085231021834
        ],
        [
            18,
            -0.002254987600062597
        ],
        [
            19,
            -0.0024610296085290084
        ],
        [
            20,
            -0.0024559861127819715
        ],
        [
            21,
            -0.002076433545500578
        ],
        [
            22,
            -0.0018635107035385179
        ],
        [
            23,
            -0.0024621946535747785
        ],
        [
            24,
            -0.0023392258170880087
        ],
        [
            25,
            -0.0016443877200060797
        ],
        [
            26,
            -0.002708183949945653
        ],
        [
            27,
            -0.0019151647675772487
        ],
        [
            28,
            -0.0017038793412256226
        ],
        [
            29,
            -0.0010599754157420982
        ],
        [
            30,
            -0.0011641280686187638
        ],
        [
            31,
            -0.0019594469725694452
        ],
        [
            32,
            -0.0007575276852505031
        ],
        [
            33,
            -0.00044663002976075283
        ],
        [
            34,
            1.2917262767864202e-05
        ],
        [
            35,
            -0.0002349667429220048
        ],
        [
            36,
            -0.00030074710433969365
        ],
        [
            37,
            -0.00010847622408816093
        ],
        [
            38,
            0.0004003129339799072
        ],
        [
            39,
            0.0007129442992840351
        ],
        [
            40,
            0.000987328840836521
        ],
        [
            41,
            0.0020530980684421163
        ],
        [
            42,
            0.001779720857618794
        ],
        [
            43,
            0.0021363798566025327
        ],
        [
            44,
            0.002172538969469673
        ],
        [
            45,
            0.0025612015104488073
        ],
        [
            46,
            0.0033129623221075377
        ],
        [
            47,
            0.004670784547981421
        ],
        [
            48,
            0.004819826101730373
        ],
        [
            49,
            0.0051089157636886595
        ],
        [
            50,
            0.006042214488364617
        ],
        [
            51,
            0.006094824903625007
        ],
        [
            52,
            0.006583047857315005
        ],
        [
            53,
            0.0066221122280472056
        ],
        [
            54,
            0.009016346974505152
        ],
        [
            55,
            0.009503868034769325
        ],
        [
            56,
            0.008614754451057305
        ],
        [
            57,
            0.011108231456517181
        ],
        [
            58,
            0.011151387351005848
        ],
        [
            59,
            0.012599731940931114
        ],
        [
            60,
            0.012400994864395831
        ],
        [
            61,
            0.012311767953568253
        ],
        [
            62,
            0.014688913885979593
        ],
        [
            63,
            0.014792609143779112
        ],
        [
            64,
            0.01662481111110814
        ],
        [
            65,
            0.017219809291298405
        ],
        [
            66,
            0.018326801245438507
        ],
        [
            67,
            0.019511766553866883
        ],
        [
            68,
            0.019466854658169663
        ],
        [
            69,
            0.019835145010673516
        ],
        [
            70,
            0.0218519393999852
        ],
        [
            71,
            0.02255242647997413
        ],
        [
            72,
            0.023143835564656157
        ],
        [
            73,
            0.02384518507322242
        ],
        [
            74,
            0.02363590816981831
        ],
        [
            75,
            0.024604573777177285
        ],
        [
            76,
            0.02515840702346217
        ],
        [
            77,
            0.02544114726221247
        ],
        [
            78,
            0.02593376973320749
        ],
        [
            79,
            0.026651753543805556
        ],
        [
            80,
            0.026277028728336815
        ],
        [
            81,
            0.02715569250817429
        ],
        [
            82,
            0.02701751381407902
        ],
        [
            83,
            0.02799535575219181
        ],
        [
            84,
            0.027400399674034293
        ],
        [
            85,
            0.02676171663815785
        ],
        [
            86,
            0.025746533095976742
        ],
        [
            87,
            0.025923680790782222
        ],
        [
            88,
            0.026065332088370585
        ],
        [
            89,
            0.024286038514260477
        ],
        [
            90,
            0.02260035581993447
        ],
        [
            91,
            0.023444131701110815
        ],
        [
            92,
            0.0230220052448461
        ],
        [
            93,
            0.020667346735910752
        ],
        [
            94,
            0.02138270911214879
        ],
        [
            95,
            0.018464001088959625
        ],
        [
            96,
            0.017446261171392585
        ],
        [
            97,
            0.014028794846198567
        ],
        [
            98,
            0.013947124915569042
        ],
        [
            99,
            0.012691684552732396
        ],
        [
            100,
            0.012988285117878862
        ],
        [
            101,
            0.012515605532945728
        ],
        [
            102,
            0.014516714036283962
        ],
        [
            103,
            0.012242774174014981
        ],
        [
            104,
            0.00621733257688073
        ],
        [
            105,
            0.007871956093506216
        ],
        [
            106,
            0.008019244533415626
        ],
        [
            107,
            0.006968589141153059
        ],
        [
            108,
            0.010311237440883914
        ],
        [
            109,
            0.008235956379481602
        ],
        [
            110,
            0.007969353758047193
        ],
        [
            111,
            0.002229623440528905
        ],
        [
            112,
            0.00772805318724934
        ],
        [
            113,
            0.008180819331501055
        ],
        [
            114,
            0.005523563179291258
        ],
        [
            115,
            0.006640098253805959
        ],
        [
            116,
            0.008152177514542669
        ],
        [
            117,
            0.004725874210398273
        ],
        [
            118,
            0.006692207616616708
        ],
        [
            119,
            0.0020951198297598506
        ],
        [
            120,
            0.003428161818925912
        ],
        [
            121,
            0.006904075883382599
        ],
        [
            122,
            0.009259955940938758
        ],
        [
            123,
            0.009048454246255333
        ],
        [
            124,
            0.002231537346528256
        ],
        [
            125,
            0.008598334307929445
        ],
        [
            126,
            0.006904970464691518
        ],
        [
            127,
            0.005921835834328482
        ],
        [
            128,
            0.008195484419542765
        ],
        [
            129,
            0.007561474506192489
        ],
        [
            130,
            0.0070149409833024645
        ],
        [
            131,
            0.008804391082563812
        ],
        [
            132,
            0.009937232688529196
        ],
        [
            133,
            0.005816483115399396
        ],
        [
            134,
            0.009774258538640779
        ],
        [
            135,
            0.009867651423788648
        ],
        [
            136,
            0.010510533675959777
        ],
        [
            137,
            0.01431846104680512
        ],
        [
            138,
            0.010509716729187879
        ],
        [
            139,
            0.00921279971882083
        ],
        [
            140,
            0.010126601328349527
        ],
        [
            141,
            0.008885643942335153
        ],
        [
            142,
            0.004578079316299166
        ],
        [
            143,
            0.010828749616604627
        ],
        [
            144,
            0.011292512572375402
        ],
        [
            145,
            0.011356750124087434
        ],
        [
            146,
            0.007812771915811756
        ],
        [
            147,
            0.008485015496628279
        ],
        [
            148,
            0.007943682942630703
        ],
        [
            149,
            0.012941643424755123
        ],
        [
            150,
            0.010188495113121254
        ],
        [
            151,
            0.011983262995014399
        ],
        [
            152,
            0.010759659613581607
        ],
        [
            153,
            0.01169600617402411
        ],
        [
            154,
            0.005958475258169615
        ],
        [
            155,
            0.011505283855186788
        ],
        [
            156,
            0.007148548399327992
        ],
        [
            157,
            0.010394591458857683
        ],
        [
            158,
            0.010359709394910377
        ],
        [
            159,
            0.009375521518877994
        ],
        [
            160,
            0.009029015557928233
        ],
        [
            161,
            0.011866630841441128
        ],
        [
            162,
            0.005107746657412093
        ],
        [
            163,
            0.011477746866554228
        ],
        [
            164,
            0.005991883871959591
        ],
        [
            165,
            0.01250187074234188
        ],
        [
            166,
            0.009181237717231006
        ],
        [
            167,
            0.0058515372788106035
        ],
        [
            168,
            0.012183398724463057
        ],
        [
            169,
            0.011024587792196838
        ],
        [
            170,
            0.008948138257913407
        ],
        [
            171,
            0.01359037172200972
        ],
        [
            172,
            0.00957422157902843
        ],
        [
            173,
            0.010892478457649513
        ],
        [
            174,
            0.009564179744566137
        ],
        [
            175,
            0.014443355491142285
        ],
        [
            176,
            0.009764553894389238
        ],
        [
            177,
            0.003619312763864402
        ],
        [
            178,
            0.007551041476134723
        ],
        [
            179,
            0.008964777835238857
        ],
        [
            180,
            0.010375363599308796
        ],
        [
            181,
            0.009002124879125883
        ],
        [
            182,
            0.006379012169275722
        ],
        [
            183,
            0.009886800615373725
        ],
        [
            184,
            0.010611001033775769
        ],
        [
            185,
            0.009795664624333345
        ],
        [
            186,
            0.00598729293990341
        ],
        [
            187,
            0.0097561214331644
        ],
        [
            188,
            0.008176791256761073
        ],
        [
            189,
            0.008518799035874491
        ],
        [
            190,
            0.009225629919310863
        ],
        [
            191,
            0.004080324848560246
        ],
        [
            192,
            0.007941449335637119
        ],
        [
            193,
            0.00830475621775617
        ],
        [
            194,
            0.004890533326298407
        ],
        [
            195,
            0.007778666789357902
        ],
        [
            196,
            0.005584831792599165
        ],
        [
            197,
            0.007845794577986065
        ],
        [
            198,
            0.005692741444026761
        ],
        [
            199,
            0.005778095733471433
        ],
        [
            200,
            0.006683036227209449
        ],
        [
            201,
            0.005726620722849907
        ],
        [
            202,
            0.009389956813724676
        ],
        [
            203,
            0.0023163068977097856
        ],
        [
            204,
            0.0013676492937061413
        ],
        [
            205,
            0.007162248784141068
        ],
        [
            206,
            0.0012762029330793956
        ],
        [
            207,
            0.006740435457979088
        ],
        [
            208,
            0.004810102150403339
        ],
        [
            209,
            0.0009052929813577748
        ],
        [
            210,
            -0.0006502536239609835
        ],
        [
            211,
            0.0028823886522429953
        ],
        [
            212,
            0.0011231942163406616
        ],
        [
            213,
            0.0073914208934424686
        ],
        [
            214,
            0.0040998085342668356
        ],
        [
            215,
            0.0009101143692479274
        ],
        [
            216,
            0.0010261864000912224
        ],
        [
            217,
            0.0020404091784121947
        ],
        [
            218,
            0.007457868687659297
        ],
        [
            219,
            0.002783193210596946
        ],
        [
            220,
            0.003940275342140698
        ],
        [
            221,
            0.0005728142241828715
        ],
        [
            222,
            0.0039208303709689085
        ],
        [
            223,
            0.0013096415742364087
        ],
        [
            224,
            0.004682071326028603
        ],
        [
            225,
            0.006105959255833016
        ],
        [
            226,
            0.0013157128373292304
        ],
        [
            227,
            -0.003344683227883616
        ],
        [
            228,
            -0.0009250073003737036
        ],
        [
            229,
            -9.036372049853192e-05
        ],
        [
            230,
            -0.0018426759954814909
        ],
        [
            231,
            -0.000948520388319718
        ],
        [
            232,
            0.002982706288838788
        ],
        [
            233,
            0.0003284601934447906
        ],
        [
            234,
            0.002494200640706712
        ],
        [
            235,
            0.0016115367141440462
        ],
        [
            236,
            0.0009213308659915542
        ],
        [
            237,
            0.002184791851022827
        ],
        [
            238,
            -0.0018054134531615418
        ],
        [
            239,
            0.0012410359276628924
        ],
        [
            240,
            -0.0005525741536163142
        ],
        [
            241,
            -0.0021268205615809777
        ],
        [
            242,
            0.0012675262723904623
        ],
        [
            243,
            -0.0016238600252418411
        ],
        [
            244,
            -0.004439304693550028
        ],
        [
            245,
            -0.001274391694622956
        ],
        [
            246,
            0.0026497582379029216
        ],
        [
            247,
            -0.002347347118938359
        ],
        [
            248,
            0.0019669664072269333
        ],
        [
            249,
            -0.001250815406579564
        ],
        [
            250,
            0.005412544980261891
        ],
        [
            251,
            -0.0007683705548180431
        ],
        [
            252,
            0.005172454590170128
        ],
        [
            253,
            0.007079329301061588
        ],
        [
            254,
            0.0024067417038236005
        ],
        [
            255,
            0.005158059772423784
        ],
        [
            256,
            0.005742219279540994
        ],
        [
            257,
            0.008016838100357027
        ],
        [
            258,
            0.004923960815751971
        ],
        [
            259,
            0.005919334701500412
        ],
        [
            260,
            0.005495632751090218
        ]
    ],
    "loss": [
        [
            1,
            56.24404678344727
        ],
        [
            2,
            53.03194961547852
        ],
        [
            3,
            45.933372497558594
        ],
        [
            4,
            32.300193786621094
        ],
        [
            5,
            19.009371280670166
        ],
        [
            6,
            13.330563354492188
        ],
        [
            7,
            11.147183895111084
        ],
        [
            8,
            10.216047763824463
        ],
        [
            9,
            9.923449754714966
        ],
        [
            10,
            9.835224723815918
        ],
        [
            11,
            9.816204261779784
        ],
        [
            12,
            9.787170314788819
        ],
        [
            13,
            9.72992000579834
        ],
        [
            14,
            9.728571605682372
        ],
        [
            15,
            9.7158203125
        ],
        [
            16,
            9.722889804840088
        ],
        [
            17,
            9.735736656188966
        ],
        [
            18,
            9.74778470993042
        ],
        [
            19,
            9.691593647003174
        ],
        [
            20,
            9.72304277420044
        ],
        [
            21,
            9.728353881835938
        ],
        [
            22,
            9.699231052398682
        ],
        [
            23,
            9.68549976348877
        ],
        [
            24,
            9.735977077484131
        ],
        [
            25,
            9.734923267364502
        ],
        [
            26,
            9.636860752105713
        ],
        [
            27,
            9.729293823242188
        ],
        [
            28,
            9.702488803863526
        ],
        [
            29,
            9.74959602355957
        ],
        [
            30,
            9.702720642089844
        ],
        [
            31,
            9.632618808746338
        ],
        [
            32,
            9.740937995910645
        ],
        [
            33,
            9.70973606109619
        ],
        [
            34,
            9.704435062408447
        ],
        [
            35,
            9.691360473632812
        ],
        [
            36,
            9.694743251800537
        ],
        [
            37,
            9.683565235137939
        ],
        [
            38,
            9.696875095367432
        ],
        [
            39,
            9.68948802947998
        ],
        [
            40,
            9.708005428314209
        ],
        [
            41,
            9.718386554718018
        ],
        [
            42,
            9.674964904785156
        ],
        [
            43,
            9.665527439117431
        ],
        [
            44,
            9.676520442962646
        ],
        [
            45,
            9.682985401153564
        ],
        [
            46,
            9.67986602783203
        ],
        [
            47,
            9.72428092956543
        ],
        [
            48,
            9.720508575439453
        ],
        [
            49,
            9.663731002807618
        ],
        [
            50,
            9.71046028137207
        ],
        [
            51,
            9.69992799758911
        ],
        [
            52,
            9.662683391571045
        ],
        [
            53,
            9.658747005462647
        ],
        [
            54,
            9.783912849426269
        ],
        [
            55,
            9.756454753875733
        ],
        [
            56,
            9.64341468811035
        ],
        [
            57,
            9.71998348236084
        ],
        [
            58,
            9.65960750579834
        ],
        [
            59,
            9.736178970336914
        ],
        [
            60,
            9.658978271484376
        ],
        [
            61,
            9.670804214477538
        ],
        [
            62,
            9.707218170166016
        ],
        [
            63,
            9.65474739074707
        ],
        [
            64,
            9.70604362487793
        ],
        [
            65,
            9.683950710296632
        ],
        [
            66,
            9.67420253753662
        ],
        [
            67,
            9.700361442565917
        ],
        [
            68,
            9.75584716796875
        ],
        [
            69,
            9.621959495544434
        ],
        [
            70,
            9.712571144104004
        ],
        [
            71,
            9.746634483337402
        ],
        [
            72,
            9.670386028289794
        ],
        [
            73,
            9.706214809417725
        ],
        [
            74,
            9.694504737854004
        ],
        [
            75,
            9.685879802703857
        ],
        [
            76,
            9.733797740936279
        ],
        [
            77,
            9.731049728393554
        ],
        [
            78,
            9.734226036071778
        ],
        [
            79,
            9.794957637786865
        ],
        [
            80,
            9.69883680343628
        ],
        [
            81,
            9.775950717926026
        ],
        [
            82,
            9.758141231536865
        ],
        [
            83,
            9.763261318206787
        ],
        [
            84,
            9.797431659698486
        ],
        [
            85,
            9.710434722900391
        ],
        [
            86,
            9.787677383422851
        ],
        [
            87,
            9.759605216979981
        ],
        [
            88,
            9.871683597564697
        ],
        [
            89,
            9.785015106201172
        ],
        [
            90,
            9.866643142700195
        ],
        [
            91,
            9.822901248931885
        ],
        [
            92,
            9.863006973266602
        ],
        [
            93,
            9.801848983764648
        ],
        [
            94,
            9.850160026550293
        ],
        [
            95,
            9.856854343414307
        ],
        [
            96,
            9.888718032836914
        ],
        [
            97,
            9.9013822555542
        ],
        [
            98,
            9.88458709716797
        ],
        [
            99,
            9.934324741363525
        ],
        [
            100,
            9.877593421936036
        ],
        [
            101,
            9.994778156280518
        ],
        [
            102,
            9.903355503082276
        ],
        [
            103,
            9.886181735992432
        ],
        [
            104,
            10.039531421661376
        ],
        [
            105,
            9.91189260482788
        ],
        [
            106,
            9.934745979309081
        ],
        [
            107,
            9.946347904205322
        ],
        [
            108,
            9.929244995117188
        ],
        [
            109,
            9.91301679611206
        ],
        [
            110,
            9.992144107818604
        ],
        [
            111,
            9.906390857696532
        ],
        [
            112,
            10.015811729431153
        ],
        [
            113,
            9.911795711517334
        ],
        [
            114,
            10.019500637054444
        ],
        [
            115,
            9.959870338439941
        ],
        [
            116,
            9.964559173583984
        ],
        [
            117,
            9.9610746383667
        ],
        [
            118,
            10.001373672485352
        ],
        [
            119,
            9.993529415130615
        ],
        [
            120,
            9.93161096572876
        ],
        [
            121,
            9.985159301757813
        ],
        [
            122,
            10.019492149353027
        ],
        [
            123,
            9.93345775604248
        ],
        [
            124,
            10.014532279968261
        ],
        [
            125,
            9.995794773101807
        ],
        [
            126,
            9.97600965499878
        ],
        [
            127,
            9.997241401672364
        ],
        [
            128,
            9.941267395019532
        ],
        [
            129,
            9.941828632354737
        ],
        [
            130,
            9.938812160491944
        ],
        [
            131,
            9.9812650680542
        ],
        [
            132,
            10.024409532546997
        ],
        [
            133,
            9.945748996734618
        ],
        [
            134,
            10.01438274383545
        ],
        [
            135,
            9.95412826538086
        ],
        [
            136,
            10.003746223449706
        ],
        [
            137,
            9.990125846862792
        ],
        [
            138,
            9.91122817993164
        ],
        [
            139,
            9.999153804779052
        ],
        [
            140,
            9.919285011291503
        ],
        [
            141,
            9.977191257476807
        ],
        [
            142,
            9.9533842086792
        ],
        [
            143,
            10.002132225036622
        ],
        [
            144,
            9.987093925476074
        ],
        [
            145,
            9.905659580230713
        ],
        [
            146,
            9.99445400238037
        ],
        [
            147,
            9.960314178466797
        ],
        [
            148,
            10.018497848510743
        ],
        [
            149,
            10.03306565284729
        ],
        [
            150,
            9.934696006774903
        ],
        [
            151,
            9.929592037200928
        ],
        [
            152,
            10.039708423614503
        ],
        [
            153,
            9.92161169052124
        ],
        [
            154,
            10.02179183959961
        ],
        [
            155,
            9.913895797729491
        ],
        [
            156,
            10.012370443344116
        ],
        [
            157,
            9.878693580627441
        ],
        [
            158,
            10.052884006500244
        ],
        [
            159,
            9.988854217529298
        ],
        [
            160,
            9.961781024932861
        ],
        [
            161,
            9.962958431243896
        ],
        [
            162,
            10.014430809020997
        ],
        [
            163,
            9.977435207366943
        ],
        [
            164,
            9.983245849609375
        ],
        [
            165,
            10.016951894760131
        ],
        [
            166,
            9.962857818603515
        ],
        [
            167,
            10.060521650314332
        ],
        [
            168,
            9.987027168273926
        ],
        [
            169,
            9.986117935180664
        ],
        [
            170,
            9.951277446746825
        ],
        [
            171,
            9.993050146102906
        ],
        [
            172,
            9.99282102584839
        ],
        [
            173,
            9.923431968688964
        ],
        [
            174,
            9.981048774719238
        ],
        [
            175,
            9.974079036712647
        ],
        [
            176,
            10.058802270889283
        ],
        [
            177,
            9.986283302307129
        ],
        [
            178,
            10.021275091171265
        ],
        [
            179,
            9.992592525482177
        ],
        [
            180,
            9.978241539001464
        ],
        [
            181,
            9.973357486724854
        ],
        [
            182,
            10.041106367111206
        ],
        [
            183,
            9.999142932891846
        ],
        [
            184,
            9.970480251312257
        ],
        [
            185,
            10.114927053451538
        ],
        [
            186,
            10.00245246887207
        ],
        [
            187,
            10.013759088516235
        ],
        [
            188,
            9.97979211807251
        ],
        [
            189,
            10.076819229125977
        ],
        [
            190,
            10.02332181930542
        ],
        [
            191,
            10.036367893218994
        ],
        [
            192,
            10.024905109405518
        ],
        [
            193,
            10.005843019485473
        ],
        [
            194,
            10.094133853912354
        ],
        [
            195,
            10.126434087753296
        ],
        [
            196,
            10.011103057861328
        ],
        [
            197,
            10.042728805541993
        ],
        [
            198,
            10.02954568862915
        ],
        [
            199,
            9.993926334381104
        ],
        [
            200,
            10.05272970199585
        ],
        [
            201,
            10.026333284378051
        ],
        [
            202,
            10.002649307250977
        ],
        [
            203,
            10.093387413024903
        ],
        [
            204,
            10.047361850738525
        ],
        [
            205,
            10.128912305831909
        ],
        [
            206,
            10.070947885513306
        ],
        [
            207,
            9.956128692626953
        ],
        [
            208,
            10.147594451904297
        ],
        [
            209,
            10.104160451889038
        ],
        [
            210,
            9.994015884399413
        ],
        [
            211,
            10.059651613235474
        ],
        [
            212,
            10.03226728439331
        ],
        [
            213,
            10.042394924163819
        ],
        [
            214,
            10.051157522201539
        ],
        [
            215,
            10.085017347335816
        ],
        [
            216,
            10.126281213760375
        ],
        [
            217,
            10.106265449523926
        ],
        [
            218,
            10.039331817626953
        ],
        [
            219,
            10.061237335205078
        ],
        [
            220,
            10.114375734329224
        ],
        [
            221,
            10.033073139190673
        ],
        [
            222,
            10.064112281799316
        ],
        [
            223,
            10.03352336883545
        ],
        [
            224,
            10.088872814178467
        ],
        [
            225,
            9.980541515350343
        ],
        [
            226,
            10.123343467712402
        ],
        [
            227,
            10.08277416229248
        ],
        [
            228,
            10.038012504577637
        ],
        [
            229,
            10.168711948394776
        ],
        [
            230,
            10.091273927688599
        ],
        [
            231,
            10.131543111801147
        ],
        [
            232,
            10.066075849533082
        ],
        [
            233,
            10.125335264205933
        ],
        [
            234,
            10.03634090423584
        ],
        [
            235,
            10.056766462326049
        ],
        [
            236,
            10.096015691757202
        ],
        [
            237,
            10.206417131423951
        ],
        [
            238,
            10.076543998718261
        ],
        [
            239,
            10.08580493927002
        ],
        [
            240,
            10.072582149505616
        ],
        [
            241,
            10.154409742355346
        ],
        [
            242,
            10.142774868011475
        ],
        [
            243,
            10.175311708450318
        ],
        [
            244,
            10.114941883087159
        ],
        [
            245,
            10.179628276824952
        ],
        [
            246,
            10.175094079971313
        ],
        [
            247,
            10.15837345123291
        ],
        [
            248,
            10.109818840026856
        ],
        [
            249,
            10.158855819702149
        ],
        [
            250,
            10.149281406402588
        ],
        [
            251,
            10.11054162979126
        ],
        [
            252,
            10.20332956314087
        ],
        [
            253,
            10.176532459259032
        ],
        [
            254,
            10.187439060211181
        ],
        [
            255,
            10.170526504516602
        ],
        [
            256,
            10.216801357269286
        ],
        [
            257,
            10.153626918792725
        ],
        [
            258,
            10.273392581939698
        ],
        [
            259,
            10.191865062713623
        ],
        [
            260,
            10.249030590057373
        ]
    ]
}

central_base_metrics = {
    "rmse_physical": [
        [
            1,
            3.745070382236059
        ],
        [
            2,
            3.66393776305368
        ],
        [
            3,
            3.5509030034768965
        ],
        [
            4,
            3.3810367960340524
        ],
        [
            5,
            3.132976326292539
        ],
        [
            6,
            2.806293521381464
        ],
        [
            7,
            2.4284654829114447
        ],
        [
            8,
            2.086226586061466
        ],
        [
            9,
            1.8818763535132337
        ],
        [
            10,
            1.7984658126621005
        ],
        [
            11,
            1.7666519521904747
        ],
        [
            12,
            1.7651319268464287
        ],
        [
            13,
            1.7722808901769953
        ],
        [
            14,
            1.7734734235609186
        ],
        [
            15,
            1.7808216078139343
        ],
        [
            16,
            1.7885283776894476
        ],
        [
            17,
            1.797902232233391
        ],
        [
            18,
            1.803650243636819
        ],
        [
            19,
            1.8039637542393472
        ],
        [
            20,
            1.8172393171675654
        ],
        [
            21,
            1.8127042009124725
        ],
        [
            22,
            1.8195847277354504
        ],
        [
            23,
            1.8187283339887277
        ],
        [
            24,
            1.827313423453215
        ],
        [
            25,
            1.828709122386978
        ],
        [
            26,
            1.832928489003437
        ],
        [
            27,
            1.8387105644780999
        ],
        [
            28,
            1.843817921456105
        ],
        [
            29,
            1.8438291359846415
        ],
        [
            30,
            1.8465668934009418
        ],
        [
            31,
            1.854444751932713
        ],
        [
            32,
            1.8590529792060986
        ],
        [
            33,
            1.8617120556485112
        ],
        [
            34,
            1.8624843369811388
        ],
        [
            35,
            1.852943247193425
        ],
        [
            36,
            1.860217050455046
        ],
        [
            37,
            1.868368375422805
        ],
        [
            38,
            1.8611749371077893
        ],
        [
            39,
            1.8721659508875528
        ],
        [
            40,
            1.868257465160152
        ],
        [
            41,
            1.881408361132501
        ],
        [
            42,
            1.8733654669635709
        ],
        [
            43,
            1.8721199258288697
        ],
        [
            44,
            1.8717024316560589
        ],
        [
            45,
            1.870554252455409
        ],
        [
            46,
            1.871226264522245
        ],
        [
            47,
            1.8815656989168352
        ],
        [
            48,
            1.8781430511001778
        ],
        [
            49,
            1.8878294460362905
        ],
        [
            50,
            1.8810533606899997
        ],
        [
            51,
            1.8816556414113244
        ],
        [
            52,
            1.885600005140226
        ],
        [
            53,
            1.8813912660644274
        ],
        [
            54,
            1.8843090248611498
        ],
        [
            55,
            1.881938389996991
        ],
        [
            56,
            1.8847247505769116
        ],
        [
            57,
            1.890736941188051
        ],
        [
            58,
            1.899338626765313
        ],
        [
            59,
            1.891457728429109
        ],
        [
            60,
            1.8976837818888337
        ],
        [
            61,
            1.9017609248995324
        ],
        [
            62,
            1.905784584905733
        ],
        [
            63,
            1.8898211631238808
        ],
        [
            64,
            1.9032879320902663
        ],
        [
            65,
            1.9149734261567153
        ],
        [
            66,
            1.9050320026958003
        ],
        [
            67,
            1.912228158317211
        ],
        [
            68,
            1.9095782254317697
        ],
        [
            69,
            1.9130197020816395
        ],
        [
            70,
            1.9068250676022078
        ],
        [
            71,
            1.9150065356090744
        ],
        [
            72,
            1.919544878555811
        ],
        [
            73,
            1.9152018529372918
        ],
        [
            74,
            1.9224792121432834
        ],
        [
            75,
            1.9276159812759608
        ],
        [
            76,
            1.9311266587233833
        ],
        [
            77,
            1.933487308569203
        ],
        [
            78,
            1.9362993681142453
        ],
        [
            79,
            1.9283478368581815
        ],
        [
            80,
            1.9463870188652352
        ],
        [
            81,
            1.9488592009324788
        ],
        [
            82,
            1.9460652057728802
        ],
        [
            83,
            1.9447490716837568
        ],
        [
            84,
            1.9527185666703986
        ],
        [
            85,
            1.951065450221303
        ],
        [
            86,
            1.9625930590730591
        ],
        [
            87,
            1.9476218241800585
        ],
        [
            88,
            1.9515617404869672
        ],
        [
            89,
            1.9573134329214223
        ],
        [
            90,
            1.964131634765298
        ],
        [
            91,
            1.9716148272379233
        ],
        [
            92,
            1.9703285182944317
        ],
        [
            93,
            1.965362925363438
        ],
        [
            94,
            1.9857530817647544
        ],
        [
            95,
            1.9818879267291951
        ],
        [
            96,
            1.973496954519878
        ],
        [
            97,
            1.9878136384206826
        ],
        [
            98,
            1.9872596387298467
        ],
        [
            99,
            1.9904772629457683
        ],
        [
            100,
            1.9866176216089193
        ],
        [
            101,
            1.9943702319026133
        ],
        [
            102,
            2.000755804554253
        ],
        [
            103,
            1.9986162351600163
        ],
        [
            104,
            2.0018377173773487
        ],
        [
            105,
            2.0089419078951622
        ],
        [
            106,
            2.006266006823884
        ],
        [
            107,
            2.0110834601694756
        ],
        [
            108,
            2.0197533932983607
        ],
        [
            109,
            2.020817841067105
        ],
        [
            110,
            2.0213920365464055
        ],
        [
            111,
            2.0287028647006213
        ],
        [
            112,
            2.0254436509725893
        ],
        [
            113,
            2.0305499221176935
        ],
        [
            114,
            2.0385647008273247
        ],
        [
            115,
            2.0412655057566886
        ],
        [
            116,
            2.041226511000301
        ],
        [
            117,
            2.043777225421173
        ],
        [
            118,
            2.045131480943626
        ],
        [
            119,
            2.0442671904304124
        ],
        [
            120,
            2.0444478853185193
        ],
        [
            121,
            2.054320168650931
        ],
        [
            122,
            2.0533206969774147
        ],
        [
            123,
            2.0537450156545582
        ],
        [
            124,
            2.0603711328949883
        ],
        [
            125,
            2.061596565264983
        ],
        [
            126,
            2.064260241939443
        ],
        [
            127,
            2.0680943443682396
        ],
        [
            128,
            2.072069019247214
        ],
        [
            129,
            2.0699162513788387
        ],
        [
            130,
            2.0687446402269773
        ],
        [
            131,
            2.0829597283106693
        ],
        [
            132,
            2.0824008698767313
        ],
        [
            133,
            2.0775001140430533
        ],
        [
            134,
            2.0927668452339954
        ],
        [
            135,
            2.103235088496425
        ],
        [
            136,
            2.0818926920534038
        ],
        [
            137,
            2.1053017856385567
        ],
        [
            138,
            2.107281561781483
        ],
        [
            139,
            2.1010139512382766
        ],
        [
            140,
            2.096292362481292
        ],
        [
            141,
            2.1148423856711243
        ],
        [
            142,
            2.1074489205552647
        ],
        [
            143,
            2.1129816943338615
        ],
        [
            144,
            2.128121127981315
        ],
        [
            145,
            2.125533296346394
        ],
        [
            146,
            2.127881700133825
        ],
        [
            147,
            2.1251815351759418
        ],
        [
            148,
            2.146735407482915
        ],
        [
            149,
            2.1256974639018296
        ],
        [
            150,
            2.1475405212734584
        ],
        [
            151,
            2.139609407675052
        ],
        [
            152,
            2.1375677788579295
        ],
        [
            153,
            2.149378824702897
        ],
        [
            154,
            2.164022872437911
        ],
        [
            155,
            2.1658110963608954
        ],
        [
            156,
            2.163969565455556
        ],
        [
            157,
            2.172596298412625
        ],
        [
            158,
            2.1773323312034245
        ],
        [
            159,
            2.1723042961092482
        ],
        [
            160,
            2.1769467370469173
        ],
        [
            161,
            2.189311038060019
        ],
        [
            162,
            2.196914467775317
        ],
        [
            163,
            2.1908772191099435
        ],
        [
            164,
            2.2113109397532673
        ],
        [
            165,
            2.1946025386319503
        ],
        [
            166,
            2.20150814569399
        ],
        [
            167,
            2.2158891618058965
        ],
        [
            168,
            2.2153119475471197
        ],
        [
            169,
            2.2077991015034657
        ],
        [
            170,
            2.221557008971191
        ],
        [
            171,
            2.233837990710378
        ],
        [
            172,
            2.235922613098818
        ],
        [
            173,
            2.237517992333297
        ],
        [
            174,
            2.2458150628291893
        ],
        [
            175,
            2.2426619026809624
        ],
        [
            176,
            2.2381256717854665
        ],
        [
            177,
            2.2490271751682474
        ],
        [
            178,
            2.26217040114128
        ],
        [
            179,
            2.252994963931456
        ],
        [
            180,
            2.2592599323319265
        ],
        [
            181,
            2.2603709506052128
        ],
        [
            182,
            2.266785082124186
        ],
        [
            183,
            2.2667574331648432
        ],
        [
            184,
            2.280474408880645
        ],
        [
            185,
            2.2735068113724837
        ],
        [
            186,
            2.2842201932347197
        ],
        [
            187,
            2.2927471028578754
        ],
        [
            188,
            2.2916921071642244
        ],
        [
            189,
            2.298683526950134
        ],
        [
            190,
            2.303461385303859
        ],
        [
            191,
            2.314158479822216
        ],
        [
            192,
            2.3203520085609344
        ],
        [
            193,
            2.316060615416894
        ],
        [
            194,
            2.3186957876559466
        ],
        [
            195,
            2.3294040805377683
        ],
        [
            196,
            2.3208768278496477
        ],
        [
            197,
            2.3250257209060194
        ],
        [
            198,
            2.335641355524332
        ],
        [
            199,
            2.316158495701274
        ],
        [
            200,
            2.3293159242761563
        ],
        [
            201,
            2.33433179664754
        ],
        [
            202,
            2.341151040805292
        ],
        [
            203,
            2.3374941693586195
        ],
        [
            204,
            2.3368306419457063
        ],
        [
            205,
            2.3428501155653865
        ],
        [
            206,
            2.347498133188492
        ],
        [
            207,
            2.3437620264160293
        ],
        [
            208,
            2.360144672552609
        ],
        [
            209,
            2.3699326424188785
        ],
        [
            210,
            2.351546242768425
        ],
        [
            211,
            2.356858449692954
        ],
        [
            212,
            2.3703804024769273
        ],
        [
            213,
            2.366161138431078
        ],
        [
            214,
            2.375525038349316
        ],
        [
            215,
            2.379439957135241
        ],
        [
            216,
            2.3708512445912873
        ],
        [
            217,
            2.3792711349388473
        ],
        [
            218,
            2.3829629380513633
        ],
        [
            219,
            2.3772632001280822
        ],
        [
            220,
            2.3717916055727994
        ],
        [
            221,
            2.37502633844781
        ],
        [
            222,
            2.381796162394773
        ],
        [
            223,
            2.3857940002721065
        ],
        [
            224,
            2.381617059808962
        ],
        [
            225,
            2.388872326880671
        ],
        [
            226,
            2.3881180872505197
        ],
        [
            227,
            2.3807578939360563
        ],
        [
            228,
            2.396769422378464
        ],
        [
            229,
            2.385824331224916
        ],
        [
            230,
            2.392786401595107
        ],
        [
            231,
            2.393792383929239
        ],
        [
            232,
            2.397428875420713
        ],
        [
            233,
            2.3919339044791155
        ],
        [
            234,
            2.3935499291471047
        ],
        [
            235,
            2.402294123576236
        ],
        [
            236,
            2.405129246295293
        ],
        [
            237,
            2.404643671232557
        ],
        [
            238,
            2.4089812824145236
        ],
        [
            239,
            2.4077043639323485
        ],
        [
            240,
            2.4127386055775726
        ],
        [
            241,
            2.4198268201123225
        ],
        [
            242,
            2.4118687403132406
        ],
        [
            243,
            2.406392541400809
        ],
        [
            244,
            2.4242474590055068
        ],
        [
            245,
            2.419229110343433
        ],
        [
            246,
            2.4187315652827373
        ],
        [
            247,
            2.4267069154141057
        ],
        [
            248,
            2.4232547251902052
        ],
        [
            249,
            2.4038097672123926
        ],
        [
            250,
            2.4172832524755927
        ],
        [
            251,
            2.416752321245322
        ],
        [
            252,
            2.431689574194228
        ],
        [
            253,
            2.426331666792604
        ],
        [
            254,
            2.422207744034147
        ],
        [
            255,
            2.426418237547233
        ],
        [
            256,
            2.4239570404805337
        ],
        [
            257,
            2.4194883829423417
        ],
        [
            258,
            2.42274729664041
        ],
        [
            259,
            2.4271893610909556
        ],
        [
            260,
            2.4302160449027492
        ]
    ],
    "mae_physical": [
        [
            1,
            3.339841375322509
        ],
        [
            2,
            3.2486352791909026
        ],
        [
            3,
            3.1201733681499153
        ],
        [
            4,
            2.9246853600079947
        ],
        [
            5,
            2.6327817516552434
        ],
        [
            6,
            2.2537248078230263
        ],
        [
            7,
            1.8055989202615377
        ],
        [
            8,
            1.485604576967858
        ],
        [
            9,
            1.3509493381590456
        ],
        [
            10,
            1.3454388968042426
        ],
        [
            11,
            1.3679401254331744
        ],
        [
            12,
            1.3719494254202456
        ],
        [
            13,
            1.3684436171441465
        ],
        [
            14,
            1.3704671867795892
        ],
        [
            15,
            1.3669906379403294
        ],
        [
            16,
            1.3656119293457754
        ],
        [
            17,
            1.3630466356470778
        ],
        [
            18,
            1.362550143454526
        ],
        [
            19,
            1.3705796011396356
        ],
        [
            20,
            1.366595610573485
        ],
        [
            21,
            1.3753660071540523
        ],
        [
            22,
            1.373815642015354
        ],
        [
            23,
            1.3811215331425537
        ],
        [
            24,
            1.379218960936005
        ],
        [
            25,
            1.3822461149177037
        ],
        [
            26,
            1.383617732170466
        ],
        [
            27,
            1.3856863516408044
        ],
        [
            28,
            1.384998642915004
        ],
        [
            29,
            1.3881753127317171
        ],
        [
            30,
            1.3917468278794676
        ],
        [
            31,
            1.3911626363122784
        ],
        [
            32,
            1.391249291800164
        ],
        [
            33,
            1.392460347027392
        ],
        [
            34,
            1.3942091988550651
        ],
        [
            35,
            1.4025136868695955
        ],
        [
            36,
            1.4011840731711
        ],
        [
            37,
            1.399258432356087
        ],
        [
            38,
            1.4074587926671311
        ],
        [
            39,
            1.4033564095561568
        ],
        [
            40,
            1.4067821736271318
        ],
        [
            41,
            1.4055181121503986
        ],
        [
            42,
            1.4072043935994845
        ],
        [
            43,
            1.4107111958233085
        ],
        [
            44,
            1.411020493990666
        ],
        [
            45,
            1.4141545110457652
        ],
        [
            46,
            1.4158136176096427
        ],
        [
            47,
            1.4094292610078245
        ],
        [
            48,
            1.4143355980112746
        ],
        [
            49,
            1.411085554876843
        ],
        [
            50,
            1.4154213674970575
        ],
        [
            51,
            1.4153795830301337
        ],
        [
            52,
            1.4195257272269275
        ],
        [
            53,
            1.421964275675851
        ],
        [
            54,
            1.4212094714512695
        ],
        [
            55,
            1.4245640707982552
        ],
        [
            56,
            1.425900632465208
        ],
        [
            57,
            1.4281566876011926
        ],
        [
            58,
            1.4211920475637592
        ],
        [
            59,
            1.430480943338291
        ],
        [
            60,
            1.4248514537875716
        ],
        [
            61,
            1.4246525273129746
        ],
        [
            62,
            1.4233892648606687
        ],
        [
            63,
            1.4311563026260685
        ],
        [
            64,
            1.42842851055635
        ],
        [
            65,
            1.4234438047215745
        ],
        [
            66,
            1.4286219372942641
        ],
        [
            67,
            1.4299360586179268
        ],
        [
            68,
            1.4350762181990855
        ],
        [
            69,
            1.4338533805834281
        ],
        [
            70,
            1.4339056747990686
        ],
        [
            71,
            1.4362728507132143
        ],
        [
            72,
            1.438497655295037
        ],
        [
            73,
            1.445513460281733
        ],
        [
            74,
            1.4420729577541351
        ],
        [
            75,
            1.4488262783836674
        ],
        [
            76,
            1.450926176599554
        ],
        [
            77,
            1.4545336397918496
        ],
        [
            78,
            1.453660407581845
        ],
        [
            79,
            1.4538811380798753
        ],
        [
            80,
            1.4563123849598136
        ],
        [
            81,
            1.4604127688987836
        ],
        [
            82,
            1.466098890111253
        ],
        [
            83,
            1.4696951089678585
        ],
        [
            84,
            1.4703809361200075
        ],
        [
            85,
            1.4741013098407436
        ],
        [
            86,
            1.4695636766987878
        ],
        [
            87,
            1.4750884964659408
        ],
        [
            88,
            1.479268939108462
        ],
        [
            89,
            1.4792866948488597
        ],
        [
            90,
            1.4824919515364878
        ],
        [
            91,
            1.4831484797838572
        ],
        [
            92,
            1.4878386186586845
        ],
        [
            93,
            1.4874869246740599
        ],
        [
            94,
            1.493299029163412
        ],
        [
            95,
            1.4924631867859814
        ],
        [
            96,
            1.4939983897917979
        ],
        [
            97,
            1.5013723929186125
        ],
        [
            98,
            1.501937677731385
        ],
        [
            99,
            1.5007580929511302
        ],
        [
            100,
            1.5035163848786741
        ],
        [
            101,
            1.5096630880961548
        ],
        [
            102,
            1.5143466809311428
        ],
        [
            103,
            1.5132344469830796
        ],
        [
            104,
            1.5150671343545656
        ],
        [
            105,
            1.5178169253710154
        ],
        [
            106,
            1.5190635935680286
        ],
        [
            107,
            1.5246927609314789
        ],
        [
            108,
            1.527913199083225
        ],
        [
            109,
            1.530805291356267
        ],
        [
            110,
            1.5357087512274046
        ],
        [
            111,
            1.536593795628161
        ],
        [
            112,
            1.5410528948178162
        ],
        [
            113,
            1.5473853180537354
        ],
        [
            114,
            1.550355049403938
        ],
        [
            115,
            1.551323134351421
        ],
        [
            116,
            1.5525715810221594
        ],
        [
            117,
            1.5536249465233571
        ],
        [
            118,
            1.5541609155165184
        ],
        [
            119,
            1.5545161865852974
        ],
        [
            120,
            1.5589347336743329
        ],
        [
            121,
            1.5619248538403898
        ],
        [
            122,
            1.563648050701296
        ],
        [
            123,
            1.5678277160670306
        ],
        [
            124,
            1.5712996052729118
        ],
        [
            125,
            1.5746244498201318
        ],
        [
            126,
            1.5755870205325049
        ],
        [
            127,
            1.5812971261707511
        ],
        [
            128,
            1.581775281880353
        ],
        [
            129,
            1.5823186797064703
        ],
        [
            130,
            1.587460560573114
        ],
        [
            131,
            1.5917169737654764
        ],
        [
            132,
            1.5922063270936142
        ],
        [
            133,
            1.5933118635738217
        ],
        [
            134,
            1.60336859322883
        ],
        [
            135,
            1.6076067641780183
        ],
        [
            136,
            1.6017721126208435
        ],
        [
            137,
            1.6175315806994568
        ],
        [
            138,
            1.6210827328063346
        ],
        [
            139,
            1.6153816544526332
        ],
        [
            140,
            1.6166623951615513
        ],
        [
            141,
            1.6284728710715834
        ],
        [
            142,
            1.625715737810006
        ],
        [
            143,
            1.6301844160299044
        ],
        [
            144,
            1.6404725023218103
        ],
        [
            145,
            1.6424526330587026
        ],
        [
            146,
            1.6442551065135647
        ],
        [
            147,
            1.642352459398476
        ],
        [
            148,
            1.6582322628111452
        ],
        [
            149,
            1.6497305389997121
        ],
        [
            150,
            1.6635442569449141
        ],
        [
            151,
            1.6625710911041982
        ],
        [
            152,
            1.6633390458049
        ],
        [
            153,
            1.6706217104518735
        ],
        [
            154,
            1.6788894282402218
        ],
        [
            155,
            1.6831993565366075
        ],
        [
            156,
            1.6810676469593435
        ],
        [
            157,
            1.6881746877286885
        ],
        [
            158,
            1.6955908603362135
        ],
        [
            159,
            1.6940801292657852
        ],
        [
            160,
            1.6983789159236728
        ],
        [
            161,
            1.7061243510326825
        ],
        [
            162,
            1.7124522801589321
        ],
        [
            163,
            1.709866636910954
        ],
        [
            164,
            1.7254381655035793
        ],
        [
            165,
            1.717116007027594
        ],
        [
            166,
            1.7210654137102332
        ],
        [
            167,
            1.7327316399865054
        ],
        [
            168,
            1.7360552975253478
        ],
        [
            169,
            1.7309499363339431
        ],
        [
            170,
            1.7419610585312586
        ],
        [
            171,
            1.7471506123909273
        ],
        [
            172,
            1.7517565057889835
        ],
        [
            173,
            1.751321610566732
        ],
        [
            174,
            1.7589284254086983
        ],
        [
            175,
            1.758965449961456
        ],
        [
            176,
            1.7590157070474044
        ],
        [
            177,
            1.7662876779968675
        ],
        [
            178,
            1.772635054648728
        ],
        [
            179,
            1.7708655939110227
        ],
        [
            180,
            1.7758857873645988
        ],
        [
            181,
            1.777180216602377
        ],
        [
            182,
            1.777690631916394
        ],
        [
            183,
            1.78195288938445
        ],
        [
            184,
            1.793274135605709
        ],
        [
            185,
            1.7876023833413381
        ],
        [
            186,
            1.7964147148100105
        ],
        [
            187,
            1.805174972257904
        ],
        [
            188,
            1.8035088590270765
        ],
        [
            189,
            1.8085033835188762
        ],
        [
            190,
            1.813267973629204
        ],
        [
            191,
            1.8203170140450065
        ],
        [
            192,
            1.8266173719957068
        ],
        [
            193,
            1.82544477122861
        ],
        [
            194,
            1.826627012040164
        ],
        [
            195,
            1.8340409268197175
        ],
        [
            196,
            1.829425778542016
        ],
        [
            197,
            1.8336869989697997
        ],
        [
            198,
            1.841068376560469
        ],
        [
            199,
            1.8318618962893616
        ],
        [
            200,
            1.838470001478453
        ],
        [
            201,
            1.8445964870420661
        ],
        [
            202,
            1.849476841957988
        ],
        [
            203,
            1.8492223495567166
        ],
        [
            204,
            1.8477407351538941
        ],
        [
            205,
            1.850644989593609
        ],
        [
            206,
            1.8523946875655972
        ],
        [
            207,
            1.854811112622957
        ],
        [
            208,
            1.8637771840031083
        ],
        [
            209,
            1.8706558860234312
        ],
        [
            210,
            1.8638765371933177
        ],
        [
            211,
            1.8648881638372266
        ],
        [
            212,
            1.8736927166983888
        ],
        [
            213,
            1.8741183198384337
        ],
        [
            214,
            1.8802680800089966
        ],
        [
            215,
            1.8811488170680162
        ],
        [
            216,
            1.879471844918019
        ],
        [
            217,
            1.8845919616319038
        ],
        [
            218,
            1.8869584786730844
        ],
        [
            219,
            1.8866434014729552
        ],
        [
            220,
            1.8846853838579074
        ],
        [
            221,
            1.884265001661874
        ],
        [
            222,
            1.8883397178472698
        ],
        [
            223,
            1.8891526947955828
        ],
        [
            224,
            1.890580126764001
        ],
        [
            225,
            1.8937735019906148
        ],
        [
            226,
            1.8928156136660963
        ],
        [
            227,
            1.8904700126196887
        ],
        [
            228,
            1.8949919930986456
        ],
        [
            229,
            1.8930358880677738
        ],
        [
            230,
            1.8940272943393603
        ],
        [
            231,
            1.899643287062645
        ],
        [
            232,
            1.8969019321171012
        ],
        [
            233,
            1.894484717398882
        ],
        [
            234,
            1.8989511480605281
        ],
        [
            235,
            1.9060705255817723
        ],
        [
            236,
            1.9051068842008307
        ],
        [
            237,
            1.909760183377846
        ],
        [
            238,
            1.9071196630798481
        ],
        [
            239,
            1.9086614649843525
        ],
        [
            240,
            1.9135740230212341
        ],
        [
            241,
            1.9150777492370155
        ],
        [
            242,
            1.915419721019429
        ],
        [
            243,
            1.9146863894688118
        ],
        [
            244,
            1.921581584356121
        ],
        [
            245,
            1.9162932425938748
        ],
        [
            246,
            1.918739564716816
        ],
        [
            247,
            1.9219212769656568
        ],
        [
            248,
            1.920836333487485
        ],
        [
            249,
            1.9143646305074562
        ],
        [
            250,
            1.914966312211913
        ],
        [
            251,
            1.9183794533884204
        ],
        [
            252,
            1.9284448529014717
        ],
        [
            253,
            1.923485547404837
        ],
        [
            254,
            1.921726132466181
        ],
        [
            255,
            1.9288621810642448
        ],
        [
            256,
            1.9197718288648773
        ],
        [
            257,
            1.916327525937074
        ],
        [
            258,
            1.9208668313316397
        ],
        [
            259,
            1.925568922045263
        ],
        [
            260,
            1.9295254075849377
        ]
    ],
    "r2_physical": [
        [
            1,
            -3.765975073327401
        ],
        [
            2,
            -3.561713173295228
        ],
        [
            3,
            -3.284591454376085
        ],
        [
            4,
            -2.884468403417302
        ],
        [
            5,
            -2.335385413854826
        ],
        [
            6,
            -1.676073183288962
        ],
        [
            7,
            -1.0039905475036792
        ],
        [
            8,
            -0.47895428458273037
        ],
        [
            9,
            -0.20341097847755063
        ],
        [
            10,
            -0.09909737854637046
        ],
        [
            11,
            -0.06055646088526134
        ],
        [
            12,
            -0.05873224283871914
        ],
        [
            13,
            -0.06732555549295771
        ],
        [
            14,
            -0.06876240401045841
        ],
        [
            15,
            -0.07763734166187541
        ],
        [
            16,
            -0.0869847953854872
        ],
        [
            17,
            -0.09840864405622063
        ],
        [
            18,
            -0.10544324015335693
        ],
        [
            19,
            -0.10582756998421572
        ],
        [
            20,
            -0.12216326529457544
        ],
        [
            21,
            -0.11656929583995068
        ],
        [
            22,
            -0.1250617630347428
        ],
        [
            23,
            -0.12400298395449005
        ],
        [
            24,
            -0.13463947214469663
        ],
        [
            25,
            -0.13637340553830768
        ],
        [
            26,
            -0.14162334652396025
        ],
        [
            27,
            -0.1488373365183009
        ],
        [
            28,
            -0.15522841466012305
        ],
        [
            29,
            -0.155242467437537
        ],
        [
            30,
            -0.15867567244385739
        ],
        [
            31,
            -0.16858308910754816
        ],
        [
            32,
            -0.17439807749211966
        ],
        [
            33,
            -0.17776005548707885
        ],
        [
            34,
            -0.1787373825190801
        ],
        [
            35,
            -0.16669150112676445
        ],
        [
            36,
            -0.1758692685350005
        ],
        [
            37,
            -0.18619698018358566
        ],
        [
            38,
            -0.1770805674794258
        ],
        [
            39,
            -0.19102391969062293
        ],
        [
            40,
            -0.1860561540858201
        ],
        [
            41,
            -0.20281251499192243
        ],
        [
            42,
            -0.192550611289809
        ],
        [
            43,
            -0.19096536048868362
        ],
        [
            44,
            -0.19043423462490305
        ],
        [
            45,
            -0.1889741600217858
        ],
        [
            46,
            -0.1898286110554428
        ],
        [
            47,
            -0.20301370020313558
        ],
        [
            48,
            -0.19864101497584574
        ],
        [
            49,
            -0.21103671663340484
        ],
        [
            50,
            -0.20235864362730616
        ],
        [
            51,
            -0.20312871573440017
        ],
        [
            52,
            -0.20817804681220542
        ],
        [
            53,
            -0.2027906568256348
        ],
        [
            54,
            -0.20652424951681647
        ],
        [
            55,
            -0.2034903212207615
        ],
        [
            56,
            -0.20705668711754188
        ],
        [
            57,
            -0.21476988738735114
        ],
        [
            58,
            -0.22584793510189005
        ],
        [
            59,
            -0.2156962537295115
        ],
        [
            60,
            -0.2237127661353553
        ],
        [
            61,
            -0.22897666907103886
        ],
        [
            62,
            -0.23418259731832602
        ],
        [
            63,
            -0.2135934252557259
        ],
        [
            64,
            -0.23095105986968534
        ],
        [
            65,
            -0.24611264225979057
        ],
        [
            66,
            -0.23320804806382256
        ],
        [
            67,
            -0.24254239858111482
        ],
        [
            68,
            -0.23910099699142018
        ],
        [
            69,
            -0.24357128233942227
        ],
        [
            70,
            -0.2355305945612809
        ],
        [
            71,
            -0.24615573263720347
        ],
        [
            72,
            -0.25206922002444276
        ],
        [
            73,
            -0.24640994401043304
        ],
        [
            74,
            -0.25590012489058034
        ],
        [
            75,
            -0.2626204968872199
        ],
        [
            76,
            -0.2672237890771083
        ],
        [
            77,
            -0.27032384473395576
        ],
        [
            78,
            -0.2740216440377672
        ],
        [
            79,
            -0.2635794340255555
        ],
        [
            80,
            -0.28733091140786593
        ],
        [
            81,
            -0.29060316659866237
        ],
        [
            82,
            -0.2869052553700724
        ],
        [
            83,
            -0.2851651624600624
        ],
        [
            84,
            -0.29571984361530745
        ],
        [
            85,
            -0.29352693250634543
        ],
        [
            86,
            -0.3088573491702664
        ],
        [
            87,
            -0.2889648179897737
        ],
        [
            88,
            -0.29418508210643224
        ],
        [
            89,
            -0.3018248340059615
        ],
        [
            90,
            -0.31091031215518994
        ],
        [
            91,
            -0.32091827831379893
        ],
        [
            92,
            -0.3191952696303815
        ],
        [
            93,
            -0.31255441527677275
        ],
        [
            94,
            -0.3399305504603931
        ],
        [
            95,
            -0.3347194302967531
        ],
        [
            96,
            -0.3234414110700956
        ],
        [
            97,
            -0.3427128050565693
        ],
        [
            98,
            -0.3419644865945026
        ],
        [
            99,
            -0.34631362446017233
        ],
        [
            100,
            -0.3410975389767905
        ],
        [
            101,
            -0.3515850059151775
        ],
        [
            102,
            -0.36025386880332544
        ],
        [
            103,
            -0.3573461662245818
        ],
        [
            104,
            -0.36172538672261934
        ],
        [
            105,
            -0.37140761234056385
        ],
        [
            106,
            -0.36775662866692027
        ],
        [
            107,
            -0.3743330393865507
        ],
        [
            108,
            -0.38620828954703845
        ],
        [
            109,
            -0.38766978989253387
        ],
        [
            110,
            -0.38845848732430244
        ],
        [
            111,
            -0.3985200068428958
        ],
        [
            112,
            -0.39403003022284633
        ],
        [
            113,
            -0.4010677655367476
        ],
        [
            114,
            -0.4121498959926275
        ],
        [
            115,
            -0.41589416553713643
        ],
        [
            116,
            -0.41584006976122456
        ],
        [
            117,
            -0.4193807450044327
        ],
        [
            118,
            -0.42126239926463427
        ],
        [
            119,
            -0.42006137717185643
        ],
        [
            120,
            -0.4203124296500529
        ],
        [
            121,
            -0.4340624313664663
        ],
        [
            122,
            -0.43266736536554085
        ],
        [
            123,
            -0.43325954790594157
        ],
        [
            124,
            -0.44252288488570524
        ],
        [
            125,
            -0.44423931344055645
        ],
        [
            126,
            -0.44797377039237163
        ],
        [
            127,
            -0.45335762210702857
        ],
        [
            128,
            -0.4589494125390754
        ],
        [
            129,
            -0.45591944796009964
        ],
        [
            130,
            -0.45427175935328146
        ],
        [
            131,
            -0.4743260717032738
        ],
        [
            132,
            -0.4735350539828729
        ],
        [
            133,
            -0.46660753287509515
        ],
        [
            134,
            -0.48824177643537325
        ],
        [
            135,
            -0.5031677024535237
        ],
        [
            136,
            -0.4728159547177535
        ],
        [
            137,
            -0.5061232624132614
        ],
        [
            138,
            -0.5089572398739937
        ],
        [
            139,
            -0.49999451572256004
        ],
        [
            140,
            -0.49326024422510173
        ],
        [
            141,
            -0.5198047958456735
        ],
        [
            142,
            -0.5091969299715076
        ],
        [
            143,
            -0.5171316476111669
        ],
        [
            144,
            -0.538949913467909
        ],
        [
            145,
            -0.535209410356517
        ],
        [
            146,
            -0.5386036486497967
        ],
        [
            147,
            -0.53470131916395
        ],
        [
            148,
            -0.5659894705746393
        ],
        [
            149,
            -0.5354465661887593
        ],
        [
            150,
            -0.5671643113491496
        ],
        [
            151,
            -0.5556102507531333
        ],
        [
            152,
            -0.5526429208888304
        ],
        [
            153,
            -0.5698484577395257
        ],
        [
            154,
            -0.5913125654716025
        ],
        [
            155,
            -0.5939435902767258
        ],
        [
            156,
            -0.591234167939735
        ],
        [
            157,
            -0.6039464669677168
        ],
        [
            158,
            -0.6109469600502566
        ],
        [
            159,
            -0.6035153472156638
        ],
        [
            160,
            -0.6103764300285355
        ],
        [
            161,
            -0.6287211353667175
        ],
        [
            162,
            -0.6400538066162653
        ],
        [
            163,
            -0.6310522656173818
        ],
        [
            164,
            -0.6616189161484332
        ],
        [
            165,
            -0.6366037923795651
        ],
        [
            166,
            -0.6469195773301781
        ],
        [
            167,
            -0.6685063557509598
        ],
        [
            168,
            -0.6676372146048288
        ],
        [
            169,
            -0.6563453898077998
        ],
        [
            170,
            -0.6770527520520258
        ],
        [
            171,
            -0.6956458217558559
        ],
        [
            172,
            -0.6988120590481843
        ],
        [
            173,
            -0.7012372023668945
        ],
        [
            174,
            -0.7138775081888078
        ],
        [
            175,
            -0.709068264004252
        ],
        [
            176,
            -0.7021613936168711
        ],
        [
            177,
            -0.7187836147590303
        ],
        [
            178,
            -0.7389313215432387
        ],
        [
            179,
            -0.7248536046561782
        ],
        [
            180,
            -0.7344596428353469
        ],
        [
            181,
            -0.736165945104184
        ],
        [
            182,
            -0.7460331721744187
        ],
        [
            183,
            -0.7459905781886476
        ],
        [
            184,
            -0.7671857658381436
        ],
        [
            185,
            -0.7564035979668524
        ],
        [
            186,
            -0.7729959018177035
        ],
        [
            187,
            -0.7862576649856181
        ],
        [
            188,
            -0.7846141687867949
        ],
        [
            189,
            -0.795519664305756
        ],
        [
            190,
            -0.8029914663734361
        ],
        [
            191,
            -0.8197762524184329
        ],
        [
            192,
            -0.829530051946159
        ],
        [
            193,
            -0.8227690325568944
        ],
        [
            194,
            -0.8269192200614892
        ],
        [
            195,
            -0.8438324864979947
        ],
        [
            196,
            -0.830357755021687
        ],
        [
            197,
            -0.8369076469613654
        ],
        [
            198,
            -0.8537198970632873
        ],
        [
            199,
            -0.822923101858954
        ],
        [
            200,
            -0.8436929295146072
        ],
        [
            201,
            -0.8516417742438636
        ],
        [
            202,
            -0.8624759158124531
        ],
        [
            203,
            -0.8566620954947337
        ],
        [
            204,
            -0.855608170859339
        ],
        [
            205,
            -0.8651802557434602
        ],
        [
            206,
            -0.8725883185233929
        ],
        [
            207,
            -0.8666325117968086
        ],
        [
            208,
            -0.8928188364799228
        ],
        [
            209,
            -0.9085511525084795
        ],
        [
            210,
            -0.8790522032841326
        ],
        [
            211,
            -0.8875514525456154
        ],
        [
            212,
            -0.9092723997611152
        ],
        [
            213,
            -0.9024814603750684
        ],
        [
            214,
            -0.9175691023314831
        ],
        [
            215,
            -0.9238947045778945
        ],
        [
            216,
            -0.9100309743223953
        ],
        [
            217,
            -0.9236217120978982
        ],
        [
            218,
            -0.9295959301299008
        ],
        [
            219,
            -0.9203762836119833
        ],
        [
            220,
            -0.9115464424960946
        ],
        [
            221,
            -0.9167640667046688
        ],
        [
            222,
            -0.9277068076916681
        ],
        [
            223,
            -0.9341835391739666
        ],
        [
            224,
            -0.9274169052236605
        ],
        [
            225,
            -0.9391780104729954
        ],
        [
            226,
            -0.9379536888593696
        ],
        [
            227,
            -0.9260265287765859
        ],
        [
            228,
            -0.9520202088121019
        ],
        [
            229,
            -0.9342327186107131
        ],
        [
            230,
            -0.9455377523179569
        ],
        [
            231,
            -0.94717399367147
        ],
        [
            232,
            -0.9530945238829922
        ],
        [
            233,
            -0.9441516947353459
        ],
        [
            234,
            -0.9467795753909021
        ],
        [
            235,
            -0.9610296342124742
        ],
        [
            236,
            -0.9656610740959533
        ],
        [
            237,
            -0.9648674538357833
        ],
        [
            238,
            -0.9719624908975135
        ],
        [
            239,
            -0.9698725053862056
        ],
        [
            240,
            -0.9781186852977171
        ],
        [
            241,
            -0.989758508792626
        ],
        [
            242,
            -0.9766925991502384
        ],
        [
            243,
            -0.9677265449402868
        ],
        [
            244,
            -0.9970350942318458
        ],
        [
            245,
            -0.9887756688578382
        ],
        [
            246,
            -0.9879577192436564
        ],
        [
            247,
            -1.0010892284087753
        ],
        [
            248,
            -0.9953998499935994
        ],
        [
            249,
            -0.9635049012217274
        ],
        [
            250,
            -0.9855776927435198
        ],
        [
            251,
            -0.9847055653802919
        ],
        [
            252,
            -1.009315175277834
        ],
        [
            253,
            -1.0004704079716435
        ],
        [
            254,
            -0.9936759744915775
        ],
        [
            255,
            -1.0006131628346702
        ],
        [
            256,
            -0.9965566436970823
        ],
        [
            257,
            -0.9892019721499212
        ],
        [
            258,
            -0.9945642656609386
        ],
        [
            259,
            -1.0018849675289716
        ],
        [
            260,
            -1.006880746372416
        ]
    ],
    "rmse_mental": [
        [
            1,
            7.802086369399507
        ],
        [
            2,
            7.681994942558031
        ],
        [
            3,
            7.519811461199775
        ],
        [
            4,
            7.278608822878736
        ],
        [
            5,
            6.915494550296691
        ],
        [
            6,
            6.3890049703582195
        ],
        [
            7,
            5.727496498575211
        ],
        [
            8,
            5.007659586129648
        ],
        [
            9,
            4.350965101954644
        ],
        [
            10,
            3.8417443168430547
        ],
        [
            11,
            3.466521652088967
        ],
        [
            12,
            3.2155514093829436
        ],
        [
            13,
            3.041896148894288
        ],
        [
            14,
            2.921726429943373
        ],
        [
            15,
            2.8393183777414848
        ],
        [
            16,
            2.780480756681278
        ],
        [
            17,
            2.7524953609939193
        ],
        [
            18,
            2.724586454361962
        ],
        [
            19,
            2.6990531471573713
        ],
        [
            20,
            2.701956804943006
        ],
        [
            21,
            2.6805218934999155
        ],
        [
            22,
            2.6756805698724335
        ],
        [
            23,
            2.668040985786096
        ],
        [
            24,
            2.6661117986135237
        ],
        [
            25,
            2.6463483350348596
        ],
        [
            26,
            2.6543838723002344
        ],
        [
            27,
            2.6608070104866854
        ],
        [
            28,
            2.6462168761433977
        ],
        [
            29,
            2.6506421635251916
        ],
        [
            30,
            2.644503125372583
        ],
        [
            31,
            2.660112350477085
        ],
        [
            32,
            2.6659302922634507
        ],
        [
            33,
            2.657667587095749
        ],
        [
            34,
            2.6720254076664705
        ],
        [
            35,
            2.645963232671897
        ],
        [
            36,
            2.659894446097811
        ],
        [
            37,
            2.658102330888786
        ],
        [
            38,
            2.6446010656772083
        ],
        [
            39,
            2.658086181184254
        ],
        [
            40,
            2.660452546682395
        ],
        [
            41,
            2.6745551942448467
        ],
        [
            42,
            2.6637533162813134
        ],
        [
            43,
            2.656043301450481
        ],
        [
            44,
            2.6436925322671523
        ],
        [
            45,
            2.649011258546832
        ],
        [
            46,
            2.64602574707763
        ],
        [
            47,
            2.664977146248808
        ],
        [
            48,
            2.654686227105722
        ],
        [
            49,
            2.6764264387698624
        ],
        [
            50,
            2.6390084310425164
        ],
        [
            51,
            2.647159574208667
        ],
        [
            52,
            2.6597321505970655
        ],
        [
            53,
            2.6447889734085175
        ],
        [
            54,
            2.6607625254917693
        ],
        [
            55,
            2.6440332718844415
        ],
        [
            56,
            2.6638988902043086
        ],
        [
            57,
            2.671677570786219
        ],
        [
            58,
            2.683702336007598
        ],
        [
            59,
            2.6559840918145974
        ],
        [
            60,
            2.6530064702267726
        ],
        [
            61,
            2.6668482135595797
        ],
        [
            62,
            2.667083093843562
        ],
        [
            63,
            2.630744555923136
        ],
        [
            64,
            2.6550731006238264
        ],
        [
            65,
            2.6648174589174065
        ],
        [
            66,
            2.64889183719391
        ],
        [
            67,
            2.658149018379661
        ],
        [
            68,
            2.6580033867875974
        ],
        [
            69,
            2.650355064183494
        ],
        [
            70,
            2.627497417269622
        ],
        [
            71,
            2.6456088471314576
        ],
        [
            72,
            2.645200143697033
        ],
        [
            73,
            2.61913374256631
        ],
        [
            74,
            2.643156056669721
        ],
        [
            75,
            2.642536358202734
        ],
        [
            76,
            2.656072409238573
        ],
        [
            77,
            2.6575964731249293
        ],
        [
            78,
            2.6618916814801974
        ],
        [
            79,
            2.6206767550661563
        ],
        [
            80,
            2.6785948711425522
        ],
        [
            81,
            2.659205933810223
        ],
        [
            82,
            2.6439995311305866
        ],
        [
            83,
            2.6278559428804282
        ],
        [
            84,
            2.6447947819580686
        ],
        [
            85,
            2.6363558838505545
        ],
        [
            86,
            2.6514105374836276
        ],
        [
            87,
            2.6070483065617585
        ],
        [
            88,
            2.623274708298026
        ],
        [
            89,
            2.607241056236989
        ],
        [
            90,
            2.6246627587596425
        ],
        [
            91,
            2.6524199460178113
        ],
        [
            92,
            2.63043939153895
        ],
        [
            93,
            2.6046828927367685
        ],
        [
            94,
            2.6677257403194954
        ],
        [
            95,
            2.653735500849955
        ],
        [
            96,
            2.646968132565832
        ],
        [
            97,
            2.6530483873148656
        ],
        [
            98,
            2.6705181590178833
        ],
        [
            99,
            2.6489088694655063
        ],
        [
            100,
            2.6305236118884627
        ],
        [
            101,
            2.6423319525339717
        ],
        [
            102,
            2.6748171520216473
        ],
        [
            103,
            2.6260662889824022
        ],
        [
            104,
            2.622923020836809
        ],
        [
            105,
            2.645024264333457
        ],
        [
            106,
            2.6098054270759072
        ],
        [
            107,
            2.6339204447430045
        ],
        [
            108,
            2.6434309646434424
        ],
        [
            109,
            2.6299818275516493
        ],
        [
            110,
            2.6306649962903013
        ],
        [
            111,
            2.6429095621359147
        ],
        [
            112,
            2.6357967137296923
        ],
        [
            113,
            2.606696541084622
        ],
        [
            114,
            2.6482757049610846
        ],
        [
            115,
            2.663976542858834
        ],
        [
            116,
            2.6489547113617133
        ],
        [
            117,
            2.6528972804271884
        ],
        [
            118,
            2.6389790282290413
        ],
        [
            119,
            2.6272395556506813
        ],
        [
            120,
            2.620219930029797
        ],
        [
            121,
            2.637722785633082
        ],
        [
            122,
            2.6094543716443765
        ],
        [
            123,
            2.622061015938137
        ],
        [
            124,
            2.6276175926376473
        ],
        [
            125,
            2.612615785560566
        ],
        [
            126,
            2.6290234176465375
        ],
        [
            127,
            2.5994133758861757
        ],
        [
            128,
            2.631659987929837
        ],
        [
            129,
            2.6130297762255856
        ],
        [
            130,
            2.6108892789699567
        ],
        [
            131,
            2.6151145622291567
        ],
        [
            132,
            2.6636613731292034
        ],
        [
            133,
            2.606289500891992
        ],
        [
            134,
            2.633855666526831
        ],
        [
            135,
            2.6642418201591416
        ],
        [
            136,
            2.5871774802930254
        ],
        [
            137,
            2.6401737762956623
        ],
        [
            138,
            2.6223292475284183
        ],
        [
            139,
            2.606228977720167
        ],
        [
            140,
            2.579731721140407
        ],
        [
            141,
            2.6141293911509345
        ],
        [
            142,
            2.6239091726092054
        ],
        [
            143,
            2.6211411448535964
        ],
        [
            144,
            2.6279910949665792
        ],
        [
            145,
            2.5865828760441185
        ],
        [
            146,
            2.5791646026970803
        ],
        [
            147,
            2.5705311650642133
        ],
        [
            148,
            2.6358146952508217
        ],
        [
            149,
            2.5436974703087567
        ],
        [
            150,
            2.6200343211582418
        ],
        [
            151,
            2.6017325378607272
        ],
        [
            152,
            2.5503655154907943
        ],
        [
            153,
            2.5650451501438467
        ],
        [
            154,
            2.6054062532549263
        ],
        [
            155,
            2.6035949277618395
        ],
        [
            156,
            2.588094022660021
        ],
        [
            157,
            2.58974264451329
        ],
        [
            158,
            2.5983618494744167
        ],
        [
            159,
            2.604476592387755
        ],
        [
            160,
            2.578375449020596
        ],
        [
            161,
            2.5871911673841117
        ],
        [
            162,
            2.576615531021795
        ],
        [
            163,
            2.562053866724148
        ],
        [
            164,
            2.616258340004448
        ],
        [
            165,
            2.538541804339457
        ],
        [
            166,
            2.588825933340328
        ],
        [
            167,
            2.625929934531928
        ],
        [
            168,
            2.586234510035797
        ],
        [
            169,
            2.553111879549033
        ],
        [
            170,
            2.578158012585378
        ],
        [
            171,
            2.6071782561935066
        ],
        [
            172,
            2.5952315977423805
        ],
        [
            173,
            2.578097988108718
        ],
        [
            174,
            2.592008253847891
        ],
        [
            175,
            2.6016323676618778
        ],
        [
            176,
            2.559618274381257
        ],
        [
            177,
            2.5681601900603988
        ],
        [
            178,
            2.637353837361481
        ],
        [
            179,
            2.5585548161748615
        ],
        [
            180,
            2.583754155439138
        ],
        [
            181,
            2.6061829575599913
        ],
        [
            182,
            2.6023840237547478
        ],
        [
            183,
            2.542212755835207
        ],
        [
            184,
            2.6282661669485505
        ],
        [
            185,
            2.582044836082867
        ],
        [
            186,
            2.5629837283630983
        ],
        [
            187,
            2.6316876374483344
        ],
        [
            188,
            2.591576092618676
        ],
        [
            189,
            2.5914290086872196
        ],
        [
            190,
            2.604244527073565
        ],
        [
            191,
            2.5872108348981047
        ],
        [
            192,
            2.617756618257196
        ],
        [
            193,
            2.599425135179466
        ],
        [
            194,
            2.610642617240761
        ],
        [
            195,
            2.633291848827191
        ],
        [
            196,
            2.5752821876221614
        ],
        [
            197,
            2.5956444872172657
        ],
        [
            198,
            2.6519637102461044
        ],
        [
            199,
            2.571345400584778
        ],
        [
            200,
            2.5833706668547793
        ],
        [
            201,
            2.5912040439668074
        ],
        [
            202,
            2.616421418755467
        ],
        [
            203,
            2.6158569895556685
        ],
        [
            204,
            2.619230217187026
        ],
        [
            205,
            2.598789187894493
        ],
        [
            206,
            2.6196820180895406
        ],
        [
            207,
            2.5888335838167826
        ],
        [
            208,
            2.600239847043503
        ],
        [
            209,
            2.621098709311445
        ],
        [
            210,
            2.636511533738297
        ],
        [
            211,
            2.598442888355816
        ],
        [
            212,
            2.627691037720719
        ],
        [
            213,
            2.62886325187314
        ],
        [
            214,
            2.6203766171195455
        ],
        [
            215,
            2.650695784231204
        ],
        [
            216,
            2.59095017992688
        ],
        [
            217,
            2.642066065040487
        ],
        [
            218,
            2.642699889616694
        ],
        [
            219,
            2.599711826003991
        ],
        [
            220,
            2.6186804777754205
        ],
        [
            221,
            2.620073484299496
        ],
        [
            222,
            2.645023892833965
        ],
        [
            223,
            2.6440470196959382
        ],
        [
            224,
            2.590836831380804
        ],
        [
            225,
            2.638930577277721
        ],
        [
            226,
            2.62956517520098
        ],
        [
            227,
            2.606807642363582
        ],
        [
            228,
            2.6415016781339515
        ],
        [
            229,
            2.6130298947876445
        ],
        [
            230,
            2.648783871770375
        ],
        [
            231,
            2.6155846804316965
        ],
        [
            232,
            2.6628039762751663
        ],
        [
            233,
            2.6507928764275115
        ],
        [
            234,
            2.609453158228727
        ],
        [
            235,
            2.6188418365886026
        ],
        [
            236,
            2.6374063578238616
        ],
        [
            237,
            2.616335688482336
        ],
        [
            238,
            2.6306535477370607
        ],
        [
            239,
            2.653763831053654
        ],
        [
            240,
            2.629726850024167
        ],
        [
            241,
            2.6494891153668987
        ],
        [
            242,
            2.618549918356815
        ],
        [
            243,
            2.6359189829251832
        ],
        [
            244,
            2.670214715796595
        ],
        [
            245,
            2.6308378929307925
        ],
        [
            246,
            2.645020374325008
        ],
        [
            247,
            2.639094852826867
        ],
        [
            248,
            2.676826615922898
        ],
        [
            249,
            2.6175199210247855
        ],
        [
            250,
            2.647426619635343
        ],
        [
            251,
            2.630781173725575
        ],
        [
            252,
            2.6499510844929106
        ],
        [
            253,
            2.6212245069081423
        ],
        [
            254,
            2.6167588036525378
        ],
        [
            255,
            2.636309066034096
        ],
        [
            256,
            2.6756934396211736
        ],
        [
            257,
            2.6566688088595347
        ],
        [
            258,
            2.614893630060504
        ],
        [
            259,
            2.6398358850614816
        ],
        [
            260,
            2.659860016937866
        ]
    ],
    "mae_mental": [
        [
            1,
            7.592886987066752
        ],
        [
            2,
            7.468928767418539
        ],
        [
            3,
            7.3011702903219176
        ],
        [
            4,
            7.051171899244592
        ],
        [
            5,
            6.6736186235337644
        ],
        [
            6,
            6.123566455132252
        ],
        [
            7,
            5.427013747595452
        ],
        [
            8,
            4.659422352507308
        ],
        [
            9,
            3.9530254570213526
        ],
        [
            10,
            3.4399827541531742
        ],
        [
            11,
            3.0519747379663826
        ],
        [
            12,
            2.808364123911471
        ],
        [
            13,
            2.642798617079451
        ],
        [
            14,
            2.5253949745281323
        ],
        [
            15,
            2.4463516216020325
        ],
        [
            16,
            2.3931750993470886
        ],
        [
            17,
            2.3685322226704777
        ],
        [
            18,
            2.3448974506275073
        ],
        [
            19,
            2.3232099655512215
        ],
        [
            20,
            2.3262268272606104
        ],
        [
            21,
            2.3084530798164575
        ],
        [
            22,
            2.3046883795712447
        ],
        [
            23,
            2.2987216485513224
        ],
        [
            24,
            2.297462592253814
        ],
        [
            25,
            2.280480562029658
        ],
        [
            26,
            2.2877657574576302
        ],
        [
            27,
            2.29394577967154
        ],
        [
            28,
            2.281082291860838
        ],
        [
            29,
            2.285340866527042
        ],
        [
            30,
            2.280326495299468
        ],
        [
            31,
            2.2940475747391984
        ],
        [
            32,
            2.299264585649645
        ],
        [
            33,
            2.2920843620558045
        ],
        [
            34,
            2.3045396160435034
        ],
        [
            35,
            2.282376975626559
        ],
        [
            36,
            2.294513235221038
        ],
        [
            37,
            2.2926159742716195
        ],
        [
            38,
            2.28165721248936
        ],
        [
            39,
            2.293351060635335
        ],
        [
            40,
            2.2951036111728564
        ],
        [
            41,
            2.307408207171672
        ],
        [
            42,
            2.297941771713463
        ],
        [
            43,
            2.2913024264412956
        ],
        [
            44,
            2.280506079261367
        ],
        [
            45,
            2.2852368838078267
        ],
        [
            46,
            2.282649227090784
        ],
        [
            47,
            2.298585795067452
        ],
        [
            48,
            2.2897686152844816
        ],
        [
            49,
            2.3081557944014266
        ],
        [
            50,
            2.2760314168156803
        ],
        [
            51,
            2.2829921310012407
        ],
        [
            52,
            2.2940839625693656
        ],
        [
            53,
            2.2810043257635995
        ],
        [
            54,
            2.2945968299298674
        ],
        [
            55,
            2.2800446394327523
        ],
        [
            56,
            2.296822709006232
        ],
        [
            57,
            2.303150070680154
        ],
        [
            58,
            2.313175623481338
        ],
        [
            59,
            2.289833329819344
        ],
        [
            60,
            2.286878395724941
        ],
        [
            61,
            2.298278276984756
        ],
        [
            62,
            2.2983432750444153
        ],
        [
            63,
            2.2672230746295
        ],
        [
            64,
            2.2876271750475907
        ],
        [
            65,
            2.295810444934948
        ],
        [
            66,
            2.2818343059436694
        ],
        [
            67,
            2.2891707549224027
        ],
        [
            68,
            2.2893373192967594
        ],
        [
            69,
            2.2824263959317594
        ],
        [
            70,
            2.263210583377529
        ],
        [
            71,
            2.2784768117440715
        ],
        [
            72,
            2.2784962782988676
        ],
        [
            73,
            2.256144227208318
        ],
        [
            74,
            2.2765927701383024
        ],
        [
            75,
            2.276321320920377
        ],
        [
            76,
            2.288147243293556
        ],
        [
            77,
            2.2894655143892444
        ],
        [
            78,
            2.293156008462648
        ],
        [
            79,
            2.2576786472990706
        ],
        [
            80,
            2.308254705893027
        ],
        [
            81,
            2.2909671841440975
        ],
        [
            82,
            2.278006138028325
        ],
        [
            83,
            2.2637824754457214
        ],
        [
            84,
            2.277909381969555
        ],
        [
            85,
            2.270775669329875
        ],
        [
            86,
            2.2836235664986275
        ],
        [
            87,
            2.2459476670703373
        ],
        [
            88,
            2.2594995627532133
        ],
        [
            89,
            2.2455908543354757
        ],
        [
            90,
            2.2607487182359436
        ],
        [
            91,
            2.283356202615274
        ],
        [
            92,
            2.2646080352164604
        ],
        [
            93,
            2.2427609868951746
        ],
        [
            94,
            2.295717165276811
        ],
        [
            95,
            2.2835375392759167
        ],
        [
            96,
            2.2789900077355876
        ],
        [
            97,
            2.283806211239583
        ],
        [
            98,
            2.298577727498235
        ],
        [
            99,
            2.2812171368985563
        ],
        [
            100,
            2.265702196069666
        ],
        [
            101,
            2.2759193858584843
        ],
        [
            102,
            2.303071221789798
        ],
        [
            103,
            2.2620035957645728
        ],
        [
            104,
            2.2602904068457113
        ],
        [
            105,
            2.2783553535873824
        ],
        [
            106,
            2.248936749793388
        ],
        [
            107,
            2.2695403550122237
        ],
        [
            108,
            2.2786114956881547
        ],
        [
            109,
            2.267041541434623
        ],
        [
            110,
            2.267446985115876
        ],
        [
            111,
            2.2773207780477165
        ],
        [
            112,
            2.2713393617320703
        ],
        [
            113,
            2.247076031324026
        ],
        [
            114,
            2.2824654385850236
        ],
        [
            115,
            2.2962099281517236
        ],
        [
            116,
            2.282865092561052
        ],
        [
            117,
            2.2856655346380697
        ],
        [
            118,
            2.272420454669643
        ],
        [
            119,
            2.2630312217248454
        ],
        [
            120,
            2.2571129025639713
        ],
        [
            121,
            2.270570471480086
        ],
        [
            122,
            2.2470543577864364
        ],
        [
            123,
            2.25717310969894
        ],
        [
            124,
            2.26070452703012
        ],
        [
            125,
            2.2472902375298576
        ],
        [
            126,
            2.2609482971397608
        ],
        [
            127,
            2.2344750004845695
        ],
        [
            128,
            2.263142730738666
        ],
        [
            129,
            2.247983674745302
        ],
        [
            130,
            2.246657932126844
        ],
        [
            131,
            2.248989994461472
        ],
        [
            132,
            2.2902337699323088
        ],
        [
            133,
            2.241114448856663
        ],
        [
            134,
            2.266010590501734
        ],
        [
            135,
            2.2915701543962634
        ],
        [
            136,
            2.2274394744151347
        ],
        [
            137,
            2.2714110709525444
        ],
        [
            138,
            2.256697609617903
        ],
        [
            139,
            2.242972750921507
        ],
        [
            140,
            2.2197902685887105
        ],
        [
            141,
            2.2503214849008097
        ],
        [
            142,
            2.259920896710576
        ],
        [
            143,
            2.258183083018741
        ],
        [
            144,
            2.2619897787635392
        ],
        [
            145,
            2.228353342494449
        ],
        [
            146,
            2.2213886074117712
        ],
        [
            147,
            2.211979920799668
        ],
        [
            148,
            2.2707581181783936
        ],
        [
            149,
            2.190539556580621
        ],
        [
            150,
            2.256270972458092
        ],
        [
            151,
            2.2418239245543607
        ],
        [
            152,
            2.1989144344587586
        ],
        [
            153,
            2.210385807462641
        ],
        [
            154,
            2.243491567469932
        ],
        [
            155,
            2.243792161748216
        ],
        [
            156,
            2.2290784040012874
        ],
        [
            157,
            2.229736099372039
        ],
        [
            158,
            2.2393491751438863
        ],
        [
            159,
            2.245413470912624
        ],
        [
            160,
            2.2221158001873946
        ],
        [
            161,
            2.2309196188643172
        ],
        [
            162,
            2.2197656792563363
        ],
        [
            163,
            2.208023904143153
        ],
        [
            164,
            2.256690545662029
        ],
        [
            165,
            2.187290660432867
        ],
        [
            166,
            2.2316251815976322
        ],
        [
            167,
            2.264992164598929
        ],
        [
            168,
            2.231447063587807
        ],
        [
            169,
            2.2026259528624044
        ],
        [
            170,
            2.2235615768948116
        ],
        [
            171,
            2.2494197407284298
        ],
        [
            172,
            2.2394581549876444
        ],
        [
            173,
            2.2258734252001786
        ],
        [
            174,
            2.23852373780431
        ],
        [
            175,
            2.247566229588277
        ],
        [
            176,
            2.2105219074197717
        ],
        [
            177,
            2.2187769960712744
        ],
        [
            178,
            2.2767380298794926
        ],
        [
            179,
            2.2121338393237138
        ],
        [
            180,
            2.2342695706599467
        ],
        [
            181,
            2.252217333059053
        ],
        [
            182,
            2.2486980863519617
        ],
        [
            183,
            2.197676961486404
        ],
        [
            184,
            2.2712346428149455
        ],
        [
            185,
            2.232830928789603
        ],
        [
            186,
            2.2193983036118583
        ],
        [
            187,
            2.276791047405552
        ],
        [
            188,
            2.243981641692084
        ],
        [
            189,
            2.2446198157362036
        ],
        [
            190,
            2.255201765008875
        ],
        [
            191,
            2.2416684611423596
        ],
        [
            192,
            2.268299664999988
        ],
        [
            193,
            2.25289003913467
        ],
        [
            194,
            2.26326521345087
        ],
        [
            195,
            2.282114961662808
        ],
        [
            196,
            2.234982234400672
        ],
        [
            197,
            2.251522864844348
        ],
        [
            198,
            2.2979075892551526
        ],
        [
            199,
            2.230752081484408
        ],
        [
            200,
            2.24189301761421
        ],
        [
            201,
            2.2482619994395487
        ],
        [
            202,
            2.2705774758313155
        ],
        [
            203,
            2.2712368449649296
        ],
        [
            204,
            2.273994769598987
        ],
        [
            205,
            2.255013828342025
        ],
        [
            206,
            2.274197844234673
        ],
        [
            207,
            2.248202257865184
        ],
        [
            208,
            2.259718129763732
        ],
        [
            209,
            2.278466181175129
        ],
        [
            210,
            2.2924107635343396
        ],
        [
            211,
            2.2595382075052
        ],
        [
            212,
            2.2824799724527307
        ],
        [
            213,
            2.286712153537853
        ],
        [
            214,
            2.281129758100252
        ],
        [
            215,
            2.3064728218155937
        ],
        [
            216,
            2.255293831631944
        ],
        [
            217,
            2.298271446614652
        ],
        [
            218,
            2.3015348041379773
        ],
        [
            219,
            2.2649633948867387
        ],
        [
            220,
            2.2818350646946883
        ],
        [
            221,
            2.2818600648158305
        ],
        [
            222,
            2.3031213492960543
        ],
        [
            223,
            2.3043347081622563
        ],
        [
            224,
            2.2603534443958386
        ],
        [
            225,
            2.2991675090145423
        ],
        [
            226,
            2.2917781842721476
        ],
        [
            227,
            2.2723799315658777
        ],
        [
            228,
            2.301418442983885
        ],
        [
            229,
            2.2836693074252152
        ],
        [
            230,
            2.3099350671510437
        ],
        [
            231,
            2.2819668779502043
        ],
        [
            232,
            2.321648634768821
        ],
        [
            233,
            2.310914709761336
        ],
        [
            234,
            2.2817364270622664
        ],
        [
            235,
            2.2901556620726713
        ],
        [
            236,
            2.3038699336954065
        ],
        [
            237,
            2.2858211768640055
        ],
        [
            238,
            2.30050735215883
        ],
        [
            239,
            2.3173208703865877
        ],
        [
            240,
            2.297627558579316
        ],
        [
            241,
            2.3125253029771753
        ],
        [
            242,
            2.2872367710680575
        ],
        [
            243,
            2.30295930198721
        ],
        [
            244,
            2.3326670720770553
        ],
        [
            245,
            2.2989927755819783
        ],
        [
            246,
            2.3097573438206234
        ],
        [
            247,
            2.3084972629675993
        ],
        [
            248,
            2.3406206014994027
        ],
        [
            249,
            2.2870713440147608
        ],
        [
            250,
            2.3126579506977185
        ],
        [
            251,
            2.300220615154988
        ],
        [
            252,
            2.316235102511741
        ],
        [
            253,
            2.2927113707001143
        ],
        [
            254,
            2.2880633682818026
        ],
        [
            255,
            2.3083303651294194
        ],
        [
            256,
            2.3392030097342826
        ],
        [
            257,
            2.3250207901000977
        ],
        [
            258,
            2.286618129627125
        ],
        [
            259,
            2.310433874259124
        ],
        [
            260,
            2.3281621449702494
        ]
    ],
    "r2_mental": [
        [
            1,
            -17.942916108017176
        ],
        [
            2,
            -17.36425702203493
        ],
        [
            3,
            -16.59702425116301
        ],
        [
            4,
            -15.486257836473182
        ],
        [
            5,
            -13.882360330612768
        ],
        [
            6,
            -11.70257537404923
        ],
        [
            7,
            -9.208336874974638
        ],
        [
            8,
            -6.80359866071715
        ],
        [
            9,
            -4.891101812653789
        ],
        [
            10,
            -3.592849806970026
        ],
        [
            11,
            -2.7394967743692713
        ],
        [
            12,
            -2.2176310435159077
        ],
        [
            13,
            -1.8794801876157319
        ],
        [
            14,
            -1.656467010717261
        ],
        [
            15,
            -1.5087276447373212
        ],
        [
            16,
            -1.405830996897739
        ],
        [
            17,
            -1.357645584968025
        ],
        [
            18,
            -1.310077312472131
        ],
        [
            19,
            -1.2669826759314091
        ],
        [
            20,
            -1.2718629671350858
        ],
        [
            21,
            -1.2359600826360517
        ],
        [
            22,
            -1.227890586307233
        ],
        [
            23,
            -1.2151866337170065
        ],
        [
            24,
            -1.211984310639779
        ],
        [
            25,
            -1.179311682469455
        ],
        [
            26,
            -1.1925665719935297
        ],
        [
            27,
            -1.2031906551146014
        ],
        [
            28,
            -1.1790951706986061
        ],
        [
            29,
            -1.1863894976155187
        ],
        [
            30,
            -1.1762736205738777
        ],
        [
            31,
            -1.202040426138951
        ],
        [
            32,
            -1.2116831413247109
        ],
        [
            33,
            -1.1979947368867148
        ],
        [
            34,
            -1.2218078427921601
        ],
        [
            35,
            -1.1786774522722352
        ],
        [
            36,
            -1.2016796785192647
        ],
        [
            37,
            -1.1987138958437722
        ],
        [
            38,
            -1.1764348220080976
        ],
        [
            39,
            -1.1986871786843265
        ],
        [
            40,
            -1.2026036908438003
        ],
        [
            41,
            -1.2260169048706309
        ],
        [
            42,
            -1.2080725328717379
        ],
        [
            43,
            -1.195308862756526
        ],
        [
            44,
            -1.1749396822520004
        ],
        [
            45,
            -1.1836998123968865
        ],
        [
            46,
            -1.1787804018018946
        ],
        [
            47,
            -1.2101019445974623
        ],
        [
            48,
            -1.1930661009447912
        ],
        [
            49,
            -1.229132846629685
        ],
        [
            50,
            -1.1672393825661307
        ],
        [
            51,
            -1.1806480250568434
        ],
        [
            52,
            -1.2014110123657846
        ],
        [
            53,
            -1.1767441189106491
        ],
        [
            54,
            -1.2031169871594138
        ],
        [
            55,
            -1.1755003646040558
        ],
        [
            56,
            -1.2083138814697323
        ],
        [
            57,
            -1.2212294228384302
        ],
        [
            58,
            -1.2412691691373317
        ],
        [
            59,
            -1.1952109863480183
        ],
        [
            60,
            -1.1902916469541989
        ],
        [
            61,
            -1.2132064373898093
        ],
        [
            62,
            -1.2135963069339715
        ],
        [
            63,
            -1.153687509257534
        ],
        [
            64,
            -1.1937053486357154
        ],
        [
            65,
            -1.2098370907630587
        ],
        [
            66,
            -1.1835029279880738
        ],
        [
            67,
            -1.1987911339058748
        ],
        [
            68,
            -1.1985502108579258
        ],
        [
            69,
            -1.18591589389833
        ],
        [
            70,
            -1.1483741799946339
        ],
        [
            71,
            -1.1780938915696506
        ],
        [
            72,
            -1.1774209835585636
        ],
        [
            73,
            -1.1347188249693687
        ],
        [
            74,
            -1.1740570654850564
        ],
        [
            75,
            -1.1730377523105595
        ],
        [
            76,
            -1.1953569801462418
        ],
        [
            77,
            -1.197877110406195
        ],
        [
            78,
            -1.2049872710077238
        ],
        [
            79,
            -1.1372348233282596
        ],
        [
            80,
            -1.232746382867119
        ],
        [
            81,
            -1.2005400187696909
        ],
        [
            82,
            -1.1754448414344032
        ],
        [
            83,
            -1.1489605171766986
        ],
        [
            84,
            -1.1767536801573772
        ],
        [
            85,
            -1.1628848563945242
        ],
        [
            86,
            -1.1876572720982597
        ],
        [
            87,
            -1.1150638640107151
        ],
        [
            88,
            -1.141474329976036
        ],
        [
            89,
            -1.1153766261045321
        ],
        [
            90,
            -1.1437411614549675
        ],
        [
            91,
            -1.1893232987165128
        ],
        [
            92,
            -1.153187885983968
        ],
        [
            93,
            -1.1112275471743773
        ],
        [
            94,
            -1.2146631887685788
        ],
        [
            95,
            -1.1914955709524806
        ],
        [
            96,
            -1.1803326290762586
        ],
        [
            97,
            -1.190360860033203
        ],
        [
            98,
            -1.2193019741348596
        ],
        [
            99,
            -1.1835310077563324
        ],
        [
            100,
            -1.1533257679971238
        ],
        [
            101,
            -1.1727015873571753
        ],
        [
            102,
            -1.2264529781269036
        ],
        [
            103,
            -1.1460344922644299
        ],
        [
            104,
            -1.1409001779940944
        ],
        [
            105,
            -1.1771314397324613
        ],
        [
            106,
            -1.119539860514819
        ],
        [
            107,
            -1.1588905995261882
        ],
        [
            108,
            -1.1745093253521675
        ],
        [
            109,
            -1.1524388586174772
        ],
        [
            110,
            -1.153557246517932
        ],
        [
            111,
            -1.1736515894683053
        ],
        [
            112,
            -1.1619674595171627
        ],
        [
            113,
            -1.1144931371442146
        ],
        [
            114,
            -1.1824872805735276
        ],
        [
            115,
            -1.2084426280485863
        ],
        [
            116,
            -1.1836065845865624
        ],
        [
            117,
            -1.1901113589774126
        ],
        [
            118,
            -1.1671910897455517
        ],
        [
            119,
            -1.147952519401061
        ],
        [
            120,
            -1.1364897812103418
        ],
        [
            121,
            -1.1651282694453617
        ],
        [
            122,
            -1.1189696831917635
        ],
        [
            123,
            -1.1394932261494048
        ],
        [
            124,
            -1.1485707073520666
        ],
        [
            125,
            -1.1241071542870715
        ],
        [
            126,
            -1.1508703741364177
        ],
        [
            127,
            -1.1026937670676364
        ],
        [
            128,
            -1.1551866266836663
        ],
        [
            129,
            -1.1247803724534982
        ],
        [
            130,
            -1.1213007154014822
        ],
        [
            131,
            -1.128172204821961
        ],
        [
            132,
            -1.2079201061107394
        ],
        [
            133,
            -1.1138328251503435
        ],
        [
            134,
            -1.1587844100150226
        ],
        [
            135,
            -1.2088824809212535
        ],
        [
            136,
            -1.0829448593148112
        ],
        [
            137,
            -1.1691538438661908
        ],
        [
            138,
            -1.139930980282759
        ],
        [
            139,
            -1.1137346515730933
        ],
        [
            140,
            -1.070972902479249
        ],
        [
            141,
            -1.1265690484511208
        ],
        [
            142,
            -1.1425103277214559
        ],
        [
            143,
            -1.1379923360286681
        ],
        [
            144,
            -1.1491815673194163
        ],
        [
            145,
            -1.0819875337743667
        ],
        [
            146,
            -1.0700624528990659
        ],
        [
            147,
            -1.0562270867429802
        ],
        [
            148,
            -1.1619969576920086
        ],
        [
            149,
            -1.0135213770595666
        ],
        [
            150,
            -1.136187106303133
        ],
        [
            151,
            -1.1064474315232538
        ],
        [
            152,
            -1.0240916979826582
        ],
        [
            153,
            -1.0474596730309744
        ],
        [
            154,
            -1.1124003507561402
        ],
        [
            155,
            -1.1094642139713553
        ],
        [
            156,
            -1.0844209431085639
        ],
        [
            157,
            -1.0870773507911764
        ],
        [
            158,
            -1.100992928837838
        ],
        [
            159,
            -1.110893126633135
        ],
        [
            160,
            -1.0687958819516088
        ],
        [
            161,
            -1.0829668984146776
        ],
        [
            162,
            -1.0659726557809113
        ],
        [
            163,
            -1.0426870778634996
        ],
        [
            164,
            -1.1300342176154614
        ],
        [
            165,
            -1.0053674802934132
        ],
        [
            166,
            -1.085600054576482
        ],
        [
            167,
            -1.1458116389738162
        ],
        [
            168,
            -1.0814267592859461
        ],
        [
            169,
            -1.028453336113468
        ],
        [
            170,
            -1.0684469702873982
        ],
        [
            171,
            -1.1152747221061499
        ],
        [
            172,
            -1.0959338372320437
        ],
        [
            173,
            -1.0683506565618837
        ],
        [
            174,
            -1.0907306638169323
        ],
        [
            175,
            -1.1062852325323456
        ],
        [
            176,
            -1.0388052014679925
        ],
        [
            177,
            -1.0524356410694864
        ],
        [
            178,
            -1.1645226224303347
        ],
        [
            179,
            -1.0371114069432772
        ],
        [
            180,
            -1.0774362466304046
        ],
        [
            181,
            -1.1136600046028993
        ],
        [
            182,
            -1.1075024920153727
        ],
        [
            183,
            -1.011171544263886
        ],
        [
            184,
            -1.149631500786751
        ],
        [
            185,
            -1.0746884407895556
        ],
        [
            186,
            -1.0441700763765804
        ],
        [
            187,
            -1.1552319138300398
        ],
        [
            188,
            -1.090033553836641
        ],
        [
            189,
            -1.0897963224241662
        ],
        [
            190,
            -1.1105169717819146
        ],
        [
            191,
            -1.082998567459354
        ],
        [
            192,
            -1.1324745712438324
        ],
        [
            193,
            -1.1027127915512867
        ],
        [
            194,
            -1.1208999178722272
        ],
        [
            195,
            -1.1578602662971642
        ],
        [
            196,
            -1.0638350157373906
        ],
        [
            197,
            -1.096600797253584
        ],
        [
            198,
            -1.1885702040177404
        ],
        [
            199,
            -1.0575299445502773
        ],
        [
            200,
            -1.076819613688507
        ],
        [
            201,
            -1.0894335032710627
        ],
        [
            202,
            -1.1302997679554965
        ],
        [
            203,
            -1.1293807465087866
        ],
        [
            204,
            -1.1348760906932185
        ],
        [
            205,
            -1.101684063392479
        ],
        [
            206,
            -1.135612659878075
        ],
        [
            207,
            -1.0856123812899017
        ],
        [
            208,
            -1.104031062653199
        ],
        [
            209,
            -1.1379231095929598
        ],
        [
            210,
            -1.1631402564526985
        ],
        [
            211,
            -1.1011239843118248
        ],
        [
            212,
            -1.1486908193195822
        ],
        [
            213,
            -1.1506083108007128
        ],
        [
            214,
            -1.136745309665387
        ],
        [
            215,
            -1.1864779568846675
        ],
        [
            216,
            -1.0890241136300305
        ],
        [
            217,
            -1.1722643484909159
        ],
        [
            218,
            -1.1733067143104319
        ],
        [
            219,
            -1.1031766338822644
        ],
        [
            220,
            -1.1339800240897246
        ],
        [
            221,
            -1.1362509683846094
        ],
        [
            222,
            -1.1771308281666788
        ],
        [
            223,
            -1.1755229879552802
        ],
        [
            224,
            -1.088841336926449
        ],
        [
            225,
            -1.16711151237494
        ],
        [
            226,
            -1.15175691651286
        ],
        [
            227,
            -1.1146733866627665
        ],
        [
            228,
            -1.1713363878877705
        ],
        [
            229,
            -1.124780565270544
        ],
        [
            230,
            -1.1833249380748478
        ],
        [
            231,
            -1.1289374350991843
        ],
        [
            232,
            -1.2064989351645163
        ],
        [
            233,
            -1.1866381365901266
        ],
        [
            234,
            -1.1189677125189634
        ],
        [
            235,
            -1.1342430169181275
        ],
        [
            236,
            -1.1646088322208343
        ],
        [
            237,
            -1.1301601664466716
        ],
        [
            238,
            -1.153538502161521
        ],
        [
            239,
            -1.1915423622376786
        ],
        [
            240,
            -1.1520215196623749
        ],
        [
            241,
            -1.1844877214428777
        ],
        [
            242,
            -1.1337672419284233
        ],
        [
            243,
            -1.1621680426286884
        ],
        [
            244,
            -1.2187976571148083
        ],
        [
            245,
            -1.1538403346719912
        ],
        [
            246,
            -1.1771250359700596
        ],
        [
            247,
            -1.1673813296392304
        ],
        [
            248,
            -1.2297994928057476
        ],
        [
            249,
            -1.132088952499962
        ],
        [
            250,
            -1.1810880147429819
        ],
        [
            251,
            -1.1537474647955266
        ],
        [
            252,
            -1.1852495693459222
        ],
        [
            253,
            -1.1381283304357095
        ],
        [
            254,
            -1.1308492033810018
        ],
        [
            255,
            -1.1628080377466934
        ],
        [
            256,
            -1.2279120182085932
        ],
        [
            257,
            -1.1963429902141787
        ],
        [
            258,
            -1.1278126321698578
        ],
        [
            259,
            -1.168598659828643
        ],
        [
            260,
            -1.2016226826551404
        ]
    ],
    "loss": [
        [
            1,
            73.49339447021484
        ],
        [
            2,
            71.06403121948242
        ],
        [
            3,
            67.82166976928711
        ],
        [
            4,
            63.129208374023435
        ],
        [
            5,
            56.4359676361084
        ],
        [
            6,
            47.5881893157959
        ],
        [
            7,
            37.70944385528564
        ],
        [
            8,
            28.55520305633545
        ],
        [
            9,
            21.700008964538576
        ],
        [
            10,
            17.295034217834473
        ],
        [
            11,
            14.492219257354737
        ],
        [
            12,
            12.842018795013427
        ],
        [
            13,
            11.799933910369873
        ],
        [
            14,
            11.103137683868407
        ],
        [
            15,
            10.662185430526733
        ],
        [
            16,
            10.36392583847046
        ],
        [
            17,
            10.241693496704102
        ],
        [
            18,
            10.109950304031372
        ],
        [
            19,
            9.976424598693848
        ],
        [
            20,
            10.032673501968384
        ],
        [
            21,
            9.905754232406617
        ],
        [
            22,
            9.901655960083009
        ],
        [
            23,
            9.858843421936035
        ],
        [
            24,
            9.875063800811768
        ],
        [
            25,
            9.777674961090089
        ],
        [
            26,
            9.830772304534912
        ],
        [
            27,
            9.881975984573364
        ],
        [
            28,
            9.8221923828125
        ],
        [
            29,
            9.84423508644104
        ],
        [
            30,
            9.82139482498169
        ],
        [
            31,
            9.925677633285522
        ],
        [
            32,
            9.969515657424926
        ],
        [
            33,
            9.935100317001343
        ],
        [
            34,
            10.011181020736695
        ],
        [
            35,
            9.846202564239501
        ],
        [
            36,
            9.940106773376465
        ],
        [
            37,
            9.956070709228516
        ],
        [
            38,
            9.863840770721435
        ],
        [
            39,
            9.967428493499757
        ],
        [
            40,
            9.965994691848755
        ],
        [
            41,
            10.080665731430054
        ],
        [
            42,
            9.998053121566773
        ],
        [
            43,
            9.954438209533691
        ],
        [
            44,
            9.88920578956604
        ],
        [
            45,
            9.912098121643066
        ],
        [
            46,
            9.89885368347168
        ],
        [
            47,
            10.028443527221679
        ],
        [
            48,
            9.964059638977051
        ],
        [
            49,
            10.107148122787475
        ],
        [
            50,
            9.89353699684143
        ],
        [
            51,
            9.936445713043213
        ],
        [
            52,
            10.01333303451538
        ],
        [
            53,
            9.922928667068481
        ],
        [
            54,
            10.013683938980103
        ],
        [
            55,
            9.920654010772704
        ],
        [
            56,
            10.030531406402588
        ],
        [
            57,
            10.089660692214967
        ],
        [
            58,
            10.179285287857056
        ],
        [
            59,
            10.011703205108642
        ],
        [
            60,
            10.015957689285278
        ],
        [
            61,
            10.098816919326783
        ],
        [
            62,
            10.11364016532898
        ],
        [
            63,
            9.875975084304809
        ],
        [
            64,
            10.044047451019287
        ],
        [
            65,
            10.132554817199708
        ],
        [
            66,
            10.018142557144165
        ],
        [
            67,
            10.08886432647705
        ],
        [
            68,
            10.079104280471801
        ],
        [
            69,
            10.050994777679444
        ],
        [
            70,
            9.915691661834718
        ],
        [
            71,
            10.035225772857666
        ],
        [
            72,
            10.048654460906983
        ],
        [
            73,
            9.903788661956787
        ],
        [
            74,
            10.04801058769226
        ],
        [
            75,
            10.064421033859253
        ],
        [
            76,
            10.146411895751953
        ],
        [
            77,
            10.163035106658935
        ],
        [
            78,
            10.194021844863892
        ],
        [
            79,
            9.95839982032776
        ],
        [
            80,
            10.315087604522706
        ],
        [
            81,
            10.225543117523193
        ],
        [
            82,
            10.140157556533813
        ],
        [
            83,
            10.055559396743774
        ],
        [
            84,
            10.167370080947876
        ],
        [
            85,
            10.120814800262451
        ],
        [
            86,
            10.236773204803466
        ],
        [
            87,
            9.962421703338624
        ],
        [
            88,
            10.058269357681274
        ],
        [
            89,
            9.998495244979859
        ],
        [
            90,
            10.111409854888915
        ],
        [
            91,
            10.27723307609558
        ],
        [
            92,
            10.163406658172608
        ],
        [
            93,
            10.01755633354187
        ],
        [
            94,
            10.409616374969483
        ],
        [
            95,
            10.324372100830079
        ],
        [
            96,
            10.261510467529297
        ],
        [
            97,
            10.345867013931274
        ],
        [
            98,
            10.434633684158324
        ],
        [
            99,
            10.337465572357178
        ],
        [
            100,
            10.229892921447753
        ],
        [
            101,
            10.319452714920043
        ],
        [
            102,
            10.508875751495362
        ],
        [
            103,
            10.255077600479126
        ],
        [
            104,
            10.252119159698486
        ],
        [
            105,
            10.389139795303345
        ],
        [
            106,
            10.204741621017456
        ],
        [
            107,
            10.34558982849121
        ],
        [
            108,
            10.427281045913697
        ],
        [
            109,
            10.365307569503784
        ],
        [
            110,
            10.372583436965943
        ],
        [
            111,
            10.461389827728272
        ],
        [
            112,
            10.414804220199585
        ],
        [
            113,
            10.290901231765748
        ],
        [
            114,
            10.531573009490966
        ],
        [
            115,
            10.621666383743285
        ],
        [
            116,
            10.546423530578613
        ],
        [
            117,
            10.57809820175171
        ],
        [
            118,
            10.509465789794922
        ],
        [
            119,
            10.449947071075439
        ],
        [
            120,
            10.418898773193359
        ],
        [
            121,
            10.542122745513916
        ],
        [
            122,
            10.397054958343507
        ],
        [
            123,
            10.46302671432495
        ],
        [
            124,
            10.51530213356018
        ],
        [
            125,
            10.448187828063965
        ],
        [
            126,
            10.540604639053345
        ],
        [
            127,
            10.408759164810181
        ],
        [
            128,
            10.585215377807618
        ],
        [
            129,
            10.484575271606445
        ],
        [
            130,
            10.473794174194335
        ],
        [
            131,
            10.546492099761963
        ],
        [
            132,
            10.789982795715332
        ],
        [
            133,
            10.484960174560547
        ],
        [
            134,
            10.681259441375733
        ],
        [
            135,
            10.87507677078247
        ],
        [
            136,
            10.411051368713379
        ],
        [
            137,
            10.761820125579835
        ],
        [
            138,
            10.683090829849244
        ],
        [
            139,
            10.579694080352784
        ],
        [
            140,
            10.430660915374755
        ],
        [
            141,
            10.672016429901124
        ],
        [
            142,
            10.694562625885009
        ],
        [
            143,
            10.704165649414062
        ],
        [
            144,
            10.794619274139404
        ],
        [
            145,
            10.584248208999634
        ],
        [
            146,
            10.557199954986572
        ],
        [
            147,
            10.50104923248291
        ],
        [
            148,
            10.915602016448975
        ],
        [
            149,
            10.378017616271972
        ],
        [
            150,
            10.838506031036378
        ],
        [
            151,
            10.721073722839355
        ],
        [
            152,
            10.462827777862548
        ],
        [
            153,
            10.580070209503173
        ],
        [
            154,
            10.834808349609375
        ],
        [
            155,
            10.836408042907715
        ],
        [
            156,
            10.751734066009522
        ],
        [
            157,
            10.791648292541504
        ],
        [
            158,
            10.861216259002685
        ],
        [
            159,
            10.873755264282227
        ],
        [
            160,
            10.76302375793457
        ],
        [
            161,
            10.851496028900147
        ],
        [
            162,
            10.832925033569335
        ],
        [
            163,
            10.73631706237793
        ],
        [
            164,
            11.08828649520874
        ],
        [
            165,
            10.639640712738037
        ],
        [
            166,
            10.911587715148926
        ],
        [
            167,
            11.158052778244018
        ],
        [
            168,
            10.964865827560425
        ],
        [
            169,
            10.764536952972412
        ],
        [
            170,
            10.94406509399414
        ],
        [
            171,
            11.136826515197754
        ],
        [
            172,
            11.089162111282349
        ],
        [
            173,
            11.010536193847656
        ],
        [
            174,
            11.116565942764282
        ],
        [
            175,
            11.15025053024292
        ],
        [
            176,
            10.924738597869872
        ],
        [
            177,
            11.013699913024903
        ],
        [
            178,
            11.408204269409179
        ],
        [
            179,
            10.989937543869019
        ],
        [
            180,
            11.137735033035279
        ],
        [
            181,
            11.25212640762329
        ],
        [
            182,
            11.258305501937866
        ],
        [
            183,
            10.968784713745118
        ],
        [
            184,
            11.449552917480469
        ],
        [
            185,
            11.188579177856445
        ],
        [
            186,
            11.145746517181397
        ],
        [
            187,
            11.51907901763916
        ],
        [
            188,
            11.3159969329834
        ],
        [
            189,
            11.348439168930053
        ],
        [
            190,
            11.429219055175782
        ],
        [
            191,
            11.395095157623292
        ],
        [
            192,
            11.57277183532715
        ],
        [
            193,
            11.461428594589233
        ],
        [
            194,
            11.53333683013916
        ],
        [
            195,
            11.692171096801758
        ],
        [
            196,
            11.364656114578247
        ],
        [
            197,
            11.48820629119873
        ],
        [
            198,
            11.816984939575196
        ],
        [
            199,
            11.334535455703735
        ],
        [
            200,
            11.447785472869873
        ],
        [
            201,
            11.509562826156616
        ],
        [
            202,
            11.664899444580078
        ],
        [
            203,
            11.648122692108155
        ],
        [
            204,
            11.659370565414429
        ],
        [
            205,
            11.584424114227295
        ],
        [
            206,
            11.708930206298827
        ],
        [
            207,
            11.54256467819214
        ],
        [
            208,
            11.668849849700928
        ],
        [
            209,
            11.820117664337157
        ],
        [
            210,
            11.814397621154786
        ],
        [
            211,
            11.651556253433228
        ],
        [
            212,
            11.856311130523682
        ],
        [
            213,
            11.845601987838744
        ],
        [
            214,
            11.845917320251464
        ],
        [
            215,
            12.01529369354248
        ],
        [
            216,
            11.679697513580322
        ],
        [
            217,
            11.975845336914062
        ],
        [
            218,
            12.00101580619812
        ],
        [
            219,
            11.761644172668458
        ],
        [
            220,
            11.825785732269287
        ],
        [
            221,
            11.848464679718017
        ],
        [
            222,
            12.00333013534546
        ],
        [
            223,
            12.01815333366394
        ],
        [
            224,
            11.73521809577942
        ],
        [
            225,
            12.005722522735596
        ],
        [
            226,
            11.954674434661865
        ],
        [
            227,
            11.812664318084718
        ],
        [
            228,
            12.060292816162109
        ],
        [
            229,
            11.868048095703125
        ],
        [
            230,
            12.081922054290771
        ],
        [
            231,
            11.923490715026855
        ],
        [
            232,
            12.171434307098389
        ],
        [
            233,
            12.094025039672852
        ],
        [
            234,
            11.891954469680787
        ],
        [
            235,
            11.978939628601074
        ],
        [
            236,
            12.086927318572998
        ],
        [
            237,
            11.979794120788574
        ],
        [
            238,
            12.076354217529296
        ],
        [
            239,
            12.184114837646485
        ],
        [
            240,
            12.088551425933838
        ],
        [
            241,
            12.218927478790283
        ],
        [
            242,
            12.030534553527833
        ],
        [
            243,
            12.095071792602539
        ],
        [
            244,
            12.345878314971923
        ],
        [
            245,
            12.124704551696777
        ],
        [
            246,
            12.193491172790527
        ],
        [
            247,
            12.201976871490478
        ],
        [
            248,
            12.37477798461914
        ],
        [
            249,
            11.998213005065917
        ],
        [
            250,
            12.199046230316162
        ],
        [
            251,
            12.111500358581543
        ],
        [
            252,
            12.284029293060303
        ],
        [
            253,
            12.111606025695801
        ],
        [
            254,
            12.076428699493409
        ],
        [
            255,
            12.19479579925537
        ],
        [
            256,
            12.369746017456055
        ],
        [
            257,
            12.25938663482666
        ],
        [
            258,
            12.070734786987305
        ],
        [
            259,
            12.214260864257813
        ],
        [
            260,
            12.33377513885498
        ]
    ]
}

metrics_keys = [
    'loss',
    'mae_physical',
    'mae_mental',
    'r2_mental',
    'r2_physical',
    'rmse_mental',
    'rmse_physical'
]

# List containing all the dictionaries
all_dicts = [afda_fl_metrics, fl_base_metrics, central_afda_metrics, central_base_metrics]
# Labels for the lines/models (customize as needed)
model_labels = ['AFDA-FL(Proposed)', 'Federated LSTM (Baseline)', 'Centralized AFDA', 'Centralized LSTM (Baseline)']

# --- Plotting Code ---

# Loop through each metric to create a separate plot
for metric in metrics_keys:
    print(f"Generating plot for: {metric}...")
    # Create a new figure and axes for each metric
    plt.figure(figsize=(12, 7)) # Slightly larger figure for 80 rounds
    ax = plt.gca() # Get current axes

    # Loop through each model's dictionary and its corresponding label
    for i, model_dic in enumerate(all_dicts):
        # Check if the metric exists in the current dictionary
        if metric in model_dic:
            # Extract rounds (x) and values (y)
            # Ensure data is sorted by round
            try:
                # Attempt to sort, assuming list of tuples format
                data_points = sorted(model_dic[metric], key=lambda item: item[0])
            except (TypeError, IndexError):
                print(f"  Warning: Could not sort data for metric '{metric}' in {model_labels[i]}. Ensure it's a list of (round, value) tuples.")
                continue # Skip plotting this line if data format is wrong

            if data_points: # Check if there's data for this metric
                rounds = [point[0] for point in data_points]
                values = [point[1] for point in data_points]

                # Check if we have data points to plot
                if rounds and values:
                    # Plot the line for the current model
                    ax.plot(rounds, values, marker='.', linestyle='-', markersize=4, label=model_labels[i]) # Use smaller markers
                else:
                    print(f"  Warning: Empty rounds or values list for metric '{metric}' in {model_labels[i]}")
            else:
                print(f"  Warning: No data points found for metric '{metric}' in {model_labels[i]}")
        else:
            print(f"  Warning: Metric '{metric}' not found in {model_labels[i]}")

    # --- Customize the plot ---
    plot_title = f'{metric.replace("_", " ").title()} Trend Across Models (260 Rounds)'
    ax.set_title(plot_title)
    ax.set_xlabel('Round / Epoch')
    ax.set_ylabel(metric.replace("_", " ").title())

    # Only show legend if lines were actually plotted
    if ax.has_data():
        ax.legend()
        ax.grid(True, linestyle='--', alpha=0.6)
    else:
        print(f"  Note: No lines were plotted for '{metric}'. Skipping legend and grid.")

    # Improve layout
    plt.tight_layout()

    # --- Save the plot ---
    # Create a filename friendly version of the metric name
    filename = f'{metric}.png'
    try:
        plt.savefig(filename)
        print(f"  Saved plot: {filename}")
    except Exception as e:
        print(f"  Error saving plot {filename}: {e}")


    # plt.show() # Uncomment this if running locally and want interactive plots

# --- Cleanup ---
# Close all figures to free memory, especially important if generating many plots
plt.close('all')

print("\nFinished generating all plots.")

Generating plot for: loss...
  Saved plot: loss.png
Generating plot for: mae_physical...
  Saved plot: mae_physical.png
Generating plot for: mae_mental...
  Saved plot: mae_mental.png
Generating plot for: r2_mental...
  Saved plot: r2_mental.png
Generating plot for: r2_physical...
  Saved plot: r2_physical.png
Generating plot for: rmse_mental...
  Saved plot: rmse_mental.png
Generating plot for: rmse_physical...
  Saved plot: rmse_physical.png

Finished generating all plots.
